In [48]:
import ollama
import json
import random
import os
import requests
import re
import datetime
import copy
import uuid
import math
import traceback
import multiprocessing as mp

from concurrent.futures import ProcessPoolExecutor, as_completed
from jinja2 import Template
from IPython.display import display, HTML, JSON, Markdown
from typing import Dict, Any, List, Optional, Tuple, Set
from abc import ABC, abstractmethod
from dataclasses import asdict, is_dataclass,dataclass, field
from enum import Enum
from dataclasses import dataclass


In [2]:
import os
import sys

# 1. מגדירים את הנתיבים
inner_folder = r"C:\Users\nivsa\Generation of Synthetic Training Data\embedded\synthetic_pipeline"
parent_folder = r"C:\Users\nivsa\Generation of Synthetic Training Data\embedded"

# 2. נכנסים פיזית לתיקייה הראשית/ההורה (השינוי שביקשת)
os.chdir(parent_folder)
print(f"📍 Working Directory: {os.getcwd()}")

# 3. קריטי: מוסיפים את התיקייה הפנימית לנתיב החיפוש
# חובה! אחרת כשתהיה בתיקייה הראשית, הקבצים הפנימיים לא יצליחו לעשות import אחד לשני
if inner_folder not in sys.path:
    sys.path.insert(0, inner_folder)
    print(f"✅ Added inner folder to path: ...\\synthetic_pipeline")

# 4. מוודאים שגם התיקייה הנוכחית (הראשית) בנתיב
if parent_folder not in sys.path:
    sys.path.insert(0, parent_folder)

📍 Working Directory: C:\Users\nivsa\Generation of Synthetic Training Data\embedded
✅ Added inner folder to path: ...\synthetic_pipeline


In [8]:
%%writefile synthetic_pipeline/strategies.py
MODEL_TO_USE = 'qwen2.5:3b' 

# ==========================================
# UNIFIED COMPONENT DATABASE - EXPANDED EDITION
# ==========================================

UNIFIED_COMPONENT_DB = {

    # ==============================================================================
    # 1. RESISTOR חלק1
    # ==============================================================================
    
    "RESISTOR": {
        "archetypes": ["Standard_SMD", "High_Power_THT", "Precision_ThinFilm"],
        
        "ABS_MAX": [
            {
                "key": "power_rating",
                "symbol": "P<sub>rated</sub>",
                "spec_type": "max_rating",
                "column_model": "MAX_ONLY",
                "engineering_class": "SAFETY_LIMIT",
                "special_semantics": "NONE",
                "llm_context": {
                    "formal_name": "Power Rating",
                    "aliases": ["Rated Power", "Power Dissipation", "Rated Dissipation", "Maximum Power"]
                },
                "possible_units": ["W", "mW"],
                "std_unit": "W",
                "scenarios": [
                    {
                        "condition": "Ambient 70°C",
                        "limits": {
                            "Standard_SMD": [0.031, 0.063, 0.1, 0.125, 0.25, 0.5, 0.75, 1.0],
                            "High_Power_THT": [1, 2, 3, 5, 7, 10, 15, 20, 25, 50, 100, 150, 225],
                            "Precision_ThinFilm": [0.063, 0.1, 0.125, 0.25, 0.4, 0.5]
                        }
                    }
                ]
            },
            {
                "key": "max_working_voltage",
                "symbol": "V<sub>max</sub>",
                "spec_type": "max_rating",
                "column_model": "MAX_ONLY",
                "engineering_class": "SAFETY_LIMIT",
                "special_semantics": "NONE",
                "llm_context": {
                    "formal_name": "Maximum Working Voltage",
                    "aliases": ["Max Working Voltage", "Rated Voltage", "Maximum Continuous Voltage", "VRMS"]
                },
                "possible_units": ["V", "kV"],
                "std_unit": "V",
                "scenarios": [
                    {
                        "condition": "DC Continuous",
                        "limits": {
                            "Standard_SMD": [25, 50, 75, 100, 150, 200, 250, 400, 500],
                            "High_Power_THT": [250, 350, 500, 750, 1000, 1500, 2000, 2500, 3500, 5000],
                            "Precision_ThinFilm": [25, 50, 75, 100, 150, 200, 250]
                        }
                    }
                ]
            },
            {
                "key": "max_overload_voltage",
                "symbol": "V<sub>OL</sub>",
                "spec_type": "max_rating",
                "column_model": "MAX_ONLY",
                "engineering_class": "SAFETY_LIMIT",
                "special_semantics": "NONE",
                "llm_context": {
                    "formal_name": "Maximum Overload Voltage",
                    "aliases": ["Overload Voltage", "Short-time Overload Voltage", "Dielectric Withstanding Voltage"]
                },
                "possible_units": ["V"],
                "std_unit": "V",
                "scenarios": [
                    {
                        "condition": "5 seconds",
                        "limits": {
                            "Standard_SMD": [50, 100, 150, 200, 300, 400, 500, 800],
                            "High_Power_THT": [500, 700, 1000, 1500, 2000, 3000, 5000, 7500],
                            "Precision_ThinFilm": [50, 100, 150, 200, 300, 400, 500]
                        }
                    }
                ]
            },
            {
                "key": "pulse_withstanding",
                "symbol": "P<sub>pulse</sub>",
                "spec_type": "max_rating",
                "column_model": "MAX_ONLY",
                "engineering_class": "SAFETY_LIMIT",
                "special_semantics": "NONE",
                "llm_context": {
                    "formal_name": "Pulse Withstanding Power",
                    "aliases": ["Peak Pulse Power", "Surge Power Capability", "Single Pulse Rating"]
                },
                "possible_units": ["W", "kW"],
                "std_unit": "W",
                "scenarios": [
                    {
                        "condition": "1ms pulse",
                        "limits": {
                            "Standard_SMD": [0.25, 0.5, 1, 2, 3, 5, 10],
                            "High_Power_THT": [25, 50, 100, 150, 200, 300, 500, 1000],
                            "Precision_ThinFilm": [0.25, 0.5, 1, 2, 3]
                        }
                    }
                ]
            },
            {
                "key": "max_operating_temp",
                "symbol": "T<sub>max</sub>",
                "spec_type": "max_rating",
                "column_model": "MAX_ONLY",
                "engineering_class": "SAFETY_LIMIT",
                "special_semantics": "NONE",
                "llm_context": {
                    "formal_name": "Maximum Operating Temperature",
                    "aliases": ["Max Operating Temp", "Upper Category Temperature", "Maximum Ambient Temperature"]
                },
                "possible_units": ["°C"],
                "std_unit": "°C",
                "scenarios": [
                    {
                        "condition": "Continuous",
                        "limits": {
                            "Standard_SMD": [125, 155, 170],
                            "High_Power_THT": [155, 175, 200, 225, 275, 350],
                            "Precision_ThinFilm": [125, 155, 175]
                        }
                    }
                ]
            },
            {
                "key": "min_operating_temp",
                "symbol": "T<sub>min</sub>",
                "spec_type": "min_rating",
                "column_model": "MIN_ONLY",
                "engineering_class": "SAFETY_LIMIT",
                "special_semantics": "NONE",
                "llm_context": {
                    "formal_name": "Minimum Operating Temperature",
                    "aliases": ["Min Operating Temp", "Lower Category Temperature"]
                },
                "possible_units": ["°C"],
                "std_unit": "°C",
                "scenarios": [
                    {
                        "condition": "Continuous",
                        "limits": {
                            "Standard_SMD": [-55, -40],
                            "High_Power_THT": [-65, -55, -40],
                            "Precision_ThinFilm": [-55, -40]
                        }
                    }
                ]
            },
            {
                "key": "max_current",
                "symbol": "I<sub>max</sub>",
                "spec_type": "max_rating",
                "column_model": "MAX_ONLY",
                "engineering_class": "SAFETY_LIMIT",
                "special_semantics": "DERIVED",
                "llm_context": {
                    "formal_name": "Maximum Continuous Current",
                    "aliases": ["Max Current", "Rated Current", "Current Rating"]
                },
                "possible_units": ["A", "mA"],
                "std_unit": "A",
                "scenarios": [
                    {
                        "condition": "Continuous DC, calculated from P=I²R",
                        "limits": {
                            "Standard_SMD": [0.05, 0.1, 0.2, 0.5, 1, 2],
                            "High_Power_THT": [1, 2, 5, 10, 15, 20, 30, 50],
                            "Precision_ThinFilm": [0.05, 0.1, 0.2, 0.5, 1]
                        }
                    }
                ]
            },
            {
                "key": "esd_rating",
                "symbol": "ESD",
                "spec_type": "max_rating",
                "column_model": "MAX_ONLY",
                "engineering_class": "SAFETY_LIMIT",
                "special_semantics": "NONE",
                "llm_context": {
                    "formal_name": "Electrostatic Discharge Rating",
                    "aliases": ["ESD Rating", "ESD Sensitivity", "HBM"]
                },
                "possible_units": ["kV"],
                "std_unit": "kV",
                "scenarios": [
                    {
                        "condition": "Human Body Model (HBM)",
                        "limits": {
                            "Standard_SMD": [0.5, 1, 2, 4, 8, 15],
                            "High_Power_THT": [2, 4, 8, 15, 25],
                            "Precision_ThinFilm": [0.5, 1, 2, 4, 8]
                        }
                    }
                ]
            },
            {
                "key": "max_element_temp",
                "symbol": "T<sub>elem</sub>",
                "spec_type": "max_rating",
                "column_model": "MAX_ONLY",
                "engineering_class": "SAFETY_LIMIT",
                "special_semantics": "NONE",
                "llm_context": {
                    "formal_name": "Maximum Element Temperature",
                    "aliases": ["Hot Spot Temperature", "Maximum Resistive Element Temperature"]
                },
                "possible_units": ["°C"],
                "std_unit": "°C",
                "scenarios": [
                    {
                        "condition": "At rated power",
                        "limits": {
                            "Standard_SMD": [155, 175],
                            "High_Power_THT": [275, 350, 400, 450, 500],
                            "Precision_ThinFilm": [155, 175]
                        }
                    }
                ]
            }
        ],
        
        "ELEC_CHAR": [
            {
                "key": "resistance",
                "symbol": "R",
                "spec_type": "nominal",
                "column_model": "MIN_TYP_MAX",
                "engineering_class": "NOMINAL_PARAMETER",
                "special_semantics": "NONE",
                "llm_context": {
                    "formal_name": "Resistance",
                    "aliases": ["Resistance Value", "Ohmic Value", "DC Resistance", "Nominal Resistance"]
                },
                "possible_units": ["Ω", "kΩ", "MΩ", "mΩ"],
                "std_unit": "Ω",
                "scenarios": [
                    {
                        "condition": "Standard E-Series (E24/E96/E192)",
                        "limits": {
                            "Standard_SMD": [0.01, 0.1, 1, 10, 22, 47, 100, 220, 470, 1000, 2200, 4700, 10000, 22000, 47000, 100000, 220000, 470000, 1000000, 10000000],
                            "High_Power_THT": [0.001, 0.01, 0.1, 0.22, 0.47, 1, 2.2, 4.7, 10, 22, 47, 100, 220, 470, 1000, 10000],
                            "Precision_ThinFilm": [10, 24.9, 49.9, 100, 249, 499, 1000, 2000, 4990, 10000, 20000, 49900, 100000, 249000, 1000000]
                        }
                    }
                ]
            },
            {
                "key": "tolerance",
                "symbol": "Tol",
                "spec_type": "nominal",
                "column_model": "TYP_ONLY",
                "engineering_class": "NOMINAL_PARAMETER",
                "special_semantics": "NONE",
                "llm_context": {
                    "formal_name": "Resistance Tolerance",
                    "aliases": ["Tolerance", "Resistance Accuracy", "Initial Tolerance"]
                },
                "possible_units": ["%"],
                "std_unit": "%",
                "scenarios": [
                    {
                        "condition": "At 25°C",
                        "limits": {
                            "Standard_SMD": [0.5, 1, 2, 5, 10],
                            "High_Power_THT": [1, 2, 5, 10, 20],
                            "Precision_ThinFilm": [0.01, 0.02, 0.05, 0.1, 0.25, 0.5, 1]
                        }
                    }
                ]
            },
            {
                "key": "tcr",
                "symbol": "TCR",
                "spec_type": "max_limit",
                "column_model": "TYP_MAX",
                "engineering_class": "PERFORMANCE_LIMIT",
                "special_semantics": "NONE",
                "llm_context": {
                    "formal_name": "Temperature Coefficient of Resistance",
                    "aliases": ["Temp. Coefficient", "TCR", "Temperature Coefficient", "Tempco"]
                },
                "possible_units": ["ppm/°C", "ppm/K"],
                "std_unit": "ppm/°C",
                "scenarios": [
                    {
                        "condition": "Over operating range",
                        "limits": {
                            "Standard_SMD": [50, 100, 200, 400, 600],
                            "High_Power_THT": [100, 200, 350, 500, 1000, 1500],
                            "Precision_ThinFilm": [2, 5, 10, 15, 25, 50, 100]
                        }
                    }
                ]
            },
            {
                "key": "load_life_stability",
                "symbol": "ΔR/R",
                "spec_type": "max_limit",
                "column_model": "MAX_ONLY",
                "engineering_class": "PERFORMANCE_LIMIT",
                "special_semantics": "NONE",
                "llm_context": {
                    "formal_name": "Load Life Stability",
                    "aliases": ["Load Life", "Long-term Drift", "Stability", "Endurance", "Life Test"]
                },
                "possible_units": ["%"],
                "std_unit": "%",
                "scenarios": [
                    {
                        "condition": "1000h @ Rated Power & 70°C",
                        "limits": {
                            "Standard_SMD": [0.5, 1, 2, 3, 5],
                            "High_Power_THT": [2, 3, 5, 10],
                            "Precision_ThinFilm": [0.02, 0.05, 0.1, 0.25, 0.5]
                        }
                    },
                    {
                        "condition": "2000h @ Rated Power & 70°C",
                        "limits": {
                            "Standard_SMD": [1, 2, 3, 5],
                            "High_Power_THT": [3, 5, 10],
                            "Precision_ThinFilm": [0.05, 0.1, 0.25, 0.5]
                        }
                    }
                ]
            },
            {
                "key": "voltage_coefficient",
                "symbol": "VCR",
                "spec_type": "max_limit",
                "column_model": "MAX_ONLY",
                "engineering_class": "PERFORMANCE_LIMIT",
                "special_semantics": "NONE",
                "llm_context": {
                    "formal_name": "Voltage Coefficient of Resistance",
                    "aliases": ["Voltage Coefficient", "VCR", "Voltage Dependence"]
                },
                "possible_units": ["ppm/V"],
                "std_unit": "ppm/V",
                "scenarios": [
                    {
                        "condition": "At rated voltage",
                        "limits": {
                            "Standard_SMD": [10, 50, 100, 200],
                            "High_Power_THT": [50, 100, 200, 500],
                            "Precision_ThinFilm": [0.1, 0.5, 1, 2, 5, 10]
                        }
                    }
                ]
            },
            {
                "key": "noise",
                "symbol": "NI",
                "spec_type": "max_limit",
                "column_model": "MAX_ONLY",
                "engineering_class": "PERFORMANCE_LIMIT",
                "special_semantics": "NONE",
                "llm_context": {
                    "formal_name": "Current Noise Index",
                    "aliases": ["Current Noise", "Excess Noise", "Noise Index", "1/f Noise"]
                },
                "possible_units": ["dB", "µV/V"],
                "std_unit": "dB",
                "scenarios": [
                    {
                        "condition": "Per voltage decade",
                        "limits": {
                            "Standard_SMD": [-20, -10, 0, 10],
                            "High_Power_THT": [-10, 0, 10, 20],
                            "Precision_ThinFilm": [-40, -35, -30, -25]
                        }
                    }
                ]
            },
            {
                "key": "thermal_resistance",
                "symbol": "θ<sub>JA</sub>",
                "spec_type": "nominal",
                "column_model": "TYP_MAX",
                "engineering_class": "NOMINAL_PARAMETER",
                "special_semantics": "NONE",
                "llm_context": {
                    "formal_name": "Thermal Resistance Junction to Ambient",
                    "aliases": ["Thermal Resistance", "Thermal Resistance J-A", "Rth(j-a)", "θJA"]
                },
                "possible_units": ["°C/W", "K/W"],
                "std_unit": "°C/W",
                "scenarios": [
                    {
                        "condition": "Junction to Ambient, Still Air",
                        "limits": {
                            "Standard_SMD": [400, 300, 200, 150, 100, 50, 35, 25],
                            "High_Power_THT": [100, 75, 50, 35, 25, 15, 10, 5, 3, 2, 1],
                            "Precision_ThinFilm": [400, 300, 200, 150, 100, 75]
                        }
                    }
                ]
            },
            {
                "key": "derating_curve",
                "symbol": "T<sub>derate</sub>",
                "spec_type": "nominal",
                "column_model": "TYP_ONLY",
                "engineering_class": "NOMINAL_PARAMETER",
                "special_semantics": "NONE",
                "llm_context": {
                    "formal_name": "Power Derating Temperature",
                    "aliases": ["Derating Temperature", "Rated Ambient Temperature", "Power Derating Start"]
                },
                "possible_units": ["°C"],
                "std_unit": "°C",
                "scenarios": [
                    {
                        "condition": "Linear derating starts",
                        "limits": {
                            "Standard_SMD": [70, 85],
                            "High_Power_THT": [25, 50, 70],
                            "Precision_ThinFilm": [70, 85, 100]
                        }
                    }
                ]
            },
            
            {
                "key": "self_resonant_freq",
                "symbol": "f<sub>res</sub>",
                "spec_type": "nominal",
                "column_model": "MIN_TYP_MAX",
                "engineering_class": "NOMINAL_PARAMETER",
                "special_semantics": "NONE",
                "llm_context": {
                    "formal_name": "Self Resonant Frequency",
                    "aliases": ["SRF", "Resonant Frequency"]
                },
                "possible_units": ["MHz", "GHz"],
                "std_unit": "MHz",
                "scenarios": [
                    {
                        "condition": "Typical",
                        "limits": {
                            "Standard_SMD": [100, 500, 1000, 2000, 5000, 10000],
                            "High_Power_THT": [1, 5, 10, 50, 100],
                            "Precision_ThinFilm": [500, 1000, 2000, 5000]
                        }
                    }
                ]
            },
            {
                "key": "parasitic_inductance",
                "symbol": "L<sub>par</sub>",
                "spec_type": "nominal",
                "column_model": "TYP_MAX",
                "engineering_class": "NOMINAL_PARAMETER",
                "special_semantics": "NONE",
                "llm_context": {
                    "formal_name": "Parasitic Inductance",
                    "aliases": ["Series Inductance", "ESL", "Self-Inductance"]
                },
                "possible_units": ["nH", "pH"],
                "std_unit": "nH",
                "scenarios": [
                    {
                        "condition": "Typical at 1MHz",
                        "limits": {
                            "Standard_SMD": [0.2, 0.5, 1, 2, 5],
                            "High_Power_THT": [5, 10, 20, 50, 100],
                            "Precision_ThinFilm": [0.2, 0.5, 1, 2]
                        }
                    }
                ]
            },
            {
                "key": "parasitic_capacitance",
                "symbol": "C<sub>par</sub>",
                "spec_type": "nominal",
                "column_model": "TYP_MAX",
                "engineering_class": "NOMINAL_PARAMETER",
                "special_semantics": "NONE",
                "llm_context": {
                    "formal_name": "Parasitic Capacitance",
                    "aliases": ["Shunt Capacitance", "Parallel Capacitance"]
                },
                "possible_units": ["pF", "fF"],
                "std_unit": "pF",
                "scenarios": [
                    {
                        "condition": "Typical",
                        "limits": {
                            "Standard_SMD": [0.05, 0.1, 0.2, 0.5, 1],
                            "High_Power_THT": [1, 2, 5, 10, 20],
                            "Precision_ThinFilm": [0.05, 0.1, 0.2, 0.5]
                        }
                    }
                ]
            }
        ],
        
        "PACKAGE": [
            {
                "key": "package_code",
                "symbol": "PKG",
                "spec_type": "mechanical",
                "column_model": "TYP_ONLY",
                "engineering_class": "MECHANICAL",
                "special_semantics": "CATEGORICAL",
                "llm_context": {
                    "formal_name": "Package Type",
                    "aliases": ["Case Size", "Package Code", "Size Code", "Form Factor"]
                },
                "possible_units": [""],
                "std_unit": "",
                "scenarios": [
                    {
                        "condition": "Standard packages",
                        "limits": {
                            "Standard_SMD": ["01005", "0201", "0402", "0603", "0805", "1206", "1210", "1218", "2010", "2512"],
                            "High_Power_THT": [
                                "Axial_0.4W", "Axial_0.6W", "Axial_1W", "Axial_2W", "Axial_3W", "Axial_5W", "Axial_7W", "Axial_10W", 
                                "TO-220", "TO-263", "TO-247"
                            ],
                            "Precision_ThinFilm": ["0402", "0603", "0805", "1206", "1210", "2010"]
                        }
                    }
                ]
            },
            {
                "key": "moisture_sensitivity",
                "symbol": "MSL",
                "spec_type": "nominal",
                "column_model": "TYP_ONLY",
                "engineering_class": "MECHANICAL",
                "special_semantics": "CATEGORICAL",
                "llm_context": {
                    "formal_name": "Moisture Sensitivity Level",
                    "aliases": ["MSL", "Moisture Sensitivity", "MSL Rating"]
                },
                "possible_units": [""],
                "std_unit": "",
                "scenarios": [
                    {
                        "condition": "Per JEDEC J-STD-020",
                        "limits": {
                            "Standard_SMD": [1, 2, 3],
                            "High_Power_THT": [1],
                            "Precision_ThinFilm": [1, 2, 3]
                        }
                    }
                ]
            },
            {
                "key": "length",
                "symbol": "L",
                "spec_type": "mechanical",
                "column_model": "MIN_TYP_MAX",
                "engineering_class": "MECHANICAL",
                "special_semantics": "NONE",
                "llm_context": {
                    "formal_name": "Package Length",
                    "aliases": ["Length", "Body Length"]
                },
                "possible_units": ["mm", "inch"],
                "std_unit": "mm",
                "scenarios": [
                    {
                        "condition": "Nominal dimensions",
                        "limits": {
                            "Standard_SMD": [0.4, 0.6, 1.0, 1.6, 2.0, 3.2, 3.2, 3.2, 5.0, 6.4],
                            "High_Power_THT": [
                                3.5, 6.5, 9.0, 11.0, 15.0, 25.0, 35.0, 52.0, 
                                15.0, 15.0, 20.0
                            ],
                            "Precision_ThinFilm": [1.0, 1.6, 2.0, 3.2, 3.2, 5.0]
                        }
                    }
                ]
            },
            {
                "key": "width",
                "symbol": "W",
                "spec_type": "mechanical",
                "column_model": "MIN_TYP_MAX",
                "engineering_class": "MECHANICAL",
                "special_semantics": "NONE",
                "llm_context": {
                    "formal_name": "Package Width",
                    "aliases": ["Width", "Body Width", "Diameter"]
                },
                "possible_units": ["mm", "inch"],
                "std_unit": "mm",
                "scenarios": [
                    {
                        "condition": "Nominal dimensions",
                        "limits": {
                            "Standard_SMD": [0.2, 0.3, 0.5, 0.8, 1.25, 1.6, 2.5, 3.2, 2.5, 3.2],
                            "High_Power_THT": [
                                1.9, 2.5, 3.5, 4.5, 5.5, 8.0, 8.5, 8.5, 
                                10.0, 10.0, 15.0
                            ],
                            "Precision_ThinFilm": [0.5, 0.8, 1.25, 1.6, 2.5, 2.5]
                        }
                    }
                ]
            },
            {
                "key": "height",
                "symbol": "H",
                "spec_type": "mechanical",
                "column_model": "MIN_TYP_MAX",
                "engineering_class": "MECHANICAL",
                "special_semantics": "NONE",
                "llm_context": {
                    "formal_name": "Package Height",
                    "aliases": ["Height", "Thickness", "Body Height"]
                },
                "possible_units": ["mm", "inch"],
                "std_unit": "mm",
                "scenarios": [
                    {
                        "condition": "Nominal dimensions",
                        "limits": {
                            "Standard_SMD": [0.13, 0.23, 0.35, 0.45, 0.55, 0.55, 0.55, 0.55, 0.55, 0.55],
                            "High_Power_THT": [
                                1.9, 2.5, 3.5, 4.5, 5.5, 8.0, 8.5, 8.5, 
                                4.5, 4.5, 5.0
                            ],
                            "Precision_ThinFilm": [0.35, 0.45, 0.55, 0.55, 0.55, 0.55]
                        }
                    }
                ]
            },
            {
                "key": "weight",
                "symbol": "m",
                "spec_type": "mechanical",
                "column_model": "TYP_MAX",
                "engineering_class": "MECHANICAL",
                "special_semantics": "NONE",
                "llm_context": {
                    "formal_name": "Package Weight",
                    "aliases": ["Weight", "Mass"]
                },
                "possible_units": ["mg", "g"],
                "std_unit": "mg",
                "scenarios": [
                    {
                        "condition": "Typical",
                        "limits": {
                            "Standard_SMD": [0.2, 0.5, 1, 3, 8, 15, 25, 40, 40, 80],
                            "High_Power_THT": [
                                150, 300, 500, 1000, 1500, 3000, 5000, 8000, 
                                2000, 1500, 5000
                            ],
                            "Precision_ThinFilm": [1, 3, 8, 15, 25, 40]
                        }
                    }
                ]
            },
            {
               "key": "terminal_finish",
               "symbol": "Finish",
               "spec_type": "mechanical",
               "column_model": "TYP_ONLY",
               "engineering_class": "MECHANICAL",
               "special_semantics": "CATEGORICAL",
               "llm_context": {
                   "formal_name": "Terminal Finish", 
                   "aliases": ["Contact Plating", "Lead Finish", "Termination Material", "Plating"]
               },
               "possible_units": [""],
               "std_unit": "",
               "scenarios": [
                   {
                       "condition": "Standard",
                       "limits": {
                           "Standard_SMD": [
                               "100% Matte Tin (Sn)", 
                               "Sn/Pb (60/40)", 
                               "Gold (Au) Flash", 
                               "Tin-Silver-Copper (SAC305)"
                           ],
                           "High_Power_THT": [
                               "100% Matte Tin", 
                               "Tin-Copper (SnCu)", 
                               "Silver (Ag)", 
                               "Nickel-plated"
                           ],
                           "Precision_ThinFilm": [
                               "Gold (Au)", 
                               "100% Matte Tin (Sn)", 
                               "Sn/Pb (63/37)", 
                               "Palladium-Silver"
                           ]
                       }
                   }
               ]
            }
        ]
      },

    # ==============================================================================
    # 2. CAPACITOR (FINAL FULL VERSION) חלק2
    # ==============================================================================    
    "CAPACITOR": {
        "archetypes": [
            "Ceramic_MLCC", 
            "Electrolytic_Alum", 
            "Electrolytic_Polymer",
            "Tantalum_Solid",
            "Tantalum_Polymer",
            "Film_Polyester",
            "Film_Polypropylene",
            "Supercapacitor"
        ],
    
    # --- TABLE 1: ABSOLUTE MAXIMUM RATINGS ---
    "ABS_MAX": [
        {
            "key": "rated_voltage",
            "symbol": "V<sub>R</sub>",
            "spec_type": "max_rating",
            "column_model": "MAX_ONLY",
            "engineering_class": "SAFETY_LIMIT",
            "special_semantics": "NONE",
            "llm_context": {"formal_name": "Rated Voltage", "aliases": ["Rated DC Voltage", "Working Voltage"]},
            "possible_units": ["V", "kV"],
            "std_unit": "V",
            "scenarios": [{"condition": "DC", "limits": {
                "Ceramic_MLCC": [6.3, 10, 16, 25, 50, 100], 
                "Electrolytic_Alum": [6.3, 10, 16, 25, 35, 50, 63, 100],
                "Electrolytic_Polymer": [2.5, 6.3, 16, 25],
                "Tantalum_Solid": [4, 6.3, 10, 16, 25, 35],
                "Tantalum_Polymer": [2.5, 6.3, 10, 16],
                "Film_Polyester": [50, 63, 100, 250, 400, 630],
                "Film_Polypropylene": [250, 400, 630, 1000],
                "Supercapacitor": [2.7, 3.0, 5.5]
            }}]
        },
        {
            "key": "surge_voltage",
            "symbol": "V<sub>S</sub>",
            "spec_type": "max_rating",
            "column_model": "MAX_ONLY",
            "engineering_class": "SAFETY_LIMIT",
            "special_semantics": "NONE",
            "llm_context": {"formal_name": "Surge Voltage", "aliases": ["Peak Voltage"]},
            "possible_units": ["V"],
            "std_unit": "V",
            "scenarios": [{"condition": "Max", "limits": {
                "Ceramic_MLCC": [15, 25, 40, 75, 150],
                "Electrolytic_Alum": [8, 13, 20, 32, 44, 79],
                "Electrolytic_Polymer": [3, 8, 20, 31],
                "Tantalum_Solid": [5, 8, 13, 20, 31, 44],
                "Tantalum_Polymer": [3, 8, 13, 20],
                "Film_Polyester": [63, 79, 125, 312, 500, 787],
                "Film_Polypropylene": [312, 500, 787, 1250],
                "Supercapacitor": [2.8, 3.1, 5.8]
            }}]
        },
        {
            "key": "operating_temp_range",
            "symbol": "T<sub>op</sub>",
            "spec_type": "operational_range",
            "column_model": "MIN_MAX",
            "engineering_class": "OPERATING_CONDITION",
            "special_semantics": "NONE",
            "llm_context": {"formal_name": "Operating Temperature Range", "aliases": ["Op Temp"]},
            "possible_units": ["°C"],
            "std_unit": "°C",
            "scenarios": [{"condition": "Standard", "limits": {
                "Ceramic_MLCC": [[-55, 125]],
                "Electrolytic_Alum": [[-40, 105]],
                "Electrolytic_Polymer": [[-55, 105]],
                "Tantalum_Solid": [[-55, 125]],
                "Tantalum_Polymer": [[-55, 105]],
                "Film_Polyester": [[-55, 105], [-55, 125]],
                "Film_Polypropylene": [[-55, 105]],
                "Supercapacitor": [[-40, 70]]
            }}]
        }
    ],
    
    # --- TABLE 2: ELECTRICAL CHARACTERISTICS (STATIC) ---
    "ELEC_CHAR": [
        {
            "key": "capacitance",
            "symbol": "C",
            "spec_type": "nominal",
            "column_model": "TYP_ONLY",
            "engineering_class": "NOMINAL_PARAMETER",
            "special_semantics": "NONE",
            "llm_context": {"formal_name": "Nominal Capacitance", "aliases": ["Capacitance"]},
            "possible_units": ["µF", "nF", "pF", "F"],
            "std_unit": "µF",
            "scenarios": [{"condition": "1kHz, 25°C", "limits": {
                "Ceramic_MLCC": [0.1, 1, 10, 22],
                "Electrolytic_Alum": [10, 47, 100, 470, 1000],
                "Electrolytic_Polymer": [10, 47, 100, 330],
                "Tantalum_Solid": [1, 10, 47, 100],
                "Tantalum_Polymer": [10, 47, 150, 330],
                "Film_Polyester": [0.01, 0.1, 0.47, 1.0, 2.2, 4.7],
                "Film_Polypropylene": [0.01, 0.1, 1.0],
                "Supercapacitor": [1000000, 5000000]
            }}]
        },
        {
            "key": "capacitance_tolerance",
            "symbol": "Tol",
            "spec_type": "nominal",
            "column_model": "TYP_ONLY",
            "engineering_class": "NOMINAL_PARAMETER",
            "special_semantics": "NONE",
            "llm_context": {"formal_name": "Capacitance Tolerance", "aliases": ["Tolerance"]},
            "possible_units": ["%"],
            "std_unit": "%",
            "scenarios": [{"condition": "Standard", "limits": {
                "Ceramic_MLCC": ["±10%", "±20%"],
                "Electrolytic_Alum": ["±20%"],
                "Electrolytic_Polymer": ["±20%"],
                "Tantalum_Solid": ["±10%", "±20%"],
                "Tantalum_Polymer": ["±20%"],
                "Film_Polyester": ["±5%", "±10%"],
                "Film_Polypropylene": ["±5%"],
                "Supercapacitor": ["-20%/+80%"]
            }}]
        },
        {
            "key": "leakage_current",
            "symbol": "I<sub>leak</sub>",
            "spec_type": "max_limit",
            "column_model": "MAX_ONLY",
            "engineering_class": "PERFORMANCE_LIMIT",
            "special_semantics": "NONE",
            "llm_context": {"formal_name": "Leakage Current", "aliases": ["DCL"]},
            "possible_units": ["µA"],
            "std_unit": "µA",
            "scenarios": [{"condition": "At rated voltage", "limits": {
                "Ceramic_MLCC": [0], # Not applicable (use IR)
                "Electrolytic_Alum": [3, 10, 50],
                "Electrolytic_Polymer": [50, 100],
                "Tantalum_Solid": [0.5, 1, 5],
                "Tantalum_Polymer": [10, 50],
                "Film_Polyester": [0], # Not applicable (use IR)
                "Film_Polypropylene": [0], # Not applicable (use IR)
                "Supercapacitor": [10, 50]
            }}]
        },
        {
            "key": "insulation_resistance",
            "symbol": "R<sub>ins</sub>",
            "spec_type": "min_limit",
            "column_model": "MIN_ONLY",
            "engineering_class": "PERFORMANCE_LIMIT",
            "special_semantics": "NONE",
            "llm_context": {"formal_name": "Insulation Resistance", "aliases": ["IR"]},
            "possible_units": ["GΩ", "MΩ"],
            "std_unit": "GΩ",
            "scenarios": [{"condition": "At rated voltage", "limits": {
                "Ceramic_MLCC": [10, 100],
                "Electrolytic_Alum": [0],
                "Electrolytic_Polymer": [0],
                "Tantalum_Solid": [0],
                "Tantalum_Polymer": [0],
                "Film_Polyester": [3.75, 7.5, 15, 30],
                "Film_Polypropylene": [30, 100],
                "Supercapacitor": [0]
            }}]
        }
    ],

    # --- TABLE 3: DYNAMIC CHARACTERISTICS ---
    "DYNAMIC_CHAR": [
        {
            "key": "esr",
            "symbol": "ESR",
            "spec_type": "max_limit",
            "column_model": "TYP_MAX",
            "engineering_class": "PERFORMANCE_LIMIT",
            "special_semantics": "NONE",
            "llm_context": {"formal_name": "Equivalent Series Resistance", "aliases": ["ESR"]},
            "possible_units": ["mΩ", "Ω"],
            "std_unit": "mΩ",
            "scenarios": [{"condition": "100kHz", "limits": {
                "Ceramic_MLCC": [2, 10],
                "Electrolytic_Alum": [50, 200, 500],
                "Electrolytic_Polymer": [10, 25, 40],
                "Tantalum_Solid": [100, 500, 1500],
                "Tantalum_Polymer": [25, 45, 60],
                "Film_Polyester": [15, 30, 60],
                "Film_Polypropylene": [5, 10, 20],
                "Supercapacitor": [30, 50, 100]
            }}]
        },
        {
            "key": "dissipation_factor",
            "symbol": "tan δ",
            "spec_type": "max_limit",
            "column_model": "TYP_MAX",
            "engineering_class": "PERFORMANCE_LIMIT",
            "special_semantics": "NONE",
            "llm_context": {"formal_name": "Dissipation Factor", "aliases": ["DF"]},
            "possible_units": ["%"],
            "std_unit": "%",
            "scenarios": [{"condition": "1kHz", "limits": {
                "Ceramic_MLCC": [2.5, 5],
                "Electrolytic_Alum": [10, 20],
                "Electrolytic_Polymer": [10, 12],
                "Tantalum_Solid": [6, 8],
                "Tantalum_Polymer": [8, 10],
                "Film_Polyester": [0.8, 1.0],
                "Film_Polypropylene": [0.1],
                "Supercapacitor": [0]
            }}]
        },
        {
            "key": "impedance",
            "symbol": "Z",
            "spec_type": "max_limit",
            "column_model": "TYP_MAX",
            "engineering_class": "PERFORMANCE_LIMIT",
            "special_semantics": "NONE",
            "llm_context": {"formal_name": "Impedance", "aliases": ["|Z|"]},
            "possible_units": ["mΩ"],
            "std_unit": "mΩ",
            "scenarios": [{"condition": "100kHz", "limits": {
                "Ceramic_MLCC": [5, 20],
                "Electrolytic_Alum": [50, 300],
                "Electrolytic_Polymer": [15, 40],
                "Tantalum_Solid": [200, 1000],
                "Tantalum_Polymer": [40, 80],
                "Film_Polyester": [20, 50],
                "Film_Polypropylene": [10, 30],
                "Supercapacitor": [100]
            }}]
        },
        {
            "key": "self_resonant_freq",
            "symbol": "SRF",
            "spec_type": "nominal",
            "column_model": "MIN_TYP",
            "engineering_class": "PERFORMANCE_LIMIT",
            "special_semantics": "NONE",
            "llm_context": {"formal_name": "Self Resonant Frequency", "aliases": ["SRF"]},
            "possible_units": ["MHz"],
            "std_unit": "MHz",
            "scenarios": [{"condition": "Typical", "limits": {
                "Ceramic_MLCC": [10, 50, 100],
                "Electrolytic_Alum": [0.1, 1],
                "Electrolytic_Polymer": [1, 5],
                "Tantalum_Solid": [1, 10],
                "Tantalum_Polymer": [5, 20],
                "Film_Polyester": [5, 15, 30],
                "Film_Polypropylene": [10, 40],
                "Supercapacitor": [0]
            }}]
        }
    ],

    # --- TABLE 4: RELIABILITY ---
    "RELIABILITY": [
        {
            "key": "rated_lifetime",
            "symbol": "L<sub>rated</sub>",
            "spec_type": "min_limit",
            "column_model": "MIN_ONLY",
            "engineering_class": "RELIABILITY",
            "special_semantics": "NONE",
            "llm_context": {"formal_name": "Rated Lifetime", "aliases": ["Load Life"]},
            "possible_units": ["hours"],
            "std_unit": "hours",
            "scenarios": [{"condition": "At rated temp", "limits": {
                "Ceramic_MLCC": [1000],
                "Electrolytic_Alum": [2000, 5000],
                "Electrolytic_Polymer": [2000, 5000],
                "Tantalum_Solid": [1000],
                "Tantalum_Polymer": [1000],
                "Film_Polyester": [1000, 2000, 5000],
                "Film_Polypropylene": [30000, 100000],
                "Supercapacitor": [1000]
            }}]
        },
        {
            "key": "capacitance_drift",
            "symbol": "ΔC/C",
            "spec_type": "max_limit",
            "column_model": "MAX_ONLY",
            "engineering_class": "RELIABILITY",
            "special_semantics": "NONE",
            "llm_context": {"formal_name": "Capacitance Drift", "aliases": ["Stability"]},
            "possible_units": ["%"],
            "std_unit": "%",
            "scenarios": [{"condition": "1000 hours", "limits": {
                "Ceramic_MLCC": [1, 2],
                "Electrolytic_Alum": [10, 20],
                "Electrolytic_Polymer": [5],
                "Tantalum_Solid": [1],
                "Tantalum_Polymer": [1],
                "Film_Polyester": [2, 5],
                "Film_Polypropylene": [0.5, 1],
                "Supercapacitor": [20]
            }}]
        },
        {
            "key": "moisture_sensitivity",
            "symbol": "MSL",
            "spec_type": "nominal",
            "column_model": "TYP_ONLY",
            "engineering_class": "RELIABILITY",
            "special_semantics": "NOT_APPLICABLE",
            "llm_context": {"formal_name": "Moisture Sensitivity Level", "aliases": ["MSL"]},
            "possible_units": [""],
            "std_unit": "",
            "scenarios": [{"condition": "J-STD-020", "limits": {
                "Ceramic_MLCC": ["1"],
                "Electrolytic_Alum": [0], # N/A for THT
                "Electrolytic_Polymer": ["3"],
                "Tantalum_Solid": ["1"],
                "Tantalum_Polymer": ["3"],
                "Film_Polyester": [0],
                "Film_Polypropylene": [0],
                "Supercapacitor": [0]
            }}]
        }
    ],

    # --- TABLE 5: PACKAGE ---
    "PACKAGE": [
        {
            "key": "package_code",
            "symbol": "PKG",
            "spec_type": "mechanical",
            "column_model": "TYP_ONLY",
            "engineering_class": "MECHANICAL",
            "special_semantics": "NONE",
            "llm_context": {"formal_name": "Package Code", "aliases": ["Case Size"]},
            "possible_units": [""],
            "std_unit": "",
            "scenarios": [{"condition": "N/A", "limits": {
                "Ceramic_MLCC": ["0402", "0603", "0805", "1206"],
                "Electrolytic_Alum": ["Radial_6.3mm", "Radial_8mm", "Radial_10mm"],
                "Electrolytic_Polymer": ["SMD_7343"],
                "Tantalum_Solid": ["Case_A", "Case_B"],
                "Tantalum_Polymer": ["Case_D"],
                "Film_Polyester": ["Radial_5mm", "Radial_7.5mm", "Radial_10mm"],
                "Film_Polypropylene": ["Radial_15mm", "Radial_22.5mm"],
                "Supercapacitor": ["Coin_Type", "Radial_Lead"]
            }}]
        },
        {
            "key": "lead_spacing",
            "symbol": "LS",
            "spec_type": "mechanical",
            "column_model": "TYP_ONLY",
            "engineering_class": "MECHANICAL",
            "special_semantics": "NONE",
            "llm_context": {"formal_name": "Lead Spacing", "aliases": ["Pitch"]},
            "possible_units": ["mm"],
            "std_unit": "mm",
            "scenarios": [{"condition": "Nominal", "limits": {
                "Ceramic_MLCC": 0,
                "Electrolytic_Alum": [2.5, 3.5, 5.0],
                "Electrolytic_Polymer": 0,
                "Tantalum_Solid": 0,
                "Tantalum_Polymer": 0,
                "Film_Polyester": [5.0, 7.5, 10.0],
                "Film_Polypropylene": [15.0, 22.5],
                "Supercapacitor": [5.0]
            }}]
        },
        {
            "key": "polarization",
            "symbol": "POL",
            "spec_type": "mechanical",
            "column_model": "TYP_ONLY",
            "engineering_class": "MECHANICAL",
            "special_semantics": "NONE",
            "llm_context": {"formal_name": "Polarization", "aliases": ["Polarity"]},
            "possible_units": [""],
            "std_unit": "",
            "scenarios": [{"condition": "N/A", "limits": {
                "Ceramic_MLCC": ["Non-Polar"],
                "Electrolytic_Alum": ["Polar"],
                "Electrolytic_Polymer": ["Polar"],
                "Tantalum_Solid": ["Polar"],
                "Tantalum_Polymer": ["Polar"],
                "Film_Polyester": ["Non-Polar"],
                "Film_Polypropylene": ["Non-Polar"],
                "Supercapacitor": ["Polar"]
  
                            }
                          } 
                         ]
        }
    ]
    },

    # ==============================================================================
    # 3. DIODE חלק3
    # ==============================================================================
    "DIODE": {
    "archetypes": [
        "Signal_Small_Signal",      # 1N4148, 1N914, BAV70
        "Power_Rectifier",          # 1N4001-1N4007, 1N5400 series
        "Schottky_Barrier",         # 1N5817-1N5819, SS14-SS54, MBRS series
        "Fast_Recovery",            # UF4001-UF4007, RHRP series
        "Ultra_Fast_Recovery",      # MUR series, STTH series
        "Zener",                    # 1N47xx series, BZX series
        "TVS_Transient",            # SMAJ, P6KE, 1.5KE series
        "Switching",                # High-speed switching diodes
        "PIN_Diode",                # RF switching, attenuators
        "Avalanche"                 # High voltage, avalanche rated
    ],
    
    # ==========================================================================
    # ABSOLUTE MAXIMUM RATINGS
    # ==========================================================================
    "ABS_MAX": [
        {
            "key": "reverse_voltage",
            "symbol": "V<sub>RRM</sub>",
            "spec_type": "max_rating",
            "column_model": "MAX_ONLY",
            "engineering_class": "SAFETY_LIMIT",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Repetitive Peak Reverse Voltage",
                "aliases": ["Peak Reverse Voltage", "Max Reverse Voltage", "VRRM", "PRV"]
            },
            "possible_units": ["V", "kV"],
            "std_unit": "V",
            "scenarios": [
                {
                    "condition": "Tj=25°C",
                    "limits": {
                        "Signal_Small_Signal": [50, 75, 100, 200, 300],
                        "Power_Rectifier": [50, 100, 200, 400, 600, 800, 1000, 1200, 1500],
                        "Schottky_Barrier": [15, 20, 30, 40, 45, 60, 100, 150, 200],
                        "Fast_Recovery": [200, 400, 600, 800, 1000, 1200],
                        "Ultra_Fast_Recovery": ["1", "2"],
                        "Zener": ["1", "2"],
                        "TVS_Transient": ["1", "2"],
                        "Switching": ["1"],
                        "PIN_Diode": ["1", "2"],
                        "Avalanche": ["2", "3"]
                    }
                }
            ]
        },
        {
            "key": "esd_rating",
            "symbol": "V<sub>ESD</sub>",
            "spec_type": "min_limit",
            "column_model": "MIN_ONLY",
            "engineering_class": "RELIABILITY",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "ESD Withstand Voltage",
                "aliases": ["ESD Rating", "HBM Rating"]
            },
            "possible_units": ["V", "kV"],
            "std_unit": "V",
            "scenarios": [
                {
                    "condition": "HBM per ANSI/ESDA/JEDEC JS-001",
                    "limits": {
                        "Signal_Small_Signal": [1000, 2000, 4000],
                        "Power_Rectifier": [1000, 2000],
                        "Schottky_Barrier": [1000, 2000],
                        "Fast_Recovery": [1000, 2000],
                        "Ultra_Fast_Recovery": [1000, 2000],
                        "Zener": [2000, 4000],
                        "TVS_Transient": [0],  # NOT_APPLICABLE - these ARE ESD protection
                        "Switching": [2000, 4000],
                        "PIN_Diode": [1000, 2000],
                        "Avalanche": [1000]
                    }
                }
            ]
        },
        {
            "key": "soldering_temp",
            "symbol": "T<sub>solder</sub>",
            "spec_type": "max_rating",
            "column_model": "MAX_ONLY",
            "engineering_class": "RELIABILITY",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Maximum Soldering Temperature",
                "aliases": ["Reflow Temp", "Soldering Temperature", "Peak Reflow"]
            },
            "possible_units": ["°C"],
            "std_unit": "°C",
            "scenarios": [
                {
                    "condition": "Peak temperature, 10s max",
                    "limits": {
                        "Signal_Small_Signal": [260],
                        "Power_Rectifier": [260, 300],
                        "Schottky_Barrier": [260],
                        "Fast_Recovery": [260],
                        "Ultra_Fast_Recovery": [260],
                        "Zener": [260],
                        "TVS_Transient": [260],
                        "Switching": [260],
                        "PIN_Diode": [260],
                        "Avalanche": [300]
                    }
                }
            ]
        },
        {
            "key": "thermal_shock",
            "symbol": "ΔT",
            "spec_type": "min_limit",
            "column_model": "MIN_ONLY",
            "engineering_class": "RELIABILITY",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Thermal Shock Test",
                "aliases": ["Temperature Cycling", "Thermal Cycle"]
            },
            "possible_units": ["cycles"],
            "std_unit": "cycles",
            "scenarios": [
                {
                    "condition": "-65°C to +150°C",
                    "limits": {
                        "Signal_Small_Signal": [1000],
                        "Power_Rectifier": [500, 1000],
                        "Schottky_Barrier": [1000],
                        "Fast_Recovery": [1000],
                        "Ultra_Fast_Recovery": [1000],
                        "Zener": [1000],
                        "TVS_Transient": [500],
                        "Switching": [1000],
                        "PIN_Diode": [500],
                        "Avalanche": [500]
                    }
                }
            ]
        },
        {
            "key": "surge_rating",
            "symbol": "N",
            "spec_type": "nominal",
            "column_model": "TYP_ONLY",
            "engineering_class": "RELIABILITY",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Surge Current Rating Derating",
                "aliases": ["Number of Surge Cycles", "Surge Life"]
            },
            "possible_units": ["cycles"],
            "std_unit": "cycles",
            "scenarios": [
                {
                    "condition": "At IFSM max",
                    "limits": {
                        "Signal_Small_Signal": [1000],
                        "Power_Rectifier": [1000, 10000],
                        "Schottky_Barrier": [1000],
                        "Fast_Recovery": [1000],
                        "Ultra_Fast_Recovery": [1000],
                        "Zener": [100],
                        "TVS_Transient": [100, 1000],
                        "Switching": [1000],
                        "PIN_Diode": [100],
                        "Avalanche": [10000]
                    }
                }
            ]
        }
    ],
    
    # ==========================================================================
    # TEST CONDITIONS & SPECIAL PARAMETERS
    # ==========================================================================
    "TEST_CONDITIONS": [
        {
            "key": "test_current_if",
            "symbol": "I<sub>F(test)</sub>",
            "spec_type": "nominal",
            "column_model": "TYP_ONLY",
            "engineering_class": "TEST_CONDITION",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Forward Test Current for VF",
                "aliases": ["Test Current", "IF test"]
            },
            "possible_units": ["A", "mA"],
            "std_unit": "A",
            "scenarios": [
                {
                    "condition": "For VF measurement",
                    "limits": {
                        "Signal_Small_Signal": [0.01, 0.05, 0.1, 0.15],
                        "Power_Rectifier": [1, 2, 5, 10],
                        "Schottky_Barrier": [1, 3, 5, 10],
                        "Fast_Recovery": [1, 5, 10],
                        "Ultra_Fast_Recovery": [1, 5, 10],
                        "Zener": [0.005, 0.01],
                        "TVS_Transient": [0.001],
                        "Switching": [0.01, 0.1],
                        "PIN_Diode": [0.01, 0.1],
                        "Avalanche": [1, 5]
                    }
                }
            ]
        },
        {
            "key": "test_current_iz",
            "symbol": "I<sub>ZT</sub>",
            "spec_type": "nominal",
            "column_model": "TYP_ONLY",
            "engineering_class": "TEST_CONDITION",
            "special_semantics": "NOT_APPLICABLE",
            "llm_context": {
                "formal_name": "Zener Test Current",
                "aliases": ["IZT", "Test Current for VZ"]
            },
            "possible_units": ["mA"],
            "std_unit": "mA",
            "scenarios": [
                {
                    "condition": "For VZ measurement",
                    "limits": {
                        "Signal_Small_Signal": [0],
                        "Power_Rectifier": [0],
                        "Schottky_Barrier": [0],
                        "Fast_Recovery": [0],
                        "Ultra_Fast_Recovery": [0],
                        "Zener": [5, 20, 50, 75, 100, 200],
                        "TVS_Transient": [0],
                        "Switching": [0],
                        "PIN_Diode": [0],
                        "Avalanche": [0]
                    }
                }
            ]
        },
        {
            "key": "test_voltage_vr",
            "symbol": "V<sub>R(test)</sub>",
            "spec_type": "nominal",
            "column_model": "TYP_ONLY",
            "engineering_class": "TEST_CONDITION",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Reverse Test Voltage",
                "aliases": ["Test Voltage", "VR test"]
            },
            "possible_units": ["V"],
            "std_unit": "V",
            "scenarios": [
                {
                    "condition": "For IR and CJ measurement",
                    "limits": {
                        "Signal_Small_Signal": [20, 50, 75],
                        "Power_Rectifier": [100, 200, 400, 600],
                        "Schottky_Barrier": [10, 20, 30, 40],
                        "Fast_Recovery": [200, 400, 600],
                        "Ultra_Fast_Recovery": [200, 400, 600],
                        "Zener": [1, 2],
                        "TVS_Transient": [5, 10, 15],
                        "Switching": [20, 50],
                        "PIN_Diode": [20, 50],
                        "Avalanche": [500, 1000]
                    }
                }
            ]
        },
        {
            "key": "test_frequency",
            "symbol": "f<sub>test</sub>",
            "spec_type": "nominal",
            "column_model": "TYP_ONLY",
            "engineering_class": "TEST_CONDITION",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Test Frequency for Capacitance",
                "aliases": ["Frequency", "f test"]
            },
            "possible_units": ["MHz", "kHz"],
            "std_unit": "MHz",
            "scenarios": [
                {
                    "condition": "For CJ measurement",
                    "limits": {
                        "Signal_Small_Signal": [1],
                        "Power_Rectifier": [1],
                        "Schottky_Barrier": [1],
                        "Fast_Recovery": [1],
                        "Ultra_Fast_Recovery": [1],
                        "Zener": [1],
                        "TVS_Transient": [1],
                        "Switching": [1],
                        "PIN_Diode": [0.001, 0.1, 1, 10],  # DC to GHz range
                        "Avalanche": [1]
                    }
                }
            ]
        },
        {
            "key": "configuration",
            "symbol": "CONFIG",
            "spec_type": "mechanical",
            "column_model": "TYP_ONLY",
            "engineering_class": "MECHANICAL",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Diode Configuration",
                "aliases": ["Configuration", "Circuit Config"]
            },
            "possible_units": [""],
            "std_unit": "",
            "scenarios": [
                {
                    "condition": "N/A",
                    "limits": {
                        "Signal_Small_Signal": ["Single", "Dual_Common_Cathode", "Dual_Common_Anode", "Dual_Series"],
                        "Power_Rectifier": ["Single", "Bridge", "Center_Tap"],
                        "Schottky_Barrier": ["Single", "Dual_Common_Cathode", "Dual_Common_Anode"],
                        "Fast_Recovery": ["Single", "Dual_Common_Cathode"],
                        "Ultra_Fast_Recovery": ["Single", "Dual_Common_Cathode"],
                        "Zener": ["Single", "Dual_Series", "Dual_Back_to_Back"],
                        "TVS_Transient": ["Unidirectional", "Bidirectional"],
                        "Switching": ["Single", "Dual_Series"],
                        "PIN_Diode": ["Single"],
                        "Avalanche": ["Single"]
                    }
                }
            ]
        },
        {
            "key": "clamping_voltage",
            "symbol": "V<sub>C</sub>",
            "spec_type": "max_limit",
            "column_model": "MAX_ONLY",
            "engineering_class": "PERFORMANCE_LIMIT",
            "special_semantics": "NOT_APPLICABLE",
            "llm_context": {
                "formal_name": "Maximum Clamping Voltage",
                "aliases": ["VC", "Peak Clamping Voltage", "Clamp Voltage"]
            },
            "possible_units": ["V"],
            "std_unit": "V",
            "scenarios": [
                {
                    "condition": "At IPP",
                    "limits": {
                        "Signal_Small_Signal": [0],
                        "Power_Rectifier": [0],
                        "Schottky_Barrier": [0],
                        "Fast_Recovery": [0],
                        "Ultra_Fast_Recovery": [0],
                        "Zener": [0],
                        "TVS_Transient": [9.2, 11.5, 17.0, 19.9, 24.4, 27.7, 38.9, 45.7, 58.1, 78.0, 106, 130, 167, 224, 300],
                        "Switching": [0],
                        "PIN_Diode": [0],
                        "Avalanche": [0]
                    }
                }
            ]
        },
        {
            "key": "reverse_recovery_time",
            "symbol": "t<sub>rr</sub>",
            "spec_type": "max_limit",
            "column_model": "TYP_MAX",
            "engineering_class": "PERFORMANCE_LIMIT",
            "special_semantics": "NEGLIGIBLE",
            "llm_context": {
                "formal_name": "Reverse Recovery Time",
                "aliases": ["trr", "Recovery Time", "Switching Time"]
            },
            "possible_units": ["ns", "µs"],
            "std_unit": "ns",
            "scenarios": [
                {
                    "condition": "IF=0.5A, IR=1A, IRR=0.25A",
                    "limits": {
                        "Signal_Small_Signal": [2, 4, 8, 10, 50],
                        "Power_Rectifier": [50, 100, 500, 1500, 3000, 5000],
                        "Schottky_Barrier": [0],
                        "Fast_Recovery": [35, 50, 75, 150, 250, 500],
                        "Ultra_Fast_Recovery": [15, 20, 25, 35, 50, 75],
                        "Zener": [100, 500],
                        "TVS_Transient": [0],
                        "Switching": [2, 4, 6, 10],
                        "PIN_Diode": [1000, 5000, 10000],
                        "Avalanche": [75, 150, 500]
                    }
                }
            ]
        },
        {
            "key": "reverse_recovery_charge",
            "symbol": "Q<sub>rr</sub>",
            "spec_type": "max_limit",
            "column_model": "TYP_MAX",
            "engineering_class": "PERFORMANCE_LIMIT",
            "special_semantics": "NEGLIGIBLE",
            "llm_context": {
                "formal_name": "Reverse Recovery Charge",
                "aliases": ["Qrr", "Recovery Charge"]
            },
            "possible_units": ["nC", "µC"],
            "std_unit": "nC",
            "scenarios": [
                {
                    "condition": "Standard test",
                    "limits": {
                        "Signal_Small_Signal": [1, 2, 5],
                        "Power_Rectifier": [50, 100, 500, 1000, 5000],
                        "Schottky_Barrier": [0],
                        "Fast_Recovery": [20, 50, 100, 250, 500],
                        "Ultra_Fast_Recovery": [10, 20, 50, 100, 200],
                        "Zener": [50, 100],
                        "TVS_Transient": [0],
                        "Switching": [1, 2, 4],
                        "PIN_Diode": [0],
                        "Avalanche": [100, 500]
                    }
                }
            ]
        },
        {
            "key": "softness_factor",
            "symbol": "S",
            "spec_type": "nominal",
            "column_model": "TYP_ONLY",
            "engineering_class": "NOMINAL_PARAMETER",
            "special_semantics": "NOT_APPLICABLE",
            "llm_context": {
                "formal_name": "Softness Factor",
                "aliases": ["S Factor", "Recovery Softness", "Snappiness"]
            },
            "possible_units": [""],
            "std_unit": "",
            "scenarios": [
                {
                    "condition": "Standard",
                    "limits": {
                        "Signal_Small_Signal": [0],
                        "Power_Rectifier": [0.3, 0.5],
                        "Schottky_Barrier": [0],
                        "Fast_Recovery": [0.3, 0.4, 0.5],
                        "Ultra_Fast_Recovery": [0.2, 0.3],
                        "Zener": [0],
                        "TVS_Transient": [0],
                        "Switching": [0],
                        "PIN_Diode": [0],
                        "Avalanche": [0.4]
                    }
                }
            ]
        },
        {
            "key": "forward_recovery_voltage",
            "symbol": "V<sub>FR</sub>",
            "spec_type": "max_limit",
            "column_model": "TYP_MAX",
            "engineering_class": "PERFORMANCE_LIMIT",
            "special_semantics": "NOT_APPLICABLE",
            "llm_context": {
                "formal_name": "Forward Recovery Voltage",
                "aliases": ["VFR", "Turn-on Overshoot"]
            },
            "possible_units": ["V"],
            "std_unit": "V",
            "scenarios": [
                {
                    "condition": "di/dt test",
                    "limits": {
                        "Signal_Small_Signal": [0],
                        "Power_Rectifier": [0],
                        "Schottky_Barrier": [0],
                        "Fast_Recovery": [1.5, 2.0, 2.5],
                        "Ultra_Fast_Recovery": [1.8, 2.2, 2.5],
                        "Zener": [0],
                        "TVS_Transient": [0],
                        "Switching": [0],
                        "PIN_Diode": [0],
                        "Avalanche": [2.0, 2.5]
                    }
                }
            ]
        },
        {
            "key": "junction_capacitance",
            "symbol": "C<sub>J</sub>",
            "spec_type": "max_limit",
            "column_model": "TYP_MAX",
            "engineering_class": "PERFORMANCE_LIMIT",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Junction Capacitance",
                "aliases": ["Total Capacitance", "Diode Capacitance", "CJ", "CT"]
            },
            "possible_units": ["pF", "nF"],
            "std_unit": "pF",
            "scenarios": [
                {
                    "condition": "VR=4V, f=1MHz",
                    "limits": {
                        "Signal_Small_Signal": [0.8, 1, 2, 4, 8, 10, 15],
                        "Power_Rectifier": [15, 30, 50, 100, 250, 500, 1000, 2500],
                        "Schottky_Barrier": [50, 100, 150, 250, 500, 1000, 1500, 2500],
                        "Fast_Recovery": [30, 50, 100, 250, 500],
                        "Ultra_Fast_Recovery": [25, 50, 100, 250, 500, 1000],
                        "Zener": [15, 50, 150, 500, 1000],
                        "TVS_Transient": [1000, 2000, 5000, 10000, 20000],
                        "Switching": [1, 2, 4],
                        "PIN_Diode": [0.5, 1, 2, 5],
                        "Avalanche": [100, 500, 1000]
                    }
                }
            ]
        },
        {
            "key": "series_resistance",
            "symbol": "R<sub>S</sub>",
            "spec_type": "max_limit",
            "column_model": "TYP_MAX",
            "engineering_class": "PERFORMANCE_LIMIT",
            "special_semantics": "NOT_APPLICABLE",
            "llm_context": {
                "formal_name": "Series Resistance",
                "aliases": ["RS", "Bulk Resistance", "On Resistance"]
            },
            "possible_units": ["Ω", "mΩ"],
            "std_unit": "Ω",
            "scenarios": [
                {
                    "condition": "At rated IF",
                    "limits": {
                        "Signal_Small_Signal": [0],
                        "Power_Rectifier": [0],
                        "Schottky_Barrier": [10, 15, 20, 30, 50, 100],  # mΩ
                        "Fast_Recovery": [0],
                        "Ultra_Fast_Recovery": [0],
                        "Zener": [0],
                        "TVS_Transient": [0],
                        "Switching": [0],
                        "PIN_Diode": [0.5, 1, 2, 5, 10],
                        "Avalanche": [0]
                    }
                }
            ]
        },
        {
            "key": "isolation_resistance",
            "symbol": "R<sub>ISO</sub>",
            "spec_type": "min_limit",
            "column_model": "MIN_ONLY",
            "engineering_class": "PERFORMANCE_LIMIT",
            "special_semantics": "NOT_APPLICABLE",
            "llm_context": {
                "formal_name": "RF Isolation (Off State)",
                "aliases": ["Isolation", "Off Resistance"]
            },
            "possible_units": ["kΩ", "MΩ"],
            "std_unit": "kΩ",
            "scenarios": [
                {
                    "condition": "VR applied, f=1GHz",
                    "limits": {
                        "Signal_Small_Signal": [0],
                        "Power_Rectifier": [0],
                        "Schottky_Barrier": [0],
                        "Fast_Recovery": [0],
                        "Ultra_Fast_Recovery": [0],
                        "Zener": [0],
                        "TVS_Transient": [0],
                        "Switching": [0],
                        "PIN_Diode": [1, 5, 10, 50],  # kΩ
                        "Avalanche": [0]
                    }
                }
            ]
        }
    ],
    
    # ==========================================================================
    # THERMAL CHARACTERISTICS
    # ==========================================================================
    "THERMAL_CHAR": [
        {
            "key": "thermal_resistance_jc",
            "symbol": "R<sub>θJC</sub>",
            "spec_type": "max_limit",
            "column_model": "TYP_MAX",
            "engineering_class": "NOMINAL_PARAMETER",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Thermal Resistance Junction to Case",
                "aliases": ["RθJC", "Rth(j-c)", "Thermal Res. J-C"]
            },
            "possible_units": ["°C/W"],
            "std_unit": "°C/W",
            "scenarios": [
                {
                    "condition": "Steady state",
                    "limits": {
                        "Signal_Small_Signal": [200, 300, 400, 500],
                        "Power_Rectifier": [0.5, 1, 1.5, 2, 3, 5, 10, 15, 20],
                        "Schottky_Barrier": [1, 2, 3, 5, 10, 15, 20],
                        "Fast_Recovery": [1, 2, 3, 5, 8, 10],
                        "Ultra_Fast_Recovery": [0.5, 1, 2, 3, 5, 8],
                        "Zener": [5, 10, 50, 100],
                        "TVS_Transient": [10, 20, 30, 50],
                        "Switching": [200, 300, 400],
                        "PIN_Diode": [50, 100, 200],
                        "Avalanche": [2, 5, 10, 20]
                    }
                }
            ]
        },
        {
            "key": "thermal_resistance_ja",
            "symbol": "R<sub>θJA</sub>",
            "spec_type": "max_limit",
            "column_model": "TYP_MAX",
            "engineering_class": "NOMINAL_PARAMETER",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Thermal Resistance Junction to Ambient",
                "aliases": ["RθJA", "Rth(j-a)", "Thermal Res. J-A"]
            },
            "possible_units": ["°C/W"],
            "std_unit": "°C/W",
            "scenarios": [
                {
                    "condition": "Standard PCB, still air",
                    "limits": {
                        "Signal_Small_Signal": [300, 400, 500],
                        "Power_Rectifier": [20, 30, 40, 50, 60, 80, 100],
                        "Schottky_Barrier": [30, 40, 50, 60, 80, 100],
                        "Fast_Recovery": [30, 40, 50, 60],
                        "Ultra_Fast_Recovery": [25, 35, 45, 60],
                        "Zener": [50, 100, 200],
                        "TVS_Transient": [50, 70, 100],
                        "Switching": [300, 400],
                        "PIN_Diode": [150, 250],
                        "Avalanche": [40, 60, 80]
                    }
                }
            ]
        },
        {
            "key": "thermal_resistance_jl",
            "symbol": "R<sub>θJL</sub>",
            "spec_type": "max_limit",
            "column_model": "TYP_MAX",
            "engineering_class": "NOMINAL_PARAMETER",
            "special_semantics": "NOT_APPLICABLE",
            "llm_context": {
                "formal_name": "Thermal Resistance Junction to Lead",
                "aliases": ["RθJL", "Rth(j-l)"]
            },
            "possible_units": ["°C/W"],
            "std_unit": "°C/W",
            "scenarios": [
                {
                    "condition": "Per lead",
                    "limits": {
                        "Signal_Small_Signal": [100, 150, 200],
                        "Power_Rectifier": [0],
                        "Schottky_Barrier": [0],
                        "Fast_Recovery": [0],
                        "Ultra_Fast_Recovery": [0],
                        "Zener": [25, 50],
                        "TVS_Transient": [0],
                        "Switching": [100, 150],
                        "PIN_Diode": [50, 100],
                        "Avalanche": [0]
                    }
                }
            ]
        },
        {
            "key": "transient_thermal_impedance",
            "symbol": "Z<sub>θJC</sub>",
            "spec_type": "nominal",
            "column_model": "TYP_ONLY",
            "engineering_class": "NOMINAL_PARAMETER",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Transient Thermal Impedance",
                "aliases": ["ZθJC", "Thermal Impedance", "Pulse Thermal Resistance"]
            },
            "possible_units": ["°C/W"],
            "std_unit": "°C/W",
            "scenarios": [
                {
                    "condition": "Various pulse widths",
                    "limits": {
                        "Signal_Small_Signal": [50, 100, 200],
                        "Power_Rectifier": [0.3, 0.5, 1, 2, 5],
                        "Schottky_Barrier": [1, 2, 5, 10],
                        "Fast_Recovery": [1, 2, 5],
                        "Ultra_Fast_Recovery": [0.5, 1, 3],
                        "Zener": [5, 10, 50],
                        "TVS_Transient": [1, 5, 10],
                        "Switching": [100, 200],
                        "PIN_Diode": [20, 50],
                        "Avalanche": [2, 5, 10]
                    }
                }
            ]
        },
        {
            "key": "operating_junction_temp_range",
            "symbol": "T<sub>J</sub>",
            "spec_type": "operational_range",
            "column_model": "MIN_MAX",
            "engineering_class": "OPERATING_CONDITION",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Operating Junction Temperature Range",
                "aliases": ["Operating Temp", "TJ Range"]
            },
            "possible_units": ["°C"],
            "std_unit": "°C",
            "scenarios": [
                {
                    "condition": "Operating",
                    "limits": {
                        "Signal_Small_Signal": [[-65, 150], [-65, 175]],
                        "Power_Rectifier": [[-65, 150], [-65, 175]],
                        "Schottky_Barrier": [[-65, 125], [-65, 150], [-65, 175]],
                        "Fast_Recovery": [[-65, 150], [-65, 175]],
                        "Ultra_Fast_Recovery": [[-65, 150], [-65, 175]],
                        "Zener": [[-65, 175], [-55, 175]],
                        "TVS_Transient": [[-65, 150], [-65, 175]],
                        "Switching": [[-65, 150], [-65, 175]],
                        "PIN_Diode": [[-65, 125], [-65, 150]],
                        "Avalanche": [[-65, 175]]
                    }
                }
            ]
        }
    ],
    
    # ==========================================================================
    # PACKAGE & MECHANICAL
    # ==========================================================================
    "PACKAGE": [
        {
            "key": "package_code",
            "symbol": "PKG",
            "spec_type": "mechanical",
            "column_model": "TYP_ONLY",
            "engineering_class": "MECHANICAL",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Package Type",
                "aliases": ["Case Type", "Package Code", "Outline"]
            },
            "possible_units": [""],
            "std_unit": "",
            "scenarios": [
                {
                    "condition": "N/A",
                    "limits": {
                        "Signal_Small_Signal": ["SOD-323", "SOD-123", "SOT-23", "SOT-323", "SC-70", "DO-35", "DO-204"],
                        "Power_Rectifier": ["DO-41", "DO-201AD", "DO-214AC(SMA)", "DO-214AA(SMB)", "DO-214AB(SMC)", "R-6", "TO-220", "TO-220AB", "TO-247", "ITO-220", "D2PAK", "DPAK"],
                        "Schottky_Barrier": ["SOD-123", "DO-214AC(SMA)", "DO-214AA(SMB)", "DO-214AB(SMC)", "DO-201AD", "TO-220", "TO-220AB", "ITO-220", "D2PAK", "DPAK"],
                        "Fast_Recovery": ["DO-41", "DO-201AD", "DO-15", "TO-220", "TO-220AB", "TO-247", "DO-214AB(SMC)", "ITO-220", "D2PAK"],
                        "Ultra_Fast_Recovery": ["DO-41", "DO-201", "TO-220", "TO-220AB", "TO-247", "DO-214AB(SMC)", "ITO-220", "D2PAK", "TO-262"],
                        "Zener": ["SOD-123", "SOD-323", "SOT-23", "DO-35", "DO-41", "DO-201", "DO-214AC(SMA)", "DO-214AA(SMB)", "SMB"],
                        "TVS_Transient": ["DO-214AC(SMA)", "DO-214AA(SMB)", "DO-214AB(SMC)", "DO-201", "DO-15", "5KP", "P600"],
                        "Switching": ["SOD-323", "SOD-123", "SOT-23", "DO-35"],
                        "PIN_Diode": ["SOD-323", "SOT-23", "DO-35", "DO-7"],
                        "Avalanche": ["DO-201", "TO-220", "TO-247", "DO-214AB"]
                    }
                }
            ]
        },
        {
            "key": "mounting_type",
            "symbol": "MNT",
            "spec_type": "mechanical",
            "column_model": "TYP_ONLY",
            "engineering_class": "MECHANICAL",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Mounting Type",
                "aliases": ["Mounting Style", "Mount"]
            },
            "possible_units": [""],
            "std_unit": "",
            "scenarios": [
                {
                    "condition": "N/A",
                    "limits": {
                        "Signal_Small_Signal": ["SMD", "Through_Hole"],
                        "Power_Rectifier": ["SMD", "Through_Hole", "Stud", "Chassis"],
                        "Schottky_Barrier": ["SMD", "Through_Hole", "Chassis"],
                        "Fast_Recovery": ["SMD", "Through_Hole", "Chassis"],
                        "Ultra_Fast_Recovery": ["SMD", "Through_Hole", "Chassis"],
                        "Zener": ["SMD", "Through_Hole"],
                        "TVS_Transient": ["SMD", "Through_Hole", "Chassis"],
                        "Switching": ["SMD", "Through_Hole"],
                        "PIN_Diode": ["SMD", "Through_Hole"],
                        "Avalanche": ["Through_Hole", "Chassis"]
                    }
                }
            ]
        },
        {
            "key": "polarity_marking",
            "symbol": "POL",
            "spec_type": "mechanical",
            "column_model": "TYP_ONLY",
            "engineering_class": "MECHANICAL",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Polarity Marking",
                "aliases": ["Cathode Mark", "Polarity"]
            },
            "possible_units": [""],
            "std_unit": "",
            "scenarios": [
                {
                    "condition": "N/A",
                    "limits": {
                        "Signal_Small_Signal": ["Band", "Line"],
                        "Power_Rectifier": ["Band", "Symbol", "Stud"],
                        "Schottky_Barrier": ["Band", "Symbol"],
                        "Fast_Recovery": ["Band", "Symbol"],
                        "Ultra_Fast_Recovery": ["Band", "Symbol"],
                        "Zener": ["Band", "Line"],
                        "TVS_Transient": ["Band", "Bidirectional_None"],
                        "Switching": ["Band", "Line"],
                        "PIN_Diode": ["Band"],
                        "Avalanche": ["Band", "Symbol"]
                    }
                }
            ]
        },
        {
            "key": "lead_finish",
            "symbol": "FINISH",
            "spec_type": "mechanical",
            "column_model": "TYP_ONLY",
            "engineering_class": "MECHANICAL",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Terminal Finish",
                "aliases": ["Lead Finish", "Plating", "Terminal Plating"]
            },
            "possible_units": [""],
            "std_unit": "",
            "scenarios": [
                {
                    "condition": "N/A",
                    "limits": {
                        "Signal_Small_Signal": ["Matte_Sn", "SnPb"],
                        "Power_Rectifier": ["Matte_Sn", "SnPb", "Solder_Dip"],
                        "Schottky_Barrier": ["Matte_Sn", "SnPb"],
                        "Fast_Recovery": ["Matte_Sn", "SnPb"],
                        "Ultra_Fast_Recovery": ["Matte_Sn", "SnPb"],
                        "Zener": ["Matte_Sn", "SnPb"],
                        "TVS_Transient": ["Matte_Sn", "SnPb"],
                        "Switching": ["Matte_Sn"],
                        "PIN_Diode": ["Au", "Matte_Sn"],
                        "Avalanche": ["Matte_Sn", "SnPb"]
                    }
                }
            ]
        },
        {
            "key": "weight",
            "symbol": "W",
            "spec_type": "mechanical",
            "column_model": "TYP_ONLY",
            "engineering_class": "MECHANICAL",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Weight",
                "aliases": ["Mass", "Part Weight"]
            },
            "possible_units": ["g", "mg"],
            "std_unit": "g",
            "scenarios": [
                {
                    "condition": "Typical",
                    "limits": {
                        "Signal_Small_Signal": [0.01, 0.02, 0.05, 0.1],
                        "Power_Rectifier": [0.5, 1, 2, 5, 10, 20, 50],
                        "Schottky_Barrier": [0.1, 0.5, 1, 2, 5, 10],
                        "Fast_Recovery": [0.5, 1, 2, 5, 10],
                        "Ultra_Fast_Recovery": [0.5, 1, 2, 5, 10, 20],
                        "Zener": [0.05, 0.1, 0.5, 1],
                        "TVS_Transient": [0.1, 0.5, 1, 5],
                        "Switching": [0.01, 0.05],
                        "PIN_Diode": [0.01, 0.05, 0.1],
                        "Avalanche": [1, 5, 10, 20]
                    }
                }
            ]
        }
    ],
    
    # ==========================================================================
    # RELIABILITY & QUALITY
    # ==========================================================================
    "RELIABILITY": [
        {
            "key": "failure_rate",
            "symbol": "λ",
            "spec_type": "max_limit",
            "column_model": "MAX_ONLY",
            "engineering_class": "RELIABILITY",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Failure Rate",
                "aliases": ["FIT Rate", "Lambda", "MTBF"]
            },
            "possible_units": ["FIT", "PPM", "%/1000h"],
            "std_unit": "FIT",
            "scenarios": [
                {
                    "condition": "Standard conditions",
                    "limits": {
                        "Signal_Small_Signal": [0.1, 1, 5],
                        "Power_Rectifier": [1, 5, 10],
                        "Schottky_Barrier": [1, 5, 10],
                        "Fast_Recovery": [1, 5],
                        "Ultra_Fast_Recovery": [1, 5],
                        "Zener": [1, 5, 10],
                        "TVS_Transient": [1, 5],
                        "Switching": [0.1, 1],
                        "PIN_Diode": [0.5, 1],
                        "Avalanche": [5, 10]
                    }
                }
            ]
        },
        {
            "key": "moisture_sensitivity_level",
            "symbol": "MSL",
            "spec_type": "nominal",
            "column_model": "TYP_ONLY",
            "engineering_class": "RELIABILITY",
            "special_semantics": "NOT_APPLICABLE",
            "llm_context": {
                "formal_name": "Moisture Sensitivity Level",
                "aliases": ["MSL", "Moisture Level"]
            },
            "possible_units": [""],
            "std_unit": "",
            "scenarios": [
                {
                    "condition": "Per J-STD-020",
                    "limits": {
                        "Signal_Small_Signal": ["1", "2"],
                        "Power_Rectifier": ["1", "2", "3"],
                        "Schottky_Barrier": ["1", "2"],
                        "Fast_Recovery": ["1", "2"],
                        "Ultra_Fast_ode": [50, 100, 200],
                        "Avalanche": [1000, 1500, 2000, 3000, 5000]
                    }
                }
            ]
        },
        {
            "key": "working_reverse_voltage",
            "symbol": "V<sub>RWM</sub>",
            "spec_type": "max_rating",
            "column_model": "MAX_ONLY",
            "engineering_class": "SAFETY_LIMIT",
            "special_semantics": "NOT_APPLICABLE",  # Only for TVS diodes
            "llm_context": {
                "formal_name": "Working Peak Reverse Voltage",
                "aliases": ["Stand-off Voltage", "VRWM", "Working Voltage"]
            },
            "possible_units": ["V"],
            "std_unit": "V",
            "scenarios": [
                {
                    "condition": "Continuous",
                    "limits": {
                        "Signal_Small_Signal": [0],
                        "Power_Rectifier": [0],
                        "Schottky_Barrier": [0],
                        "Fast_Recovery": [0],
                        "Ultra_Fast_Recovery": [0],
                        "Zener": [0],
                        "TVS_Transient": [5, 6.8, 10, 12, 15, 17, 24, 28, 36, 51, 70, 85, 110, 150, 200],
                        "Switching": [0],
                        "PIN_Diode": [0],
                        "Avalanche": [0]
                    }
                }
            ]
        },
        {
            "key": "breakdown_voltage",
            "symbol": "V<sub>BR</sub>",
            "spec_type": "min_max_range",
            "column_model": "MIN_MAX",
            "engineering_class": "SAFETY_LIMIT",
            "special_semantics": "NOT_APPLICABLE",  # Only for TVS and Zener
            "llm_context": {
                "formal_name": "Breakdown Voltage",
                "aliases": ["VBR", "Clamping Voltage", "Avalanche Voltage"]
            },
            "possible_units": ["V"],
            "std_unit": "V",
            "scenarios": [
                {
                    "condition": "At test current",
                    "limits": {
                        "Signal_Small_Signal": [0],
                        "Power_Rectifier": [0],
                        "Schottky_Barrier": [0],
                        "Fast_Recovery": [0],
                        "Ultra_Fast_Recovery": [0],
                        "Zener": [[2.28, 2.52], [3.14, 3.47], [4.85, 5.36], [6.46, 7.14], [8.65, 9.56], [11.4, 12.6], [14.25, 15.75]],
                        "TVS_Transient": [[6.4, 7.1], [9.5, 10.5], [13.3, 14.7], [16.2, 17.9], [20.9, 23.1], [30.4, 33.6], [48.4, 53.6]],
                        "Switching": [0],
                        "PIN_Diode": [0],
                        "Avalanche": [[1050, 1150], [1550, 1650], [2050, 2150]]
                    }
                }
            ]
        },
        {
            "key": "forward_current",
            "symbol": "I<sub>F(AV)</sub>",
            "spec_type": "max_rating",
            "column_model": "MAX_ONLY",
            "engineering_class": "SAFETY_LIMIT",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Average Rectified Forward Current",
                "aliases": ["Average Forward Current", "Rectified Current", "IF(AV)", "IO"]
            },
            "possible_units": ["A", "mA"],
            "std_unit": "A",
            "scenarios": [
                {
                    "condition": "Tc=specified",
                    "limits": {
                        "Signal_Small_Signal": [0.075, 0.1, 0.15, 0.2, 0.3, 0.5],
                        "Power_Rectifier": [1, 1.5, 2, 3, 5, 6, 10, 15, 20, 30, 40, 50],
                        "Schottky_Barrier": [0.5, 1, 2, 3, 5, 10, 15, 20, 30, 40],
                        "Fast_Recovery": [1, 2, 3, 5, 8, 10, 15, 20, 30],
                        "Ultra_Fast_Recovery": [1, 2, 4, 6, 8, 10, 15, 20, 30, 50],
                        "Zener": [0.5, 1, 5],
                        "TVS_Transient": [0],  # NOT_APPLICABLE - rated by pulse power
                        "Switching": [0.1, 0.15, 0.2, 0.5],
                        "PIN_Diode": [0.01, 0.1, 0.5],
                        "Avalanche": [0.5, 1, 2, 5]
                    }
                }
            ]
        },
        {
            "key": "peak_forward_current",
            "symbol": "I<sub>FSM</sub>",
            "spec_type": "max_rating",
            "column_model": "MAX_ONLY",
            "engineering_class": "SAFETY_LIMIT",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Non-Repetitive Peak Forward Surge Current",
                "aliases": ["Surge Current", "Peak Forward Surge Current", "IFSM", "Surge Rating"]
            },
            "possible_units": ["A"],
            "std_unit": "A",
            "scenarios": [
                {
                    "condition": "8.3ms half-sine, Tj=25°C",
                    "limits": {
                        "Signal_Small_Signal": [0.5, 1, 2, 4, 5],
                        "Power_Rectifier": [30, 50, 70, 100, 150, 200, 300, 400, 500],
                        "Schottky_Barrier": [20, 30, 50, 75, 100, 150, 200, 300],
                        "Fast_Recovery": [30, 50, 75, 100, 150, 200, 300],
                        "Ultra_Fast_Recovery": [30, 50, 75, 100, 150, 200, 300, 400],
                        "Zener": [10, 25, 100],
                        "TVS_Transient": [0],  # NOT_APPLICABLE
                        "Switching": [1, 2, 4],
                        "PIN_Diode": [1, 5, 10],
                        "Avalanche": [10, 20, 50]
                    }
                }
            ]
        },
        {
            "key": "repetitive_peak_forward_current",
            "symbol": "I<sub>FRM</sub>",
            "spec_type": "max_rating",
            "column_model": "MAX_ONLY",
            "engineering_class": "SAFETY_LIMIT",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Repetitive Peak Forward Current",
                "aliases": ["IFRM", "Repetitive Peak Current"]
            },
            "possible_units": ["A"],
            "std_unit": "A",
            "scenarios": [
                {
                    "condition": "Square wave",
                    "limits": {
                        "Signal_Small_Signal": [0.5, 0.75, 1],
                        "Power_Rectifier": [5, 10, 15, 30, 60, 100],
                        "Schottky_Barrier": [5, 10, 20, 30, 60, 100],
                        "Fast_Recovery": [5, 10, 20, 30, 60],
                        "Ultra_Fast_Recovery": [5, 10, 20, 40, 60],
                        "Zener": [2, 5],
                        "TVS_Transient": [0],
                        "Switching": [0.5, 1, 2],
                        "PIN_Diode": [0.1, 0.5],
                        "Avalanche": [2, 5, 10]
                    }
                }
            ]
        },
        {
            "key": "power_dissipation",
            "symbol": "P<sub>tot</sub>",
            "spec_type": "max_rating",
            "column_model": "MAX_ONLY",
            "engineering_class": "SAFETY_LIMIT",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Total Power Dissipation",
                "aliases": ["Power Dissipation", "Power Loss", "Ptot", "PD"]
            },
            "possible_units": ["W", "mW"],
            "std_unit": "W",
            "scenarios": [
                {
                    "condition": "Tc=25°C or Ta=25°C",
                    "limits": {
                        "Signal_Small_Signal": [0.15, 0.2, 0.25, 0.33, 0.5],
                        "Power_Rectifier": [1, 1.5, 2, 3, 5, 10, 25, 50, 100],
                        "Schottky_Barrier": [0.5, 1, 1.5, 2, 5, 10, 25],
                        "Fast_Recovery": [1.5, 2.5, 4, 5, 10, 25],
                        "Ultra_Fast_Recovery": [2, 3, 5, 10, 20, 40],
                        "Zener": [0.5, 1, 5, 10, 50],
                        "TVS_Transient": [0],  # NOT_APPLICABLE - rated by pulse
                        "Switching": [0.2, 0.25, 0.5],
                        "PIN_Diode": [0.1, 0.25, 0.5],
                        "Avalanche": [1, 5, 10, 25]
                    }
                }
            ]
        },
        {
            "key": "peak_pulse_power",
            "symbol": "P<sub>PPM</sub>",
            "spec_type": "max_rating",
            "column_model": "MAX_ONLY",
            "engineering_class": "SAFETY_LIMIT",
            "special_semantics": "NOT_APPLICABLE",  # Only for TVS
            "llm_context": {
                "formal_name": "Peak Pulse Power Dissipation",
                "aliases": ["PPPM", "Pulse Power", "Transient Power"]
            },
            "possible_units": ["W", "kW"],
            "std_unit": "W",
            "scenarios": [
                {
                    "condition": "10/1000µs waveform",
                    "limits": {
                        "Signal_Small_Signal": [0],
                        "Power_Rectifier": [0],
                        "Schottky_Barrier": [0],
                        "Fast_Recovery": [0],
                        "Ultra_Fast_Recovery": [0],
                        "Zener": [0],
                        "TVS_Transient": [400, 600, 1000, 1500, 3000, 5000, 10000, 15000],
                        "Switching": [0],
                        "PIN_Diode": [0],
                        "Avalanche": [0]
                    }
                }
            ]
        },
        {
            "key": "tj_max",
            "symbol": "T<sub>j(max)</sub>",
            "spec_type": "max_rating",
            "column_model": "MAX_ONLY",
            "engineering_class": "SAFETY_LIMIT",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Maximum Junction Temperature",
                "aliases": ["Max Junction Temp", "Tj max"]
            },
            "possible_units": ["°C"],
            "std_unit": "°C",
            "scenarios": [
                {
                    "condition": "Max",
                    "limits": {
                        "Signal_Small_Signal": [150, 175, 200],
                        "Power_Rectifier": [150, 175, 200],
                        "Schottky_Barrier": [125, 150, 175],
                        "Fast_Recovery": [150, 175],
                        "Ultra_Fast_Recovery": [150, 175],
                        "Zener": [150, 175, 200],
                        "TVS_Transient": [150, 175],
                        "Switching": [150, 175],
                        "PIN_Diode": [125, 150],
                        "Avalanche": [175, 200]
                    }
                }
            ]
        },
        {
            "key": "storage_temp_range",
            "symbol": "T<sub>stg</sub>",
            "spec_type": "operational_range",
            "column_model": "MIN_MAX",
            "engineering_class": "OPERATING_CONDITION",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Storage Temperature Range",
                "aliases": ["Storage Temp", "Tstg"]
            },
            "possible_units": ["°C"],
            "std_unit": "°C",
            "scenarios": [
                {
                    "condition": "Non-operating",
                    "limits": {
                        "Signal_Small_Signal": [[-65, 175], [-65, 200]],
                        "Power_Rectifier": [[-65, 175], [-55, 200]],
                        "Schottky_Barrier": [[-65, 150], [-65, 175]],
                        "Fast_Recovery": [[-65, 175]],
                        "Ultra_Fast_Recovery": [[-65, 175]],
                        "Zener": [[-65, 175], [-65, 200]],
                        "TVS_Transient": [[-65, 175]],
                        "Switching": [[-65, 175]],
                        "PIN_Diode": [[-65, 150]],
                        "Avalanche": [[-65, 175]]
                    }
                }
            ]
        },
        {
            "key": "avalanche_energy",
            "symbol": "E<sub>AS</sub>",
            "spec_type": "max_rating",
            "column_model": "MAX_ONLY",
            "engineering_class": "SAFETY_LIMIT",
            "special_semantics": "NOT_APPLICABLE",  # Only for avalanche-rated
            "llm_context": {
                "formal_name": "Single Pulse Avalanche Energy",
                "aliases": ["EAS", "Avalanche Energy", "Single Pulse Energy"]
            },
            "possible_units": ["mJ", "J"],
            "std_unit": "mJ",
            "scenarios": [
                {
                    "condition": "Single pulse",
                    "limits": {
                        "Signal_Small_Signal": [0],
                        "Power_Rectifier": [0],
                        "Schottky_Barrier": [0],
                        "Fast_Recovery": [10, 30, 50, 100],
                        "Ultra_Fast_Recovery": [20, 50, 100, 200, 500],
                        "Zener": [0],
                        "TVS_Transient": [0],
                        "Switching": [0],
                        "PIN_Diode": [0],
                        "Avalanche": [100, 250, 500, 1000, 2000]
                    }
                }
            ]
        }
    ],
    
    # ==========================================================================
    # ELECTRICAL CHARACTERISTICS
    # ==========================================================================
    "ELEC_CHAR": [
        {
            "key": "forward_voltage",
            "symbol": "V<sub>F</sub>",
            "spec_type": "max_limit",
            "column_model": "TYP_MAX",
            "engineering_class": "PERFORMANCE_LIMIT",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Forward Voltage Drop",
                "aliases": ["Forward Voltage", "Max Forward Voltage", "VF", "On Voltage"]
            },
            "possible_units": ["V", "mV"],
            "std_unit": "V",
            "scenarios": [
                {
                    "condition": "IF=rated, Tj=25°C",
                    "limits": {
                        "Signal_Small_Signal": [0.715, 1.0, 1.25],
                        "Power_Rectifier": [0.93, 1.0, 1.1, 1.2, 1.3],
                        "Schottky_Barrier": [0.35, 0.45, 0.50, 0.55, 0.70, 0.85, 1.0],
                        "Fast_Recovery": [1.0, 1.2, 1.5, 1.7],
                        "Ultra_Fast_Recovery": [1.1, 1.3, 1.5, 1.7, 2.0],
                        "Zener": [1.0, 1.2],
                        "TVS_Transient": [3.3, 3.5, 5.0],  # At 1mA test
                        "Switching": [0.715, 1.0],
                        "PIN_Diode": [0.9, 1.0, 1.2],
                        "Avalanche": [1.2, 1.5, 1.7]
                    }
                }
            ]
        },
        {
            "key": "forward_voltage_temp_coeff",
            "symbol": "dV<sub>F</sub>/dT",
            "spec_type": "nominal",
            "column_model": "TYP_ONLY",
            "engineering_class": "NOMINAL_PARAMETER",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Forward Voltage Temperature Coefficient",
                "aliases": ["VF Temp Coeff", "Thermal Coefficient VF"]
            },
            "possible_units": ["mV/°C"],
            "std_unit": "mV/°C",
            "scenarios": [
                {
                    "condition": "At constant IF",
                    "limits": {
                        "Signal_Small_Signal": [-2.0, -2.5],
                        "Power_Rectifier": [-2.0, -2.5, -3.0],
                        "Schottky_Barrier": [-1.0, -1.5, -2.0],
                        "Fast_Recovery": [-2.0, -2.5],
                        "Ultra_Fast_Recovery": [-2.0, -2.5],
                        "Zener": [-2.0],
                        "TVS_Transient": [-2.0],
                        "Switching": [-2.0],
                        "PIN_Diode": [-2.0],
                        "Avalanche": [-2.5]
                    }
                }
            ]
        },
        {
            "key": "reverse_current",
            "symbol": "I<sub>R</sub>",
            "spec_type": "max_limit",
            "column_model": "TYP_MAX",
            "engineering_class": "PERFORMANCE_LIMIT",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Reverse Leakage Current",
                "aliases": ["Reverse Current", "Leakage Current", "IR", "IRRM"]
            },
            "possible_units": ["µA", "mA", "nA"],
            "std_unit": "µA",
            "scenarios": [
                {
                    "condition": "At VR rated, Tj=25°C",
                    "limits": {
                        "Signal_Small_Signal": [0.005, 0.010, 0.025, 0.050, 0.1],
                        "Power_Rectifier": [0.5, 1, 5, 10, 30, 50, 100],
                        "Schottky_Barrier": [0.05, 0.1, 0.5, 1, 2, 5, 10, 50, 100, 500],
                        "Fast_Recovery": [0.5, 5, 10, 50],
                        "Ultra_Fast_Recovery": [0.5, 5, 10, 50, 100],
                        "Zener": [1, 10, 50, 100],
                        "TVS_Transient": [1, 5, 10, 100],
                        "Switching": [0.005, 0.025],
                        "PIN_Diode": [0.01, 0.1, 1],
                        "Avalanche": [10, 50, 100]
                    }
                }
            ]
        },
        {
            "key": "reverse_current_at_tj_max",
            "symbol": "I<sub>R</sub>",
            "spec_type": "max_limit",
            "column_model": "MAX_ONLY",
            "engineering_class": "PERFORMANCE_LIMIT",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Reverse Current at Tj(max)",
                "aliases": ["IR at Max Temp", "Hot Leakage"]
            },
            "possible_units": ["µA", "mA"],
            "std_unit": "µA",
            "scenarios": [
                {
                    "condition": "At VR rated, Tj=Tj(max)",
                    "limits": {
                        "Signal_Small_Signal": [5, 30, 50],
                        "Power_Rectifier": [30, 50, 100, 500],
                        "Schottky_Barrier": [500, 1000, 2000, 5000],
                        "Fast_Recovery": [50, 100, 500],
                        "Ultra_Fast_Recovery": [100, 500],
                        "Zener": [100, 500],
                        "TVS_Transient": [1000],
                        "Switching": [30, 50],
                        "PIN_Diode": [10, 100],
                        "Avalanche": [500, 1000]
                    }
                }
            ]
        },
        {
            "key": "zener_voltage",
            "symbol": "V<sub>Z</sub>",
            "spec_type": "min_max_range",
            "column_model": "MIN_TYP_MAX",
            "engineering_class": "NOMINAL_PARAMETER",
            "special_semantics": "NOT_APPLICABLE",  # Only for Zener
            "llm_context": {
                "formal_name": "Zener Voltage",
                "aliases": ["VZ", "Breakdown Voltage", "Reference Voltage"]
            },
            "possible_units": ["V"],
            "std_unit": "V",
            "scenarios": [
                {
                    "condition": "At IZT",
                    "limits": {
                        "Signal_Small_Signal": [0],
                        "Power_Rectifier": [0],
                        "Schottky_Barrier": [0],
                        "Fast_Recovery": [0],
                        "Ultra_Fast_Recovery": [0],
                        "Zener": [2.4, 3.3, 3.9, 4.7, 5.1, 5.6, 6.2, 6.8, 7.5, 8.2, 9.1, 10, 11, 12, 13, 15, 16, 18, 20, 22, 24, 27, 30, 33, 36, 39, 43, 47, 51, 56, 62, 68, 75, 82, 91, 100, 110, 120, 130, 150, 160, 180, 200],
                        "TVS_Transient": [0],
                        "Switching": [0],
                        "PIN_Diode": [0],
                        "Avalanche": [0]
                    }
                }
            ]
        },
        {
            "key": "zener_impedance",
            "symbol": "Z<sub>ZT</sub>",
            "spec_type": "max_limit",
            "column_model": "TYP_MAX",
            "engineering_class": "PERFORMANCE_LIMIT",
            "special_semantics": "NOT_APPLICABLE",  # Only for Zener
            "llm_context": {
                "formal_name": "Zener Dynamic Impedance",
                "aliases": ["ZZT", "Dynamic Impedance", "Zener Resistance"]
            },
            "possible_units": ["Ω"],
            "std_unit": "Ω",
            "scenarios": [
                {
                    "condition": "At IZT",
                    "limits": {
                        "Signal_Small_Signal": [0],
                        "Power_Rectifier": [0],
                        "Schottky_Barrier": [0],
                        "Fast_Recovery": [0],
                        "Ultra_Fast_Recovery": [0],
                        "Zener": [2, 5, 10, 20, 30, 50, 80, 100, 150, 200, 400, 600, 800, 1000],
                        "TVS_Transient": [0],
                        "Switching": [0],
                        "PIN_Diode": [0],
                        "Avalanche": [0]
                    }
                }
            ]
        },
        {
            "key": "zener_temp_coefficient",
            "symbol": "α<sub>VZ</sub>",
            "spec_type": "nominal",
            "column_model": "TYP_ONLY",
            "engineering_class": "NOMINAL_PARAMETER",
            "special_semantics": "NOT_APPLICABLE",  # Only for Zener
            "llm_context": {
                "formal_name": "Zener Voltage Temperature Coefficient",
                "aliases": ["Temp Coeff", "αVZ", "TCV"]
            },
            "possible_units": ["%/°C", "mV/°C"],
            "std_unit": "%/°C",
            "scenarios": [
                {
                    "condition": "Standard",
                    "limits": {
                        "Signal_Small_Signal": [0],
                        "Power_Rectifier": [0],
                        "Schottky_Barrier": [0],
                        "Fast_Recovery": [0],
                        "Ultra_Fast_Recovery": [0],
                        "Zener": [-0.08, -0.05, -0.03, 0, 0.02, 0.03, 0.05, 0.07, 0.08, 0.10],  
                        "TVS_Transient": [0],
                        "Switching": [0],
                        "PIN_Diode": [0],
                        "Avalanche": [0]
                        }
                    }
                ]
            }
        ]
    },

# ==============================================================================
# MOSFET - Enhanced and Comprehensive Parameter Dictionary חלק4
# ==============================================================================

    "MOSFET": {
    "archetypes": [
        "HV_Power_THT",             # High Voltage Power Through-Hole (500V+)
        "MV_Power_SMD",             # Medium Voltage Power SMD (40-250V)
        "LV_Logic_Level",           # Low Voltage Logic Level (12-30V)
        "SiC_High_Voltage",         # Silicon Carbide (900V-1700V)
        "GaN_High_Frequency",       # Gallium Nitride (100V-650V)
        "RF_Power",                 # RF Power MOSFETs (VHF/UHF)
        "Dual_N_Channel",           # Dual N-Channel configuration
        "Dual_P_Channel",           # Dual P-Channel configuration
        "Complementary_Pair",       # N+P Channel pair
        "Depletion_Mode"            # Normally-ON devices
    ],
    
    # ==========================================================================
    # ABSOLUTE MAXIMUM RATINGS (הוספתי את החלק הזה שהיה חסר)
    # ==========================================================================
    "ABS_MAX": [
        {
            "key": "vdss",
            "symbol": "V<sub>DSS</sub>",
            "spec_type": "max_rating",
            "column_model": "MAX_ONLY",
            "engineering_class": "SAFETY_LIMIT",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Drain-Source Voltage",
                "aliases": ["Drain-Source Breakdown Voltage", "Max VDS", "VDSS", "BV_DSS"]
            },
            "possible_units": ["V", "kV"],
            "std_unit": "V",
            "scenarios": [
                {
                    "condition": "VGS=0V, Tj=25°C",
                    "limits": {
                        "HV_Power_THT": [400, 500, 600, 650, 700, 800, 900, 1000, 1200, 1500, 1700],
                        "MV_Power_SMD": [30, 40, 55, 60, 75, 80, 100, 120, 150, 200, 250, 300],
                        "LV_Logic_Level": [12, 20, 25, 30, 40, 60],
                        "SiC_High_Voltage": [650, 900, 1000, 1200, 1700, 3300],
                        "GaN_High_Frequency": [100, 150, 200, 650, 900],
                        "RF_Power": [28, 50, 65],
                        "Dual_N_Channel": [20, 30, 60, 100],
                        "Dual_P_Channel": [-20, -30, -60],
                        "Complementary_Pair": [60, 100],
                        "Depletion_Mode": [25, 60, 200, 500]
                    }
                }
            ]
        },
        {
            "key": "id_cont",
            "symbol": "I<sub>D</sub>",
            "spec_type": "max_rating",
            "column_model": "MAX_ONLY",
            "engineering_class": "SAFETY_LIMIT",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Continuous Drain Current",
                "aliases": ["Drain Current", "Continuous Current", "ID", "ID(cont)"]
            },
            "possible_units": ["A", "mA"],
            "std_unit": "A",
            "scenarios": [
                {
                    "condition": "Tc=25°C, VGS=10V",
                    "limits": {
                        "HV_Power_THT": [7, 10, 15, 20, 24, 30, 40, 46, 60, 80, 100, 120],
                        "MV_Power_SMD": [15, 20, 30, 40, 50, 60, 70, 80, 100, 120, 140, 180, 200],
                        "LV_Logic_Level": [0.5, 1, 2, 3, 4, 5, 8, 10, 12, 20, 30],
                        "SiC_High_Voltage": [17, 23, 36, 49, 63, 90, 117],
                        "GaN_High_Frequency": [10, 15, 30, 45, 60, 90],
                        "RF_Power": [1, 2, 4, 6, 10],
                        "Dual_N_Channel": [1, 2, 4, 8, 12],
                        "Dual_P_Channel": [-1, -2, -4, -6],
                        "Complementary_Pair": [4, 8, 12],
                        "Depletion_Mode": [0.2, 0.5, 1, 5, 10]
                    }
                }
            ]
        },
        {
            "key": "id_pulse",
            "symbol": "I<sub>DM</sub>",
            "spec_type": "max_rating",
            "column_model": "MAX_ONLY",
            "engineering_class": "SAFETY_LIMIT",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Pulsed Drain Current",
                "aliases": ["Pulsed Current", "Max Pulsed Drain Current", "IDM", "ID(pulse)"]
            },
            "possible_units": ["A"],
            "std_unit": "A",
            "scenarios": [
                {
                    "condition": "10µs pulse, Tj=25°C",
                    "limits": {
                        "HV_Power_THT": [28, 40, 60, 80, 96, 120, 160, 184, 240, 320, 400, 480],
                        "MV_Power_SMD": [60, 80, 120, 160, 200, 240, 280, 320, 400, 480, 560, 720, 800],
                        "LV_Logic_Level": [2, 4, 8, 12, 16, 20, 32, 40, 48, 80, 120],
                        "SiC_High_Voltage": [51, 69, 108, 147, 189, 270, 351],
                        "GaN_High_Frequency": [40, 60, 120, 180, 240, 360],
                        "RF_Power": [4, 8, 16, 24, 40],
                        "Dual_N_Channel": [4, 8, 16, 32, 48],
                        "Dual_P_Channel": [-4, -8, -16, -24],
                        "Complementary_Pair": [16, 32, 48],
                        "Depletion_Mode": [0.8, 2, 4, 20, 40]
                    }
                }
            ]
        },
        {
            "key": "vgs_max",
            "symbol": "V<sub>GS</sub>",
            "spec_type": "max_rating",
            "column_model": "MIN_MAX",
            "engineering_class": "SAFETY_LIMIT",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Gate-Source Voltage",
                "aliases": ["Max Gate-Source Voltage", "VGS Rating", "Gate Voltage Range"]
            },
            "possible_units": ["V"],
            "std_unit": "V",
            "scenarios": [
                {
                    "condition": "Continuous",
                    "limits": {
                        "HV_Power_THT": [[-20, 20], [-30, 30]],
                        "MV_Power_SMD": [[-20, 20], [-10, 20]],
                        "LV_Logic_Level": [[-8, 8], [-12, 12], [-20, 20]],
                        "SiC_High_Voltage": [[-5, 20], [-6, 22]],
                        "GaN_High_Frequency": [[-10, 6], [-6, 10]],
                        "RF_Power": [[-10, 10]],
                        "Dual_N_Channel": [[-20, 20]],
                        "Dual_P_Channel": [[-20, 20]],
                        "Complementary_Pair": [[-20, 20]],
                        "Depletion_Mode": [[-30, 10], [-20, 5]]
                    }
                }
            ]
        },
        {
            "key": "power_dissipation",
            "symbol": "P<sub>D</sub>",
            "spec_type": "max_rating",
            "column_model": "MAX_ONLY",
            "engineering_class": "SAFETY_LIMIT",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Maximum Power Dissipation",
                "aliases": ["Power Dissipation", "Max Power", "PD", "Ptot"]
            },
            "possible_units": ["W", "kW"],
            "std_unit": "W",
            "scenarios": [
                {
                    "condition": "Tc=25°C",
                    "limits": {
                        "HV_Power_THT": [125, 150, 200, 250, 300, 375, 500, 600, 800, 1000],
                        "MV_Power_SMD": [40, 50, 63, 80, 100, 125, 150, 200, 250, 300],
                        "LV_Logic_Level": [0.7, 1, 1.4, 2, 2.5, 5, 8, 10, 15, 20],
                        "SiC_High_Voltage": [208, 278, 417, 556, 694, 833],
                        "GaN_High_Frequency": [50, 75, 100, 150, 200],
                        "RF_Power": [50, 100, 150, 300, 500],
                        "Dual_N_Channel": [1, 2, 3.5, 5, 10],
                        "Dual_P_Channel": [1, 2, 3.5],
                        "Complementary_Pair": [2.5, 5],
                        "Depletion_Mode": [1, 5, 25, 100, 300]
                    }
                }
            ]
        },
        {
            "key": "tj_max",
            "symbol": "T<sub>j(max)</sub>",
            "spec_type": "max_rating",
            "column_model": "MAX_ONLY",
            "engineering_class": "SAFETY_LIMIT",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Maximum Junction Temperature",
                "aliases": ["Max Junction Temp", "Tj max", "Operating Temperature Max"]
            },
            "possible_units": ["°C"],
            "std_unit": "°C",
            "scenarios": [
                {
                    "condition": "Max",
                    "limits": {
                        "HV_Power_THT": [150, 175],
                        "MV_Power_SMD": [150, 175],
                        "LV_Logic_Level": [150, 175],
                        "SiC_High_Voltage": [175, 200],
                        "GaN_High_Frequency": [150, 175, 200],
                        "RF_Power": [200, 225],
                        "Dual_N_Channel": [150, 175],
                        "Dual_P_Channel": [150, 175],
                        "Complementary_Pair": [150, 175],
                        "Depletion_Mode": [150, 175, 200]
                    }
                }
            ]
        }
    ],
    
    # ==========================================================================
    # ELECTRICAL CHARACTERISTICS - STATIC PARAMETERS (הוספתי את ההתחלה החסרה)
    # ==========================================================================
    "ELEC_CHAR": [
        {
            "key": "bvdss",
            "symbol": "BV<sub>DSS</sub>",
            "spec_type": "min_limit",
            "column_model": "MIN_ONLY",
            "engineering_class": "PERFORMANCE_LIMIT",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Drain-Source Breakdown Voltage",
                "aliases": ["BVDSS", "Breakdown Voltage", "V(BR)DSS"]
            },
            "possible_units": ["V"],
            "std_unit": "V",
            "scenarios": [
                {
                    "condition": "VGS=0V, ID=250µA",
                    "limits": {
                        "HV_Power_THT": [400, 500, 600, 650, 800, 1000, 1200],
                        "MV_Power_SMD": [30, 40, 60, 80, 100, 150, 200, 250],
                        "LV_Logic_Level": [12, 20, 25, 30, 40],
                        "SiC_High_Voltage": [650, 900, 1200, 1700],
                        "GaN_High_Frequency": [100, 150, 200, 650],
                        "RF_Power": [28, 50, 65],
                        "Dual_N_Channel": [20, 30, 60],
                        "Dual_P_Channel": [-20, -30, -60],
                        "Complementary_Pair": [60],
                        "Depletion_Mode": [25, 60, 200, 500]
                    }
                }
            ]
        },
        {
            "key": "rds_on",
            "symbol": "R<sub>DS(on)</sub>",
            "spec_type": "max_limit",
            "column_model": "TYP_MAX",
            "engineering_class": "PERFORMANCE_LIMIT",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Static Drain-Source On-Resistance",
                "aliases": ["On-Resistance", "RDS(on)", "Drain-Source On-State Resistance", "Ron"]
            },
            "possible_units": ["mΩ", "Ω"],
            "std_unit": "mΩ",
            "scenarios": [
                {
                    "condition": "VGS=10V, ID=specified, Tj=25°C",
                    "limits": {
                        "HV_Power_THT": [150, 200, 250, 350, 500, 750, 1000, 1200, 1500, 2000],
                        "MV_Power_SMD": [2, 3, 5, 7, 10, 15, 20, 25, 30, 40, 50, 70],
                        "LV_Logic_Level": [2, 3, 5, 8, 10, 15, 20, 30, 45],
                        "SiC_High_Voltage": [25, 40, 60, 80, 120, 160],
                        "GaN_High_Frequency": [15, 25, 50, 100, 150],
                        "RF_Power": [500, 800, 1200, 2000], 
                        "Dual_N_Channel": [10, 20, 40, 60, 100],
                        "Dual_P_Channel": [15, 30, 60, 90],
                        "Complementary_Pair": [20, 40, 80],
                        "Depletion_Mode": [50, 100, 500, 1000, 2000]
                    }
                }
            ]
        },
        {
            "key": "idss",
            "symbol": "I<sub>DSS</sub>",
            "spec_type": "max_limit",
            "column_model": "TYP_MAX",
            "engineering_class": "PERFORMANCE_LIMIT",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Zero Gate Voltage Drain Current",
                "aliases": ["Drain-Source Leakage Current", "Zero Gate Voltage Drain Current", "IDSS", "Leakage"]
            },
            "possible_units": ["µA", "nA", "mA"],
            "std_unit": "µA",
            "scenarios": [
                {
                    "condition": "VDS=rated, VGS=0V, Tj=25°C",
                    "limits": {
                        "HV_Power_THT": [1, 10, 25, 50, 100],
                        "MV_Power_SMD": [1, 10, 50, 100],
                        "LV_Logic_Level": [0.001, 0.01, 0.1, 1, 10],
                        "SiC_High_Voltage": [1, 10, 100],
                        "GaN_High_Frequency": [0.1, 1, 10, 100],
                        "RF_Power": [1, 10, 50],
                        "Dual_N_Channel": [0.01, 1, 10],
                        "Dual_P_Channel": [0.01, 1, 10],
                        "Complementary_Pair": [1, 10],
                        "Depletion_Mode": [0]
                    }
                }
            ]
        },
        {
            "key": "igss",
            "symbol": "I<sub>GSS</sub>",
            "spec_type": "max_limit",
            "column_model": "MAX_ONLY",
            "engineering_class": "PERFORMANCE_LIMIT",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Gate-Source Leakage Current",
                "aliases": ["Gate Leakage", "IGSS", "Gate Leakage Current"]
            },
            "possible_units": ["nA", "µA"],
            "std_unit": "nA",
            "scenarios": [
                {
                    "condition": "VGS=±20V, VDS=0V",
                    "limits": {
                        "HV_Power_THT": [100, 500, 1000],
                        "MV_Power_SMD": [100, 500],
                        "LV_Logic_Level": [10, 100, 500],
                        "SiC_High_Voltage": [100, 500],
                        "GaN_High_Frequency": [100, 500, 1000],
                        "RF_Power": [100, 500],
                        "Dual_N_Channel": [100, 500],
                        "Dual_P_Channel": [100, 500],
                        "Complementary_Pair": [500],
                        "Depletion_Mode": [100, 500]
                    }
                }
            ]
        },
        {
            "key": "gfs",
            "symbol": "g<sub>fs</sub>",
            "spec_type": "min_limit",
            "column_model": "TYP_ONLY",
            "engineering_class": "NOMINAL_PARAMETER",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Forward Transconductance",
                "aliases": ["Transconductance", "gfs", "gm"]
            },
            "possible_units": ["S", "mS"],
            "std_unit": "S",
            "scenarios": [
                {
                    "condition": "VDS=specified, ID=specified",
                    "limits": {
                        "HV_Power_THT": [5, 10, 15, 20, 30, 40],
                        "MV_Power_SMD": [10, 20, 40, 60, 80, 120],
                        "LV_Logic_Level": [1, 2, 5, 10, 15],
                        "SiC_High_Voltage": [10, 15, 25, 35],
                        "GaN_High_Frequency": [15, 25, 40, 60],
                        "RF_Power": [2, 5, 8, 12],
                        "Dual_N_Channel": [2, 5, 10],
                        "Dual_P_Channel": [2, 4, 8],
                        "Complementary_Pair": [5, 10],
                        "Depletion_Mode": [0.5, 1, 5, 10]
                    }
                }
            ]
        },
        {
            "key": "vsd",
            "symbol": "V<sub>SD</sub>",
            "spec_type": "max_limit",
            "column_model": "TYP_MAX",
            "engineering_class": "PERFORMANCE_LIMIT",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Body Diode Forward Voltage",
                "aliases": ["Source-Drain Diode Voltage", "Body Diode Forward Voltage", "Diode VF"]
            },
            "possible_units": ["V"],
            "std_unit": "V",
            "scenarios": [
                {
                    "condition": "IS=ID, VGS=0V",
                    "limits": {
                        "HV_Power_THT": [1.2, 1.3, 1.5, 1.7],
                        "MV_Power_SMD": [0.7, 0.9, 1.0, 1.2, 1.3],
                        "LV_Logic_Level": [0.6, 0.7, 0.8, 1.0],
                        "SiC_High_Voltage": [2.8, 3.2, 3.5, 4.0],
                        "GaN_High_Frequency": [0], 
                        "RF_Power": [1.2, 1.5],
                        "Dual_N_Channel": [0.7, 1.0],
                        "Dual_P_Channel": [0.7, 1.0],
                        "Complementary_Pair": [1.0],
                        "Depletion_Mode": [1.2, 1.5]
                    }
                }
            ]
        },
        {
            "key": "trr",
            "symbol": "t<sub>rr</sub>",
            "spec_type": "max_limit",
            "column_model": "TYP_MAX",
            "engineering_class": "PERFORMANCE_LIMIT",
            "special_semantics": "NOT_APPLICABLE",
            "llm_context": {
                "formal_name": "Body Diode Reverse Recovery Time",
                "aliases": ["trr", "Reverse Recovery Time", "Body Diode Recovery"]
            },
            "possible_units": ["ns"],
            "std_unit": "ns",
            "scenarios": [
                {
                    "condition": "IF=ID, di/dt=100A/µs",
                    "limits": {
                        "HV_Power_THT": [200, 300, 500, 750, 1000],
                        "MV_Power_SMD": [50, 75, 100, 150, 250],
                        "LV_Logic_Level": [20, 30, 50, 75],
                        "SiC_High_Voltage": [15, 20, 30, 40],
                        "GaN_High_Frequency": [0],
                        "RF_Power": [0],
                        "Dual_N_Channel": [30, 50, 100],
                        "Dual_P_Channel": [30, 50],
                        "Complementary_Pair": [50],
                        "Depletion_Mode": [100, 200]
                    }
                }
            ]
        },
        {
            "key": "qrr",
            "symbol": "Q<sub>rr</sub>",
            "spec_type": "max_limit",
            "column_model": "TYP_MAX",
            "engineering_class": "PERFORMANCE_LIMIT",
            "special_semantics": "NOT_APPLICABLE",
            "llm_context": {
                "formal_name": "Body Diode Reverse Recovery Charge",
                "aliases": ["Qrr", "Recovery Charge"]
            },
            "possible_units": ["nC", "µC"],
            "std_unit": "nC",
            "scenarios": [
                {
                    "condition": "IF=ID, di/dt=100A/µs",
                    "limits": {
                        "HV_Power_THT": [500, 1000, 2000, 4000],
                        "MV_Power_SMD": [100, 200, 500, 1000],
                        "LV_Logic_Level": [10, 20, 50, 100],
                        "SiC_High_Voltage": [50, 100, 200],
                        "GaN_High_Frequency": [0],
                        "RF_Power": [0],
                        "Dual_N_Channel": [20, 50, 100],
                        "Dual_P_Channel": [20, 50],
                        "Complementary_Pair": [50],
                        "Depletion_Mode": [100, 500]
                    }
                }
            ]
        },
        {
            "key": "vgs_th",
            "symbol": "V<sub>GS(th)</sub>",
            "spec_type": "min_max_range",
            "column_model": "MIN_TYP_MAX",
            "engineering_class": "NOMINAL_PARAMETER",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Gate Threshold Voltage",
                "aliases": ["Gate Threshold", "Threshold Voltage", "VGS(th)", "Vth"]
            },
            "possible_units": ["V"],
            "std_unit": "V",
            "scenarios": [
                {
                    "condition": "VDS=VGS, ID=250µA, Tj=25°C",
                    "limits": {
                        "HV_Power_THT": [[2, 4], [2.5, 4.5], [3, 5]],
                        "MV_Power_SMD": [[1, 2.5], [1.5, 3], [2, 4]],
                        "LV_Logic_Level": [[0.4, 1], [0.5, 1.5], [0.8, 2], [1, 2.5]],
                        "SiC_High_Voltage": [[2, 4], [2.5, 5]],
                        "GaN_High_Frequency": [[0.5, 2.5], [1, 3]],
                        "RF_Power": [[2, 4], [2.5, 4.5]],
                        "Dual_N_Channel": [[0.5, 1.5], [1, 2.5]],
                        "Dual_P_Channel": [[-2.5, -1], [-2.5, -0.5]],
                        "Complementary_Pair": [[1, 2.5]],
                        "Depletion_Mode": [[-2.5, -0.3], [-4, -1], [-6, -2]]
                    }
                }
            ]
        }
    ],

    # ==========================================================================
    # DYNAMIC CHARACTERISTICS (הוספתי את החלק הזה שהיה חסר)
    # ==========================================================================
    "DYNAMIC_CHAR": [
        {
            "key": "ciss",
            "symbol": "C<sub>iss</sub>",
            "spec_type": "nominal",
            "column_model": "TYP_MAX",
            "engineering_class": "NOMINAL_PARAMETER",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Input Capacitance",
                "aliases": ["Input Capacitance", "Ciss", "Gate Capacitance"]
            },
            "possible_units": ["pF", "nF"],
            "std_unit": "pF",
            "scenarios": [
                {
                    "condition": "VGS=0V, VDS=25V, f=1MHz",
                    "limits": {
                        "HV_Power_THT": [2000, 3000, 5000, 8000, 10000, 15000],
                        "MV_Power_SMD": [1000, 2000, 3000, 5000, 8000, 10000],
                        "LV_Logic_Level": [200, 400, 600, 1000, 1500, 2500],
                        "SiC_High_Voltage": [1400, 2300, 3700, 4900],
                        "GaN_High_Frequency": [500, 900, 1400, 2100],
                        "RF_Power": [100, 200, 400, 600],
                        "Dual_N_Channel": [400, 800, 1500],
                        "Dual_P_Channel": [500, 1000],
                        "Complementary_Pair": [1000, 2000],
                        "Depletion_Mode": [500, 1000, 3000, 5000]
                    }
                }
            ]
        },
        {
            "key": "coss",
            "symbol": "C<sub>oss</sub>",
            "spec_type": "nominal",
            "column_model": "TYP_MAX",
            "engineering_class": "NOMINAL_PARAMETER",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Output Capacitance",
                "aliases": ["Output Capacitance", "Coss", "Drain Capacitance"]
            },
            "possible_units": ["pF", "nF"],
            "std_unit": "pF",
            "scenarios": [
                {
                    "condition": "VGS=0V, VDS=25V, f=1MHz",
                    "limits": {
                        "HV_Power_THT": [200, 300, 500, 800, 1000, 1500],
                        "MV_Power_SMD": [100, 200, 400, 600, 1000, 1500],
                        "LV_Logic_Level": [50, 100, 150, 300, 500],
                        "SiC_High_Voltage": [90, 140, 230, 310],
                        "GaN_High_Frequency": [80, 120, 190, 280],
                        "RF_Power": [30, 60, 100, 150],
                        "Dual_N_Channel": [100, 200, 400],
                        "Dual_P_Channel": [120, 240],
                        "Complementary_Pair": [200, 400],
                        "Depletion_Mode": [100, 300, 800, 1200]
                    }
                }
            ]
        },
        {
            "key": "crss",
            "symbol": "C<sub>rss</sub>",
            "spec_type": "nominal",
            "column_model": "TYP_MAX",
            "engineering_class": "NOMINAL_PARAMETER",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Reverse Transfer Capacitance",
                "aliases": ["Reverse Transfer Capacitance", "Crss", "Miller Capacitance", "Cgd"]
            },
            "possible_units": ["pF"],
            "std_unit": "pF",
            "scenarios": [
                {
                    "condition": "VGS=0V, VDS=25V, f=1MHz",
                    "limits": {
                        "HV_Power_THT": [50, 80, 100, 150, 200, 300],
                        "MV_Power_SMD": [20, 40, 60, 100, 150, 250],
                        "LV_Logic_Level": [10, 20, 30, 50, 80, 150],
                        "SiC_High_Voltage": [6, 12, 19, 25],
                        "GaN_High_Frequency": [4, 8, 12, 18],
                        "RF_Power": [5, 10, 20, 30],
                        "Dual_N_Channel": [20, 40, 80],
                        "Dual_P_Channel": [25, 50],
                        "Complementary_Pair": [40, 80],
                        "Depletion_Mode": [20, 50, 150, 250]
                    }
                }
            ]
        },
        {
            "key": "qg",
            "symbol": "Q<sub>g</sub>",
            "spec_type": "nominal",
            "column_model": "TYP_MAX",
            "engineering_class": "NOMINAL_PARAMETER",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Total Gate Charge",
                "aliases": ["Gate Charge", "Total Gate Charge", "Qg"]
            },
            "possible_units": ["nC", "µC"],
            "std_unit": "nC",
            "scenarios": [
                {
                    "condition": "VDD=rated/2, ID=rated, VGS=10V",
                    "limits": {
                        "HV_Power_THT": [50, 80, 100, 150, 200, 300],
                        "MV_Power_SMD": [20, 40, 60, 80, 120, 180],
                        "LV_Logic_Level": [3, 5, 10, 15, 20, 30],
                        "SiC_High_Voltage": [49, 82, 123, 164],
                        "GaN_High_Frequency": [5, 10, 20, 30, 45],
                        "RF_Power": [10, 20, 40],
                        "Dual_N_Channel": [5, 10, 20],
                        "Dual_P_Channel": [8, 15],
                        "Complementary_Pair": [15, 30],
                        "Depletion_Mode": [10, 30, 80, 150]
                    }
                }
            ]
        },
        {
            "key": "qgs",
            "symbol": "Q<sub>gs</sub>",
            "spec_type": "nominal",
            "column_model": "TYP_ONLY",
            "engineering_class": "NOMINAL_PARAMETER",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Gate-Source Charge",
                "aliases": ["Qgs", "Gate to Source Charge"]
            },
            "possible_units": ["nC"],
            "std_unit": "nC",
            "scenarios": [
                {
                    "condition": "VDD=rated/2, ID=rated",
                    "limits": {
                        "HV_Power_THT": [15, 25, 35, 50, 70],
                        "MV_Power_SMD": [8, 15, 25, 35, 50],
                        "LV_Logic_Level": [1, 2, 4, 6, 10],
                        "SiC_High_Voltage": [18, 30, 45, 60],
                        "GaN_High_Frequency": [2, 4, 8, 12],
                        "RF_Power": [4, 8, 15],
                        "Dual_N_Channel": [2, 4, 8],
                        "Dual_P_Channel": [3, 6],
                        "Complementary_Pair": [6, 12],
                        "Depletion_Mode": [4, 12, 30, 50]
                    }
                }
            ]
        },
        {
            "key": "qgd",
            "symbol": "Q<sub>gd</sub>",
            "spec_type": "nominal",
            "column_model": "TYP_ONLY",
            "engineering_class": "NOMINAL_PARAMETER",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Gate-Drain Charge (Miller Charge)",
                "aliases": ["Qgd", "Miller Charge", "Gate to Drain Charge"]
            },
            "possible_units": ["nC"],
            "std_unit": "nC",
            "scenarios": [
                {
                    "condition": "VDD=rated/2, ID=rated",
                    "limits": {
                        "HV_Power_THT": [20, 35, 50, 75, 100],
                        "MV_Power_SMD": [8, 15, 25, 35, 55],
                        "LV_Logic_Level": [1, 2, 4, 6, 10],
                        "SiC_High_Voltage": [14, 23, 35, 47],
                        "GaN_High_Frequency": [1.5, 3, 6, 9],
                        "RF_Power": [3, 6, 12],
                        "Dual_N_Channel": [2, 4, 8],
                        "Dual_P_Channel": [3, 5],
                        "Complementary_Pair": [5, 10],
                        "Depletion_Mode": [3, 10, 30, 50]
                    }
                }
            ]
        },
        {
            "key": "rg",
            "symbol": "R<sub>g</sub>",
            "spec_type": "nominal",
            "column_model": "TYP_ONLY",
            "engineering_class": "NOMINAL_PARAMETER",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Gate Resistance (Internal)",
                "aliases": ["Rg", "Internal Gate Resistance", "Gate Resistor"]
            },
            "possible_units": ["Ω"],
            "std_unit": "Ω",
            "scenarios": [
                {
                    "condition": "f=1MHz, open drain",
                    "limits": {
                        "HV_Power_THT": [1, 1.5, 2, 3, 5],
                        "MV_Power_SMD": [0.5, 1, 1.5, 2, 3],
                        "LV_Logic_Level": [0.3, 0.5, 1, 1.5, 2],
                        "SiC_High_Voltage": [2, 3, 4, 5],
                        "GaN_High_Frequency": [0.5, 1, 2, 3],
                        "RF_Power": [1, 2, 3],
                        "Dual_N_Channel": [1, 2, 3],
                        "Dual_P_Channel": [1.5, 3],
                        "Complementary_Pair": [2, 4],
                        "Depletion_Mode": [2, 4, 8]
                    }
                }
            ]
        }
    ],

    # ==========================================================================
    # SWITCHING CHARACTERISTICS (הוספתי את ההתחלה, שילבתי את המידע שלך בהמשך)
    # ==========================================================================
    "SWITCHING_CHAR": [
        {
            "key": "td_on",
            "symbol": "t<sub>d(on)</sub>",
            "spec_type": "nominal",
            "column_model": "TYP_MAX",
            "engineering_class": "NOMINAL_PARAMETER",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Turn-On Delay Time",
                "aliases": ["Turn-On Delay Time", "td(on)", "Delay Time On"]
            },
            "possible_units": ["ns"],
            "std_unit": "ns",
            "scenarios": [
                {
                    "condition": "VDD=rated/2, ID=rated, RG=specified, VGS=10V",
                    "limits": {
                        "HV_Power_THT": [20, 30, 50, 75, 100, 150],
                        "MV_Power_SMD": [10, 15, 20, 30, 50, 75],
                        "LV_Logic_Level": [3, 5, 8, 10, 15, 25],
                        "SiC_High_Voltage": [12, 18, 25, 35],
                        "GaN_High_Frequency": [2, 4, 6, 10, 15],
                        "RF_Power": [5, 10, 15],
                        "Dual_N_Channel": [5, 10, 20],
                        "Dual_P_Channel": [8, 15],
                        "Complementary_Pair": [12, 25],
                        "Depletion_Mode": [10, 30, 75]
                    }
                }
            ]
        },
        {
            "key": "tr",
            "symbol": "t<sub>r</sub>",
            "spec_type": "nominal",
            "column_model": "TYP_MAX",
            "engineering_class": "NOMINAL_PARAMETER",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Rise Time",
                "aliases": ["tr", "Rise Time", "Turn-On Rise Time"]
            },
            "possible_units": ["ns"],
            "std_unit": "ns",
            "scenarios": [
                {
                    "condition": "VDD=rated/2, ID=rated, RG=specified",
                    "limits": {
                        "HV_Power_THT": [30, 50, 75, 100, 150, 200],
                        "MV_Power_SMD": [20, 30, 50, 80, 120, 150],
                        "LV_Logic_Level": [3, 5, 10, 15, 25, 40],
                        "SiC_High_Voltage": [15, 23, 35, 50],
                        "GaN_High_Frequency": [2, 4, 8, 12, 20],
                        "RF_Power": [8, 15, 25],
                        "Dual_N_Channel": [8, 15, 30],
                        "Dual_P_Channel": [12, 25],
                        "Complementary_Pair": [20, 40],
                        "Depletion_Mode": [20, 50, 150]
                    }
                }
            ]
        },
        {
            "key": "td_off",
            "symbol": "t<sub>d(off)</sub>",
            "spec_type": "nominal",
            "column_model": "TYP_MAX",
            "engineering_class": "NOMINAL_PARAMETER",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Turn-Off Delay Time",
                "aliases": ["Turn-Off Delay Time", "td(off)", "Delay Time Off"]
            },
            "possible_units": ["ns"],
            "std_unit": "ns",
            "scenarios": [
                {
                    "condition": "VDD=rated/2, ID=rated, RG=specified",
                    "limits": {
                        "HV_Power_THT": [50, 75, 100, 150, 200, 300],
                        "MV_Power_SMD": [20, 30, 50, 75, 100, 150],
                        "LV_Logic_Level": [5, 10, 15, 20, 30, 50],
                        "SiC_High_Voltage": [20, 30, 45, 60],
                        "GaN_High_Frequency": [3, 6, 10, 15, 25],
                        "RF_Power": [10, 20, 30],
                        "Dual_N_Channel": [10, 20, 40],
                        "Dual_P_Channel": [15, 30],
                        "Complementary_Pair": [25, 50],
                        "Depletion_Mode": [20, 50, 150]
                    }
                }
            ]
        },
        {
            "key": "tf",
            "symbol": "t<sub>f</sub>",
            "spec_type": "nominal",
            "column_model": "TYP_MAX",
            "engineering_class": "NOMINAL_PARAMETER",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Fall Time",
                "aliases": ["tf", "Fall Time", "Turn-Off Fall Time"]
            },
            "possible_units": ["ns"],
            "std_unit": "ns",
            "scenarios": [
                {
                    "condition": "VDD=rated/2, ID=rated, RG=specified",
                    "limits": {
                        "HV_Power_THT": [40, 60, 100, 150, 200, 300],
                        "MV_Power_SMD": [20, 30, 50, 75, 120, 180],
                        "LV_Logic_Level": [4, 8, 12, 20, 30, 50],
                        "SiC_High_Voltage": [18, 28, 42, 56],
                        "GaN_High_Frequency": [3, 5, 10, 15, 25],
                        "RF_Power": [10, 20, 30],
                        "Dual_N_Channel": [10, 20, 40],
                        "Dual_P_Channel": [15, 30],
                        "Complementary_Pair": [25, 50],
                        "Depletion_Mode": [25, 60, 180]
                    }
                }
            ]
        },
        {
            "key": "eon",
            "symbol": "E<sub>on</sub>",
            "spec_type": "nominal",
            "column_model": "TYP_MAX",
            "engineering_class": "NOMINAL_PARAMETER",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Turn-On Switching Energy",
                "aliases": ["Eon", "Turn-On Energy", "Switching Loss On"]
            },
            "possible_units": ["µJ", "mJ"],
            "std_unit": "µJ",
            "scenarios": [
                {
                    "condition": "VDD=rated/2, ID=rated, RG=specified, Tj=25°C",
                    "limits": {
                        "HV_Power_THT": [500, 1000, 2000, 3000, 5000, 8000],
                        "MV_Power_SMD": [50, 100, 200, 400, 800, 1500],
                        "LV_Logic_Level": [5, 10, 20, 40, 80, 150],
                        "SiC_High_Voltage": [200, 350, 600, 900],
                        "GaN_High_Frequency": [10, 20, 40, 80, 150],
                        "RF_Power": [50, 100, 200],
                        "Dual_N_Channel": [20, 50, 100],
                        "Dual_P_Channel": [30, 75],
                        "Complementary_Pair": [60, 150],
                        "Depletion_Mode": [100, 300, 1000]
                    }
                }
            ]
        },
        {
            "key": "eoff",
            "symbol": "E<sub>off</sub>",
            "spec_type": "nominal",
            "column_model": "TYP_MAX",
            "engineering_class": "NOMINAL_PARAMETER",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Turn-Off Switching Energy",
                "aliases": ["Eoff", "Turn-Off Energy", "Switching Loss Off"]
            },
            "possible_units": ["µJ", "mJ"],
            "std_unit": "µJ",
            "scenarios": [
                {
                    "condition": "VDD=rated/2, ID=rated, RG=specified, Tj=25°C",
                    "limits": {
                        "HV_Power_THT": [300, 600, 1200, 2000, 3500, 6000],
                        "MV_Power_SMD": [30, 60, 120, 250, 500, 1000],
                        "LV_Logic_Level": [3, 6, 12, 25, 50, 100],
                        "SiC_High_Voltage": [100, 180, 320, 480],
                        "GaN_High_Frequency": [5, 10, 20, 40, 80],
                        "RF_Power": [30, 60, 120],
                        "Dual_N_Channel": [12, 30, 60],
                        "Dual_P_Channel": [18, 45],
                        "Complementary_Pair": [36, 90],
                        "Depletion_Mode": [60, 180, 600]
                    }
                }
            ]
        },
        {
            "key": "etot",
            "symbol": "E<sub>tot</sub>",
            "spec_type": "nominal",
            "column_model": "TYP_MAX",
            "engineering_class": "NOMINAL_PARAMETER",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Total Switching Energy",
                "aliases": ["Etot", "Total Energy Loss", "Eon + Eoff"]
            },
            "possible_units": ["µJ", "mJ"],
            "std_unit": "µJ",
            "scenarios": [
                {
                    "condition": "VDD=rated/2, ID=rated, RG=specified, Tj=25°C",
                    "limits": {
                        "HV_Power_THT": [800, 1600, 3200, 5000, 8500, 14000],
                        "MV_Power_SMD": [80, 160, 320, 650, 1300, 2500],
                        "LV_Logic_Level": [8, 16, 32, 65, 130, 250],
                        "SiC_High_Voltage": [300, 530, 920, 1380],
                        "GaN_High_Frequency": [15, 30, 60, 120, 230],
                        "RF_Power": [80, 160, 320],
                        "Dual_N_Channel": [32, 80, 160],
                        "Dual_P_Channel": [48, 120],
                        "Complementary_Pair": [96, 240],
                        "Depletion_Mode": [160, 480, 1600]
                    }
                }
            ]
        }
    ],

    # ==========================================================================
    # THERMAL CHARACTERISTICS
    # ==========================================================================
    "THERMAL_CHAR": [
        {
            "key": "thermal_resistance_jc",
            "symbol": "R<sub>θJC</sub>",
            "spec_type": "max_limit",
            "column_model": "TYP_MAX",
            "engineering_class": "NOMINAL_PARAMETER",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Thermal Resistance Junction to Case",
                "aliases": ["RθJC", "Rth(j-c)", "Thermal Res. J-C", "Theta JC"]
            },
            "possible_units": ["°C/W"],
            "std_unit": "°C/W",
            "scenarios": [
                {
                    "condition": "Steady state, drain-tab",
                    "limits": {
                        "HV_Power_THT": [0.3, 0.4, 0.5, 0.6, 0.8, 1.0, 1.5, 2.0],
                        "MV_Power_SMD": [0.5, 0.8, 1.0, 1.2, 1.5, 2.0, 2.5, 3.0],
                        "LV_Logic_Level": [10, 15, 20, 30, 50, 80, 125],
                        "SiC_High_Voltage": [0.6, 0.9, 1.2, 1.5, 1.8],
                        "GaN_High_Frequency": [1.0, 1.5, 2.5, 4.0, 6.0],
                        "RF_Power": [1.0, 2.0, 3.0, 5.0],
                        "Dual_N_Channel": [20, 40, 60, 100],
                        "Dual_P_Channel": [30, 60],
                        "Complementary_Pair": [50, 100],
                        "Depletion_Mode": [5, 10, 20, 50, 100]
                    }
                }
            ]
        },
        {
            "key": "thermal_resistance_ja",
            "symbol": "R<sub>θJA</sub>",
            "spec_type": "max_limit",
            "column_model": "TYP_MAX",
            "engineering_class": "NOMINAL_PARAMETER",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Thermal Resistance Junction to Ambient",
                "aliases": ["RθJA", "Rth(j-a)", "Thermal Res. J-A", "Theta JA"]
            },
            "possible_units": ["°C/W"],
            "std_unit": "°C/W",
            "scenarios": [
                {
                    "condition": "Standard PCB, still air",
                    "limits": {
                        "HV_Power_THT": [40, 50, 60, 70, 85, 100],
                        "MV_Power_SMD": [40, 50, 62, 80, 100, 125],
                        "LV_Logic_Level": [100, 125, 150, 200, 250, 400],
                        "SiC_High_Voltage": [35, 45, 55, 70],
                        "GaN_High_Frequency": [50, 70, 100, 150],
                        "RF_Power": [40, 60, 80],
                        "Dual_N_Channel": [150, 200, 300],
                        "Dual_P_Channel": [200, 300],
                        "Complementary_Pair": [250, 350],
                        "Depletion_Mode": [100, 150, 200, 300]
                    }
                }
            ]
        },
        {
            "key": "thermal_resistance_cs",
            "symbol": "R<sub>θCS</sub>",
            "spec_type": "nominal",
            "column_model": "TYP_ONLY",
            "engineering_class": "NOMINAL_PARAMETER",
            "special_semantics": "NOT_APPLICABLE",
            "llm_context": {
                "formal_name": "Thermal Resistance Case to Sink",
                "aliases": ["RθCS", "Rth(c-s)", "Thermal Res. C-S"]
            },
            "possible_units": ["°C/W"],
            "std_unit": "°C/W",
            "scenarios": [
                {
                    "condition": "With thermal interface material",
                    "limits": {
                        "HV_Power_THT": [0.1, 0.15, 0.2, 0.25, 0.3, 0.5],
                        "MV_Power_SMD": [0],
                        "LV_Logic_Level": [0],
                        "SiC_High_Voltage": [0.15, 0.2, 0.3],
                        "GaN_High_Frequency": [0],
                        "RF_Power": [0.2, 0.3, 0.5],
                        "Dual_N_Channel": [0],
                        "Dual_P_Channel": [0],
                        "Complementary_Pair": [0],
                        "Depletion_Mode": [0.25, 0.4]
                    }
                }
            ]
        },
        {
            "key": "transient_thermal_impedance_jc",
            "symbol": "Z<sub>θJC</sub>",
            "spec_type": "nominal",
            "column_model": "TYP_ONLY",
            "engineering_class": "NOMINAL_PARAMETER",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Transient Thermal Impedance Junction to Case",
                "aliases": ["ZθJC", "Pulse Thermal Impedance", "Thermal Impedance"]
            },
            "possible_units": ["°C/W"],
            "std_unit": "°C/W",
            "scenarios": [
                {
                    "condition": "Single pulse, various durations",
                    "limits": {
                        "HV_Power_THT": [0.15, 0.2, 0.25, 0.3, 0.4, 0.5],
                        "MV_Power_SMD": [0.25, 0.4, 0.5, 0.6, 0.75, 1.0],
                        "LV_Logic_Level": [5, 7.5, 10, 15, 25, 40],
                        "SiC_High_Voltage": [0.3, 0.45, 0.6, 0.75],
                        "GaN_High_Frequency": [0.5, 0.75, 1.25, 2.0],
                        "RF_Power": [0.5, 1.0, 1.5],
                        "Dual_N_Channel": [10, 20, 30],
                        "Dual_P_Channel": [15, 30],
                        "Complementary_Pair": [25, 50],
                        "Depletion_Mode": [2.5, 5, 10, 25]
                    }
                }
            ]
        },
        {
            "key": "operating_junction_temp_range",
            "symbol": "T<sub>J</sub>",
            "spec_type": "operational_range",
            "column_model": "MIN_MAX",
            "engineering_class": "OPERATING_CONDITION",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Operating Junction Temperature Range",
                "aliases": ["Operating Temp", "TJ Range", "Junction Temp Range"]
            },
            "possible_units": ["°C"],
            "std_unit": "°C",
            "scenarios": [
                {
                    "condition": "Operating",
                    "limits": {
                        "HV_Power_THT": [[-55, 150], [-55, 175]],
                        "MV_Power_SMD": [[-55, 150], [-55, 175]],
                        "LV_Logic_Level": [[-55, 150], [-55, 175]],
                        "SiC_High_Voltage": [[-55, 175], [-40, 200]],
                        "GaN_High_Frequency": [[-55, 150], [-55, 175], [-40, 200]],
                        "RF_Power": [[-65, 200], [-55, 225]],
                        "Dual_N_Channel": [[-55, 150], [-55, 175]],
                        "Dual_P_Channel": [[-55, 150]],
                        "Complementary_Pair": [[-55, 150]],
                        "Depletion_Mode": [[-55, 150], [-55, 175]]
                    }
                }
            ]
        }
    ],

    # ==========================================================================
    # PACKAGE & MECHANICAL  
    # ==========================================================================
    "PACKAGE": [
        {
            "key": "package_code",
            "symbol": "PKG",
            "spec_type": "mechanical",
            "column_model": "TYP_ONLY",
            "engineering_class": "MECHANICAL",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Package Type",
                "aliases": ["Case Style", "Package Code", "Outline"]
            },
            "possible_units": [""],
            "std_unit": "",
            "scenarios": [
                {
                    "condition": "N/A",
                    "limits": {
                        "HV_Power_THT": ["TO-220", "TO-220AB", "TO-247", "TO-3P", "TO-264", "ITO-220", "TO-220FP"],
                        "MV_Power_SMD": ["DPAK(TO-252)", "D2PAK(TO-263)", "TO-263-7", "PowerPAK_SO-8", "PQFN_5x6", "DFN_5x6", "TOLL", "DirectFET"],
                        "LV_Logic_Level": ["SOT-23", "SOT-23-3", "SOT-23-6", "SOT-323", "SC-70", "SOT-223", "TSOP-6", "DFN_2x2", "DFN_3x3", "SON"],
                        "SiC_High_Voltage": ["TO-247-3", "TO-247-4", "TO-263-7", "D2PAK-7L"],
                        "GaN_High_Frequency": ["PQFN_5x6", "DFN_5x6", "DFN_8x8", "TOLL"],
                        "RF_Power": ["TO-220", "TO-247", "Flange_Mount", "NI-780", "NI-1230"],
                        "Dual_N_Channel": ["SOT-23-6", "SOT-363", "SC-70-6", "TSOP-6", "DFN_3x3", "PowerPAK_SO-8"],
                        "Dual_P_Channel": ["SOT-23-6", "SOT-363", "SC-70-6", "TSOP-6"],
                        "Complementary_Pair": ["SOT-23-6", "SOT-363", "SC-70-6"],
                        "Depletion_Mode": ["TO-220", "TO-252", "SOT-23", "SOT-89"]
                    }
                }
            ]
        },
        {
            "key": "pin_configuration",
            "symbol": "PIN",
            "spec_type": "mechanical",
            "column_model": "TYP_ONLY",
            "engineering_class": "MECHANICAL",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Pin Configuration",
                "aliases": ["Pinout", "Pin Assignment"]
            },
            "possible_units": [""],
            "std_unit": "",
            "scenarios": [
                {
                    "condition": "N/A",
                    "limits": {
                        "HV_Power_THT": ["G-D-S", "G-D-S-S"],
                        "MV_Power_SMD": ["G-D-S", "S-G-S-D-D-D-D-D"],
                        "LV_Logic_Level": ["G-S-D", "G-D-S", "S-G-D"],
                        "SiC_High_Voltage": ["G-D-S-Kelvin_S"],
                        "GaN_High_Frequency": ["G-S-D", "S-G-D-S"],
                        "RF_Power": ["G-D-S"],
                        "Dual_N_Channel": ["G1-S1-S2-G2-D1-D2", "G1-S1-D1-D2-S2-G2"],
                        "Dual_P_Channel": ["G1-S1-S2-G2-D1-D2"],
                        "Complementary_Pair": ["GN-SN-SP-GP-DN-DP"],
                        "Depletion_Mode": ["G-D-S"]
                    }
                }
            ]
        },
        {
            "key": "mounting_type",
            "symbol": "MNT",
            "spec_type": "mechanical",
            "column_model": "TYP_ONLY",
            "engineering_class": "MECHANICAL",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Mounting Type",
                "aliases": ["Mounting Style", "Mount"]
            },
            "possible_units": [""],
            "std_unit": "",
            "scenarios": [
                {
                    "condition": "N/A",
                    "limits": {
                        "HV_Power_THT": ["Through_Hole", "Chassis_Mount"],
                        "MV_Power_SMD": ["SMD", "SMD_with_Tab"],
                        "LV_Logic_Level": ["SMD"],
                        "SiC_High_Voltage": ["Through_Hole", "Chassis_Mount"],
                        "GaN_High_Frequency": ["SMD"],
                        "RF_Power": ["Through_Hole", "Flange_Mount", "Stud_Mount"],
                        "Dual_N_Channel": ["SMD"],
                        "Dual_P_Channel": ["SMD"],
                        "Complementary_Pair": ["SMD"],
                        "Depletion_Mode": ["Through_Hole", "SMD"]
                    }
                }
            ]
        },
        {
            "key": "polarity",
            "symbol": "POL",
            "spec_type": "mechanical",
            "column_model": "TYP_ONLY",
            "engineering_class": "MECHANICAL",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Channel Polarity",
                "aliases": ["N-Channel", "P-Channel", "Type"]
            },
            "possible_units": [""],
            "std_unit": "",
            "scenarios": [
                {
                    "condition": "N/A",
                    "limits": {
                        "HV_Power_THT": ["N-Channel"],
                        "MV_Power_SMD": ["N-Channel", "P-Channel"],
                        "LV_Logic_Level": ["N-Channel", "P-Channel"],
                        "SiC_High_Voltage": ["N-Channel"],
                        "GaN_High_Frequency": ["N-Channel", "P-Channel"],
                        "RF_Power": ["N-Channel"],
                        "Dual_N_Channel": ["Dual_N-Channel"],
                        "Dual_P_Channel": ["Dual_P-Channel"],
                        "Complementary_Pair": ["N+P_Channel"],
                        "Depletion_Mode": ["N-Channel", "P-Channel"]
                    }
                }
            ]
        },
        {
            "key": "lead_finish",
            "symbol": "FINISH",
            "spec_type": "mechanical",
            "column_model": "TYP_ONLY",
            "engineering_class": "MECHANICAL",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Terminal Finish",
                "aliases": ["Lead Finish", "Plating", "Terminal Plating"]
            },
            "possible_units": [""],
            "std_unit": "",
            "scenarios": [
                {
                    "condition": "N/A",
                    "limits": {
                        "HV_Power_THT": ["Matte_Sn", "SnPb"],
                        "MV_Power_SMD": ["Matte_Sn", "NiPdAu"],
                        "LV_Logic_Level": ["Matte_Sn", "NiPdAu"],
                        "SiC_High_Voltage": ["Matte_Sn"],
                        "GaN_High_Frequency": ["NiPdAu", "Matte_Sn"],
                        "RF_Power": ["Au", "Matte_Sn"],
                        "Dual_N_Channel": ["Matte_Sn"],
                        "Dual_P_Channel": ["Matte_Sn"],
                        "Complementary_Pair": ["Matte_Sn"],
                        "Depletion_Mode": ["Matte_Sn", "SnPb"]
                    }
                }
            ]
        },
        {
            "key": "weight",
            "symbol": "W",
            "spec_type": "mechanical",
            "column_model": "TYP_ONLY",
            "engineering_class": "MECHANICAL",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Weight",
                "aliases": ["Mass", "Part Weight"]
            },
            "possible_units": ["g", "mg"],
            "std_unit": "g",
            "scenarios": [
                {
                    "condition": "Typical",
                    "limits": {
                        "HV_Power_THT": [1.5, 2, 3, 5, 8, 15],
                        "MV_Power_SMD": [0.1, 0.2, 0.4, 0.8, 1.5],
                        "LV_Logic_Level": [0.002, 0.005, 0.01, 0.02, 0.05],
                        "SiC_High_Voltage": [3, 5, 8],
                        "GaN_High_Frequency": [0.2, 0.5, 1],
                        "RF_Power": [10, 20, 50, 100],
                        "Dual_N_Channel": [0.01, 0.02, 0.05],
                        "Dual_P_Channel": [0.01, 0.02],
                        "Complementary_Pair": [0.02, 0.05],
                        "Depletion_Mode": [1, 2, 5]
                    }
                }
            ]
        }
    ],

    # ==========================================================================
    # RELIABILITY & QUALITY
    # ==========================================================================
    "RELIABILITY": [
        {
            "key": "failure_rate",
            "symbol": "λ",
            "spec_type": "max_limit",
            "column_model": "MAX_ONLY",
            "engineering_class": "RELIABILITY",
            "special_semantics": "NONE",
            "llm_context": {
                "formal_name": "Failure Rate",
                "aliases": ["FIT Rate", "Lambda", "MTBF"]
            },
            "possible_units": ["FIT", "PPM"],
            "std_unit": "FIT",
            "scenarios": [
                {
                    "condition": "Standard conditions",
                    "limits": {
                        "HV_Power_THT": [1, 5, 10],
                        "MV_Power_SMD": [1, 5, 10],
                        "LV_Logic_Level": [0.5, 1, 5],
                        "SiC_High_Voltage": [5, 10],
                        "GaN_High_Frequency": [5, 10],
                        "RF_Power": [5, 10],
                        "Dual_N_Channel": [1, 5],
                        "Dual_P_Channel": [1, 5],
                        "Complementary_Pair": [2, 10],
                        "Depletion_Mode": [5, 10]
                    }
                }
            ]
        },
        {
            "key": "moisture_sensitivity_level",
            "symbol": "MSL",
            "spec_type": "nominal",
            "column_model": "TYP_ONLY",
            "engineering_class": "RELIABILITY",
            "special_semantics": "NOT_APPLICABLE",
            "llm_context": {
                "formal_name": "Moisture Sensitivity Level",
                "aliases": ["MSL", "Moisture Level"]
            },
            "possible_units": [""],
            "std_unit": "",
            "scenarios": [
                {
                    "condition": "Per J-STD-020",
                    "limits": {
                        "HV_Power_THT": [0],
                        "MV_Power_SMD": ["1", "2", "3"],
                        "LV_Logic_Level": ["1", "2"],
                        "SiC_High_Voltage": ["1", "2"],
                        "GaN_High_Frequency": ["1", "2", "3"],
                        "RF_Power": [0],
                        "Dual_N_Channel": ["1"],
                        "Dual_P_Channel": ["1"],
                        "Complementary_Pair": ["1"],
                        "Depletion_Mode": [0]
                    }
                }
            ]
        }
    ]
},
    # ==========================================================================
    # 5. VOLTAGE_REGULATOR חלק5
    # ==========================================================================
    
  "VOLTAGE_REGULATOR": {
    "archetypes": [
      "LDO_Low_Voltage_CMOS",
      "LDO_High_Voltage_Bipolar"
    ],
    "ABS_MAX": [
      {
        "key": "vin_max",
        "symbol": "V<sub>IN(max)</sub>",
        "spec_type": "max_rating",
        "column_model": "MAX_ONLY",
        "engineering_class": "SAFETY_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Maximum Input Voltage",
          "aliases": ["Input Voltage Max", "Abs Max Input Voltage"]
        },
        "possible_units": ["V"],
        "std_unit": "V",
        "scenarios": [
          {
            "condition": "Absolute Max",
            "limits": {
              "LDO_Low_Voltage_CMOS": [6.0, 7.0],
              "LDO_High_Voltage_Bipolar": [40, 60]
            }
          }
        ]
      },
      {
        "key": "vout_max",
        "symbol": "V<sub>OUT(max)</sub>",
        "spec_type": "max_rating",
        "column_model": "MAX_ONLY",
        "engineering_class": "SAFETY_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Maximum Output Voltage",
          "aliases": ["Output Voltage Max", "Abs Max Output"]
        },
        "possible_units": ["V"],
        "std_unit": "V",
        "scenarios": [
          {
            "condition": "Absolute Max",
            "limits": {
              "LDO_Low_Voltage_CMOS": [5.5, 6.0],
              "LDO_High_Voltage_Bipolar": [30, 40]
            }
          }
        ]
      },
      {
        "key": "power_dissipation",
        "symbol": "P<sub>D</sub>",
        "spec_type": "max_rating",
        "column_model": "MAX_ONLY",
        "engineering_class": "SAFETY_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Maximum Power Dissipation",
          "aliases": ["Power Dissipation", "Max Power"]
        },
        "possible_units": ["W", "mW"],
        "std_unit": "W",
        "scenarios": [
          {
            "condition": "Ta=25°C",
            "limits": {
              "LDO_Low_Voltage_CMOS": [0.3, 0.5, 1],
              "LDO_High_Voltage_Bipolar": [1, 2, 20]
            }
          }
        ]
      },
      {
        "key": "tj_max",
        "symbol": "T<sub>j(max)</sub>",
        "spec_type": "max_rating",
        "column_model": "MAX_ONLY",
        "engineering_class": "SAFETY_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Maximum Junction Temperature",
          "aliases": ["Max Junction Temp"]
        },
        "possible_units": ["°C"],
        "std_unit": "°C",
        "scenarios": [
          {
            "condition": "Max",
            "limits": {
              "LDO_Low_Voltage_CMOS": [125, 150],
              "LDO_High_Voltage_Bipolar": [125, 150]
            }
          }
        ]
      },
      {
        "key": "storage_temp",
        "symbol": "T<sub>STG</sub>",
        "spec_type": "max_rating",
        "column_model": "MIN_MAX",
        "engineering_class": "SAFETY_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Storage Temperature Range",
          "aliases": ["Storage Temperature"]
        },
        "possible_units": ["°C"],
        "std_unit": "°C",
        "scenarios": [
          {
            "condition": "Range",
            "limits": {
              "LDO_Low_Voltage_CMOS": ["-65 to 150"],
              "LDO_High_Voltage_Bipolar": ["-65 to 150"]
            }
          }
        ]
      },
      {
        "key": "esd_hbm",
        "symbol": "V<sub>ESD</sub>",
        "spec_type": "max_rating",
        "column_model": "MIN_ONLY",
        "engineering_class": "SAFETY_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "ESD Human Body Model",
          "aliases": ["ESD HBM"]
        },
        "possible_units": ["kV"],
        "std_unit": "kV",
        "scenarios": [
          {
            "condition": "HBM",
            "limits": {
              "LDO_Low_Voltage_CMOS": [1, 2, 4],
              "LDO_High_Voltage_Bipolar": [2, 4]
            }
          }
        ]
      },
      {
        "key": "reverse_current",
        "symbol": "I<sub>REV</sub>",
        "spec_type": "max_rating",
        "column_model": "MAX_ONLY",
        "engineering_class": "SAFETY_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Maximum Reverse Current",
          "aliases": ["Reverse Current", "Max Reverse Current"]
        },
        "possible_units": ["mA"],
        "std_unit": "mA",
        "scenarios": [
          {
            "condition": "Vout > Vin",
            "limits": {
              "LDO_Low_Voltage_CMOS": [10, 50],
              "LDO_High_Voltage_Bipolar": [50, 100]
            }
          }
        ]
      },
      {
        "key": "lead_temp",
        "symbol": "T<sub>LEAD</sub>",
        "spec_type": "max_rating",
        "column_model": "MAX_ONLY",
        "engineering_class": "SAFETY_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Lead Temperature (Soldering)",
          "aliases": ["Soldering Temperature"]
        },
        "possible_units": ["°C"],
        "std_unit": "°C",
        "scenarios": [
          {
            "condition": "10s",
            "limits": {
              "LDO_Low_Voltage_CMOS": [260, 300],
              "LDO_High_Voltage_Bipolar": [260, 300]
            }
          }
        ]
      }
    ],
    "DYNAMIC_CHAR": [
      {
        "key": "loop_bandwidth",
        "symbol": "f<sub>BW</sub>",
        "spec_type": "nominal",
        "column_model": "TYP_ONLY",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Control Loop Bandwidth",
          "aliases": ["Loop Bandwidth"]
        },
        "possible_units": ["kHz", "MHz"],
        "std_unit": "kHz",
        "scenarios": [
          {
            "condition": "Nominal load",
            "limits": {
              "LDO_Low_Voltage_CMOS": [10, 50, 500],
              "LDO_High_Voltage_Bipolar": [1, 10, 50]
            }
          }
        ]
      },
      {
        "key": "phase_margin",
        "symbol": "φ<sub>M</sub>",
        "spec_type": "nominal",
        "column_model": "TYP_ONLY",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Phase Margin",
          "aliases": ["PM"]
        },
        "possible_units": ["deg", "°"],
        "std_unit": "deg",
        "scenarios": [
          {
            "condition": "Nominal load",
            "limits": {
              "LDO_Low_Voltage_CMOS": [45, 60, 70],
              "LDO_High_Voltage_Bipolar": [45, 60]
            }
          }
        ]
      },
      {
        "key": "gain_margin",
        "symbol": "GM",
        "spec_type": "nominal",
        "column_model": "TYP_ONLY",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Gain Margin",
          "aliases": ["Loop Gain Margin"]
        },
        "possible_units": ["dB"],
        "std_unit": "dB",
        "scenarios": [
          {
            "condition": "Nominal load",
            "limits": {
              "LDO_Low_Voltage_CMOS": [10, 15, 20],
              "LDO_High_Voltage_Bipolar": [10, 15]
            }
          }
        ]
      }
    ],
    "PACKAGE": [
      {
        "key": "package_code",
        "symbol": "PKG",
        "spec_type": "mechanical",
        "column_model": "TYP_ONLY",
        "engineering_class": "MECHANICAL",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Package",
          "aliases": ["Case", "Package Code"]
        },
        "possible_units": [""],
        "std_unit": "",
        "scenarios": [
          {
            "condition": "Type",
            "limits": {
              "LDO_Low_Voltage_CMOS": ["SOT-23", "DFN", "CSP"],
              "LDO_High_Voltage_Bipolar": ["TO-220", "SOT-223", "DPAK"]
            }
          }
        ]
      },
      {
        "key": "thermal_resistance_ja",
        "symbol": "θ<sub>JA</sub>",
        "spec_type": "nominal",
        "column_model": "TYP_MAX",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Thermal Resistance Junction to Ambient",
          "aliases": ["Rth(j-a)"]
        },
        "possible_units": ["°C/W"],
        "std_unit": "°C/W",
        "scenarios": [
          {
            "condition": "Standard PCB",
            "limits": {
              "LDO_Low_Voltage_CMOS": [150, 200, 300],
              "LDO_High_Voltage_Bipolar": [40, 60, 100]
            }
          }
        ]
      },
      {
        "key": "thermal_resistance_jc",
        "symbol": "θ<sub>JC</sub>",
        "spec_type": "nominal",
        "column_model": "TYP_MAX",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Thermal Resistance Junction to Case",
          "aliases": ["Rth(j-c)"]
        },
        "possible_units": ["°C/W"],
        "std_unit": "°C/W",
        "scenarios": [
          {
            "condition": "N/A",
            "limits": {
              "LDO_Low_Voltage_CMOS": [50, 80, 120],
              "LDO_High_Voltage_Bipolar": [2, 5, 15]
            }
          }
        ]
      }
    ],
    "ELEC_CHAR": [
      {
        "key": "vin_operating",
        "symbol": "V<sub>IN</sub>",
        "spec_type": "nominal",
        "column_model": "MIN_MAX",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Operating Input Voltage Range",
          "aliases": ["Input Voltage Range"]
        },
        "possible_units": ["V"],
        "std_unit": "V",
        "scenarios": [
          {
            "condition": "Operating",
            "limits": {
              "LDO_Low_Voltage_CMOS": ["1.8 to 5.5", "2.5 to 6.0"],
              "LDO_High_Voltage_Bipolar": ["3.0 to 40"]
            }
          }
        ]
      },
      {
        "key": "minimum_input_voltage",
        "symbol": "V<sub>IN(min)</sub>",
        "spec_type": "min_limit",
        "column_model": "MIN_ONLY",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Minimum Input Voltage",
          "aliases": ["Min Vin", "Minimum Operating Voltage"]
        },
        "possible_units": ["V"],
        "std_unit": "V",
        "scenarios": [
          {
            "condition": "For regulation",
            "limits": {
              "LDO_Low_Voltage_CMOS": [1.6, 1.8, 2.2],
              "LDO_High_Voltage_Bipolar": [2.5, 3.0, 4.5]
            }
          }
        ]
      },
      {
        "key": "output_voltage",
        "symbol": "V<sub>OUT</sub>",
        "spec_type": "nominal",
        "column_model": "MIN_TYP_MAX",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Output Voltage",
          "aliases": ["Nominal Output Voltage"]
        },
        "possible_units": ["V"],
        "std_unit": "V",
        "scenarios": [
          {
            "condition": "Nominal",
            "limits": {
              "LDO_Low_Voltage_CMOS": [1.2, 1.8, 3.3],
              "LDO_High_Voltage_Bipolar": [5.0, 12.0, 24.0]
            }
          }
        ]
      },
      {
        "key": "iout_max",
        "symbol": "I<sub>OUT</sub>",
        "spec_type": "max_rating",
        "column_model": "MAX_ONLY",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Rated Output Current",
          "aliases": ["Max Output Current"]
        },
        "possible_units": ["mA", "A"],
        "std_unit": "mA",
        "scenarios": [
          {
            "condition": "Nominal",
            "limits": {
              "LDO_Low_Voltage_CMOS": [150, 300, 500],
              "LDO_High_Voltage_Bipolar": [1000, 3000, 5000]
            }
          }
        ]
      },
      {
        "key": "iout_min",
        "symbol": "I<sub>OUT(min)</sub>",
        "spec_type": "min_limit",
        "column_model": "MIN_ONLY",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Minimum Load Current",
          "aliases": ["Min Load Current"]
        },
        "possible_units": ["µA", "mA"],
        "std_unit": "µA",
        "scenarios": [
          {
            "condition": "For regulation",
            "limits": {
              "LDO_Low_Voltage_CMOS": [0, 1, 10],
              "LDO_High_Voltage_Bipolar": [0, 5, 50]
            }
          }
        ]
      },
      {
        "key": "efficiency",
        "symbol": "η",
        "spec_type": "nominal",
        "column_model": "TYP_ONLY",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Efficiency",
          "aliases": ["Power Efficiency"]
        },
        "possible_units": ["%"],
        "std_unit": "%",
        "scenarios": [
          {
            "condition": "Nominal load",
            "limits": {
              "LDO_Low_Voltage_CMOS": [70, 80, 90],
              "LDO_High_Voltage_Bipolar": [60, 70, 80]
            }
          }
        ]
      },
      {
        "key": "dropout_voltage",
        "symbol": "V<sub>DO</sub>",
        "spec_type": "max_limit",
        "column_model": "TYP_MAX",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Dropout Voltage",
          "aliases": ["Min Input-Output Voltage"]
        },
        "possible_units": ["mV", "V"],
        "std_unit": "mV",
        "scenarios": [
          {
            "condition": "At Iout Max",
            "limits": {
              "LDO_Low_Voltage_CMOS": [100, 200, 300],
              "LDO_High_Voltage_Bipolar": [300, 500, 1500]
            }
          }
        ]
      },
      {
        "key": "vin_to_vout_differential",
        "symbol": "V<sub>IN</sub>-V<sub>OUT</sub>",
        "spec_type": "min_limit",
        "column_model": "MIN_ONLY",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Minimum Input-Output Differential",
          "aliases": ["Headroom Voltage"]
        },
        "possible_units": ["V", "mV"],
        "std_unit": "mV",
        "scenarios": [
          {
            "condition": "For regulation",
            "limits": {
              "LDO_Low_Voltage_CMOS": [100, 200, 300],
              "LDO_High_Voltage_Bipolar": [300, 500, 1000]
            }
          }
        ]
      },
      {
        "key": "output_voltage_accuracy",
        "symbol": "ΔV<sub>OUT</sub>",
        "spec_type": "nominal",
        "column_model": "MIN_TYP_MAX",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Output Voltage Accuracy",
          "aliases": ["Output Tolerance"]
        },
        "possible_units": ["%"],
        "std_unit": "%",
        "scenarios": [
          {
            "condition": "25°C",
            "limits": {
              "LDO_Low_Voltage_CMOS": [0.5, 1, 2],
              "LDO_High_Voltage_Bipolar": [1, 2, 4]
            }
          }
        ]
      },
      {
        "key": "line_regulation",
        "symbol": "ΔV<sub>OUT</sub>/ΔV<sub>IN</sub>",
        "spec_type": "max_limit",
        "column_model": "TYP_MAX",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Line Regulation",
          "aliases": ["Line Regulation"]
        },
        "possible_units": ["mV/V", "%/V"],
        "std_unit": "mV/V",
        "scenarios": [
          {
            "condition": "Vin range",
            "limits": {
              "LDO_Low_Voltage_CMOS": [0.01, 0.1, 1],
              "LDO_High_Voltage_Bipolar": [1, 5, 10]
            }
          }
        ]
      },
      {
        "key": "load_regulation",
        "symbol": "ΔV<sub>OUT</sub>/ΔI<sub>OUT</sub>",
        "spec_type": "max_limit",
        "column_model": "TYP_MAX",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Load Regulation",
          "aliases": ["Load Regulation"]
        },
        "possible_units": ["mV", "%"],
        "std_unit": "mV",
        "scenarios": [
          {
            "condition": "0mA to Imax",
            "limits": {
              "LDO_Low_Voltage_CMOS": [5, 10, 20],
              "LDO_High_Voltage_Bipolar": [10, 50, 100]
            }
          }
        ]
      },
      {
        "key": "psrr",
        "symbol": "PSRR",
        "spec_type": "min_limit",
        "column_model": "MIN_TYP_MAX",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Power Supply Rejection Ratio",
          "aliases": ["Ripple Rejection"]
        },
        "possible_units": ["dB"],
        "std_unit": "dB",
        "scenarios": [
          {
            "condition": "1kHz",
            "limits": {
              "LDO_Low_Voltage_CMOS": [45, 55, 75],
              "LDO_High_Voltage_Bipolar": [35, 45, 55]
            }
          }
        ]
      },
      {
        "key": "quiescent_current",
        "symbol": "I<sub>Q</sub>",
        "spec_type": "max_limit",
        "column_model": "TYP_MAX",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Quiescent Current",
          "aliases": ["Ground Current"]
        },
        "possible_units": ["µA", "mA"],
        "std_unit": "µA",
        "scenarios": [
          {
            "condition": "No load",
            "limits": {
              "LDO_Low_Voltage_CMOS": [1, 5, 100],
              "LDO_High_Voltage_Bipolar": [500, 1000, 5000]
            }
          }
        ]
      },
      {
        "key": "shutdown_current",
        "symbol": "I<sub>SD</sub>",
        "spec_type": "max_limit",
        "column_model": "TYP_MAX",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Shutdown Current",
          "aliases": ["Standby Current"]
        },
        "possible_units": ["µA", "nA"],
        "std_unit": "µA",
        "scenarios": [
          {
            "condition": "Enable Low",
            "limits": {
              "LDO_Low_Voltage_CMOS": [0.01, 0.1, 1],
              "LDO_High_Voltage_Bipolar": [1, 5, 10]
            }
          }
        ]
      },
      {
        "key": "input_supply_current",
        "symbol": "I<sub>IN</sub>",
        "spec_type": "nominal",
        "column_model": "TYP_ONLY",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Input Supply Current",
          "aliases": ["Total Input Current"]
        },
        "possible_units": ["mA", "A"],
        "std_unit": "mA",
        "scenarios": [
          {
            "condition": "At Iout Max",
            "limits": {
              "LDO_Low_Voltage_CMOS": [150, 300, 500],
              "LDO_High_Voltage_Bipolar": [500, 1000, 5000]
            }
          }
        ]
      },
      {
        "key": "reverse_leakage_current",
        "symbol": "I<sub>R(LEAK)</sub>",
        "spec_type": "max_limit",
        "column_model": "TYP_MAX",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Reverse Leakage Current",
          "aliases": ["Reverse Current Leakage"]
        },
        "possible_units": ["µA", "nA"],
        "std_unit": "µA",
        "scenarios": [
          {
            "condition": "Vout = Vin, device off",
            "limits": {
              "LDO_Low_Voltage_CMOS": [0.001, 0.01, 0.1],
              "LDO_High_Voltage_Bipolar": [0.1, 1, 10]
            }
          }
        ]
      },
      {
        "key": "output_noise",
        "symbol": "V<sub>n</sub>",
        "spec_type": "max_limit",
        "column_model": "TYP_MAX",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Output Noise Voltage",
          "aliases": ["RMS Noise"]
        },
        "possible_units": ["µVrms"],
        "std_unit": "µVrms",
        "scenarios": [
          {
            "condition": "10Hz to 100kHz",
            "limits": {
              "LDO_Low_Voltage_CMOS": [10, 30, 100],
              "LDO_High_Voltage_Bipolar": [50, 100, 200]
            }
          }
        ]
      },
      {
        "key": "output_noise_density",
        "symbol": "e<sub>n</sub>",
        "spec_type": "max_limit",
        "column_model": "TYP_MAX",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Output Noise Spectral Density",
          "aliases": ["Noise Density"]
        },
        "possible_units": ["µV/√Hz", "nV/√Hz"],
        "std_unit": "nV/√Hz",
        "scenarios": [
          {
            "condition": "At 1kHz",
            "limits": {
              "LDO_Low_Voltage_CMOS": [10, 30, 100],
              "LDO_High_Voltage_Bipolar": [50, 100, 200]
            }
          }
        ]
      },
      {
        "key": "enable_threshold_high",
        "symbol": "V<sub>EN(H)</sub>",
        "spec_type": "nominal",
        "column_model": "MIN_TYP_MAX",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Enable Threshold Voltage High",
          "aliases": ["Enable Input High"]
        },
        "possible_units": ["V"],
        "std_unit": "V",
        "scenarios": [
          {
            "condition": "Logic high",
            "limits": {
              "LDO_Low_Voltage_CMOS": [0.8, 1.2, 2.0],
              "LDO_High_Voltage_Bipolar": [2.0, 2.4]
            }
          }
        ]
      },
      {
        "key": "enable_threshold_low",
        "symbol": "V<sub>EN(L)</sub>",
        "spec_type": "nominal",
        "column_model": "MIN_TYP_MAX",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Enable Threshold Voltage Low",
          "aliases": ["Disable Threshold"]
        },
        "possible_units": ["V"],
        "std_unit": "V",
        "scenarios": [
          {
            "condition": "Logic low",
            "limits": {
              "LDO_Low_Voltage_CMOS": [0.3, 0.4, 0.5],
              "LDO_High_Voltage_Bipolar": [0.4, 0.8]
            }
          }
        ]
      },
      {
        "key": "enable_input_current",
        "symbol": "I<sub>EN</sub>",
        "spec_type": "max_limit",
        "column_model": "TYP_MAX",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Enable Pin Input Current",
          "aliases": ["Enable Pin Leakage"]
        },
        "possible_units": ["µA", "nA"],
        "std_unit": "nA",
        "scenarios": [
          {
            "condition": "Enable High/Low",
            "limits": {
              "LDO_Low_Voltage_CMOS": [1, 10, 100],
              "LDO_High_Voltage_Bipolar": [10, 100, 1000]
            }
          }
        ]
      },
      {
        "key": "uvlo_threshold",
        "symbol": "V<sub>UVLO</sub>",
        "spec_type": "nominal",
        "column_model": "MIN_TYP_MAX",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Under-Voltage Lockout Threshold",
          "aliases": ["UVLO"]
        },
        "possible_units": ["V"],
        "std_unit": "V",
        "scenarios": [
          {
            "condition": "Rising",
            "limits": {
              "LDO_Low_Voltage_CMOS": [1.5, 1.8, 2.0],
              "LDO_High_Voltage_Bipolar": [2.5, 3.0, 4.0]
            }
          }
        ]
      },
      {
        "key": "uvlo_hysteresis",
        "symbol": "ΔV<sub>UVLO</sub>",
        "spec_type": "nominal",
        "column_model": "TYP_ONLY",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "UVLO Hysteresis",
          "aliases": ["Under-Voltage Lockout Hysteresis"]
        },
        "possible_units": ["mV"],
        "std_unit": "mV",
        "scenarios": [
          {
            "condition": "Typ",
            "limits": {
              "LDO_Low_Voltage_CMOS": [50, 100, 200],
              "LDO_High_Voltage_Bipolar": [100, 200, 500]
            }
          }
        ]
      },
      {
        "key": "soft_start_time",
        "symbol": "t<sub>SS</sub>",
        "spec_type": "nominal",
        "column_model": "MIN_TYP_MAX",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Soft-Start Time",
          "aliases": ["Soft Start"]
        },
        "possible_units": ["ms", "µs"],
        "std_unit": "ms",
        "scenarios": [
          {
            "condition": "0V to 90% Vout",
            "limits": {
              "LDO_Low_Voltage_CMOS": [0.5, 1, 5],
              "LDO_High_Voltage_Bipolar": [1, 5, 20]
            }
          }
        ]
      },
      {
        "key": "current_limit",
        "symbol": "I<sub>LIM</sub>",
        "spec_type": "nominal",
        "column_model": "MIN_TYP_MAX",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Current Limit",
          "aliases": ["Current Limit Threshold"]
        },
        "possible_units": ["mA", "A"],
        "std_unit": "mA",
        "scenarios": [
          {
            "condition": "Typ",
            "limits": {
              "LDO_Low_Voltage_CMOS": [200, 400, 600],
              "LDO_High_Voltage_Bipolar": [600, 1200, 6000]
            }
          }
        ]
      },
      {
        "key": "short_circuit_current",
        "symbol": "I<sub>SC</sub>",
        "spec_type": "nominal",
        "column_model": "MIN_TYP_MAX",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Short Circuit Current",
          "aliases": ["ISC"]
        },
        "possible_units": ["mA", "A"],
        "std_unit": "mA",
        "scenarios": [
          {
            "condition": "Vout = 0V",
            "limits": {
              "LDO_Low_Voltage_CMOS": [200, 400, 600],
              "LDO_High_Voltage_Bipolar": [600, 1200, 3000]
            }
          }
        ]
      },
      {
        "key": "current_limit_foldback",
        "symbol": "I<sub>FOLD</sub>",
        "spec_type": "nominal",
        "column_model": "MIN_TYP_MAX",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Foldback Current Limit",
          "aliases": ["Foldback Current"]
        },
        "possible_units": ["mA", "A"],
        "std_unit": "mA",
        "scenarios": [
          {
            "condition": "Vout < 50% nominal",
            "limits": {
              "LDO_Low_Voltage_CMOS": [50, 100, 150],
              "LDO_High_Voltage_Bipolar": [100, 200, 500]
            }
          }
        ]
      },
      {
        "key": "current_limit_response_time",
        "symbol": "t<sub>CL</sub>",
        "spec_type": "max_limit",
        "column_model": "TYP_MAX",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Current Limit Response Time",
          "aliases": ["Overcurrent Response"]
        },
        "possible_units": ["µs", "ns"],
        "std_unit": "µs",
        "scenarios": [
          {
            "condition": "From overload to limit",
            "limits": {
              "LDO_Low_Voltage_CMOS": [1, 5, 10],
              "LDO_High_Voltage_Bipolar": [5, 10, 50]
            }
          }
        ]
      },
      {
        "key": "thermal_shutdown",
        "symbol": "T<sub>SD</sub>",
        "spec_type": "nominal",
        "column_model": "MIN_TYP_MAX",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Thermal Shutdown Temperature",
          "aliases": ["Over-Temperature Shutdown"]
        },
        "possible_units": ["°C"],
        "std_unit": "°C",
        "scenarios": [
          {
            "condition": "Typ",
            "limits": {
              "LDO_Low_Voltage_CMOS": [150, 160, 165],
              "LDO_High_Voltage_Bipolar": [150, 160, 170]
            }
          }
        ]
      },
      {
        "key": "thermal_shutdown_hysteresis",
        "symbol": "ΔT<sub>SD</sub>",
        "spec_type": "nominal",
        "column_model": "TYP_ONLY",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Thermal Shutdown Hysteresis",
          "aliases": ["TSD Hysteresis"]
        },
        "possible_units": ["°C"],
        "std_unit": "°C",
        "scenarios": [
          {
            "condition": "Typ",
            "limits": {
              "LDO_Low_Voltage_CMOS": [10, 15, 20],
              "LDO_High_Voltage_Bipolar": [15, 20, 30]
            }
          }
        ]
      },
      {
        "key": "startup_time",
        "symbol": "t<sub>START</sub>",
        "spec_type": "max_limit",
        "column_model": "TYP_MAX",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Startup Time",
          "aliases": ["Turn-on Time"]
        },
        "possible_units": ["µs", "ms"],
        "std_unit": "µs",
        "scenarios": [
          {
            "condition": "Enable high",
            "limits": {
              "LDO_Low_Voltage_CMOS": [50, 100, 200],
              "LDO_High_Voltage_Bipolar": [100, 500, 1000]
            }
          }
        ]
      },
      {
        "key": "settling_time",
        "symbol": "t<sub>SETTLING</sub>",
        "spec_type": "max_limit",
        "column_model": "TYP_MAX",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Output Settling Time",
          "aliases": ["Regulation Time"]
        },
        "possible_units": ["µs", "ms"],
        "std_unit": "µs",
        "scenarios": [
          {
            "condition": "To 1% of Vout",
            "limits": {
              "LDO_Low_Voltage_CMOS": [10, 50, 100],
              "LDO_High_Voltage_Bipolar": [50, 100, 500]
            }
          }
        ]
      },
      {
        "key": "load_transient_response",
        "symbol": "ΔV<sub>LOAD</sub>",
        "spec_type": "max_limit",
        "column_model": "TYP_MAX",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Load Transient Response",
          "aliases": ["Load Step Response", "Transient Response"]
        },
        "possible_units": ["mV", "%"],
        "std_unit": "mV",
        "scenarios": [
          {
            "condition": "Iout step 1mA to Imax",
            "limits": {
              "LDO_Low_Voltage_CMOS": [20, 50, 100],
              "LDO_High_Voltage_Bipolar": [50, 100, 200]
            }
          }
        ]
      },
      {
        "key": "load_transient_recovery_time",
        "symbol": "t<sub>REC</sub>",
        "spec_type": "max_limit",
        "column_model": "TYP_MAX",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Load Transient Recovery Time",
          "aliases": ["Recovery Time"]
        },
        "possible_units": ["µs", "ms"],
        "std_unit": "µs",
        "scenarios": [
          {
            "condition": "Iout step 10% to 90%",
            "limits": {
              "LDO_Low_Voltage_CMOS": [10, 50, 100],
              "LDO_High_Voltage_Bipolar": [50, 100, 500]
            }
          }
        ]
      },
      {
        "key": "output_voltage_overshoot",
        "symbol": "V<sub>OS</sub>",
        "spec_type": "max_limit",
        "column_model": "TYP_MAX",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Output Voltage Overshoot",
          "aliases": ["Startup Overshoot"]
        },
        "possible_units": ["%", "mV"],
        "std_unit": "%",
        "scenarios": [
          {
            "condition": "During startup",
            "limits": {
              "LDO_Low_Voltage_CMOS": [1, 2, 5],
              "LDO_High_Voltage_Bipolar": [2, 5, 10]
            }
          }
        ]
      },
      {
        "key": "output_voltage_undershoot",
        "symbol": "V<sub>US</sub>",
        "spec_type": "max_limit",
        "column_model": "TYP_MAX",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Output Voltage Undershoot",
          "aliases": ["Load Transient Undershoot"]
        },
        "possible_units": ["%", "mV"],
        "std_unit": "%",
        "scenarios": [
          {
            "condition": "Load step",
            "limits": {
              "LDO_Low_Voltage_CMOS": [1, 2, 5],
              "LDO_High_Voltage_Bipolar": [2, 5, 10]
            }
          }
        ]
      },
      {
        "key": "slew_rate",
        "symbol": "SR",
        "spec_type": "nominal",
        "column_model": "TYP_ONLY",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Output Voltage Slew Rate",
          "aliases": ["Slew Rate"]
        },
        "possible_units": ["V/µs", "mV/µs"],
        "std_unit": "mV/µs",
        "scenarios": [
          {
            "condition": "During startup",
            "limits": {
              "LDO_Low_Voltage_CMOS": [0.1, 1, 10],
              "LDO_High_Voltage_Bipolar": [1, 10, 100]
            }
          }
        ]
      },
      {
        "key": "monotonic_startup",
        "symbol": "MONO",
        "spec_type": "nominal",
        "column_model": "TYP_ONLY",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Monotonic Startup",
          "aliases": ["Monotonic Turn-on"]
        },
        "possible_units": [""],
        "std_unit": "",
        "scenarios": [
          {
            "condition": "Guaranteed",
            "limits": {
              "LDO_Low_Voltage_CMOS": ["Yes"],
              "LDO_High_Voltage_Bipolar": ["Yes"]
            }
          }
        ]
      },
      {
        "key": "pgood_threshold_high",
        "symbol": "V<sub>PG(H)</sub>",
        "spec_type": "nominal",
        "column_model": "MIN_TYP_MAX",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Power Good Threshold High",
          "aliases": ["PG High Threshold"]
        },
        "possible_units": ["%"],
        "std_unit": "%",
        "scenarios": [
          {
            "condition": "% of Vout nominal",
            "limits": {
              "LDO_Low_Voltage_CMOS": [90, 92, 95],
              "LDO_High_Voltage_Bipolar": [90, 92, 95]
            }
          }
        ]
      },
      {
        "key": "pgood_output_low",
        "symbol": "V<sub>PG(OL)</sub>",
        "spec_type": "max_limit",
        "column_model": "MAX_ONLY",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Power Good Output Low Voltage",
          "aliases": ["PG Output Low"]
        },
        "possible_units": ["V"],
        "std_unit": "V",
        "scenarios": [
          {
            "condition": "Isink = 1mA",
            "limits": {
              "LDO_Low_Voltage_CMOS": [0.3, 0.4],
              "LDO_High_Voltage_Bipolar": [0.4, 0.5]
            }
          }
        ]
      },
      {
        "key": "pgood_leakage",
        "symbol": "I<sub>PG(LEAK)</sub>",
        "spec_type": "max_limit",
        "column_model": "TYP_MAX",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Power Good Leakage Current",
          "aliases": ["PG Leakage"]
        },
        "possible_units": ["nA", "µA"],
        "std_unit": "nA",
        "scenarios": [
          {
            "condition": "PG high",
            "limits": {
              "LDO_Low_Voltage_CMOS": [1, 10, 100],
              "LDO_High_Voltage_Bipolar": [10, 100, 1000]
            }
          }
        ]
      },
      {
        "key": "pgood_delay",
        "symbol": "t<sub>PG</sub>",
        "spec_type": "nominal",
        "column_model": "TYP_MAX",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Power Good Delay",
          "aliases": ["PG Delay"]
        },
        "possible_units": ["µs", "ms"],
        "std_unit": "µs",
        "scenarios": [
          {
            "condition": "Vout crosses threshold",
            "limits": {
              "LDO_Low_Voltage_CMOS": [10, 50, 100],
              "LDO_High_Voltage_Bipolar": [50, 100, 500]
            }
          }
        ]
      },
      {
        "key": "open_loop_gain",
        "symbol": "A<sub>OL</sub>",
        "spec_type": "min_limit",
        "column_model": "MIN_TYP",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Open Loop Gain",
          "aliases": ["DC Gain"]
        },
        "possible_units": ["dB"],
        "std_unit": "dB",
        "scenarios": [
          {
            "condition": "DC",
            "limits": {
              "LDO_Low_Voltage_CMOS": [60, 70, 80],
              "LDO_High_Voltage_Bipolar": [50, 60, 70]
            }
          }
        ]
      },
      {
        "key": "pass_element_resistance",
        "symbol": "R<sub>DS(ON)</sub>",
        "spec_type": "max_limit",
        "column_model": "TYP_MAX",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Pass Element On-Resistance",
          "aliases": ["RDS(ON)", "On-Resistance"]
        },
        "possible_units": ["mΩ", "Ω"],
        "std_unit": "mΩ",
        "scenarios": [
          {
            "condition": "At Iout Max",
            "limits": {
              "LDO_Low_Voltage_CMOS": [100, 200, 1000],
              "LDO_High_Voltage_Bipolar": [200, 500, 2000]
            }
          }
        ]
      },
      {
        "key": "output_discharge_resistance",
        "symbol": "R<sub>DISCH</sub>",
        "spec_type": "nominal",
        "column_model": "TYP_ONLY",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Output Discharge Resistance",
          "aliases": ["Discharge Resistor"]
        },
        "possible_units": ["Ω", "kΩ"],
        "std_unit": "Ω",
        "scenarios": [
          {
            "condition": "When disabled",
            "limits": {
              "LDO_Low_Voltage_CMOS": [100, 200, 500],
              "LDO_High_Voltage_Bipolar": [500, 1000, 5000]
            }
          }
        ]
      },
      {
        "key": "power_on_reset_threshold",
        "symbol": "V<sub>POR</sub>",
        "spec_type": "nominal",
        "column_model": "MIN_TYP_MAX",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Power-On Reset Threshold",
          "aliases": ["POR Threshold"]
        },
        "possible_units": ["V"],
        "std_unit": "V",
        "scenarios": [
          {
            "condition": "Vin rising",
            "limits": {
              "LDO_Low_Voltage_CMOS": [1.0, 1.5, 2.0],
              "LDO_High_Voltage_Bipolar": [2.0, 2.5, 3.0]
            }
          }
        ]
      },
      {
        "key": "tracking_accuracy",
        "symbol": "ΔV<sub>TRACK</sub>",
        "spec_type": "max_limit",
        "column_model": "TYP_MAX",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Tracking Accuracy",
          "aliases": ["Voltage Tracking Error"]
        },
        "possible_units": ["mV", "%"],
        "std_unit": "mV",
        "scenarios": [
          {
            "condition": "During tracking",
            "limits": {
              "LDO_Low_Voltage_CMOS": [5, 10, 20],
              "LDO_High_Voltage_Bipolar": [10, 20, 50]
            }
          }
        ]
      },
      {
        "key": "reference_voltage",
        "symbol": "V<sub>REF</sub>",
        "spec_type": "nominal",
        "column_model": "MIN_TYP_MAX",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Internal Reference Voltage",
          "aliases": ["Bandgap Reference"]
        },
        "possible_units": ["V", "mV"],
        "std_unit": "V",
        "scenarios": [
          {
            "condition": "25°C",
            "limits": {
              "LDO_Low_Voltage_CMOS": [0.6, 0.8, 1.2],
              "LDO_High_Voltage_Bipolar": [1.2, 1.25, 2.5]
            }
          }
        ]
      },
      {
        "key": "reference_tempco",
        "symbol": "TC<sub>VREF</sub>",
        "spec_type": "max_limit",
        "column_model": "TYP_MAX",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Reference Voltage Temperature Coefficient",
          "aliases": ["Vref Tempco"]
        },
        "possible_units": ["ppm/°C"],
        "std_unit": "ppm/°C",
        "scenarios": [
          {
            "condition": "Over temp",
            "limits": {
              "LDO_Low_Voltage_CMOS": [20, 50, 100],
              "LDO_High_Voltage_Bipolar": [50, 100, 200]
            }
          }
        ]
      },
      {
        "key": "feedback_pin_voltage",
        "symbol": "V<sub>FB</sub>",
        "spec_type": "nominal",
        "column_model": "MIN_TYP_MAX",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Feedback Pin Voltage",
          "aliases": ["FB Pin Voltage"]
        },
        "possible_units": ["V"],
        "std_unit": "V",
        "scenarios": [
          {
            "condition": "Regulation",
            "limits": {
              "LDO_Low_Voltage_CMOS": [0.6, 0.8, 1.2],
              "LDO_High_Voltage_Bipolar": [1.2, 1.25, 2.5]
            }
          }
        ]
      },
      {
        "key": "feedback_pin_current",
        "symbol": "I<sub>FB</sub>",
        "spec_type": "max_limit",
        "column_model": "TYP_MAX",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Feedback Pin Bias Current",
          "aliases": ["FB Pin Current"]
        },
        "possible_units": ["nA", "µA"],
        "std_unit": "nA",
        "scenarios": [
          {
            "condition": "Typ",
            "limits": {
              "LDO_Low_Voltage_CMOS": [1, 10, 50],
              "LDO_High_Voltage_Bipolar": [10, 50, 100]
            }
          }
        ]
      },
      {
        "key": "adj_pin_voltage",
        "symbol": "V<sub>ADJ</sub>",
        "spec_type": "nominal",
        "column_model": "MIN_TYP_MAX",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Adjust Pin Voltage",
          "aliases": ["ADJ Pin Voltage"]
        },
        "possible_units": ["V"],
        "std_unit": "V",
        "scenarios": [
          {
            "condition": "Regulation",
            "limits": {
              "LDO_Low_Voltage_CMOS": [0.6, 0.8, 1.2],
              "LDO_High_Voltage_Bipolar": [1.2, 1.25, 2.5]
            }
          }
        ]
      },
      {
        "key": "adj_pin_current",
        "symbol": "I<sub>ADJ</sub>",
        "spec_type": "max_limit",
        "column_model": "TYP_MAX",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Adjust Pin Current",
          "aliases": ["ADJ Pin Current"]
        },
        "possible_units": ["nA", "µA"],
        "std_unit": "nA",
        "scenarios": [
          {
            "condition": "Typ",
            "limits": {
              "LDO_Low_Voltage_CMOS": [1, 10, 50],
              "LDO_High_Voltage_Bipolar": [10, 50, 200]
            }
          }
        ]
      },
      {
        "key": "output_capacitor_min",
        "symbol": "C<sub>OUT(min)</sub>",
        "spec_type": "min_limit",
        "column_model": "MIN_ONLY",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Minimum Output Capacitor",
          "aliases": ["Cout Min"]
        },
        "possible_units": ["µF"],
        "std_unit": "µF",
        "scenarios": [
          {
            "condition": "For stability",
            "limits": {
              "LDO_Low_Voltage_CMOS": [1, 2.2, 4.7],
              "LDO_High_Voltage_Bipolar": [4.7, 10, 22]
            }
          }
        ]
      },
      {
        "key": "output_capacitor_esr",
        "symbol": "ESR<sub>OUT</sub>",
        "spec_type": "nominal",
        "column_model": "MIN_MAX",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Output Capacitor ESR Range",
          "aliases": ["Cout ESR"]
        },
        "possible_units": ["mΩ", "Ω"],
        "std_unit": "mΩ",
        "scenarios": [
          {
            "condition": "For stability",
            "limits": {
              "LDO_Low_Voltage_CMOS": ["5 to 500"],
              "LDO_High_Voltage_Bipolar": ["10 to 5000"]
            }
          }
        ]
      },
      {
        "key": "input_capacitor_min",
        "symbol": "C<sub>IN(min)</sub>",
        "spec_type": "min_limit",
        "column_model": "MIN_ONLY",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Minimum Input Capacitor",
          "aliases": ["Cin Min"]
        },
        "possible_units": ["µF"],
        "std_unit": "µF",
        "scenarios": [
          {
            "condition": "Recommended",
            "limits": {
              "LDO_Low_Voltage_CMOS": [0.1, 1, 2.2],
              "LDO_High_Voltage_Bipolar": [1, 4.7, 10]
            }
          }
        ]
      }
    ]
  },

    # ==============================================================================
    # 6. INDUCTOR - CORRECTED WITH METADATA FIELDS חלק6
    # ==============================================================================
  "INDUCTOR": {
    "archetypes": ["Power_Ferrite", "RF_Ceramic", "Common_Mode_Choke"],
    
    "ABS_MAX": [
      {
        "key": "rated_current",
        "symbol": "I<sub>R</sub>",
        "spec_type": "max_rating",
        "column_model": "MAX_ONLY",
        "engineering_class": "SAFETY_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Rated Current", "aliases": ["Rated Current", "Max DC Current"]},
        "possible_units": ["mA", "A"],
        "std_unit": "mA",
        "scenarios": [
          {
            "condition": "Temp Rise 40°C",
            "limits": {
              "Power_Ferrite": [1000, 2000, 5000, 10000, 20000],
              "RF_Ceramic": [100, 200, 500],
              "Common_Mode_Choke": [500, 1000, 5000]
            }
          },
          {
            "condition": "Temp Rise 20°C",
            "limits": {
              "Power_Ferrite": [800, 1500, 4000, 8000, 15000],
              "RF_Ceramic": [80, 150, 400],
              "Common_Mode_Choke": [400, 800, 4000]
            }
          }
        ]
      },
      {
        "key": "saturation_current",
        "symbol": "I<sub>sat</sub>",
        "spec_type": "max_rating",
        "column_model": "MAX_ONLY",
        "engineering_class": "SAFETY_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Saturation Current", "aliases": ["Saturation Current", "Isat"]},
        "possible_units": ["mA", "A"],
        "std_unit": "mA",
        "scenarios": [
          {
            "condition": "30% L drop",
            "limits": {
              "Power_Ferrite": [1200, 2500, 6000, 12000, 25000],
              "RF_Ceramic": [150, 300, 700],
              "Common_Mode_Choke": [600, 1200, 6000]
            }
          },
          {
            "condition": "10% L drop",
            "limits": {
              "Power_Ferrite": [800, 1500, 4000, 8000, 18000],
              "RF_Ceramic": [100, 200, 500],
              "Common_Mode_Choke": [400, 800, 4000]
            }
          }
        ]
      },
      {
        "key": "max_operating_temp",
        "symbol": "T<sub>max</sub>",
        "spec_type": "max_rating",
        "column_model": "MAX_ONLY",
        "engineering_class": "SAFETY_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Maximum Operating Temperature", "aliases": ["Max Operating Temp"]},
        "possible_units": ["°C"],
        "std_unit": "°C",
        "scenarios": [
          {
            "condition": "Max",
            "limits": {
              "Power_Ferrite": [125, 155, 175],
              "RF_Ceramic": [125, 150],
              "Common_Mode_Choke": [125, 155]
            }
          }
        ]
      },
      {
        "key": "storage_temp",
        "symbol": "T<sub>STG</sub>",
        "spec_type": "max_rating",
        "column_model": "MIN_MAX",
        "engineering_class": "SAFETY_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Storage Temperature Range", "aliases": ["Storage Temperature", "Storage Temp"]},
        "possible_units": ["°C"],
        "std_unit": "°C",
        "scenarios": [
          {
            "condition": "Min to Max",
            "limits": {
              "Power_Ferrite": ["-55 to 155", "-55 to 175"],
              "RF_Ceramic": ["-55 to 125", "-55 to 150"],
              "Common_Mode_Choke": ["-55 to 155"]
            }
          }
        ]
      },
      {
        "key": "max_ac_voltage",
        "symbol": "V<sub>AC(max)</sub>",
        "spec_type": "max_rating",
        "column_model": "MAX_ONLY",
        "engineering_class": "SAFETY_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Maximum AC Voltage", "aliases": ["Max AC Voltage", "Rated Voltage"]},
        "possible_units": ["V", "Vrms"],
        "std_unit": "Vrms",
        "scenarios": [
          {
            "condition": "RMS",
            "limits": {
              "Power_Ferrite": [10, 25, 50, 100],
              "RF_Ceramic": [5, 10, 25],
              "Common_Mode_Choke": [50, 100, 250, 500]
            }
          }
        ]
      },
      {
        "key": "test_voltage",
        "symbol": "V<sub>TEST</sub>",
        "spec_type": "max_rating",
        "column_model": "TYP_ONLY",
        "engineering_class": "SAFETY_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Test Voltage (Hi-Pot)", "aliases": ["Dielectric Test Voltage", "Hi-Pot Voltage"]},
        "possible_units": ["V", "Vrms"],
        "std_unit": "Vrms",
        "scenarios": [
          {
            "condition": "1 second",
            "limits": {
              "Power_Ferrite": [100, 250, 500],
              "RF_Ceramic": [100],
              "Common_Mode_Choke": [500, 1000, 2500, 4000]
            }
          }
        ]
      },
      {
        "key": "insulation_resistance",
        "symbol": "R<sub>INS</sub>",
        "spec_type": "min_limit",
        "column_model": "MIN_ONLY",
        "engineering_class": "SAFETY_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Insulation Resistance", "aliases": ["Isolation Resistance", "Rins"]},
        "possible_units": ["MΩ", "GΩ"],
        "std_unit": "MΩ",
        "scenarios": [
          {
            "condition": "At 500VDC",
            "limits": {
              "Power_Ferrite": [10, 100],
              "RF_Ceramic": [100],
              "Common_Mode_Choke": [100, 1000]
            }
          }
        ]
      }
    ],
    
    "ELEC_CHAR": [
      {
        "key": "inductance",
        "symbol": "L",
        "spec_type": "nominal",
        "column_model": "MIN_TYP_MAX",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Inductance", "aliases": ["Inductance", "Inductance Value"]},
        "possible_units": ["nH", "µH", "mH"],
        "std_unit": "µH",
        "scenarios": [
          {
            "condition": "Test Freq Defined",
            "limits": {
              "Power_Ferrite": [0.1, 0.22, 0.47, 1.0, 2.2, 4.7, 10, 22, 47, 100, 220, 470, 1000],
              "RF_Ceramic": [0.001, 0.0022, 0.0047, 0.01, 0.022, 0.047, 0.1, 0.22, 0.47, 1.0],
              "Common_Mode_Choke": [10, 47, 100, 220, 470, 1000, 2200, 4700, 10000]
            }
          }
        ]
      },
      {
        "key": "inductance_frequency",
        "symbol": "f<sub>TEST</sub>",
        "spec_type": "nominal",
        "column_model": "TYP_ONLY",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Inductance Test Frequency", "aliases": ["Test Frequency", "Measurement Frequency"]},
        "possible_units": ["kHz", "MHz"],
        "std_unit": "kHz",
        "scenarios": [
          {
            "condition": "Standard",
            "limits": {
              "Power_Ferrite": [100, 250, 1000],
              "RF_Ceramic": [1000, 10000, 100000],
              "Common_Mode_Choke": [100, 1000]
            }
          }
        ]
      },
      {
        "key": "tolerance",
        "symbol": "Tol",
        "spec_type": "nominal",
        "column_model": "TYP_ONLY",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Inductance Tolerance", "aliases": ["Tolerance", "Inductance Tolerance"]},
        "possible_units": ["%"],
        "std_unit": "%",
        "scenarios": [
          {
            "condition": "Standard",
            "limits": {
              "Power_Ferrite": [10, 20, 30],
              "RF_Ceramic": [0.5, 1, 2, 5, 10],
              "Common_Mode_Choke": [20, 30]
            }
          }
        ]
      },
      {
        "key": "dcr",
        "symbol": "DCR",
        "spec_type": "max_limit",
        "column_model": "TYP_MAX",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "DC Resistance", "aliases": ["DC Resistance", "Max DCR"]},
        "possible_units": ["mΩ", "Ω"],
        "std_unit": "mΩ",
        "scenarios": [
          {
            "condition": "Max at 25°C",
            "limits": {
              "Power_Ferrite": [2, 5, 10, 20, 50, 100, 200, 500],
              "RF_Ceramic": [50, 100, 200, 500, 1000, 2000, 5000],
              "Common_Mode_Choke": [10, 20, 50, 100, 200]
            }
          }
        ]
      },
      {
        "key": "dcr_tolerance",
        "symbol": "ΔR<sub>DC</sub>",
        "spec_type": "nominal",
        "column_model": "TYP_ONLY",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "DCR Tolerance", "aliases": ["Resistance Tolerance", "DCR Tol"]},
        "possible_units": ["%"],
        "std_unit": "%",
        "scenarios": [
          {
            "condition": "Typical",
            "limits": {
              "Power_Ferrite": [10, 15, 20],
              "RF_Ceramic": [15, 20],
              "Common_Mode_Choke": [10, 20]
            }
          }
        ]
      },
      {
        "key": "srf",
        "symbol": "f<sub>res</sub>",
        "spec_type": "min_limit",
        "column_model": "MIN_TYP_MAX",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Self Resonant Frequency", "aliases": ["Self Resonant Frequency", "SRF"]},
        "possible_units": ["MHz", "GHz"],
        "std_unit": "MHz",
        "scenarios": [
          {
            "condition": "Typ",
            "limits": {
              "Power_Ferrite": [2, 5, 10, 20, 50, 100],
              "RF_Ceramic": [100, 500, 1000, 2000, 5000, 10000],
              "Common_Mode_Choke": [5, 10, 50, 100, 200]
            }
          }
        ]
      },
      {
        "key": "quality_factor",
        "symbol": "Q",
        "spec_type": "min_limit",
        "column_model": "MIN_TYP_MAX",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Quality Factor", "aliases": ["Quality Factor", "Q Factor"]},
        "possible_units": [""],
        "std_unit": "",
        "scenarios": [
          {
            "condition": "At test freq",
            "limits": {
              "Power_Ferrite": [15, 20, 30, 40, 50],
              "RF_Ceramic": [30, 40, 60, 80, 100, 120],
              "Common_Mode_Choke": [5, 10, 20]
            }
          }
        ]
      },
      {
        "key": "impedance",
        "symbol": "Z",
        "spec_type": "nominal",
        "column_model": "MIN_TYP_MAX",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Impedance", "aliases": ["Impedance at Frequency", "Z"]},
        "possible_units": ["Ω", "kΩ"],
        "std_unit": "Ω",
        "scenarios": [
          {
            "condition": "At 100MHz",
            "limits": {
              "Power_Ferrite": [50, 100, 220, 470, 1000],
              "RF_Ceramic": [10, 22, 47, 100, 220],
              "Common_Mode_Choke": [100, 220, 470, 1000, 2200]
            }
          }
        ]
      },
      {
        "key": "parasitic_capacitance",
        "symbol": "C<sub>P</sub>",
        "spec_type": "nominal",
        "column_model": "TYP_MAX",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Parasitic Capacitance", "aliases": ["Stray Capacitance", "Self Capacitance"]},
        "possible_units": ["pF", "nF"],
        "std_unit": "pF",
        "scenarios": [
          {
            "condition": "Typical",
            "limits": {
              "Power_Ferrite": [5, 10, 20, 50],
              "RF_Ceramic": [0.1, 0.5, 1, 2],
              "Common_Mode_Choke": [10, 20, 50, 100]
            }
          }
        ]
      },
      {
        "key": "leakage_inductance",
        "symbol": "L<sub>leak</sub>",
        "spec_type": "max_limit",
        "column_model": "TYP_MAX",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Leakage Inductance", "aliases": ["Stray Inductance", "Lleak"]},
        "possible_units": ["nH", "µH"],
        "std_unit": "nH",
        "scenarios": [
          {
            "condition": "Typical",
            "limits": {
              "Power_Ferrite": [10, 50, 100],
              "RF_Ceramic": [1, 5, 10],
              "Common_Mode_Choke": [5, 10, 50]
            }
          }
        ]
      },
      {
        "key": "differential_mode_inductance",
        "symbol": "L<sub>DM</sub>",
        "spec_type": "nominal",
        "column_model": "MIN_TYP_MAX",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Differential Mode Inductance", "aliases": ["DM Inductance", "Ldm"]},
        "possible_units": ["µH", "nH"],
        "std_unit": "µH",
        "scenarios": [
          {
            "condition": "At test frequency",
            "limits": {
              "Power_Ferrite": [0.1, 0.5, 1],
              "RF_Ceramic": [0.01, 0.05, 0.1],
              "Common_Mode_Choke": [0.5, 1, 5, 10]
            }
          }
        ]
      },
      {
        "key": "common_mode_inductance",
        "symbol": "L<sub>CM</sub>",
        "spec_type": "nominal",
        "column_model": "MIN_TYP_MAX",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Common Mode Inductance", "aliases": ["CM Inductance", "Lcm"]},
        "possible_units": ["µH", "mH"],
        "std_unit": "µH",
        "scenarios": [
          {
            "condition": "At test frequency",
            "limits": {
              "Power_Ferrite": [10, 47, 100],
              "RF_Ceramic": [1, 10, 47],
              "Common_Mode_Choke": [100, 470, 1000, 4700, 10000]
            }
          }
        ]
      },
      {
        "key": "insertion_loss",
        "symbol": "IL",
        "spec_type": "min_limit",
        "column_model": "MIN_TYP",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Insertion Loss", "aliases": ["Attenuation", "IL"]},
        "possible_units": ["dB"],
        "std_unit": "dB",
        "scenarios": [
          {
            "condition": "At 100MHz",
            "limits": {
              "Power_Ferrite": [10, 20, 30],
              "RF_Ceramic": [5, 10, 20],
              "Common_Mode_Choke": [20, 30, 40, 60]
            }
          }
        ]
      },
      {
        "key": "inductance_vs_current",
        "symbol": "ΔL/I",
        "spec_type": "max_limit",
        "column_model": "TYP_MAX",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Inductance Change vs DC Current", "aliases": ["L vs I", "Inductance Droop"]},
        "possible_units": ["%"],
        "std_unit": "%",
        "scenarios": [
          {
            "condition": "At rated current",
            "limits": {
              "Power_Ferrite": [100, 200, 500, 1000, 2000],
              "RF_Ceramic": [10, 50, 100],
              "Common_Mode_Choke": [100, 200, 500]
            }
          }
        ]
      }
    ],
    
    "MAGNETIC_PROPERTIES": [
      {
        "key": "permeability",
        "symbol": "µ<sub>r</sub>",
        "spec_type": "nominal",
        "column_model": "TYP_ONLY",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Relative Permeability", "aliases": ["Permeability", "µr"]},
        "possible_units": [""],
        "std_unit": "",
        "scenarios": [
          {
            "condition": "Initial",
            "limits": {
              "Power_Ferrite": [20, 40, 60, 90, 125],
              "RF_Ceramic": [5, 10, 20],
              "Common_Mode_Choke": [2000, 5000, 10000]
            }
          }
        ]
      },
      {
        "key": "saturation_flux_density",
        "symbol": "B<sub>sat</sub>",
        "spec_type": "nominal",
        "column_model": "TYP_ONLY",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Saturation Flux Density", "aliases": ["Bsat", "Saturation B"]},
        "possible_units": ["mT", "T", "G"],
        "std_unit": "mT",
        "scenarios": [
          {
            "condition": "At 25°C",
            "limits": {
              "Power_Ferrite": [300, 400, 500],
              "RF_Ceramic": [200, 300],
              "Common_Mode_Choke": [400, 500]
            }
          }
        ]
      },
      {
        "key": "al_value",
        "symbol": "A<sub>L</sub>",
        "spec_type": "nominal",
        "column_model": "MIN_TYP_MAX",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Inductance Factor", "aliases": ["AL Value", "Inductance per Turn"]},
        "possible_units": ["nH/turn²", "µH/turn²"],
        "std_unit": "nH/turn²",
        "scenarios": [
          {
            "condition": "Per turn squared",
            "limits": {
              "Power_Ferrite": [10, 20, 50, 100, 200, 500],
              "RF_Ceramic": [1, 5, 10, 20],
              "Common_Mode_Choke": [50, 100, 200, 500]
            }
          }
        ]
      },
      {
        "key": "core_loss",
        "symbol": "P<sub>core</sub>",
        "spec_type": "max_limit",
        "column_model": "TYP_MAX",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Core Loss", "aliases": ["Core Power Loss", "AC Loss"]},
        "possible_units": ["mW", "W"],
        "std_unit": "mW",
        "scenarios": [
          {
            "condition": "At rated current, 100kHz",
            "limits": {
              "Power_Ferrite": [10, 50, 100, 200, 500],
              "RF_Ceramic": [1, 5, 10],
              "Common_Mode_Choke": [20, 100, 200]
            }
          }
        ]
      },
      {
        "key": "core_loss_density",
        "symbol": "P<sub>cv</sub>",
        "spec_type": "max_limit",
        "column_model": "TYP_MAX",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Core Loss Density", "aliases": ["Core Loss per Volume", "Pcv"]},
        "possible_units": ["mW/cm³", "kW/m³"],
        "std_unit": "mW/cm³",
        "scenarios": [
          {
            "condition": "At 100kHz, 100mT",
            "limits": {
              "Power_Ferrite": [100, 200, 500],
              "RF_Ceramic": [50, 100],
              "Common_Mode_Choke": [150, 300]
            }
          }
        ]
      },
      {
        "key": "curie_temperature",
        "symbol": "T<sub>C</sub>",
        "spec_type": "nominal",
        "column_model": "TYP_ONLY",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Curie Temperature", "aliases": ["Curie Point", "Tc"]},
        "possible_units": ["°C"],
        "std_unit": "°C",
        "scenarios": [
          {
            "condition": "Core material",
            "limits": {
              "Power_Ferrite": [130, 200, 240],
              "RF_Ceramic": [200, 300],
              "Common_Mode_Choke": [130, 200]
            }
          }
        ]
      }
    ],
    
    "MECHANICAL_CHAR": [
      {
        "key": "core_volume",
        "symbol": "V<sub>e</sub>",
        "spec_type": "nominal",
        "column_model": "TYP_ONLY",
        "engineering_class": "MECHANICAL",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Effective Core Volume", "aliases": ["Core Volume", "Ve"]},
        "possible_units": ["mm³", "cm³"],
        "std_unit": "mm³",
        "scenarios": [
          {
            "condition": "Typical",
            "limits": {
              "Power_Ferrite": [5, 10, 50, 100, 500, 1000],
              "RF_Ceramic": [0.5, 1, 5, 10],
              "Common_Mode_Choke": [50, 100, 500]
            }
          }
        ]
      },
      {
        "key": "magnetic_path_length",
        "symbol": "l<sub>e</sub>",
        "spec_type": "nominal",
        "column_model": "TYP_ONLY",
        "engineering_class": "MECHANICAL",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Effective Magnetic Path Length", "aliases": ["Path Length", "le"]},
        "possible_units": ["mm", "cm"],
        "std_unit": "mm",
        "scenarios": [
          {
            "condition": "Typical",
            "limits": {
              "Power_Ferrite": [5, 10, 20, 30, 50],
              "RF_Ceramic": [1, 2, 5],
              "Common_Mode_Choke": [10, 20, 40]
            }
          }
        ]
      },
      {
        "key": "core_area",
        "symbol": "A<sub>e</sub>",
        "spec_type": "nominal",
        "column_model": "TYP_ONLY",
        "engineering_class": "MECHANICAL",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Effective Core Area", "aliases": ["Core Area", "Ae"]},
        "possible_units": ["mm²", "cm²"],
        "std_unit": "mm²",
        "scenarios": [
          {
            "condition": "Typical",
            "limits": {
              "Power_Ferrite": [1, 2, 5, 10, 20, 50],
              "RF_Ceramic": [0.1, 0.5, 1],
              "Common_Mode_Choke": [5, 10, 20]
            }
          }
        ]
      },
      {
        "key": "wire_gauge",
        "symbol": "AWG",
        "spec_type": "nominal",
        "column_model": "TYP_ONLY",
        "engineering_class": "MECHANICAL",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Wire Gauge", "aliases": ["Wire Size", "AWG"]},
        "possible_units": [""],
        "std_unit": "",
        "scenarios": [
          {
            "condition": "Typical",
            "limits": {
              "Power_Ferrite": [18, 20, 22, 24, 26, 28, 30],
              "RF_Ceramic": [26, 28, 30, 32, 34],
              "Common_Mode_Choke": [20, 22, 24, 26, 28]
            }
          }
        ]
      },
      {
        "key": "turns_count",
        "symbol": "N",
        "spec_type": "nominal",
        "column_model": "TYP_ONLY",
        "engineering_class": "MECHANICAL",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Number of Turns", "aliases": ["Turns", "Winding Turns"]},
        "possible_units": [""],
        "std_unit": "",
        "scenarios": [
          {
            "condition": "Typical",
            "limits": {
              "Power_Ferrite": [5, 10, 15, 20, 30, 50],
              "RF_Ceramic": [2, 3, 5, 10],
              "Common_Mode_Choke": [10, 20, 50, 100]
            }
          }
        ]
      },
      {
        "key": "core_material",
        "symbol": "Material",
        "spec_type": "nominal",
        "column_model": "TYP_ONLY",
        "engineering_class": "MECHANICAL",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Core Material Type", "aliases": ["Core Material"]},
        "possible_units": [""],
        "std_unit": "",
        "scenarios": [
          {
            "condition": "N/A",
            "limits": {
              "Power_Ferrite": ["NiZn", "MnZn", "Iron_Powder", "Sendust", "Molypermalloy"],
              "RF_Ceramic": ["Ceramic", "Air_Core", "Ferrite"],
              "Common_Mode_Choke": ["NiZn", "MnZn", "Nanocrystalline"]
            }
          }
        ]
      },
      {
        "key": "shielding",
        "symbol": "Shielding",
        "spec_type": "nominal",
        "column_model": "TYP_ONLY",
        "engineering_class": "MECHANICAL",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Magnetic Shielding", "aliases": ["Shielding Type"]},
        "possible_units": [""],
        "std_unit": "",
        "scenarios": [
          {
            "condition": "N/A",
            "limits": {
              "Power_Ferrite": ["Shielded", "Semi-Shielded", "Unshielded"],
              "RF_Ceramic": ["Unshielded"],
              "Common_Mode_Choke": ["Shielded", "Unshielded"]
            }
          }
        ]
      },
      {
        "key": "mounting_type",
        "symbol": "Mount",
        "spec_type": "mechanical",
        "column_model": "TYP_ONLY",
        "engineering_class": "MECHANICAL",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Mounting Type", "aliases": ["Mount Type", "Mounting"]},
        "possible_units": [""],
        "std_unit": "",
        "scenarios": [
          {
            "condition": "N/A",
            "limits": {
              "Power_Ferrite": ["SMD", "Through-Hole", "Chassis"],
              "RF_Ceramic": ["SMD", "Chip"],
              "Common_Mode_Choke": ["SMD", "Through-Hole", "PCB"]
            }
          }
        ]
      },
      {
        "key": "footprint",
        "symbol": "Footprint",
        "spec_type": "mechanical",
        "column_model": "TYP_ONLY",
        "engineering_class": "MECHANICAL",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "PCB Footprint", "aliases": ["Footprint Size", "PCB Area"]},
        "possible_units": ["mm²"],
        "std_unit": "mm²",
        "scenarios": [
          {
            "condition": "Typical",
            "limits": {
              "Power_Ferrite": [4, 9, 16, 36, 64, 100],
              "RF_Ceramic": [0.5, 1, 2, 4],
              "Common_Mode_Choke": [9, 16, 36]
            }
          }
        ]
      },
      {
        "key": "height",
        "symbol": "H",
        "spec_type": "mechanical",
        "column_model": "TYP_ONLY",
        "engineering_class": "MECHANICAL",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Component Height", "aliases": ["Height", "Profile"]},
        "possible_units": ["mm"],
        "std_unit": "mm",
        "scenarios": [
          {
            "condition": "Max",
            "limits": {
              "Power_Ferrite": [0.8, 1.0, 1.5, 2.0, 3.0, 4.0, 5.0, 10.0],
              "RF_Ceramic": [0.3, 0.5, 0.6, 0.8, 1.0],
              "Common_Mode_Choke": [1.5, 2.0, 3.0, 5.0, 10.0]
            }
          }
        ]
      },
      {
        "key": "weight",
        "symbol": "W",
        "spec_type": "mechanical",
        "column_model": "TYP_ONLY",
        "engineering_class": "MECHANICAL",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Weight", "aliases": ["Mass", "Component Weight"]},
        "possible_units": ["mg", "g"],
        "std_unit": "mg",
        "scenarios": [
          {
            "condition": "Typical",
            "limits": {
              "Power_Ferrite": [10, 50, 100, 500, 1000, 5000],
              "RF_Ceramic": [1, 5, 10, 50],
              "Common_Mode_Choke": [100, 500, 1000]
            }
          }
        ]
      }
    ],
    
    "RELIABILITY": [
      {
        "key": "mtbf",
        "symbol": "MTBF",
        "spec_type": "min_limit",
        "column_model": "MIN_ONLY",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Mean Time Between Failures", "aliases": ["MTBF", "Reliability"]},
        "possible_units": ["hours"],
        "std_unit": "hours",
        "scenarios": [
          {
            "condition": "At rated conditions",
            "limits": {
              "Power_Ferrite": [100000, 500000, 1000000],
              "RF_Ceramic": [1000000],
              "Common_Mode_Choke": [500000, 1000000]
            }
          }
        ]
      },
      {
        "key": "humidity_resistance",
        "symbol": "RH",
        "spec_type": "max_rating",
        "column_model": "MAX_ONLY",
        "engineering_class": "SAFETY_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Humidity Resistance", "aliases": ["Max Humidity", "RH"]},
        "possible_units": ["%RH"],
        "std_unit": "%RH",
        "scenarios": [
          {
            "condition": "Operating",
            "limits": {
              "Power_Ferrite": [85, 95],
              "RF_Ceramic": [85, 95],
              "Common_Mode_Choke": [85, 95]
            }
          }
        ]
      },
      {
        "key": "vibration_resistance",
        "symbol": "Vib",
        "spec_type": "nominal",
        "column_model": "TYP_ONLY",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Vibration Resistance", "aliases": ["Vibration Spec", "Vib Rating"]},
        "possible_units": ["G"],
        "std_unit": "G",
        "scenarios": [
          {
            "condition": "10-2000Hz",
            "limits": {
              "Power_Ferrite": [10, 20, 50],
              "RF_Ceramic": [20, 50],
              "Common_Mode_Choke": [10, 20]
            }
          }
        ]
      },
      {
        "key": "shock_resistance",
        "symbol": "Shock",
        "spec_type": "nominal",
        "column_model": "TYP_ONLY",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Shock Resistance", "aliases": ["Shock Rating", "Mechanical Shock"]},
        "possible_units": ["G"],
        "std_unit": "G",
        "scenarios": [
          {
            "condition": "Half-sine 11ms",
            "limits": {
              "Power_Ferrite": [50, 100, 500],
              "RF_Ceramic": [100, 500, 1000],
              "Common_Mode_Choke": [50, 100]
            }
          }
        ]
      },
      {
        "key": "solderability",
        "symbol": "Solder",
        "spec_type": "nominal",
        "column_model": "TYP_ONLY",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Solderability", "aliases": ["Solder Compatibility", "Reflow Compatible"]},
        "possible_units": [""],
        "std_unit": "",
        "scenarios": [
          {
            "condition": "Reflow profile",
            "limits": {
              "Power_Ferrite": ["Lead-Free", "SnPb", "Both"],
              "RF_Ceramic": ["Lead-Free", "SnPb", "Both"],
              "Common_Mode_Choke": ["Lead-Free", "SnPb", "Both"]
            }
          }
        ]
      },
      {
        "key": "msl_rating",
        "symbol": "MSL",
        "spec_type": "nominal",
        "column_model": "TYP_ONLY",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Moisture Sensitivity Level", "aliases": ["MSL", "Moisture Rating"]},
        "possible_units": [""],
        "std_unit": "",
        "scenarios": [
          {
            "condition": "Per JEDEC",
            "limits": {
              "Power_Ferrite": ["MSL1", "MSL2", "MSL3"],
              "RF_Ceramic": ["MSL1"],
              "Common_Mode_Choke": ["MSL1", "MSL2"]
            }
          }
        ]
      }
    ],
    
    "PACKAGE": [
      {
        "key": "package_code",
        "symbol": "PKG",
        "spec_type": "mechanical",
        "column_model": "TYP_ONLY",
        "engineering_class": "MECHANICAL",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Size Code", "aliases": ["Package Size", "Case Code"]},
        "possible_units": [""],
        "std_unit": "",
        "scenarios": [
          {
            "condition": "N/A",
            "limits": {
              "Power_Ferrite": ["0201", "0402", "0603", "0805", "1008", "1206", "1210", "1812", "2520", "3225", "4040", "5050", "6060", "Radial", "Axial"],
              "RF_Ceramic": ["01005", "0201", "0402", "0603", "0805"],
              "Common_Mode_Choke": ["0805", "1206", "1210", "Radial", "Toroid"]
            }
          }
        ]
      },
      {
        "key": "termination",
        "symbol": "Term",
        "spec_type": "mechanical",
        "column_model": "TYP_ONLY",
        "engineering_class": "MECHANICAL",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Termination Type", "aliases": ["Terminal Finish", "Termination"]},
        "possible_units": [""],
        "std_unit": "",
        "scenarios": [
          {
            "condition": "N/A",
            "limits": {
              "Power_Ferrite": ["Sn", "Ag", "Ni/Au", "Bare_Cu"],
              "RF_Ceramic": ["Sn", "Ag", "Ni/Au"],
              "Common_Mode_Choke": ["Sn", "Ni/Au"]
            }
          }
        ]
      },
      {
        "key": "pin_count",
        "symbol": "Pins",
        "spec_type": "mechanical",
        "column_model": "TYP_ONLY",
        "engineering_class": "MECHANICAL",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Number of Pins/Terminals", "aliases": ["Pin Count", "Terminals"]},
        "possible_units": [""],
        "std_unit": "",
        "scenarios": [
          {
            "condition": "N/A",
            "limits": {
              "Power_Ferrite": [2],
              "RF_Ceramic": [2],
              "Common_Mode_Choke": [4, 6, 8]
            }
          }
        ]
      }
    ],
    
    "OPERATING_CONDITIONS": [
      {
        "key": "operating_temp_range",
        "symbol": "T<sub>A</sub>",
        "spec_type": "nominal",
        "column_model": "MIN_MAX",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Operating Temperature Range", "aliases": ["Ambient Temperature Range", "Operating Temp"]},
        "possible_units": ["°C"],
        "std_unit": "°C",
        "scenarios": [
          {
            "condition": "Standard",
            "limits": {
              "Power_Ferrite": ["-40 to 85", "-40 to 105", "-40 to 125", "-55 to 155"],
              "RF_Ceramic": ["-55 to 125", "-55 to 150"],
              "Common_Mode_Choke": ["-40 to 85", "-40 to 125"]
            }
          }
        ]
      },
      {
        "key": "ambient_pressure",
        "symbol": "P<sub>AMB</sub>",
        "spec_type": "nominal",
        "column_model": "MIN_MAX",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Ambient Pressure Range", "aliases": ["Operating Pressure", "Pressure Range"]},
        "possible_units": ["kPa", "mmHg"],
        "std_unit": "kPa",
        "scenarios": [
          {
            "condition": "Operating",
            "limits": {
              "Power_Ferrite": ["70 to 106"],
              "RF_Ceramic": ["70 to 106"],
              "Common_Mode_Choke": ["70 to 106"]
            }
          }
        ]
      }
    ],
    
    "AC_CHARACTERISTICS": [
      {
        "key": "frequency_range",
        "symbol": "f<sub>RANGE</sub>",
        "spec_type": "nominal",
        "column_model": "MIN_MAX",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Operating Frequency Range", "aliases": ["Frequency Range", "Working Frequency"]},
        "possible_units": ["kHz", "MHz", "GHz"],
        "std_unit": "kHz",
        "scenarios": [
          {
            "condition": "Typical",
            "limits": {
              "Power_Ferrite": ["1 to 1000", "10 to 2000"],
              "RF_Ceramic": ["100 to 10000", "1000 to 100000"],
              "Common_Mode_Choke": ["0.1 to 100", "1 to 1000"]
            }
          }
        ]
      },
      {
        "key": "ripple_current",
        "symbol": "ΔI<sub>L</sub>",
        "spec_type": "max_limit",
        "column_model": "MAX_ONLY",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Rated Ripple Current", "aliases": ["AC Ripple Current", "Delta I"]},
        "possible_units": ["mA", "A"],
        "std_unit": "mA",
        "scenarios": [
          {
            "condition": "At switching frequency",
            "limits": {
              "Power_Ferrite": [500, 1000, 2000, 5000],
              "RF_Ceramic": [50, 100, 200],
              "Common_Mode_Choke": [200, 500, 1000]
            }
          }
        ]
      },
      {
         "key": "coupling_coefficient",
         "symbol": "k",
         "spec_type": "min_limit",
         "column_model": "MIN_TYP",
         "engineering_class": "PERFORMANCE_LIMIT",
         "special_semantics": "NONE",
         "llm_context": {"formal_name": "Coupling Coefficient", "aliases": ["Magnetic Coupling", "k Factor"]},
         "possible_units": [""],
         "std_unit": "",
         "scenarios": [
           {
             "condition": "Between windings",
             "limits": {
               "Power_Ferrite": [0.95, 0.98],
               "RF_Ceramic": [0.90, 0.95],
               "Common_Mode_Choke": [0.90, 0.95, 0.98, 0.99]
             }
           }
         ]
       }
    ],

    "THERMAL_CHAR": [
      {
        "key": "thermal_resistance",
        "symbol": "θ<sub>JA</sub>",
        "spec_type": "nominal",
        "column_model": "TYP_MAX",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Thermal Resistance", "aliases": ["Rth", "Thermal Resistance to Ambient"]},
        "possible_units": ["°C/W"],
        "std_unit": "°C/W",
        "scenarios": [
          {
            "condition": "Standard PCB",
            "limits": {
              "Power_Ferrite": [20, 30, 50, 80],
              "RF_Ceramic": [100, 150, 200],
              "Common_Mode_Choke": [40, 60, 100]
            }
          }
        ]
      },
      {
        "key": "power_dissipation",
        "symbol": "P<sub>D</sub>",
        "spec_type": "max_limit",
        "column_model": "TYP_MAX",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Power Dissipation", "aliases": ["Max Power Loss", "PD"]},
        "possible_units": ["mW", "W"],
        "std_unit": "mW",
        "scenarios": [
          {
            "condition": "At rated current",
            "limits": {
              "Power_Ferrite": [100, 250, 500, 1000, 2000],
              "RF_Ceramic": [50, 100, 200],
              "Common_Mode_Choke": [100, 300, 500]
            }
          }
        ]
      }
    ],

    "COMPLIANCE": [
      {
        "key": "rohs_status",
        "symbol": "RoHS",
        "spec_type": "nominal",
        "column_model": "TYP_ONLY",
        "engineering_class": "REGULATORY",
        "special_semantics": "BOOLEAN_OR_ENUM",
        "llm_context": {"formal_name": "RoHS Compliance", "aliases": ["RoHS", "Lead Free Status"]},
        "possible_units": [""],
        "std_unit": "",
        "scenarios": [
          {
            "condition": "N/A",
            "limits": {
              "Power_Ferrite": ["Compliant", "Compliant with Exemption", "Non-Compliant"],
              "RF_Ceramic": ["Compliant"],
              "Common_Mode_Choke": ["Compliant"]
            }
          }
        ]
      },
      {
        "key": "automotive_grade",
        "symbol": "AEC-Q200",
        "spec_type": "nominal",
        "column_model": "TYP_ONLY",
        "engineering_class": "REGULATORY",
        "special_semantics": "BOOLEAN",
        "llm_context": {"formal_name": "AEC-Q200 Qualified", "aliases": ["Automotive Grade", "AEC-Q200"]},
        "possible_units": [""],
        "std_unit": "",
        "scenarios": [
          {
            "condition": "N/A",
            "limits": {
              "Power_Ferrite": ["Yes", "No"],
              "RF_Ceramic": ["Yes", "No"],
              "Common_Mode_Choke": ["Yes", "No"]
            }
          }
        ]
      },
      {
        "key": "reach_status",
        "symbol": "REACH",
        "spec_type": "nominal",
        "column_model": "TYP_ONLY",
        "engineering_class": "REGULATORY",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "REACH Compliance", "aliases": ["REACH", "SVHC Free"]},
        "possible_units": [""],
        "std_unit": "",
        "scenarios": [
          {
            "condition": "N/A",
            "limits": {
              "Power_Ferrite": ["Compliant", "Affected"],
              "RF_Ceramic": ["Compliant"],
              "Common_Mode_Choke": ["Compliant"]
            }
          }
        ]
      },
      {
        "key": "halogen_free",
        "symbol": "HF",
        "spec_type": "nominal",
        "column_model": "TYP_ONLY",
        "engineering_class": "REGULATORY",
        "special_semantics": "BOOLEAN",
        "llm_context": {"formal_name": "Halogen Free", "aliases": ["Halogen Free", "HF"]},
        "possible_units": [""],
        "std_unit": "",
        "scenarios": [
          {
            "condition": "N/A",
            "limits": {
              "Power_Ferrite": ["Yes", "No"],
              "RF_Ceramic": ["Yes"],
              "Common_Mode_Choke": ["Yes", "No"]
            }
          }
        ]
      }
    ],

    "LOGISTICS": [
      {
        "key": "packaging_type",
        "symbol": "Pkg",
        "spec_type": "mechanical",
        "column_model": "TYP_ONLY",
        "engineering_class": "LOGISTICS",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Packaging Type", "aliases": ["Packing", "Delivery Form"]},
        "possible_units": [""],
        "std_unit": "",
        "scenarios": [
          {
            "condition": "N/A",
            "limits": {
              "Power_Ferrite": ["Tape & Reel", "Tray", "Bulk"],
              "RF_Ceramic": ["Tape & Reel", "Paper Tape"],
              "Common_Mode_Choke": ["Tape & Reel", "Tray"]
            }
          }
        ]
      },
      {
        "key": "reel_quantity",
        "symbol": "Qty",
        "spec_type": "nominal",
        "column_model": "TYP_ONLY",
        "engineering_class": "LOGISTICS",
        "special_semantics": "INTEGER",
        "llm_context": {"formal_name": "Reel Quantity", "aliases": ["Pack Qty", "Units per Reel"]},
        "possible_units": ["pcs"],
        "std_unit": "pcs",
        "scenarios": [
          {
            "condition": "Standard Reel",
            "limits": {
              "Power_Ferrite": [500, 1000, 2000, 3000],
              "RF_Ceramic": [10000, 15000],
              "Common_Mode_Choke": [500, 1000]
            }
          }
        ]
      },
      {
        "key": "tape_width",
        "symbol": "W<sub>tape</sub>",
        "spec_type": "mechanical",
        "column_model": "TYP_ONLY",
        "engineering_class": "LOGISTICS",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Carrier Tape Width", "aliases": ["Tape Width"]},
        "possible_units": ["mm"],
        "std_unit": "mm",
        "scenarios": [
          {
            "condition": "N/A",
            "limits": {
              "Power_Ferrite": [8, 12, 16, 24],
              "RF_Ceramic": [8],
              "Common_Mode_Choke": [12, 16, 24]
            }
          }
        ]
      }
    ]
  },
    # ==============================================================================
    # 7. BJT - CORRECTED WITH METADATA FIELDS חלק7
    # ==============================================================================
  "BJT": {
    "archetypes": ["Small_Signal", "Power_BJT"],
    
    "ABS_MAX": [
      {
        "key": "vceo",
        "symbol": "V<sub>CEO</sub>",
        "spec_type": "max_rating",
        "column_model": "MAX_ONLY",
        "engineering_class": "SAFETY_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Collector-Emitter Voltage", "aliases": ["Collector-Emitter Voltage", "Max VCEO"]},
        "possible_units": ["V"],
        "std_unit": "V",
        "scenarios": [
          {
            "condition": "Open Base",
            "limits": {
              "Small_Signal": [15, 20, 30, 40, 45, 60, 80, 100],
              "Power_BJT": [40, 60, 80, 100, 120, 140, 200, 250, 400, 600, 1000, 1500]
            }
          }
        ]
      },
      {
        "key": "vcbo",
        "symbol": "V<sub>CBO</sub>",
        "spec_type": "max_rating",
        "column_model": "MAX_ONLY",
        "engineering_class": "SAFETY_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Collector-Base Voltage", "aliases": ["Collector-Base Voltage", "Max VCBO"]},
        "possible_units": ["V"],
        "std_unit": "V",
        "scenarios": [
          {
            "condition": "Open Emitter",
            "limits": {
              "Small_Signal": [20, 30, 40, 60, 80, 100, 120],
              "Power_BJT": [60, 80, 100, 120, 160, 200, 300, 450, 700, 1000, 1700]
            }
          }
        ]
      },
      {
        "key": "vebo",
        "symbol": "V<sub>EBO</sub>",
        "spec_type": "max_rating",
        "column_model": "MAX_ONLY",
        "engineering_class": "SAFETY_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Emitter-Base Voltage", "aliases": ["Emitter-Base Voltage", "Max VEBO"]},
        "possible_units": ["V"],
        "std_unit": "V",
        "scenarios": [
          {
            "condition": "Open Collector",
            "limits": {
              "Small_Signal": [5, 6, 7, 8],
              "Power_BJT": [5, 6, 7, 8, 10]
            }
          }
        ]
      },
      {
        "key": "ic_max",
        "symbol": "I<sub>C</sub>",
        "spec_type": "max_rating",
        "column_model": "MAX_ONLY",
        "engineering_class": "SAFETY_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Collector Current", "aliases": ["Collector Current", "Max Collector Current"]},
        "possible_units": ["mA", "A"],
        "std_unit": "mA",
        "scenarios": [
          {
            "condition": "Continuous",
            "limits": {
              "Small_Signal": [50, 100, 200, 500, 800, 1000],
              "Power_BJT": [1000, 2000, 3000, 5000, 8000, 10000, 15000, 20000, 30000, 50000]
            }
          }
        ]
      },
      {
        "key": "ic_peak",
        "symbol": "I<sub>C(peak)</sub>",
        "spec_type": "max_rating",
        "column_model": "MAX_ONLY",
        "engineering_class": "SAFETY_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Peak Collector Current", "aliases": ["Pulsed Collector Current", "IC Peak"]},
        "possible_units": ["mA", "A"],
        "std_unit": "mA",
        "scenarios": [
          {
            "condition": "Pulsed",
            "limits": {
              "Small_Signal": [200, 500, 1000],
              "Power_BJT": [3000, 6000, 10000, 20000, 40000, 80000]
            }
          }
        ]
      },
      {
        "key": "ib_max",
        "symbol": "I<sub>B</sub>",
        "spec_type": "max_rating",
        "column_model": "MAX_ONLY",
        "engineering_class": "SAFETY_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Base Current", "aliases": ["Max Base Current", "IB Max"]},
        "possible_units": ["mA", "A"],
        "std_unit": "mA",
        "scenarios": [
          {
            "condition": "Continuous",
            "limits": {
              "Small_Signal": [10, 20, 50, 100],
              "Power_BJT": [100, 200, 500, 1000, 2000, 5000]
            }
          }
        ]
      },
      {
        "key": "power_dissipation",
        "symbol": "P<sub>D</sub>",
        "spec_type": "max_rating",
        "column_model": "MAX_ONLY",
        "engineering_class": "SAFETY_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Maximum Power Dissipation", "aliases": ["Total Power Dissipation", "Max Power"]},
        "possible_units": ["W", "mW"],
        "std_unit": "W",
        "scenarios": [
          {
            "condition": "Tc=25°C",
            "limits": {
              "Small_Signal": [0.15, 0.2, 0.3, 0.35, 0.5, 0.625, 1, 1.5],
              "Power_BJT": [1, 2, 5, 10, 20, 25, 40, 50, 75, 100, 150, 200, 300]
            }
          },
          {
            "condition": "Ta=25°C",
            "limits": {
              "Small_Signal": [0.1, 0.15, 0.2, 0.3, 0.4],
              "Power_BJT": [0.5, 1, 2, 5]
            }
          }
        ]
      },
      {
        "key": "tj_max",
        "symbol": "T<sub>j(max)</sub>",
        "spec_type": "max_rating",
        "column_model": "MAX_ONLY",
        "engineering_class": "SAFETY_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Maximum Junction Temperature", "aliases": ["Max Junction Temp"]},
        "possible_units": ["°C"],
        "std_unit": "°C",
        "scenarios": [
          {
            "condition": "Max",
            "limits": {
              "Small_Signal": [125, 150, 175],
              "Power_BJT": [150, 175, 200]
            }
          }
        ]
      },
      {
        "key": "storage_temp",
        "symbol": "T<sub>STG</sub>",
        "spec_type": "max_rating",
        "column_model": "MIN_MAX",
        "engineering_class": "SAFETY_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Storage Temperature Range", "aliases": ["Storage Temperature", "Storage Temp Range"]},
        "possible_units": ["°C"],
        "std_unit": "°C",
        "scenarios": [
          {
            "condition": "Min to Max",
            "limits": {
              "Small_Signal": ["-55 to 150", "-65 to 150"],
              "Power_BJT": ["-55 to 150", "-65 to 175"]
            }
          }
        ]
      },
      {
        "key": "vcex",
        "symbol": "V<sub>CEX</sub>",
        "spec_type": "max_rating",
        "column_model": "MAX_ONLY",
        "engineering_class": "SAFETY_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Collector-Emitter Voltage with External Resistor", "aliases": ["VCEX", "Collector-Emitter with RBE"]},
        "possible_units": ["V"],
        "std_unit": "V",
        "scenarios": [
          {
            "condition": "With RBE resistor",
            "limits": {
              "Small_Signal": [20, 30, 45, 60],
              "Power_BJT": [60, 80, 100, 150, 250, 400]
            }
          }
        ]
      },
      {
        "key": "soa_dc",
        "symbol": "SOA",
        "spec_type": "max_rating",
        "column_model": "GRAPH",
        "engineering_class": "SAFETY_LIMIT",
        "special_semantics": "GRAPH_REFERENCE",
        "llm_context": {"formal_name": "Safe Operating Area - DC", "aliases": ["DC Safe Operating Area", "DC SOA"]},
        "possible_units": [""],
        "std_unit": "",
        "scenarios": [
          {
            "condition": "DC Operation",
            "limits": {
              "Small_Signal": ["See_Graph"],
              "Power_BJT": ["See_Graph"]
            }
          }
        ]
      },
      {
        "key": "soa_pulsed",
        "symbol": "SOA<sub>pulse</sub>",
        "spec_type": "max_rating",
        "column_model": "GRAPH",
        "engineering_class": "SAFETY_LIMIT",
        "special_semantics": "GRAPH_REFERENCE",
        "llm_context": {"formal_name": "Safe Operating Area - Pulsed", "aliases": ["Pulsed Safe Operating Area", "Pulsed SOA"]},
        "possible_units": [""],
        "std_unit": "",
        "scenarios": [
          {
            "condition": "Pulsed Operation",
            "limits": {
              "Small_Signal": ["See_Graph"],
              "Power_BJT": ["See_Graph"]
            }
          }
        ]
      },
      {
        "key": "secondary_breakdown",
        "symbol": "V<sub>(BR)CES</sub>",
        "spec_type": "max_rating",
        "column_model": "MIN_ONLY",
        "engineering_class": "SAFETY_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Secondary Breakdown Voltage", "aliases": ["Second Breakdown", "VCEO(sus)"]},
        "possible_units": ["V"],
        "std_unit": "V",
        "scenarios": [
          {
            "condition": "At specified conditions",
            "limits": {
              "Small_Signal": [20, 30, 45],
              "Power_BJT": [40, 60, 80, 100, 150]
            }
          }
        ]
      },
      {
        "key": "lead_temp",
        "symbol": "T<sub>LEAD</sub>",
        "spec_type": "max_rating",
        "column_model": "MAX_ONLY",
        "engineering_class": "SAFETY_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Lead Temperature (Soldering)", "aliases": ["Soldering Temperature", "Lead Temp"]},
        "possible_units": ["°C"],
        "std_unit": "°C",
        "scenarios": [
          {
            "condition": "10 seconds",
            "limits": {
              "Small_Signal": [260, 300],
              "Power_BJT": [260, 300]
            }
          }
        ]
      }
    ],
    
    "ELEC_CHAR": [
      {
        "key": "bvceo",
        "symbol": "BV<sub>CEO</sub>",
        "spec_type": "min_limit",
        "column_model": "MIN_ONLY",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Collector-Emitter Breakdown Voltage", "aliases": ["BVCEO", "Breakdown Voltage"]},
        "possible_units": ["V"],
        "std_unit": "V",
        "scenarios": [
          {
            "condition": "At IC=1mA to 10mA",
            "limits": {
              "Small_Signal": [20, 30, 45, 60, 80],
              "Power_BJT": [50, 60, 80, 100, 140, 250, 400]
            }
          }
        ]
      },
      {
        "key": "bvcbo",
        "symbol": "BV<sub>CBO</sub>",
        "spec_type": "min_limit",
        "column_model": "MIN_ONLY",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Collector-Base Breakdown Voltage", "aliases": ["BVCBO"]},
        "possible_units": ["V"],
        "std_unit": "V",
        "scenarios": [
          {
            "condition": "At IC=10µA to 100µA",
            "limits": {
              "Small_Signal": [30, 40, 60, 80, 100],
              "Power_BJT": [80, 100, 120, 160, 300, 450]
            }
          }
        ]
      },
      {
        "key": "bvebo",
        "symbol": "BV<sub>EBO</sub>",
        "spec_type": "min_limit",
        "column_model": "MIN_ONLY",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Emitter-Base Breakdown Voltage", "aliases": ["BVEBO"]},
        "possible_units": ["V"],
        "std_unit": "V",
        "scenarios": [
          {
            "condition": "At IE=10µA",
            "limits": {
              "Small_Signal": [5, 6, 7],
              "Power_BJT": [5, 6, 7, 8]
            }
          }
        ]
      },
      {
        "key": "hfe",
        "symbol": "h<sub>FE</sub>",
        "spec_type": "min_limit",
        "column_model": "MIN_TYP_MAX",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "DC Current Gain", "aliases": ["DC Current Gain", "hFE", "Beta"]},
        "possible_units": [""],
        "std_unit": "",
        "scenarios": [
          {
            "condition": "VCE=5V, IC=2mA",
            "limits": {
              "Small_Signal": [100, 150, 200, 250, 300, 400, 600],
              "Power_BJT": [10, 20, 30, 40, 50, 60, 100, 150]
            }
          },
          {
            "condition": "VCE=5V, IC=10mA",
            "limits": {
              "Small_Signal": [110, 150, 200, 300, 400],
              "Power_BJT": [15, 25, 40, 60, 100]
            }
          },
          {
            "condition": "VCE=2V, IC=500mA",
            "limits": {
              "Small_Signal": [],
              "Power_BJT": [10, 20, 30, 50, 80]
            }
          }
        ]
      },
      {
        "key": "vce_sat",
        "symbol": "V<sub>CE(sat)</sub>",
        "spec_type": "max_limit",
        "column_model": "TYP_MAX",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Saturation Voltage", "aliases": ["Collector-Emitter Saturation Voltage", "Saturation Voltage"]},
        "possible_units": ["mV", "V"],
        "std_unit": "mV",
        "scenarios": [
          {
            "condition": "IC=10mA, IB=1mA",
            "limits": {
              "Small_Signal": [100, 150, 200, 250, 300],
              "Power_BJT": [200, 300, 500]
            }
          },
          {
            "condition": "IC=500mA, IB=50mA",
            "limits": {
              "Small_Signal": [200, 300, 500],
              "Power_BJT": [500, 800, 1000, 1500, 2000]
            }
          },
          {
            "condition": "IC=3A, IB=300mA",
            "limits": {
              "Small_Signal": [],
              "Power_BJT": [1000, 1500, 2000, 3000]
            }
          }
        ]
      },
      {
        "key": "vbe_sat",
        "symbol": "V<sub>BE(sat)</sub>",
        "spec_type": "max_limit",
        "column_model": "TYP_MAX",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Base-Emitter Saturation Voltage", "aliases": ["VBE Saturation", "VBE(sat)"]},
        "possible_units": ["V", "mV"],
        "std_unit": "V",
        "scenarios": [
          {
            "condition": "IC=10mA, IB=1mA",
            "limits": {
              "Small_Signal": [0.7, 0.8, 0.9],
              "Power_BJT": [0.8, 0.9, 1.0]
            }
          },
          {
            "condition": "IC=500mA, IB=50mA",
            "limits": {
              "Small_Signal": [0.8, 0.9, 1.0],
              "Power_BJT": [0.9, 1.0, 1.2, 1.5]
            }
          }
        ]
      },
      {
        "key": "vbe_on",
        "symbol": "V<sub>BE(on)</sub>",
        "spec_type": "nominal",
        "column_model": "MIN_TYP_MAX",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Base-Emitter On Voltage", "aliases": ["Base-Emitter On Voltage", "VBE On"]},
        "possible_units": ["V"],
        "std_unit": "V",
        "scenarios": [
          {
            "condition": "VCE=5V, IC=2mA",
            "limits": {
              "Small_Signal": [0.55, 0.6, 0.65, 0.7, 0.75],
              "Power_BJT": [0.6, 0.65, 0.7, 0.8]
            }
          },
          {
            "condition": "VCE=5V, IC=10mA",
            "limits": {
              "Small_Signal": [0.6, 0.65, 0.7, 0.75, 0.8],
              "Power_BJT": [0.65, 0.7, 0.8, 0.9]
            }
          },
          {
            "condition": "VCE=2V, IC=500mA",
            "limits": {
              "Small_Signal": [],
              "Power_BJT": [0.7, 0.8, 0.9, 1.0, 1.2]
            }
          }
        ]
      },
      {
        "key": "icbo",
        "symbol": "I<sub>CBO</sub>",
        "spec_type": "max_limit",
        "column_model": "MAX_ONLY",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Collector-Base Leakage Current", "aliases": ["Collector Cutoff Current", "Leakage Current"]},
        "possible_units": ["nA", "µA"],
        "std_unit": "nA",
        "scenarios": [
          {
            "condition": "VCB=30V to 60V, 25°C",
            "limits": {
              "Small_Signal": [10, 50, 100],
              "Power_BJT": [100, 500, 1000, 5000]
            }
          },
          {
            "condition": "VCB=30V to 60V, 150°C",
            "limits": {
              "Small_Signal": [1000, 5000],
              "Power_BJT": [10000, 50000, 100000]
            }
          }
        ]
      },
      {
        "key": "iceo",
        "symbol": "I<sub>CEO</sub>",
        "spec_type": "max_limit",
        "column_model": "MAX_ONLY",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Collector-Emitter Leakage Current", "aliases": ["ICEO", "Collector Cutoff Current"]},
        "possible_units": ["nA", "µA", "mA"],
        "std_unit": "nA",
        "scenarios": [
          {
            "condition": "VCE=30V to 60V",
            "limits": {
              "Small_Signal": [50, 100, 500, 1000],
              "Power_BJT": [500, 1000, 5000, 10000]
            }
          }
        ]
      },
      {
        "key": "iebo",
        "symbol": "I<sub>EBO</sub>",
        "spec_type": "max_limit",
        "column_model": "MAX_ONLY",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Emitter-Base Leakage Current", "aliases": ["IEBO", "Emitter Cutoff Current"]},
        "possible_units": ["nA", "µA"],
        "std_unit": "nA",
        "scenarios": [
          {
            "condition": "VEB=5V",
            "limits": {
              "Small_Signal": [10, 50, 100],
              "Power_BJT": [100, 500, 1000]
            }
          }
        ]
      },
      {
        "key": "ft",
        "symbol": "f<sub>T</sub>",
        "spec_type": "nominal",
        "column_model": "MIN_TYP_MAX",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Transition Frequency", "aliases": ["Transition Frequency", "Gain Bandwidth Product", "fT"]},
        "possible_units": ["MHz", "GHz"],
        "std_unit": "MHz",
        "scenarios": [
          {
            "condition": "VCE=10V, IC=10mA",
            "limits": {
              "Small_Signal": [100, 150, 200, 250, 300, 400, 500, 600],
              "Power_BJT": [1, 3, 5, 10, 20, 30, 50]
            }
          }
        ]
      },
      {
        "key": "cob",
        "symbol": "C<sub>ob</sub>",
        "spec_type": "nominal",
        "column_model": "MIN_TYP_MAX",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Output Capacitance", "aliases": ["Output Capacitance", "Collector Output Capacitance"]},
        "possible_units": ["pF"],
        "std_unit": "pF",
        "scenarios": [
          {
            "condition": "VCB=10V, f=1MHz",
            "limits": {
              "Small_Signal": [1, 2, 3, 5, 8, 10, 15, 20],
              "Power_BJT": [30, 50, 100, 200, 500, 1000, 2000]
            }
          }
        ]
      },
      {
        "key": "cib",
        "symbol": "C<sub>ib</sub>",
        "spec_type": "nominal",
        "column_model": "TYP_MAX",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Input Capacitance", "aliases": ["Input Capacitance", "Base Input Capacitance"]},
        "possible_units": ["pF"],
        "std_unit": "pF",
        "scenarios": [
          {
            "condition": "VEB=0.5V, f=1MHz",
            "limits": {
              "Small_Signal": [2, 5, 8, 10, 15],
              "Power_BJT": [50, 100, 200, 500, 1000]
            }
          }
        ]
      },
      {
        "key": "cre",
        "symbol": "C<sub>re</sub>",
        "spec_type": "nominal",
        "column_model": "TYP_MAX",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Reverse Transfer Capacitance", "aliases": ["Reverse Capacitance", "Collector-Base Capacitance"]},
        "possible_units": ["pF"],
        "std_unit": "pF",
        "scenarios": [
          {
            "condition": "VCB=10V, f=1MHz",
            "limits": {
              "Small_Signal": [0.5, 1, 1.5, 2, 3],
              "Power_BJT": [5, 10, 20, 50, 100]
            }
          }
        ]
      },
      {
        "key": "hfe_linearity",
        "symbol": "Δh<sub>FE</sub>",
        "spec_type": "max_limit",
        "column_model": "TYP_MAX",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "hFE Variation", "aliases": ["Gain Variation", "Beta Variation"]},
        "possible_units": ["%"],
        "std_unit": "%",
        "scenarios": [
          {
            "condition": "Over IC range",
            "limits": {
              "Small_Signal": [10, 20, 30],
              "Power_BJT": [20, 30, 50]
            }
          }
        ]
      },
      {
        "key": "thermal_resistance_jc",
        "symbol": "θ<sub>JC</sub>",
        "spec_type": "nominal",
        "column_model": "TYP_MAX",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Thermal Resistance Junction to Case", "aliases": ["Rth(j-c)", "Thermal Res. J-C"]},
        "possible_units": ["°C/W"],
        "std_unit": "°C/W",
        "scenarios": [
          {
            "condition": "N/A",
            "limits": {
              "Small_Signal": [200, 150, 100, 50],
              "Power_BJT": [5, 3, 2, 1.5, 1, 0.7, 0.5]
            }
          }
        ]
      },
      {
        "key": "thermal_resistance_ja",
        "symbol": "θ<sub>JA</sub>",
        "spec_type": "nominal",
        "column_model": "TYP_MAX",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {"formal_name": "Thermal Resistance Junction to Ambient", "aliases": ["Rth(j-a)", "Thermal Res. J-A"]},
        "possible_units": ["°C/W"],
        "std_unit": "°C/W",
        "scenarios": [
          {
            "condition": "Free air",
            "limits": {
              "Small_Signal": [200, 300, 400, 500],
              "Power_BJT": [40, 50, 62.5, 80, 100]
            }
          }
        ]
      },
      {
        "key": "noise_figure",
      "symbol": "NF",
      "spec_type": "max_limit",
      "column_model": "TYP_MAX",
      "engineering_class": "PERFORMANCE_LIMIT",
      "special_semantics": "NONE",
      "llm_context": {"formal_name": "Noise Figure", "aliases": ["Noise Figure", "NF", "Spot Noise Figure"]},
      "possible_units": ["dB"],
      "std_unit": "dB",
      "scenarios": [
        {
          "condition": "f=1kHz, Rg=1kΩ",
          "limits": {
            "Small_Signal": [1, 2, 4, 6, 10],
            "Power_BJT": [] 
          }
        }
      ]
    },
    {
      "key": "delay_time",
      "symbol": "t<sub>d</sub>",
      "spec_type": "nominal",
      "column_model": "TYP_MAX",
      "engineering_class": "DYNAMIC_PARAMETER",
      "special_semantics": "SWITCHING",
      "llm_context": {"formal_name": "Delay Time", "aliases": ["Delay Time", "td"]},
      "possible_units": ["ns", "µs"],
      "std_unit": "ns",
      "scenarios": [
        {
          "condition": "VCC=30V, IC=1A, IB1=-IB2",
          "limits": {
            "Small_Signal": [5, 10, 35],
            "Power_BJT": [10, 50, 100, 200]
          }
        }
      ]
    },
    {
      "key": "rise_time",
      "symbol": "t<sub>r</sub>",
      "spec_type": "nominal",
      "column_model": "TYP_MAX",
      "engineering_class": "DYNAMIC_PARAMETER",
      "special_semantics": "SWITCHING",
      "llm_context": {"formal_name": "Rise Time", "aliases": ["Rise Time", "tr"]},
      "possible_units": ["ns", "µs"],
      "std_unit": "ns",
      "scenarios": [
        {
          "condition": "Resistive Load",
          "limits": {
            "Small_Signal": [10, 40, 100],
            "Power_BJT": [100, 300, 700, 1500]
          }
        }
      ]
    },
    {
      "key": "storage_time",
      "symbol": "t<sub>s</sub>",
      "spec_type": "nominal",
      "column_model": "TYP_MAX",
      "engineering_class": "DYNAMIC_PARAMETER",
      "special_semantics": "SWITCHING",
      "llm_context": {"formal_name": "Storage Time", "aliases": ["Storage Time", "ts", "Saturation Storage Time"]},
      "possible_units": ["ns", "µs"],
      "std_unit": "ns",
      "scenarios": [
        {
          "condition": "VCC=30V, IC=1A, IB1=-IB2",
          "limits": {
            "Small_Signal": [150, 200, 400],
            "Power_BJT": [500, 1000, 2500, 4000]
          }
        }
      ]
    },
    {
      "key": "fall_time",
      "symbol": "t<sub>f</sub>",
      "spec_type": "nominal",
      "column_model": "TYP_MAX",
      "engineering_class": "DYNAMIC_PARAMETER",
      "special_semantics": "SWITCHING",
      "llm_context": {"formal_name": "Fall Time", "aliases": ["Fall Time", "tf"]},
      "possible_units": ["ns", "µs"],
      "std_unit": "ns",
      "scenarios": [
        {
          "condition": "Resistive Load",
          "limits": {
            "Small_Signal": [20, 50, 100],
            "Power_BJT": [100, 400, 800, 2000]
          }
        }
      ]
    },
    {
      "key": "input_impedance_hie",
      "symbol": "h<sub>ie</sub>",
      "spec_type": "nominal",
      "column_model": "MIN_TYP_MAX",
      "engineering_class": "SMALL_SIGNAL_PARAMETER",
      "special_semantics": "HYBRID_PARAM",
      "llm_context": {"formal_name": "Input Impedance (Small Signal)", "aliases": ["Input Impedance", "hie"]},
      "possible_units": ["kΩ", "Ω"],
      "std_unit": "kΩ",
      "scenarios": [
        {
          "condition": "VCE=10V, IC=1mA, f=1kHz",
          "limits": {
            "Small_Signal": [1, 2, 5, 10],
            "Power_BJT": [0.5, 1, 2]
          }
        }
      ]
    },
    {
      "key": "output_admittance_hoe",
      "symbol": "h<sub>oe</sub>",
      "spec_type": "nominal",
      "column_model": "MIN_TYP_MAX",
      "engineering_class": "SMALL_SIGNAL_PARAMETER",
      "special_semantics": "HYBRID_PARAM",
      "llm_context": {"formal_name": "Output Admittance (Small Signal)", "aliases": ["Output Admittance", "hoe"]},
      "possible_units": ["µS", "µmhos"],
      "std_unit": "µS",
      "scenarios": [
        {
          "condition": "VCE=10V, IC=1mA, f=1kHz",
          "limits": {
            "Small_Signal": [5, 10, 30, 50, 100],
            "Power_BJT": [50, 100, 200]
          }
        }
      ]
    },
    {
      "key": "reverse_voltage_ratio_hre",
      "symbol": "h<sub>re</sub>",
      "spec_type": "nominal",
      "column_model": "TYP_MAX",
      "engineering_class": "SMALL_SIGNAL_PARAMETER",
      "special_semantics": "HYBRID_PARAM",
      "llm_context": {"formal_name": "Reverse Voltage Ratio", "aliases": ["Reverse Voltage Feedback Ratio", "hre"]},
      "possible_units": ["x10^-4", ""],
      "std_unit": "x10^-4",
      "scenarios": [
        {
          "condition": "VCE=10V, IC=1mA, f=1kHz",
          "limits": {
            "Small_Signal": [0.1, 1, 2, 5, 10],
            "Power_BJT": [5, 10, 20]
           }
          }
        ]
      }
    ]
  },

    # ==============================================================================
    # 8. OPAMP חלק8
    # ==============================================================================
    "OPAMP": {
    "archetypes": ["General_Purpose", "Precision_Zero_Drift", "High_Speed"],

    "ABS_MAX": [
      {
        "key": "supply_voltage",
        "symbol": "V<sub>S</sub>",
        "spec_type": "max_rating",
        "column_model": "MAX_ONLY",
        "engineering_class": "SAFETY_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Total Supply Voltage",
          "aliases": ["Supply Voltage", "Max Supply Voltage"]
        },
        "possible_units": ["V"],
        "std_unit": "V",
        "scenarios": [
          {
            "condition": "Total (V+ - V-)",
            "limits": {
              "General_Purpose": [16, 32, 36, 40, 44],
              "Precision_Zero_Drift": [5.5, 6, 12, 16, 18],
              "High_Speed": [5.5, 6, 12, 15]
            }
          }
        ]
      },
      {
        "key": "differential_input_voltage",
        "symbol": "V<sub>ID</sub>",
        "spec_type": "max_rating",
        "column_model": "MAX_ONLY",
        "engineering_class": "SAFETY_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Differential Input Voltage",
          "aliases": ["Max Differential Input", "Diff. Input Voltage"]
        },
        "possible_units": ["V"],
        "std_unit": "V",
        "scenarios": [
          {
            "condition": "Max",
            "limits": {
              "General_Purpose": [32, 36, 40, 44],
              "Precision_Zero_Drift": [5.5, 6, 12, 16],
              "High_Speed": [5.5, 6, 12]
            }
          }
        ]
      },
      {
        "key": "input_voltage_range",
        "symbol": "V<sub>ICM</sub>",
        "spec_type": "max_rating",
        "column_model": "MIN_MAX",
        "engineering_class": "SAFETY_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Input Common-Mode Voltage Range",
          "aliases": ["Common Mode Input Voltage", "Input Voltage Range"]
        },
        "possible_units": ["V"],
        "std_unit": "V",
        "scenarios": [
          {
            "condition": "Absolute Max",
            "limits": {
              "General_Purpose": ["V- to V+", "V--0.3 to V++0.3"],
              "Precision_Zero_Drift": ["V--0.3 to V++0.3"],
              "High_Speed": ["V- to V+"]
            }
          }
        ]
      },
      {
        "key": "output_short_circuit_duration",
        "symbol": "t<sub>SC</sub>",
        "spec_type": "max_rating",
        "column_model": "MAX_ONLY",
        "engineering_class": "SAFETY_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Output Short Circuit Duration",
          "aliases": ["Short Circuit Duration", "Output Short Time"]
        },
        "possible_units": ["s"],
        "std_unit": "s",
        "scenarios": [
          {
            "condition": "Continuous",
            "limits": {
              "General_Purpose": ["Indefinite"],
              "Precision_Zero_Drift": ["Indefinite", "10"],
              "High_Speed": ["Indefinite", "10"]
            }
          }
        ]
      },
      {
        "key": "power_dissipation",
        "symbol": "P<sub>D</sub>",
        "spec_type": "max_rating",
        "column_model": "MAX_ONLY",
        "engineering_class": "SAFETY_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Maximum Power Dissipation",
          "aliases": ["Power Dissipation", "Max Power"]
        },
        "possible_units": ["mW", "W"],
        "std_unit": "mW",
        "scenarios": [
          {
            "condition": "Ta=25°C",
            "limits": {
              "General_Purpose": [500, 625, 1000, 1250],
              "Precision_Zero_Drift": [300, 500, 625],
              "High_Speed": [500, 1000, 1500]
            }
          }
        ]
      },
      {
        "key": "junction_temperature",
        "symbol": "T<sub>J</sub>",
        "spec_type": "max_rating",
        "column_model": "MAX_ONLY",
        "engineering_class": "SAFETY_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Maximum Junction Temperature",
          "aliases": ["Max Junction Temp", "Tj Max"]
        },
        "possible_units": ["°C"],
        "std_unit": "°C",
        "scenarios": [
          {
            "condition": "Max",
            "limits": {
              "General_Purpose": [125, 150],
              "Precision_Zero_Drift": [125, 150],
              "High_Speed": [125, 150]
            }
          }
        ]
      },
      {
        "key": "storage_temperature",
        "symbol": "T<sub>STG</sub>",
        "spec_type": "max_rating",
        "column_model": "MIN_MAX",
        "engineering_class": "SAFETY_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Storage Temperature Range",
          "aliases": ["Storage Temp Range", "Storage Temperature"]
        },
        "possible_units": ["°C"],
        "std_unit": "°C",
        "scenarios": [
          {
            "condition": "Min to Max",
            "limits": {
              "General_Purpose": ["-65 to 150"],
              "Precision_Zero_Drift": ["-65 to 150"],
              "High_Speed": ["-65 to 150"]
            }
          }
        ]
      },
      {
        "key": "esd_rating",
        "symbol": "V<sub>ESD</sub>",
        "spec_type": "max_rating",
        "column_model": "MIN_ONLY",
        "engineering_class": "SAFETY_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "ESD Rating",
          "aliases": ["Electrostatic Discharge", "ESD HBM"]
        },
        "possible_units": ["kV"],
        "std_unit": "kV",
        "scenarios": [
          {
            "condition": "Human Body Model",
            "limits": {
              "General_Purpose": [1, 2, 4],
              "Precision_Zero_Drift": [2, 4],
              "High_Speed": [1, 2]
            }
          }
        ]
      },
      {
        "key": "lead_temperature",
        "symbol": "T<sub>LEAD</sub>",
        "spec_type": "max_rating",
        "column_model": "MAX_ONLY",
        "engineering_class": "SAFETY_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Lead Temperature (Soldering)",
          "aliases": ["Soldering Temperature", "Lead Temp"]
        },
        "possible_units": ["°C"],
        "std_unit": "°C",
        "scenarios": [
          {
            "condition": "10 seconds",
            "limits": {
              "General_Purpose": [260, 300],
              "Precision_Zero_Drift": [260, 300],
              "High_Speed": [260, 300]
            }
          }
        ]
      }
    ],

    "ELEC_CHAR": [
      {
        "key": "supply_voltage_range",
        "symbol": "V<sub>S</sub>",
        "spec_type": "nominal",
        "column_model": "MIN_MAX",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Operating Supply Voltage Range",
          "aliases": ["Supply Voltage Range", "Operating Voltage"]
        },
        "possible_units": ["V"],
        "std_unit": "V",
        "scenarios": [
          {
            "condition": "Single Supply",
            "limits": {
              "General_Purpose": ["3 to 32", "5 to 36"],
              "Precision_Zero_Drift": ["1.8 to 5.5", "2.7 to 5.5", "4.5 to 12"],
              "High_Speed": ["2.7 to 5.5", "4.5 to 12"]
            }
          },
          {
            "condition": "Dual Supply",
            "limits": {
              "General_Purpose": ["±1.5 to ±16", "±2.5 to ±18"],
              "Precision_Zero_Drift": ["±0.9 to ±2.75", "±2.25 to ±6"],
              "High_Speed": ["±1.35 to ±2.75", "±2.25 to ±6"]
            }
          }
        ]
      },
      {
        "key": "offset_voltage",
        "symbol": "V<sub>OS</sub>",
        "spec_type": "max_limit",
        "column_model": "TYP_MAX",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Input Offset Voltage",
          "aliases": ["Input Offset Voltage", "Vos", "Offset Voltage"]
        },
        "possible_units": ["µV", "mV"],
        "std_unit": "µV",
        "scenarios": [
          {
            "condition": "25°C",
            "limits": {
              "General_Purpose": [500, 1000, 2000, 5000, 7000],
              "Precision_Zero_Drift": [1, 5, 10, 25, 50, 100],
              "High_Speed": [200, 500, 1000, 3000, 5000]
            }
          },
          {
            "condition": "Over temperature",
            "limits": {
              "General_Purpose": [1000, 3000, 6000],
              "Precision_Zero_Drift": [5, 15, 50, 150],
              "High_Speed": [500, 2000, 4000]
            }
          }
        ]
      },
      {
        "key": "offset_drift",
        "symbol": "ΔV<sub>OS</sub>/ΔT",
        "spec_type": "max_limit",
        "column_model": "TYP_MAX",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Input Offset Voltage Drift",
          "aliases": ["Offset Voltage Drift", "TCV", "Temp Coeff of Vos"]
        },
        "possible_units": ["µV/°C", "nV/°C"],
        "std_unit": "µV/°C",
        "scenarios": [
          {
            "condition": "Typical",
            "limits": {
              "General_Purpose": [1, 5, 10, 15, 20, 30],
              "Precision_Zero_Drift": [0.005, 0.01, 0.02, 0.05, 0.1],
              "High_Speed": [0.5, 1, 2, 5, 10]
            }
          }
        ]
      },
      {
        "key": "input_bias_current",
        "symbol": "I<sub>B</sub>",
        "spec_type": "max_limit",
        "column_model": "TYP_MAX",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Input Bias Current",
          "aliases": ["Input Bias Current", "Bias Current", "IB"]
        },
        "possible_units": ["pA", "nA", "µA"],
        "std_unit": "nA",
        "scenarios": [
          {
            "condition": "25°C",
            "limits": {
              "General_Purpose": [3, 20, 45, 50, 80, 100, 200, 500],
              "Precision_Zero_Drift": [0.001, 0.005, 0.01, 0.1, 0.5, 1, 5],
              "High_Speed": [0.1, 1, 5, 10, 50, 100]
            }
          },
          {
            "condition": "Over temperature",
            "limits": {
              "General_Purpose": [10, 50, 100, 300, 800],
              "Precision_Zero_Drift": [0.01, 0.1, 1, 10],
              "High_Speed": [1, 10, 50, 200]
            }
          }
        ]
      },
      {
        "key": "input_offset_current",
        "symbol": "I<sub>OS</sub>",
        "spec_type": "max_limit",
        "column_model": "TYP_MAX",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Input Offset Current",
          "aliases": ["Input Offset Current", "Offset Current", "IOS"]
        },
        "possible_units": ["pA", "nA"],
        "std_unit": "nA",
        "scenarios": [
          {
            "condition": "25°C",
            "limits": {
              "General_Purpose": [0.5, 3, 5, 10, 20, 50, 100],
              "Precision_Zero_Drift": [0.001, 0.005, 0.01, 0.1, 0.5, 2],
              "High_Speed": [0.1, 1, 5, 10, 20]
            }
          }
        ]
      },
      {
        "key": "input_impedance_diff",
        "symbol": "Z<sub>ID</sub>",
        "spec_type": "min_limit",
        "column_model": "MIN_TYP",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Differential Input Impedance",
          "aliases": ["Input Impedance", "Differential Input Z"]
        },
        "possible_units": ["MΩ", "GΩ"],
        "std_unit": "MΩ",
        "scenarios": [
          {
            "condition": "Typical",
            "limits": {
              "General_Purpose": [1, 10, 100, 1000],
              "Precision_Zero_Drift": [100, 1000, 10000],
              "High_Speed": [1, 10, 100]
            }
          }
        ]
      },
      {
        "key": "input_impedance_common",
        "symbol": "Z<sub>IC</sub>",
        "spec_type": "min_limit",
        "column_model": "MIN_TYP",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Common-Mode Input Impedance",
          "aliases": ["Common Mode Input Z", "Input Z CM"]
        },
        "possible_units": ["MΩ", "GΩ"],
        "std_unit": "MΩ",
        "scenarios": [
          {
            "condition": "Typical",
            "limits": {
              "General_Purpose": [100, 1000, 10000],
              "Precision_Zero_Drift": [1000, 10000],
              "High_Speed": [100, 1000]
            }
          }
        ]
      },
      {
        "key": "input_capacitance",
        "symbol": "C<sub>IN</sub>",
        "spec_type": "nominal",
        "column_model": "TYP_ONLY",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Input Capacitance",
          "aliases": ["Input Cap", "Cin"]
        },
        "possible_units": ["pF"],
        "std_unit": "pF",
        "scenarios": [
          {
            "condition": "Typical",
            "limits": {
              "General_Purpose": [1, 2, 3, 5, 8],
              "Precision_Zero_Drift": [2, 5, 10],
              "High_Speed": [1, 2, 3, 5]
            }
          }
        ]
      },
      {
        "key": "cmrr",
        "symbol": "CMRR",
        "spec_type": "min_limit",
        "column_model": "MIN_TYP",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Common Mode Rejection Ratio",
          "aliases": ["CMRR", "Common Mode Rejection"]
        },
        "possible_units": ["dB"],
        "std_unit": "dB",
        "scenarios": [
          {
            "condition": "DC",
            "limits": {
              "General_Purpose": [70, 80, 90, 100, 110],
              "Precision_Zero_Drift": [100, 110, 120, 130, 140, 160],
              "High_Speed": [65, 70, 80, 90]
            }
          },
          {
            "condition": "At 60Hz",
            "limits": {
              "General_Purpose": [70, 80, 90],
              "Precision_Zero_Drift": [90, 100, 120],
              "High_Speed": [60, 70, 80]
            }
          }
        ]
      },
      {
        "key": "psrr",
        "symbol": "PSRR",
        "spec_type": "min_limit",
        "column_model": "MIN_TYP",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Power Supply Rejection Ratio",
          "aliases": ["PSRR", "Supply Rejection Ratio", "SVRR"]
        },
        "possible_units": ["dB"],
        "std_unit": "dB",
        "scenarios": [
          {
            "condition": "DC, V+ or V-",
            "limits": {
              "General_Purpose": [70, 80, 90, 100, 110],
              "Precision_Zero_Drift": [100, 110, 120, 130, 140],
              "High_Speed": [60, 70, 80, 90]
            }
          },
          {
            "condition": "At 60Hz",
            "limits": {
              "General_Purpose": [70, 80, 90],
              "Precision_Zero_Drift": [90, 100, 110],
              "High_Speed": [60, 70, 80]
            }
          }
        ]
      },
      {
        "key": "open_loop_gain",
        "symbol": "A<sub>OL</sub>",
        "spec_type": "min_limit",
        "column_model": "MIN_TYP",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Open Loop Voltage Gain",
          "aliases": ["Open Loop Gain", "Large Signal Voltage Gain", "AOL"]
        },
        "possible_units": ["dB", "V/mV"],
        "std_unit": "dB",
        "scenarios": [
          {
            "condition": "DC, RL=2kΩ",
            "limits": {
              "General_Purpose": [90, 100, 106, 110, 120],
              "Precision_Zero_Drift": [110, 120, 130, 140, 160],
              "High_Speed": [80, 90, 100, 110]
            }
          },
          {
            "condition": "DC, RL=10kΩ",
            "limits": {
              "General_Purpose": [100, 110, 120],
              "Precision_Zero_Drift": [120, 130, 140],
              "High_Speed": [90, 100, 110]
            }
          }
        ]
      },
      {
        "key": "gbw",
        "symbol": "GBW",
        "spec_type": "nominal",
        "column_model": "MIN_TYP_MAX",
        "engineering_class": "NOMINAL_PARAMETER",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Gain Bandwidth Product",
          "aliases": ["Gain Bandwidth", "GBW", "Unity Gain Bandwidth"]
        },
        "possible_units": ["kHz", "MHz", "GHz"],
        "std_unit": "MHz",
        "scenarios": [
          {
            "condition": "Standard",
            "limits": {
              "General_Purpose": [0.5, 1, 1.2, 3, 4, 10, 18],
              "Precision_Zero_Drift": [0.05, 0.1, 0.5, 1, 2, 5, 10],
              "High_Speed": [10, 50, 100, 200, 500, 1000, 2000, 5000]
            }
          }
        ]
      },
      {
        "key": "phase_margin",
        "symbol": "φ<sub>M</sub>",
        "spec_type": "min_limit",
        "column_model": "MIN_TYP",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Phase Margin",
          "aliases": ["Phase Margin", "PM"]
        },
        "possible_units": ["deg", "°"],
        "std_unit": "deg",
        "scenarios": [
          {
            "condition": "Unity gain",
            "limits": {
              "General_Purpose": [45, 60, 65, 70],
              "Precision_Zero_Drift": [45, 60, 70],
              "High_Speed": [45, 50, 60]
            }
          }
        ]
      },
      {
        "key": "gain_margin",
        "symbol": "GM",
        "spec_type": "min_limit",
        "column_model": "MIN_TYP",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Gain Margin",
          "aliases": ["Gain Margin", "GM"]
        },
        "possible_units": ["dB"],
        "std_unit": "dB",
        "scenarios": [
          {
            "condition": "Unity gain",
            "limits": {
              "General_Purpose": [10, 12, 15],
              "Precision_Zero_Drift": [10, 15],
              "High_Speed": [10, 12]
            }
          }
        ]
      },
      {
        "key": "slew_rate",
        "symbol": "SR",
        "spec_type": "min_limit",
        "column_model": "MIN_TYP",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Slew Rate",
          "aliases": ["Slew Rate", "SR"]
        },
        "possible_units": ["V/µs", "V/ms"],
        "std_unit": "V/µs",
        "scenarios": [
          {
            "condition": "Unity Gain, Rising",
            "limits": {
              "General_Purpose": [0.3, 0.5, 0.6, 1, 3, 10, 13],
              "Precision_Zero_Drift": [0.01, 0.05, 0.1, 0.5, 1, 2],
              "High_Speed": [10, 50, 100, 300, 500, 1000, 2000, 5000]
            }
          },
          {
            "condition": "Unity Gain, Falling",
            "limits": {
              "General_Purpose": [0.3, 0.5, 0.6, 1, 3, 10],
              "Precision_Zero_Drift": [0.01, 0.05, 0.1, 0.5, 1],
              "High_Speed": [10, 50, 100, 300, 500, 1000, 2000]
            }
          }
        ]
      },
      {
        "key": "settling_time",
        "symbol": "t<sub>S</sub>",
        "spec_type": "max_limit",
        "column_model": "TYP_MAX",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Settling Time",
          "aliases": ["Settling Time", "tS"]
        },
        "possible_units": ["ns", "µs"],
        "std_unit": "µs",
        "scenarios": [
          {
            "condition": "To 0.1%, G=1",
            "limits": {
              "General_Purpose": [1, 5, 10, 20],
              "Precision_Zero_Drift": [10, 50, 100],
              "High_Speed": [0.05, 0.1, 0.5, 1]
            }
          },
          {
            "condition": "To 0.01%, G=1",
            "limits": {
              "General_Purpose": [5, 10, 30],
              "Precision_Zero_Drift": [50, 100, 200],
              "High_Speed": [0.1, 0.5, 1, 2]
            }
          }
        ]
      },
      {
        "key": "rise_time",
        "symbol": "t<sub>R</sub>",
        "spec_type": "max_limit",
        "column_model": "TYP_MAX",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Rise Time",
          "aliases": ["Rise Time", "tR"]
        },
        "possible_units": ["ns", "µs"],
        "std_unit": "ns",
        "scenarios": [
          {
            "condition": "10% to 90%",
            "limits": {
              "General_Purpose": [100, 300, 500, 1000],
              "Precision_Zero_Drift": [500, 1000, 5000],
              "High_Speed": [1, 5, 10, 50]
            }
          }
        ]
      },
      {
        "key": "overshoot",
        "symbol": "OS",
        "spec_type": "max_limit",
        "column_model": "TYP_MAX",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Overshoot",
          "aliases": ["Overshoot", "OS"]
        },
        "possible_units": ["%"],
        "std_unit": "%",
        "scenarios": [
          {
            "condition": "Unity gain",
            "limits": {
              "General_Purpose": [5, 10, 20],
              "Precision_Zero_Drift": [5, 10],
              "High_Speed": [10, 20, 30]
            }
          }
        ]
      },
      {
        "key": "supply_current",
        "symbol": "I<sub>SY</sub>",
        "spec_type": "max_limit",
        "column_model": "TYP_MAX",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Supply Current per Amplifier",
          "aliases": ["Supply Current", "Quiescent Current per Amp", "ISY"]
        },
        "possible_units": ["µA", "mA"],
        "std_unit": "mA",
        "scenarios": [
          {
            "condition": "No load, VS=5V",
            "limits": {
              "General_Purpose": [0.03, 0.05, 0.3, 0.7, 1, 1.4, 3, 5],
              "Precision_Zero_Drift": [0.045, 0.5, 0.8, 1, 1.7, 2, 3],
              "High_Speed": [2, 5, 7, 10, 15, 20, 25]
            }
          }
        ]
      },
      {
        "key": "shutdown_current",
        "symbol": "I<sub>SD</sub>",
        "spec_type": "max_limit",
        "column_model": "TYP_MAX",
        "engineering_class": "PERFORMANCE_LIMIT",
        "special_semantics": "NONE",
        "llm_context": {
          "formal_name": "Shutdown Supply Current",
          "aliases": ["Shutdown Current", "ISD"]
        },
        "possible_units": ["nA", "µA"],
        "std_unit": "µA",
        "scenarios": [
          {
            "condition": "Shutdown mode",
            "limits":{
            "General_Purpose": [1, 5, 10],
            "Precision_Zero_Drift": [0.1, 1, 2],
            "High_Speed": [10, 50, 100, 500]
          }
        }
      ]
    },
    {
      "key": "output_voltage_swing_high",
      "symbol": "V<sub>OH</sub>",
      "spec_type": "min_limit",
      "column_model": "MIN_TYP",
      "engineering_class": "PERFORMANCE_LIMIT",
      "special_semantics": "RAIL_TO_RAIL",
      "llm_context": {
        "formal_name": "High Level Output Voltage",
        "aliases": ["Output Voltage Swing High", "VOH", "Maximum Output Voltage"]
      },
      "possible_units": ["V", "mV"],
      "std_unit": "mV",
      "scenarios": [
        {
          "condition": "Output swing from positive rail (V+ - VOH), RL=10kΩ",
          "limits": {
            "General_Purpose": [1000, 1500, 2000],
            "Precision_Zero_Drift": [5, 10, 20, 50],
            "High_Speed": [100, 500, 1000, 1500]
          }
        }
      ]
    },
    {
      "key": "output_voltage_swing_low",
      "symbol": "V<sub>OL</sub>",
      "spec_type": "max_limit",
      "column_model": "TYP_MAX",
      "engineering_class": "PERFORMANCE_LIMIT",
      "special_semantics": "RAIL_TO_RAIL",
      "llm_context": {
        "formal_name": "Low Level Output Voltage",
        "aliases": ["Output Voltage Swing Low", "VOL", "Minimum Output Voltage"]
      },
      "possible_units": ["V", "mV"],
      "std_unit": "mV",
      "scenarios": [
        {
          "condition": "Output swing from negative rail (VOL - V-), RL=10kΩ",
          "limits": {
            "General_Purpose": [500, 1000, 1500],
            "Precision_Zero_Drift": [5, 10, 20, 50],
            "High_Speed": [100, 500, 1000]
          }
        }
      ]
    },
    {
      "key": "output_short_circuit_current",
      "symbol": "I<sub>SC</sub>",
      "spec_type": "min_limit",
      "column_model": "TYP_ONLY",
      "engineering_class": "PERFORMANCE_LIMIT",
      "special_semantics": "NONE",
      "llm_context": {
        "formal_name": "Output Short Circuit Current",
        "aliases": ["Short Circuit Current", "Output Current Capability", "Iout"]
      },
      "possible_units": ["mA"],
      "std_unit": "mA",
      "scenarios": [
        {
          "condition": "Sourcing/Sinking",
          "limits": {
            "General_Purpose": [10, 20, 30, 40],
            "Precision_Zero_Drift": [10, 20, 30],
            "High_Speed": [50, 80, 100, 150, 200, 400]
          }
        }
      ]
    },
    {
      "key": "voltage_noise_density",
      "symbol": "e<sub>n</sub>",
      "spec_type": "max_limit",
      "column_model": "TYP_MAX",
      "engineering_class": "NOISE_PARAM",
      "special_semantics": "NONE",
      "llm_context": {
        "formal_name": "Input Voltage Noise Density",
        "aliases": ["Voltage Noise", "Input Noise Voltage Density", "Spot Noise"]
      },
      "possible_units": ["nV/√Hz"],
      "std_unit": "nV/√Hz",
      "scenarios": [
        {
          "condition": "f=1kHz",
          "limits": {
            "General_Purpose": [20, 30, 40, 50, 80],
            "Precision_Zero_Drift": [3, 5, 10, 15, 22],
            "High_Speed": [1, 2, 5, 8, 12]
          }
        }
      ]
    },
    {
      "key": "current_noise_density",
      "symbol": "i<sub>n</sub>",
      "spec_type": "max_limit",
      "column_model": "TYP_MAX",
      "engineering_class": "NOISE_PARAM",
      "special_semantics": "NONE",
      "llm_context": {
        "formal_name": "Input Current Noise Density",
        "aliases": ["Current Noise", "Input Noise Current Density"]
      },
      "possible_units": ["fA/√Hz", "pA/√Hz"],
      "std_unit": "fA/√Hz",
      "scenarios": [
        {
          "condition": "f=1kHz",
          "limits": {
            "General_Purpose": [500, 1000],
            "Precision_Zero_Drift": [10, 50, 100, 500],
            "High_Speed": [1000, 2000, 5000, 10000]
          }
        }
      ]
    },
    {
      "key": "voltage_noise_pp",
      "symbol": "e<sub>n(p-p)</sub>",
      "spec_type": "max_limit",
      "column_model": "TYP_MAX",
      "engineering_class": "NOISE_PARAM",
      "special_semantics": "LOW_FREQ_NOISE",
      "llm_context": {
        "formal_name": "Low Frequency Noise (0.1Hz to 10Hz)",
        "aliases": ["0.1Hz to 10Hz Noise", "Peak-to-Peak Noise", "Flicker Noise"]
      },
      "possible_units": ["µVpp", "nVpp"],
      "std_unit": "µVpp",
      "scenarios": [
        {
          "condition": "0.1Hz to 10Hz",
          "limits": {
            "General_Purpose": [1, 2, 5, 10],
            "Precision_Zero_Drift": [0.1, 0.2, 0.5, 1.5],
            "High_Speed": [5, 10, 20]
          }
        }
      ]
    },
    {
      "key": "thd_plus_n",
      "symbol": "THD+N",
      "spec_type": "max_limit",
      "column_model": "TYP_ONLY",
      "engineering_class": "PERFORMANCE_LIMIT",
      "special_semantics": "AUDIO_PARAM",
      "llm_context": {
        "formal_name": "Total Harmonic Distortion + Noise",
        "aliases": ["THD+N", "Distortion"]
      },
      "possible_units": ["%", "dB"],
      "std_unit": "dB",
      "scenarios": [
        {
          "condition": "f=1kHz, G=1, Vout=2Vpp",
          "limits": {
            "General_Purpose": [-60, -70, -80],
            "Precision_Zero_Drift": [-90, -100, -110, -120],
            "High_Speed": [-70, -80, -90]
          }
        }
      ]
    },
    {
      "key": "channel_separation",
      "symbol": "CS",
      "spec_type": "min_limit",
      "column_model": "TYP_ONLY",
      "engineering_class": "PERFORMANCE_LIMIT",
      "special_semantics": "MULTI_CHANNEL",
      "llm_context": {
        "formal_name": "Channel Separation",
        "aliases": ["Crosstalk", "Channel Isolation"]
      },
      "possible_units": ["dB"],
      "std_unit": "dB",
      "scenarios": [
        {
          "condition": "f=1kHz to 100kHz",
          "limits": {
            "General_Purpose": [90, 100, 110],
            "Precision_Zero_Drift": [120, 130, 140],
            "High_Speed": [60, 70, 80, 100]
          }
        }
        ]
      }
    ]
  }
}
    
    


Overwriting synthetic_pipeline/strategies.py


In [9]:
%%writefile synthetic_pipeline/data_types.py
import uuid
import datetime
from enum import Enum
from dataclasses import dataclass, field
from typing import Dict, Any, List, Optional
from threading import Lock


# ============================================================================
# UNIFIED ENUMS (MASTER)
# ============================================================================
class SpecType(Enum):
    """Specification type classification"""
    NOMINAL = "nominal"
    TYPICAL = "typical"
    
    # Limits
    MAX_RATING = "max_rating"   # Absolute Maximum (נזק אם עוברים)
    MAX_LIMIT = "max_limit"     # Performance Max (מובטח שלא יעבור את זה)
    MIN_LIMIT = "min_limit"     # Performance Min
    MIN_RATING = "min_rating"   # Absolute Minimum
    
    # Ranges
    OPERATIONAL_RANGE = "operational_range" # טווח עבודה מומלץ

class EngineeringClass(Enum):
    """Engineering classification"""
    SAFETY_LIMIT = "SAFETY_LIMIT"           # גבול בטיחות (Absolute Max)
    
    # Performance
    PERFORMANCE = "PERFORMANCE"             # ביצועים (ברירת מחדל)
    PERFORMANCE_LIMIT = "PERFORMANCE_LIMIT" # גבול ביצועים (אותו דבר, לתאימות)
    
    # Operation
    OPERATING_CONDITION = "OPERATING_CONDITION" # תנאי סביבה/עבודה
    NOMINAL_PARAMETER = "NOMINAL_PARAMETER"     # ערך נקוב
    
    # Others
    RELIABILITY = "RELIABILITY" # אמינות
    MECHANICAL = "MECHANICAL"   # מכאני
    THERMAL = "THERMAL"         # תרמי
    GENERIC = "GENERIC"         # כללי
    
class RoundingMode(Enum):
    ADAPTIVE = "adaptive"
    HIGH = "high"
    MINIMAL = "minimal"

# ==========================================
# 2. Data Models (Immutable Context & Results)
# ==========================================

@dataclass()
class GenerationContext:
    """מחזיק את כל המצב של הדגימה הנוכחית. Immutable."""
    sample_id: str
    component_type: str
    package: str
    archetype: str
    tolerance: float
    process_corner: str = "TYPICAL"
    temp_c: int = 25
    package_index: Optional[int] = None 
    extras: Dict[str, Any] = field(default_factory=dict)
    _lock: Any = field(default=None, repr=False, compare=False)

    def __post_init__(self):  
        """Initialize thread lock for parallel processing"""
        if self._lock is None:
            object.__setattr__(self, '_lock', Lock())


@dataclass
class GeneratedParameter:
    """תוצאה של חישוב פרמטר בודד (לפני פירמוט לתצוגה)"""
    # שדות חובה (חייבים לתת להם ערך ביצירה)
    key: str
    label: str
    symbol: str
    section: str
    value_min: Optional[float] = None
    value_typ: Optional[float] = None
    value_max: Optional[float] = None
    unit: str = ""
    condition: str = ""
    spec_type: SpecType = SpecType.NOMINAL
    engineering_class: EngineeringClass = EngineeringClass.PERFORMANCE

@dataclass
class DatasheetResult:
    """האובייקט המלא שמוחזר מהגנרטור"""
    context: GenerationContext
    parameters: List[GeneratedParameter]
    # ניתן להוסיף כאן לוגיקה של קורלציות גלובליות בעתיד


Overwriting synthetic_pipeline/data_types.py


In [10]:
%%writefile synthetic_pipeline/strategy_base.py
from abc import ABC, abstractmethod
from typing import Dict, Any, Optional, Tuple, List
import math

from synthetic_pipeline.data_types import (
    GenerationContext,
    DatasheetResult,
    GeneratedParameter,
)

from synthetic_pipeline.strategy_utils import apply_final_rounding

class ComponentStrategy(ABC):
    """
    המוח ההנדסי. כל רכיב (נגד, מוספט) חייב לממש את זה.
    הגנרטור מדבר רק עם הממשק הזה.
    """
    
    @abstractmethod
    def create_context(self, schema: Dict, requested_corner: Optional[str]) -> GenerationContext:
        """חישוב ראשוני של נתוני הדגימה (Package, Archetype)"""
        pass

    @abstractmethod
    def calculate_base_value(self, key: str, limits: Any, context: GenerationContext) -> Optional[float]:
        """שליפת הערך הגולמי (לוגיקה פיזיקלית)"""
        pass

    @abstractmethod
    def apply_correlations(self, result: DatasheetResult) -> None:
        """
        הזדמנות לתקן ערכים על סמך ערכים אחרים.
        למשל: עדכון RDSon על בסיס VDS שחושב כבר.
        """
        pass
   

    def create_custom_parameter(self, key: str, context: GenerationContext, param_def: Dict) -> Optional[GeneratedParameter]:
        """
        Hook method: מאפשרת לאסטרטגיה לייצר פרמטר שלם בעצמה, 
        ובכך לעקוף את הלוגיקה הגנרית של הגנרטור.
        אם מחזיר None - הגנרטור ימשיך בלוגיקה הרגילה.
        """
        return None

    from typing import Tuple, Optional

    def apply_final_rounding_atomic(
        self, 
        values_triple: Tuple[Optional[float], Optional[float], Optional[float]], 
        tolerance: float,
        unit: str,
        param_key: str = "",
        auto_calculate_bounds: bool = True  # פרמטר חדש לשליטה
    ) -> Tuple[Optional[float], Optional[float], Optional[float]]:
      
        
        raw_min, raw_typ, raw_max = values_triple
        
        # ============================================================
        # 1. פונקציית עיגול מרכזית
        # ============================================================
        def _smart_round(val: Optional[float]) -> Optional[float]:
            """
            עיגול חכם המשתמש ב-apply_final_rounding אם קיים,
            אחרת חוזר ל-fallback פשוט
            """
            if val is None:
                return None
            
            # נסה להשתמש בפונקציה המתקדמת אם קיימת
            try:
                return apply_final_rounding(val, "adaptive", unit, param_key)
            except (NameError, AttributeError):
                # Fallback: עיגול פשוט אבל חכם
                if abs(val) >= 10:
                    return round(float(val), 2)  # מספרים גדולים: 2 ספרות
                elif abs(val) >= 1:
                    return round(float(val), 3)  # בינוניים: 3 ספרות
                else:
                    return round(float(val), 4)  # קטנים: 4 ספרות
        
        # ============================================================
        # 2. עיגול הערכים הקיימים
        # ============================================================
        rounded_min = _smart_round(raw_min)
        rounded_typ = _smart_round(raw_typ)
        rounded_max = _smart_round(raw_max)
        
        # ============================================================
        # 3. חישוב אוטומטי של גבולות (רק אם מבוקש)
        # ============================================================
        if auto_calculate_bounds and rounded_typ is not None and tolerance is not None and tolerance > 0:
            # חישוב Min אם חסר
            if rounded_min is None:
                calculated_min = raw_typ * (1 - tolerance)
                rounded_min = _smart_round(calculated_min)
            
            # חישוב Max אם חסר
            if rounded_max is None:
                calculated_max = raw_typ * (1 + tolerance)
                rounded_max = _smart_round(calculated_max)
        
        return (rounded_min, rounded_typ, rounded_max)
    
    


Overwriting synthetic_pipeline/strategy_base.py


In [11]:
%%writefile synthetic_pipeline/logic.py
from typing import Dict, Any, List, Optional

from synthetic_pipeline.data_types import (
    GenerationContext,
    GeneratedParameter,
    DatasheetResult,
    SpecType,
    EngineeringClass,
)

from synthetic_pipeline.strategy_base import ComponentStrategy


class DatasheetGenerator:
    """
    The Engine: Orchestrates data generation.
    Updated to support MULTIPLE SCENARIOS per parameter AND Strategy Hooks.
    """
    def __init__(self, db_schema: Dict[str, Any]):
        self.db = db_schema

    def generate(self, component_type: str, strategy: ComponentStrategy, 
                 process_corner: Optional[str] = None) -> DatasheetResult:
        
        if component_type not in self.db:
            raise ValueError(f"Unknown component: {component_type}")
        
        # 1. Create Context
        schema = self.db[component_type]
        context = strategy.create_context(schema, process_corner)
        
        parameters: List[GeneratedParameter] = []

        # ✅ Step 2A: Process resistance FIRST (if exists)
        resistance_param = None
        for section_name, params_config in schema.items():
            if section_name == "archetypes": 
                continue
                
            for config in params_config:
                key = config["key"]
                
                if key == "resistance":
                    custom_param = strategy.create_custom_parameter(key, context, config)
                    if custom_param is not None:
                        resistance_param = custom_param
                    break
            
            if resistance_param:
                break
        
        # Add resistance to list
        if resistance_param:
            parameters.append(resistance_param)
        
        # ✅ Step 2B: Process all OTHER parameters
        for section_name, params_config in schema.items():
            if section_name == "archetypes": 
                continue
            
            for config in params_config:
                key = config["key"]
                
                # Skip resistance (already processed)
                if key == "resistance":
                    continue
                
                # Check Hook
                custom_param = strategy.create_custom_parameter(key, context, config)
                if custom_param is not None:
                    parameters.append(custom_param)
                    continue
                
                # Fallback logic
                param_results = self._process_parameter(config, section_name, context, strategy)
                if param_results:
                    parameters.extend(param_results) 
        
        # 3. Finalize
        result = DatasheetResult(context=context, parameters=parameters)
        strategy.apply_correlations(result)
        return result

    def _process_parameter(self, config: Dict, section: str, 
                           ctx: GenerationContext, strategy: ComponentStrategy) -> List[GeneratedParameter]:
        """
        מעבד פרמטר ומחזיר רשימה של תוצאות - אחת לכל תרחיש (Scenario) רלוונטי.
        """
        results = []
        
        # A. Get all relevant scenarios
        scenarios = self._filter_scenarios(config.get("scenarios", []), ctx.package)
        
        if not scenarios: return []

        # B. Metadata Extraction
        formal_name = config.get("llm_context", {}).get("formal_name")
        if not formal_name:
            formal_name = config.get("label", config["key"])
        symbol = config.get("symbol", "")
        
        try: s_type = SpecType(config.get("spec_type", "nominal"))
        except: s_type = SpecType.NOMINAL
        
        try: e_class = EngineeringClass(config.get("engineering_class", "PERFORMANCE"))
        except: e_class = EngineeringClass.PERFORMANCE

        # Loop over all scenarios
        for scenario in scenarios:
            
            # 1. Engineering Calculation
            base_val = strategy.calculate_base_value(config["key"], scenario["limits"], ctx)
            if base_val is None: continue 

            # 2. Calculate Limits using Strategy Logic
            # התיקון: מעבירים את ה-strategy לפונקציה
            min_v, typ_v, max_v = self._calculate_smart_limits(
                base_val, s_type, ctx.tolerance, config["std_unit"], config["key"], strategy
            )

            # 3. Create Parameter Object
            param_obj = GeneratedParameter(
                key=config["key"],
                label=formal_name,
                symbol=symbol,
                section=section,
                value_min=min_v,
                value_typ=typ_v,
                value_max=max_v,
                unit=config["std_unit"],
                condition=scenario.get("condition", ""),
                spec_type=s_type,
                engineering_class=e_class
            )
            
            results.append(param_obj)
        
        return results

    # --- Helpers ---

    def _filter_scenarios(self, scenarios, package):
        return scenarios 

    def _calculate_smart_limits(self, val, spec_type, tol, unit, key, strategy: ComponentStrategy):
        """
        פונקציית עזר המרכזת את לוגיקת ה-Atomic Rounding.
        תיקון: שימוש במתודות של strategy במקום globals.
        """
        if not isinstance(val, (int, float)):
            return None, val, None

        min_v, typ_v, max_v = None, None, None
        tol_decimal = tol / 100.0 

        # קריאה לפונקציית העיגול מתוך האסטרטגיה
        if spec_type == SpecType.MAX_RATING or spec_type == SpecType.MAX_LIMIT:
            max_v = val
            _, _, max_v = strategy.apply_final_rounding_atomic((None, None, max_v), tol, unit, key)
            
        elif spec_type == SpecType.MIN_LIMIT:
            min_v = val
            min_v, _, _ = strategy.apply_final_rounding_atomic((min_v, None, None), tol, unit, key)
            
        else: # NOMINAL
            typ_v = val
            min_v, typ_v, max_v = strategy.apply_final_rounding_atomic((None, typ_v, None), tol, unit, key)
            
            # Fallback Logic (רק אם האסטרטגיה החזירה None לטווח)
            if min_v is None and max_v is None and typ_v is not None:
                typ_v = round(val, 2)
                min_v = typ_v * (1 - tol_decimal)
                max_v = typ_v * (1 + tol_decimal)

        return min_v, typ_v, max_v

Overwriting synthetic_pipeline/logic.py


In [12]:
%%writefile synthetic_pipeline/strategy_utils.py
import random
import re
import math
from typing import Optional, Any, List, Tuple, Dict


# ==========================================
# 1. PHYSICAL CORRELATION ENGINE AND HELPER FUNCTIONS
# ==========================================

class PhysicalCorrelationEngine:
    """
    Ensures physically realistic relationships between parameters.
    Example: High VDS → High RDS(on), Low Capacitance → High SRF
    """
    
    @staticmethod
    def adjust_rds_based_on_vds(vds_value: float, base_rds: float) -> float:
        """MOSFETs: Higher voltage rating → Higher on-resistance"""
        if vds_value > 500:
            return base_rds * random.uniform(2.0, 5.0)
        elif vds_value > 200:
            return base_rds * random.uniform(1.5, 3.0)
        elif vds_value > 100:
            return base_rds * random.uniform(1.0, 2.0)
        else:
            return base_rds * random.uniform(0.5, 1.2)
    
    @staticmethod
    def adjust_forward_current_based_on_package(package: str, base_current: float) -> float:
        """Diodes: Larger packages → Higher current capability"""
        if any(p in package for p in ["TO-220", "TO-247", "D2PAK"]):
            return base_current * random.uniform(1.5, 3.0)
        elif any(p in package for p in ["SMC", "DO-214"]):
            return base_current * random.uniform(1.0, 1.5)
        else:
            return base_current * random.uniform(0.5, 1.0)
            
    @staticmethod        
    def adjust_capacitance_vs_voltage(cap_base, voltage_rating, archetype):
        if "Electrolytic" in archetype and voltage_rating > 50:
            return cap_base * random.uniform(0.8, 0.95)
        return cap_base
    
    @staticmethod
    def adjust_esr_based_on_capacitance(cap_value: float, base_esr: float) -> float:
        """Capacitors: Higher capacitance → Lower ESR (usually)"""
        if cap_value > 100e6:  # >100µF
            return base_esr * random.uniform(0.3, 0.7)
        elif cap_value > 10e6:  # >10µF
            return base_esr * random.uniform(0.6, 1.0)
        else:
            return base_esr * random.uniform(1.0, 2.0)
    
    @staticmethod
    def adjust_power_dissipation_based_on_package(package: str, base_power: float) -> float:
        """Larger packages → Better thermal performance"""
        thermal_map = {
            "TO-220": 2.5, "TO-247": 3.5, "TO-264": 4.0,
            "DPAK": 1.5, "D2PAK": 2.0,
            "SOT-23": 0.3, "SOT-223": 0.8,
            "0805": 0.2, "1206": 0.4, "2512": 1.0
        }
        for pkg, factor in thermal_map.items():
            if pkg in package:
                return base_power * factor
        return base_power


    # E-series standards
E24_SERIES = [
    1.0, 1.1, 1.2, 1.3, 1.5, 1.6, 1.8, 2.0, 2.2, 2.4, 2.7, 3.0,
    3.3, 3.6, 3.9, 4.3, 4.7, 5.1, 5.6, 6.2, 6.8, 7.5, 8.2, 9.1
    ]
    
E96_SERIES = [
    1.00, 1.02, 1.05, 1.07, 1.10, 1.13, 1.15, 1.18, 1.21, 1.24,
    1.27, 1.30, 1.33, 1.37, 1.40, 1.43, 1.47, 1.50, 1.54, 1.58,
    1.62, 1.65, 1.69, 1.74, 1.78, 1.82, 1.87, 1.91, 1.96, 2.00,
    2.05, 2.10, 2.15, 2.21, 2.26, 2.32, 2.37, 2.43, 2.49, 2.55,
    2.61, 2.67, 2.74, 2.80, 2.87, 2.94, 3.01, 3.09, 3.16, 3.24,
    3.32, 3.40, 3.48, 3.57, 3.65, 3.74, 3.83, 3.92, 4.02, 4.12,
    4.22, 4.32, 4.42, 4.53, 4.64, 4.75, 4.87, 4.99, 5.11, 5.23,
    5.36, 5.49, 5.62, 5.76, 5.90, 6.04, 6.19, 6.34, 6.49, 6.65,
    6.81, 6.98, 7.15, 7.32, 7.50, 7.68, 7.87, 8.06, 8.25, 8.45,
    8.66, 8.87, 9.09, 9.31, 9.53, 9.76
    ]
    
E192_SERIES = E96_SERIES  # Simplified for now
    
E_SERIES = {
    "E24": E24_SERIES,
    "E96": E96_SERIES,
    "E192": E192_SERIES
    }

def snap_to_e_series(value: float, series: str = "E24") -> float:
    """Snap a value to nearest E-series standard value"""
    if value <= 0:
        return value
     
    # Get mantissa and exponent
    exponent = 0
    mantissa = value
     
    # נרמול לטווח [1, 10)
    while mantissa >= 10:
        mantissa /= 10.0  # הוספתי .0 ליתר ביטחון
        exponent += 1
     
    while mantissa < 1:
        mantissa *= 10.0
        exponent -= 1
     
    # Find nearest E-series value
    e_values = E_SERIES.get(series, E_SERIES["E24"])
    nearest = min(e_values, key=lambda x: abs(x - mantissa))
    
    # Reconstruct value
    result = nearest * (10 ** exponent)
    
    # --- התיקון: עיגול כדי להעיף שגיאות דיוק ---
    # אם זה נגד ב-Ohm, בדרך כלל 2-3 ספרות אחרי הנקודה זה די והותר,
    # או 10 ספרות משמעותיות כדי לנקות את ה"זבל" הבינארי.
    if exponent >= 0:
        # למשל: 4700.0 במקום 4700.0000004
        return round(result, 2) 
    else:
        # למשל: 0.047 (מספרים קטנים דורשים יותר ספרות)
        return round(result, abs(exponent) + 2)
    
    
def extract_temperature_from_condition(condition_str: str) -> Optional[int]:
    """Extract temperature value from condition string"""
    import re
    match = re.search(r'(\d+)\s*°C', condition_str)
    if match:
        return int(match.group(1))
    match = re.search(r'T[acj]\s*=\s*(\d+)', condition_str, re.IGNORECASE)
    if match:
        return int(match.group(1))
    return None

def apply_temperature_derating(base_value: float, temp: int) -> float:
    """Apply derating based on temperature"""
    closest_temp = min(TEMP_DERATING.keys(), key=lambda x: abs(x - temp))
    factor = TEMP_DERATING[closest_temp]
    return base_value * factor



def snap_to_standard_tolerance(value: float) -> float:
    """Snap tolerance value to standard values"""
    return min(ResistorConstants.STANDARD_TOLERANCES, key=lambda x: abs(x - value))

def apply_final_rounding(value: Any, precision_level: str = "adaptive", 
                        unit: str = "", param_key: str = "") -> Any:
    """
    FIXED VERSION: Human-like rounding logic.
    Enforces "Datasheet Style" numbers:
    - > 100: No decimals (e.g., 350, not 349.9)
    - 10-100: Max 1 decimal (e.g., 12.5)
    - 1-10: Max 2 decimals (e.g., 3.3, 1.25)
    - < 1: 2 significant digits (e.g., 0.05, 0.0012)
    """
    if not isinstance(value, (int, float)):
        return value
    if value == 0:
        return 0
    
    # 1. Rounding Logic helper
    def human_round(val):
        abs_v = abs(val)
        
        # Large numbers (>100) -> Integer
        if abs_v >= 100:
            return int(round(val))
            
        # Medium numbers (10-100) -> Max 1 decimal
        if abs_v >= 10:
            return round(val, 1)
            
        # Small numbers (1-10) -> Max 2 decimals
        if abs_v >= 1:
            return round(val, 2)
            
        # Tiny numbers (<1) -> 2 Significant digits (e.g., 0.0047)
        return float(f"{val:.2g}")

    # 2. Force Integers for specific types
    if unit in ["°C", "K"] or "temp" in param_key.lower():
        return int(round(value))
        
    if unit == "ppm/°C" or unit == "ppm/V":
        # Snap to nearest 5
        return int(round(value / 5) * 5)

    # 3. Special handling for E-Series (Resistors/Capacitors)
    if param_key in ["resistance", "capacitance", "inductance"]:
        # First, snap to standard value if close enough
        if precision_level != "high":
            series = "E24" if precision_level == "adaptive" else "E12"
            snapped = snap_to_e_series(value, series)
            # Only use snapped if it's reasonably close (within 5%)
            if abs(value - snapped) / value < 0.05:
                # But adhere to human formatting for the snapped value too!
                return human_round(snapped)
    
    # 4. Apply generic human rounding
    return human_round(value)
    
def validate_min_typ_max_spacing(
    min_val: Optional[float], 
    typ_val: Optional[float], 
    max_val: Optional[float],
    tolerance_pct: float,  # שים לב: מצפים פה לאחוז (למשל 1 או 0.5)
    decimals: int = 2      # ברירת מחדל 2, אבל אפשר לשנות לכל רכיב
) -> Tuple[Optional[float], Optional[float], Optional[float]]:
    """
    Ensure min/max values respect the tolerance relative to typ_val.
    Recalculates min/max if they deviate too much (e.g. due to bad rounding elsewhere).
    """
    # 1. בדיקות תקינות בסיסיות
    if typ_val is None:
        return min_val, typ_val, max_val
        
    if min_val is None and max_val is None:
        return min_val, typ_val, max_val

    # המרה מאחוזים (0.5) לשבר עשרוני (0.005)
    tol_fraction = tolerance_pct / 100.0

    # 2. טיפול ב-Min (Low Side)
    if min_val is not None:
        # חישוב הטולרנס בפועל
        actual_tol_low = abs(typ_val - min_val) / typ_val
        
        # אם יש סטייה של יותר מ-10% מהטולרנס המוגדר (למשל טולרנס 1% יצא 0.8%)
        # או שהערך "התנגש" (min == typ) בגלל עיגול אגרסיבי
        if abs(actual_tol_low - tol_fraction) > (tol_fraction * 0.1) or min_val == typ_val:
            # מחשבים מחדש בצורה מדויקת
            new_min = typ_val * (1 - tol_fraction)
            min_val = round(new_min, decimals)

    # 3. טיפול ב-Max (High Side)
    if max_val is not None:
        actual_tol_high = abs(max_val - typ_val) / typ_val
        
        if abs(actual_tol_high - tol_fraction) > (tol_fraction * 0.1) or max_val == typ_val:
            new_max = typ_val * (1 + tol_fraction)
            max_val = round(new_max, decimals)

    return min_val, typ_val, max_val

def select_best_display_unit(value: float, base_unit: str, possible_units: List[str]) -> str:
    """Select best unit for display based on value magnitude"""
    if not isinstance(value, (int, float)):
        return base_unit
    
    abs_val = abs(value)
    
    # Voltage
    if base_unit == "V":
        if abs_val < 0.1 and "mV" in possible_units:
            return "mV"
        if abs_val < 0.0001 and "µV" in possible_units:
            return "µV"
    
    # Current
    if base_unit == "A":
        if abs_val < 0.001 and "µA" in possible_units:
            return "µA"
        elif abs_val < 1 and "mA" in possible_units:
            return "mA"
    
    # Resistance
    if base_unit == "Ω":
        if abs_val >= 1e6 and "MΩ" in possible_units:
            return "MΩ"
        elif abs_val >= 1000 and "kΩ" in possible_units:
            return "kΩ"
        elif abs_val < 1 and "mΩ" in possible_units:
            return "mΩ"
    
    # Capacitance
    if base_unit in ["pF", "F"]:
        if abs_val >= 1e6 and "µF" in possible_units:
            return "µF"
        elif abs_val >= 1000 and "nF" in possible_units:
            return "nF"
        elif base_unit == "F" and abs_val < 1e-6 and "pF" in possible_units:
            return "pF"
    
    # Inductance
    if base_unit == "µH":
        if abs_val >= 1000 and "mH" in possible_units:
            return "mH"
        elif abs_val < 1 and "nH" in possible_units:
            return "nH"
    
    # Frequency
    if base_unit == "MHz":
        if abs_val >= 1000 and "GHz" in possible_units:
            return "GHz"
        elif abs_val < 1 and "kHz" in possible_units:
            return "kHz"
    
    return base_unit

def convert_for_display(value: Any, base_unit: str, target_unit: str, 
                       precision: str = "adaptive", param_key: str = "") -> Any:
    """Convert value to target unit and round for display"""
    if value == "-" or value is None:
        return "-"
    if not isinstance(value, (int, float)):
        return value
    
    # Get conversion factor
    factor = UnitConverters.get_factor(base_unit, target_unit)
    converted = value * factor
    
    # Round the converted value
    rounded = apply_final_rounding(converted, precision, target_unit, param_key)
    
    return rounded

def validate_min_typ_max(min_val: Optional[float], typ_val: Optional[float], 
                        max_val: Optional[float]) -> Tuple[Optional[float], 
                                                            Optional[float], 
                                                            Optional[float]]:
    """Ensure min <= typ <= max constraint"""
    values = [v for v in [min_val, typ_val, max_val] if isinstance(v, (int, float))]
    
    if len(values) < 2:
        return min_val, typ_val, max_val
    
    values_sorted = sorted(values)
    
    result = [None, None, None]
    idx = 0
    if min_val is not None and isinstance(min_val, (int, float)):
        result[0] = values_sorted[idx]
        idx += 1
    if typ_val is not None and isinstance(typ_val, (int, float)):
        result[1] = values_sorted[idx]
        idx += 1
    if max_val is not None and isinstance(max_val, (int, float)):
        result[2] = values_sorted[idx]
    
    return tuple(result)


class UnitConverters:
    """מילון המרות אוניברסלי לכל סוגי הרכיבים"""
    
    # המרה מיחידת בסיס ליחידת יעד (המכפיל)
    CONVERSION_MAP = {
        # Voltage
        ("V", "mV"): 1000, ("V", "µV"): 1e6,
        ("mV", "V"): 0.001, ("µV", "V"): 1e-6,
        
        # Current
        ("A", "mA"): 1000, ("A", "µA"): 1e6, ("mA", "µA"): 1000,
        ("mA", "A"): 0.001, ("µA", "A"): 1e-6, ("µA", "mA"): 0.001,
        
        # Resistance
        ("Ω", "mΩ"): 1000, ("Ω", "kΩ"): 0.001, ("Ω", "MΩ"): 1e-6,
        ("mΩ", "Ω"): 0.001, ("kΩ", "Ω"): 1000, ("MΩ", "Ω"): 1e6,
        
        # Capacitance
        ("F", "mF"): 1000, ("F", "µF"): 1e6, ("F", "nF"): 1e9, ("F", "pF"): 1e12,
        ("µF", "nF"): 1000, ("µF", "pF"): 1e6, ("nF", "pF"): 1000,
        ("pF", "nF"): 0.001, ("nF", "µF"): 0.001, ("µF", "F"): 1e-6,
        
        # Frequency
        ("Hz", "kHz"): 0.001, ("Hz", "MHz"): 1e-6, ("Hz", "GHz"): 1e-9,
        ("kHz", "Hz"): 1000, ("kHz", "MHz"): 0.001,
        ("MHz", "kHz"): 1000, ("MHz", "Hz"): 1e6, ("MHz", "GHz"): 0.001,
        
        # Time
        ("s", "ms"): 1000, ("s", "µs"): 1e6, ("s", "ns"): 1e9,
        ("ms", "s"): 0.001, ("ms", "µs"): 1000,
        ("µs", "ms"): 0.001, ("µs", "ns"): 1000, ("ns", "µs"): 0.001
    }

    @staticmethod
    def get_factor(from_unit, to_unit):
        if from_unit == to_unit: return 1.0
        return UnitConverters.CONVERSION_MAP.get((from_unit, to_unit), 1.0)

Overwriting synthetic_pipeline/strategy_utils.py


In [3]:
%%writefile synthetic_pipeline/generators.py

import random
import json
import re
import html
import requests
import matplotlib.pyplot as plt
import io
import cv2
import numpy as np
import random

from typing import Dict, Any, List, Optional
from dataclasses import asdict
from datetime import datetime
from typing import TYPE_CHECKING
from html2image import Html2Image


# === אימפורטים קריטיים ===
from synthetic_pipeline.strategy_utils import (
    apply_final_rounding,
    select_best_display_unit,
    convert_for_display,
)


        

if TYPE_CHECKING:
    from synthetic_pipeline.data_types import DatasheetResult

# ==========================================
# 0. BASE RENDERER (SHARED LOGIC) - ✅ הוספה חדשה
# ==========================================
class PackageCharacterizer:
    """
    מאפיין כל package לפי תכונות פיזיקליות אמיתיות.
    מאפשר יצירת תיאורים מדויקים ורלוונטיים.
    """
    
    PACKAGE_CATEGORIES = {
        # === Through-Hole (THT) ===
        "TO-220": {
            "form_factor": "through_hole",
            "size_class": "medium_power",
            "primary_benefit": "thermal",
            "mounting": "screw_mount",
            "key_features": ["heatsink_tab", "high_power", "traditional"]
        },
        "TO-220AB": {
            "form_factor": "through_hole",
            "size_class": "medium_power",
            "primary_benefit": "thermal",
            "mounting": "screw_mount",
            "key_features": ["heatsink_tab", "high_power", "traditional"]
        },
        "TO-247": {
            "form_factor": "through_hole",
            "size_class": "high_power",
            "primary_benefit": "thermal",
            "mounting": "screw_mount",
            "key_features": ["heatsink_tab", "ultra_high_power", "industrial_grade"]
        },
        "TO-247-3": {
            "form_factor": "through_hole",
            "size_class": "high_power",
            "primary_benefit": "thermal",
            "mounting": "screw_mount",
            "key_features": ["heatsink_tab", "ultra_high_power", "sic_optimized"]
        },
        "TO-247-4": {
            "form_factor": "through_hole",
            "size_class": "high_power",
            "primary_benefit": "thermal",
            "mounting": "screw_mount",
            "key_features": ["kelvin_source", "ultra_high_power", "low_inductance"]
        },
        "TO-3P": {
            "form_factor": "through_hole",
            "size_class": "high_power",
            "primary_benefit": "thermal",
            "mounting": "screw_mount",
            "key_features": ["heatsink_tab", "high_power", "legacy_compatible"]
        },
        "TO-264": {
            "form_factor": "through_hole",
            "size_class": "ultra_high_power",
            "primary_benefit": "thermal",
            "mounting": "screw_mount",
            "key_features": ["heatsink_tab", "extreme_power", "industrial"]
        },
        "ITO-220": {
            "form_factor": "through_hole",
            "size_class": "medium_power",
            "primary_benefit": "reliability",
            "mounting": "isolated_mount",
            "key_features": ["electrical_isolation", "safety", "high_voltage"]
        },
        "TO-220FP": {
            "form_factor": "through_hole",
            "size_class": "medium_power",
            "primary_benefit": "reliability",
            "mounting": "isolated_mount",
            "key_features": ["full_pack", "electrical_isolation", "high_voltage"]
        },
        
        # === Surface Mount - Power ===
        "DPAK(TO-252)": {
            "form_factor": "surface_mount",
            "size_class": "medium_power_smd",
            "primary_benefit": "space",
            "mounting": "smd_reflow",
            "key_features": ["compact", "automated_assembly", "cost_effective"]
        },
        "TO-252": {
            "form_factor": "surface_mount",
            "size_class": "medium_power_smd",
            "primary_benefit": "space",
            "mounting": "smd_reflow",
            "key_features": ["compact", "automated_assembly", "cost_effective"]
        },
        "D2PAK(TO-263)": {
            "form_factor": "surface_mount",
            "size_class": "high_power_smd",
            "primary_benefit": "thermal",
            "mounting": "smd_reflow",
            "key_features": ["large_pad", "high_power_smd", "excellent_thermal"]
        },
        "TO-263-7": {
            "form_factor": "surface_mount",
            "size_class": "high_power_smd",
            "primary_benefit": "thermal",
            "mounting": "smd_reflow",
            "key_features": ["kelvin_connection", "high_power_smd", "precision_sensing"]
        },
        "D2PAK-7L": {
            "form_factor": "surface_mount",
            "size_class": "high_power_smd",
            "primary_benefit": "thermal",
            "mounting": "smd_reflow",
            "key_features": ["kelvin_connection", "high_power_smd", "low_inductance"]
        },
        "PowerPAK_SO-8": {
            "form_factor": "surface_mount",
            "size_class": "medium_power_smd",
            "primary_benefit": "space",
            "mounting": "smd_reflow",
            "key_features": ["low_profile", "dual_cool", "optimized_footprint"]
        },
        "PQFN_5x6": {
            "form_factor": "surface_mount",
            "size_class": "small_power_smd",
            "primary_benefit": "space",
            "mounting": "smd_reflow",
            "key_features": ["ultra_compact", "land_grid", "low_profile"]
        },
        "DFN_5x6": {
            "form_factor": "surface_mount",
            "size_class": "small_power_smd",
            "primary_benefit": "space",
            "mounting": "smd_reflow",
            "key_features": ["ultra_compact", "exposed_pad", "thermal_efficient"]
        },
        "DFN_8x8": {
            "form_factor": "surface_mount",
            "size_class": "medium_power_smd",
            "primary_benefit": "thermal",
            "mounting": "smd_reflow",
            "key_features": ["gan_optimized", "exposed_pad", "high_frequency"]
        },
        "TOLL": {
            "form_factor": "surface_mount",
            "size_class": "high_power_smd",
            "primary_benefit": "thermal",
            "mounting": "smd_reflow",
            "key_features": ["leadless", "ultra_low_inductance", "high_current"]
        },
        "DirectFET": {
            "form_factor": "surface_mount",
            "size_class": "small_power_smd",
            "primary_benefit": "space",
            "mounting": "smd_reflow",
            "key_features": ["top_cool", "bottom_cool", "ultra_compact"]
        },
        
        # === Surface Mount - Small Signal ===
        "SOT-23": {
            "form_factor": "surface_mount",
            "size_class": "small_signal",
            "primary_benefit": "space",
            "mounting": "smd_reflow",
            "key_features": ["ultra_compact", "low_cost", "high_density"]
        },
        "SOT-23-3": {
            "form_factor": "surface_mount",
            "size_class": "small_signal",
            "primary_benefit": "space",
            "mounting": "smd_reflow",
            "key_features": ["ultra_compact", "low_cost", "high_density"]
        },
        "SOT-23-5": {
            "form_factor": "surface_mount",
            "size_class": "small_signal",
            "primary_benefit": "space",
            "mounting": "smd_reflow",
            "key_features": ["ultra_compact", "low_cost", "high_density"]
        },
        "SOT-23-6": {
            "form_factor": "surface_mount",
            "size_class": "small_signal",
            "primary_benefit": "space",
            "mounting": "smd_reflow",
            "key_features": ["ultra_compact", "low_cost", "high_density"]
        },
        "SOT-89": {
            "form_factor": "surface_mount",
            "size_class": "small_signal",
            "primary_benefit": "space",
            "mounting": "smd_reflow",
            "key_features": ["compact", "thermal_tab", "standard"]
        },
        "SOT-223": {
            "form_factor": "surface_mount",
            "size_class": "small_signal",
            "primary_benefit": "thermal",
            "mounting": "smd_reflow",
            "key_features": ["thermal_tab", "medium_power", "standard"]
        },
        "SOD-123": {
            "form_factor": "surface_mount",
            "size_class": "small_signal",
            "primary_benefit": "space",
            "mounting": "smd_reflow",
            "key_features": ["ultra_compact", "diode_package", "standard"]
        },
        "DO-214": {
            "form_factor": "surface_mount",
            "size_class": "small_signal",
            "primary_benefit": "space",
            "mounting": "smd_reflow",
            "key_features": ["compact", "diode_package", "standard"]
        },
        
        # === Passive Components (Resistors, Capacitors, Inductors) ===
        "0201": {
            "form_factor": "surface_mount",
            "size_class": "ultra_small",
            "primary_benefit": "space",
            "mounting": "smd_reflow",
            "key_features": ["ultra_compact", "high_density", "miniature"]
        },
        "0402": {
            "form_factor": "surface_mount",
            "size_class": "very_small",
            "primary_benefit": "space",
            "mounting": "smd_reflow",
            "key_features": ["compact", "high_density", "standard"]
        },
        "0603": {
            "form_factor": "surface_mount",
            "size_class": "small",
            "primary_benefit": "space",
            "mounting": "smd_reflow",
            "key_features": ["compact", "standard", "versatile"]
        },
        "0805": {
            "form_factor": "surface_mount",
            "size_class": "small",
            "primary_benefit": "space",
            "mounting": "smd_reflow",
            "key_features": ["standard", "easy_handling", "versatile"]
        },
        "1206": {
            "form_factor": "surface_mount",
            "size_class": "medium",
            "primary_benefit": "mechanical",
            "mounting": "smd_reflow",
            "key_features": ["standard", "robust", "easy_handling"]
        },
        "1210": {
            "form_factor": "surface_mount",
            "size_class": "medium",
            "primary_benefit": "mechanical",
            "mounting": "smd_reflow",
            "key_features": ["robust", "higher_power", "standard"]
        },
        "1812": {
            "form_factor": "surface_mount",
            "size_class": "large",
            "primary_benefit": "mechanical",
            "mounting": "smd_reflow",
            "key_features": ["robust", "high_power", "large_footprint"]
        },
        "2010": {
            "form_factor": "surface_mount",
            "size_class": "large",
            "primary_benefit": "mechanical",
            "mounting": "smd_reflow",
            "key_features": ["robust", "high_power", "large_footprint"]
        },
        "2512": {
            "form_factor": "surface_mount",
            "size_class": "large",
            "primary_benefit": "thermal",
            "mounting": "smd_reflow",
            "key_features": ["high_power", "thermal_mass", "industrial"]
        }
    }

    # תבניות משפטים לפי קטגוריית Benefit
    PACKAGE_BENEFIT_TEMPLATES = {
        "thermal": {
            "heatsink_tab": [
                "provides superior thermal coupling to the PCB via an integrated heatsink tab, achieving junction-to-ambient thermal resistance (Rθ_JA) of {rth_ja} °C/W",
                "features a large metal tab for direct heatsink mounting, delivering junction-to-case thermal resistance (Rθ_JC) as low as {rth_jc} °C/W",
                "ensures efficient heat dissipation through a dedicated thermal pad that transfers heat directly to external cooling solutions"
            ],
            "exposed_pad": [
                "utilizes an exposed thermal pad on the bottom for direct heat transfer to the PCB, minimizing thermal resistance",
                "incorporates a large exposed die pad that enables low-impedance thermal paths to ground planes"
            ],
            "large_pad": [
                "features an expansive thermal pad designed for maximum heat spreading across the PCB copper area"
            ],
            "default": [
                "provides reliable thermal performance suitable for standard operating conditions",
                "ensures adequate heat dissipation for typical application requirements"
            ]
        },
        "space": {
            "ultra_compact": [
                "measures just {dimensions} mm, enabling ultra-high component density on space-constrained PCBs",
                "offers a miniature footprint of {dimensions} mm for applications demanding minimal board area"
            ],
            "compact": [
                "features a compact {dimensions} mm footprint, balancing size with ease of assembly",
                "provides space-efficient design in a {dimensions} mm package suitable for automated pick-and-place"
            ],
            "default": [
                "optimizes board space utilization without compromising manufacturability",
                "delivers a practical form factor for modern high-density layouts"
            ]
        },
        "mechanical": {
            "robust": [
                "delivers robust mechanical stability with reinforced terminations that withstand vibration and thermal cycling",
                "features strengthened solder joints and body construction for harsh industrial environments"
            ],
            "default": [
                "ensures reliable mechanical performance in standard operating conditions",
                "provides adequate mechanical strength for typical assembly processes"
            ]
        },
        "reliability": {
            "electrical_isolation": [
                "incorporates electrical isolation between the die and the mounting tab, enabling safe chassis grounding without insulating pads",
                "features an isolated mounting surface that eliminates the need for additional thermal interface insulators"
            ],
            "default": [
                "meets industry-standard reliability specifications for long-term operation",
                "ensures consistent performance across rated environmental conditions"
            ]
        }
    }

    @classmethod
    def get_package_info(cls, package_name: str) -> dict:
        """מחזיר מידע על package או fallback אם לא נמצא"""
        return cls.PACKAGE_CATEGORIES.get(
            package_name,
            {
                "form_factor": "surface_mount",
                "size_class": "generic",
                "primary_benefit": "space",
                "mounting": "smd_reflow",
                "key_features": ["standard"]
            }
        )
    
    @classmethod
    def get_package_benefit(cls, package_name: str, package_specs: dict) -> str:
        """
        מייצר תיאור benefit ספציפי לpackage בהתבסס על מאפייניו האמיתיים
        עם הגנה מפני ערכים חסרים (N/A)
        """
        pkg_info = cls.get_package_info(package_name)
        benefit_category = pkg_info["primary_benefit"]
        key_features = pkg_info["key_features"]
        
        # חילוץ נתונים מתוך ה-specs
        limits = package_specs.get("limits", {})
        dims = package_specs.get("dimensions", [])
        
        # הכנת המשתנים להזרקה
        rth_jc_val = limits.get("rth_jc")
        rth_ja_val = limits.get("rth_ja")
        
        # פירמוט מידות בצורה בטוחה
        dim_str = "generic"
        if dims and isinstance(dims, list) and len(dims) >= 2:
            dim_str = f"{dims[0]}×{dims[1]}"
        
        # בחירת קטגוריית תבניות
        templates = cls.PACKAGE_BENEFIT_TEMPLATES.get(benefit_category, {})
        
        # סינון תבניות: נשתמש רק בתבניות שאנחנו יכולים למלא את הנתונים שלהן!
        valid_templates = []
        
        # 1. נסה למצוא תבניות ספציפיות לפי key_features
        potential_templates = []
        for feature in key_features:
            if feature in templates:
                potential_templates.extend(templates[feature])
        
        # אם לא מצאנו לפי פיצ'ר, נלך לדיפולט של הקטגוריה
        if not potential_templates:
            potential_templates = templates.get("default", [])

        # 2. בדיקת היתכנות (האם יש לי את הנתון הדרוש לתבנית?)
        for t in potential_templates:
            if "{rth_jc}" in t and not rth_jc_val:
                continue # דלג על תבנית שדורשת נתון חסר
            if "{rth_ja}" in t and not rth_ja_val:
                continue
            if "{dimensions}" in t and dim_str == "generic":
                continue
            
            valid_templates.append(t)
            
        # אם סיננו את הכל (למשל חסרים נתונים), נשתמש ב-Generic Fallback מוחלט
        if not valid_templates:
            valid_templates = [
                "designed for optimal performance in standard applications",
                "offers a robust solution for modern circuit designs",
                f"utilizes industry-standard {package_name} packaging"
            ]

        # בחירה והחלפה
        selected_template = random.choice(valid_templates)
        
        # ביצוע ההחלפה (רק אם הערכים קיימים, אחרת זה לא היה נבחר)
        if rth_jc_val:
            selected_template = selected_template.replace("{rth_jc}", str(rth_jc_val))
        if rth_ja_val:
            selected_template = selected_template.replace("{rth_ja}", str(rth_ja_val))
        
        selected_template = selected_template.replace("{dimensions}", dim_str)
        
        return selected_template
        
class BaseDatasheetRenderer:
    """
    מחלקת בסיס המכילה לוגיקה משותפת לכל ה-Renderers.
    מונעת שכפול קוד בפונקציות כמו _format_row.
    """

    SINGLE_VALUE_PARAMS = ["Package Type", "Pin Configuration", "Mounting Type", "Polarization", "Terminal Finish", "Moisture Sensitivity Level"]

    def _get_possible_units_generic(self, unit: str) -> List[str]:
        """מיפוי משפחות יחידות - תומך בכל הרכיבים"""
        families = [
            ['pF', 'nF', 'µF', 'uF', 'mF', 'F'],          # קיבול
            ['nV', 'µV', 'mV', 'V', 'kV'],                # מתח
            ['nA', 'µA', 'mA', 'A'],                      # זרם
            ['mΩ', 'Ω', 'kΩ', 'MΩ'],                      # התנגדות
            ['Hz', 'kHz', 'MHz', 'GHz'],                  # תדר
            ['ns', 'µs', 'ms', 's'],                      # זמן
            ['W', 'mW', 'kW'],                            # הספק
            ['°C/W']                                      # התנגדות תרמית
        ]
        for fam in families:
            if unit in fam:
                return fam
        return [unit]

    def _format_row(self, param) -> Dict:
        """
        גרסה אוניברסלית ומתוקנת (כולל תיקון לבאג המשקל האפסי).
        משמשת גם את ה-JSON וגם את ה-HTML.
        """
        # --- תיקון לבאג המשקל (Weight Bug Fix) ---
        if (param.label == "Weight" or param.key == "weight"):
             if param.value_typ is None or (isinstance(param.value_typ, (int, float)) and param.value_typ <= 0):
                 param.value_typ = None 
        def extract_numeric_value(val):
            """מחלץ ערך מספרי מ-dictionary, list, או מחזיר את הערך כמו שהוא"""
            if val is None:
                return None
            
            # אם זה dictionary - נסה לחלץ את הערך הראשון
            if isinstance(val, dict):
                if len(val) > 0:
                    return list(val.values())[0]
                return None
            
            # אם זה list - קח את הערך הראשון
            if isinstance(val, list):
                if len(val) > 0:
                    return val[0]
                return None
            
            return val

        # חילוץ ערכים בצורה בטוחה
        min_val = extract_numeric_value(param.value_min)
        typ_val = extract_numeric_value(param.value_typ)
        max_val = extract_numeric_value(param.value_max)

        # 1. Atomic Rounding (לפני המרת יחידות)
        min_val, typ_val, max_val = apply_final_rounding((min_val, typ_val, max_val), 5.0, param.unit, param.key)

        # 2. בחירת יחידת תצוגה
        # 2. בחירת יחידת תצוגה
        candidate_vals = [v for v in [min_val, typ_val, max_val] if isinstance(v, (int, float))]
        
        if candidate_vals:
            # --- השינוי (שלב 1): ייצור הרשימה החסרה ---
            possible_units = self._get_possible_units_generic(param.unit)
            
            # --- השינוי (שלב 2): שליחת הרשימה כארגומנט שלישי ---
            display_unit = select_best_display_unit(candidate_vals, param.unit, possible_units)
        else:
            display_unit = param.unit

        # 3. פונקציית המרה + פורמט למחרוזת
        def to_disp(val):
            if val is None:
                return None
            if not isinstance(val, (int, float)):
                return val
            
            # המרת יחידות
            converted = convert_for_display(val, param.unit, display_unit)
            
            # המרה למחרוזת מעוצבת (זה הערך הסופי שיישמר ב-JSON ויוצג ב-HTML)
            if isinstance(converted, (int, float)):
                f_val = float(converted)
                
                # פורמט למספרים קטנים מאוד
                if 0 < abs(f_val) < 0.05:
                    return f"{f_val:.3g}"
                
                # פורמט למספרים שלמים או קרובים לשלמים
                if abs(f_val - round(f_val)) < 0.05: 
                    return str(int(round(f_val)))
                    
                if f_val.is_integer():
                    return str(int(f_val))
                
                return f"{f_val:.3g}"
            
            return str(converted)

        # 4. המרת הערכים (כעת הם מחרוזות מעוצבות)
        disp_min = to_disp(param.value_min)
        disp_typ = to_disp(param.value_typ)
        disp_max = to_disp(param.value_max)

        # 5. בניית השורה (כל הערכים כמחרוזות או None)
        row = {
            "Parameter": param.label,
            "symbol": param.symbol if (hasattr(param, 'symbol') and param.symbol) else "—", 
            "Condition": param.condition if param.condition else "",
            "Unit": display_unit, 
            "Min": disp_min,
            "Typ": disp_typ,
            "Max": disp_max
        }
        
        # לוגיקה להסתרת Typ אם זה Max Rating
        try:
            # בודק אם הערך הוא Enum או מחרוזת
            spec_val = param.spec_type.value if hasattr(param.spec_type, 'value') else param.spec_type
            if spec_val in ["max_rating", "max_limit"]:
                 if not disp_typ: row["Typ"] = ""
        except: pass

        # בדיקה אם להציג ערך בודד (Value)
        if param.key in self.SINGLE_VALUE_PARAMS:
             row["Value"] = disp_typ if disp_typ else "—"

        return row


# ==========================================
# 1. CONFIGURATION & MARKETING (ללא שינוי מהותי)
# ==========================================

class HtmlRenderingConstants:
    """מאגר הסגנונות ומבני הטבלאות"""
    
    SECTION_TITLES = {
        "ABS_MAX": "Absolute Maximum Ratings",
        "ELEC_CHAR": "Electrical Characteristics",
        "PACKAGE": "Mechanical Information",
        "THERMAL": "Thermal Characteristics"
    }

    TABLE_STRUCTURES = {
        "standard_full": { "name": "Standard Electrical Characteristics", "columns": ["Parameter", "symbol", "Condition", "Min", "Typ", "Max", "Unit"], "merge_limits": False, "show_symbol": True, "split_sections": True },
        "max_ratings_only": { "name": "Absolute Maximum Ratings", "columns": ["Parameter", "symbol", "Condition", "Rating", "Unit"], "merge_limits": "max_only", "show_symbol": True, "split_sections": True },
        "thermal_data": { "name": "Thermal Characteristics", "columns": ["Parameter", "symbol", "Conditions", "Typ", "Max", "Unit"], "merge_limits": False, "show_symbol": True, "split_sections": True },
        "operating_range": { "name": "Recommended Operating Conditions", "columns": ["Parameter", "symbol", "Min", "Max", "Unit"], "merge_limits": False, "show_symbol": True, "split_sections": True },
        "compact_commercial": { "name": "Mechanical / General Information", "columns": ["Parameter", "Condition", "Value", "Unit"], "merge_limits": "typ_preferred", "show_symbol": False, "split_sections": True },
        "reliability_specs": { "name": "Reliability & Test Specifications", "columns": ["Test Item", "Test Conditions", "Duration", "Performance / Limit"], "merge_limits": "custom_text", "show_symbol": False, "split_sections": True },
        "legacy_databook": { "name": "Legacy Databook Style", "columns": ["Characteristic", "symbol", "Min", "Typ", "Max", "Unit"], "merge_limits": False, "show_symbol": True, "condition_in_name": True }
    }

    VISUAL_STYLES = {
        "classic_standard": { "name": "Classic Standard (TI Style)", "css": "body { font-family: Arial, Helvetica, sans-serif; color: #333; line-height: 1.4; } h1 { color: #003366; border-bottom: 2px solid #003366; } table { width: 100%; border-collapse: collapse; margin-bottom: 20px; font-size: 13px; } th { background-color: #003366; color: white; text-align: left; padding: 8px; } td { border-bottom: 1px solid #ddd; padding: 6px; } tr:nth-child(even) { background-color: #f2f2f2; }" },
        "vintage_databook": { "name": "Vintage Databook (80s Style)", "css": "body { font-family: 'Courier New', Courier, monospace; color: #000; } h1 { text-transform: uppercase; text-decoration: underline; } table { width: 100%; border: 2px solid #000; border-collapse: collapse; margin-bottom: 25px; } th { border-bottom: 2px solid #000; border-right: 1px solid #000; padding: 5px; text-transform: uppercase; } td { border-right: 1px solid #000; border-bottom: 1px solid #000; padding: 5px; }" },
        "modern_tech": { "name": "Modern Tech (IoT Style)", "css": "body { font-family: 'Segoe UI', Roboto, sans-serif; color: #444; background: #fcfcfc; } h1 { color: #00BCD4; font-weight: 300; letter-spacing: 1px; } table { width: 100%; border-collapse: separate; border-spacing: 0; box-shadow: 0 2px 15px rgba(0,0,0,0.05); border-radius: 8px; overflow: hidden; } th { background-color: #f8f9fa; color: #555; font-weight: 600; padding: 12px; border-bottom: 2px solid #00BCD4; } td { padding: 12px; border-bottom: 1px solid #eee; }" },
        "industrial_heavy": { "name": "Industrial Heavy", "css": "body { font-family: Verdana, sans-serif; color: #222; } h1 { background-color: #FF5722; color: white; padding: 10px; font-family: Impact, sans-serif; } table { width: 100%; border: 3px solid #444; border-collapse: collapse; } th { background-color: #444; color: #FF5722; padding: 10px; border: 1px solid #666; } td { border: 1px solid #999; padding: 8px; font-weight: bold; }" },
        "scientific_precision": { "name": "Scientific Precision", "css": "body { font-family: 'Times New Roman', Times, serif; color: #111; } h1 { font-style: italic; border-bottom: 1px solid #000; } table { width: 100%; border-top: 2px solid #000; border-bottom: 2px solid #000; border-collapse: collapse; } th { border-bottom: 1px solid #000; padding: 4px; font-style: italic; } td { padding: 4px; border-right: 1px dotted #ccc; } td:last-child { border-right: none; }" },
        "compact_japanese": { "name": "Compact Japanese", "css": "body { font-family: Tahoma, sans-serif; font-size: 11px; color: #000033; } h1 { font-size: 16px; color: #000080; border-left: 5px solid #000080; padding-left: 10px; } table { width: 100%; border-collapse: collapse; border: 1px solid #999; } th { background-color: #e6e6ff; color: #000080; padding: 3px; border: 1px solid #999; font-size: 10px; } td { border: 1px solid #ccc; padding: 2px 4px; }" },
        "euro_minimalist": { "name": "Euro Minimalist", "css": "body { font-family: 'Helvetica Neue', Helvetica, sans-serif; color: #2c3e50; } h1 { color: #c0392b; font-size: 24px; margin-bottom: 30px; } table { width: 100%; border-collapse: collapse; } th { text-align: left; padding: 15px 5px; border-bottom: 2px solid #c0392b; color: #7f8c8d; font-weight: normal; } td { padding: 10px 5px; border-bottom: 1px solid #ecf0f1; }" },
        "high_contrast_print": { "name": "High Contrast (Print)", "css": "body { font-family: Arial, sans-serif; color: #000; background: #fff; } h1 { background: #000; color: #fff; padding: 5px 10px; } table { width: 100%; border: 4px solid #000; border-collapse: collapse; } th { background: #000; color: #fff; padding: 10px; font-weight: bold; border: 1px solid #fff; } td { border: 1px solid #000; padding: 8px; font-weight: bold; }" },
        "dark_mode_dev": { "name": "Dark Mode (Developer)", "css": "body { font-family: 'Consolas', 'Monaco', monospace; background-color: #1e1e1e; color: #d4d4d4; } h1 { color: #569cd6; } table { width: 100%; border-collapse: collapse; } th { color: #4ec9b0; text-align: left; padding: 10px; border-bottom: 1px solid #333; } td { padding: 10px; border-bottom: 1px solid #2d2d2d; color: #ce9178; } tr:hover { background-color: #2a2d2e; }" },
        "eco_green": { "name": "Eco Green", "css": "body { font-family: 'Trebuchet MS', sans-serif; color: #2e7d32; } h1 { color: #1b5e20; } table { width: 100%; border-collapse: separate; border-spacing: 0; border: 1px solid #a5d6a7; border-radius: 10px; } th { background-color: #e8f5e9; color: #1b5e20; padding: 10px; first-child: border-top-left-radius: 10px; } td { padding: 8px; border-top: 1px solid #a5d6a7; }" }
    }

class MarketingSentenceBuilder:
    """
    מנוע בניית משפטים מודולרי לייצור מיליוני וריאציות של תיאורים.
    """
    
    # === אוצר מילים דינמי ===
    
    OPENERS = [
        "The {name} from {mfr}",
        "Designed by {mfr}, the {name}",
        "{mfr} introduces the {name}, which",
        "Representing a leap in {tech} technology, the {name}",
        "The {name} is a {quality} {comp_type} that",
        "Engineered for {application}, the {name}",
        "{mfr}'s {name} sets a new standard as a",
        "Featuring advanced {tech} architecture, the {name}"
    ]

    VERBS = [
        "integrates", "combines", "features", "utilizes", 
        "incorporates", "leverages", "delivers", "provides",
        "employs", "harnesses"
    ]

    # מילים לחיבור בין הטכנולוגיה לאריזה
    CONNECTORS = [
        "within a", "housed in a", "encapsulated in a", 
        "delivered in a standard", "packaged in a robust",
        "assembled in a space-saving", "offered in a"
    ]

    # תיאורים טכנולוגיים לפי סוג רכיב
    TECH_DESCRIPTORS = {
        "OPAMP": [
            "advanced low-noise architecture", "precision rail-to-rail topology",
            "high-bandwidth amplification circuitry", "ultra-stable input stage design",
            "drift-compensated internal reference"
        ],
        "BJT": [
            "robust planar bipolar technology", "high-gain epitaxial structure",
            "optimized saturation characteristics", "fast-switching silicon lattice",
            "low-leakage junction design"
        ],
        "MOSFET": [
            "trench-gate field effect technology", "ultra-low Rdson lattice structure",
            "high-speed switching topology", "avalanche-rugged cell design",
            "charge-balanced structure"
        ],
        "DIODE": [
            "low-forward-drop junction", "fast-recovery epitaxial design",
            "controlled-avalanche structure", "high-surge capability die"
        ],
        "RESISTOR": [
            "precision thin-film element", "high-stability thick-film composition",
            "pulse-withstanding resistive material", "low-inductance element design"
        ],
        "CAPACITOR": [
            "multi-layer dielectric structure", "low-ESR electrode configuration",
            "high-stability dielectric formulation", "self-healing film technology"
        ],
        "INDUCTOR": [
            "high-permeability core material", "low-loss wire winding",
            "shielded magnetic construction", "saturation-resistant core design"
        ],
        "VOLTAGE_REGULATOR": [
            "fast-transient response architecture", "low-dropout pass element",
            "precision bandgap reference", "high-efficiency control loop"
        ],
        "generic": [
            "industry-proven semiconductor technology", "cutting-edge reliable architecture",
            "high-performance solid-state design", "optimized internal circuitry"
        ]
    }

    # משפטי סיום (Value Proposition)
    CLOSERS = [
        "ensuring consistent performance over lifetime.",
        "making it ideal for demanding modern designs.",
        "offering reliable operation in harsh conditions.",
        "optimized for cost-sensitive and high-volume applications.",
        "delivering superior linearity and stability.",
        "guaranteeing minimal signal degradation.",
        "providing a perfect balance of performance and efficiency.",
        "enabling next-generation power density.",
        "simplifying thermal management in compact systems."
    ]

    @classmethod
    def generate(cls, context, manufacturer: str, product_name: str, package_desc: str) -> str:
        """
        בונה פסקה מלאה המורכבת מ-2-3 משפטים מודולריים.
        """
        # 1. חילוץ נתונים
        comp_type = context.component_type.upper()
        # מנרמל את שם הרכיב למפתח במילון (למשל POWER_BJT -> BJT)
        tech_key = "generic"
        for key in cls.TECH_DESCRIPTORS.keys():
            if key in comp_type:
                tech_key = key
                break
        
        # 2. בחירת רכיבי המשפט הראשון
        tech_desc = random.choice(cls.TECH_DESCRIPTORS[tech_key])
        
        # בחירת Application רנדומלית כדי לגוון את הפתיחה
        apps = ["industrial systems", "automotive electronics", "portable devices", 
                "power management", "signal processing", "telecom infrastructure"]
        
        opener = random.choice(cls.OPENERS).format(
            name=product_name, 
            mfr=manufacturer, 
            tech=tech_key.lower().replace("_", " "),
            quality=random.choice(["high-performance", "precision", "industrial-grade", "reliable", "rugged"]),
            comp_type=context.component_type.replace("_", " ").lower(),
            application=random.choice(apps)
        )
        
        verb = random.choice(cls.VERBS)
        connector = random.choice(cls.CONNECTORS)
        
        # משפט 1: הזהות והאריזה
        # "The Ultra-BJT from Apex integrates robust planar technology within a TO-220 package."
        sentence_1 = f"{opener} {verb} {tech_desc} {connector} {context.package} package."

        # משפט 2: התיאור הפיזיקלי (מגיע מ-PackageCharacterizer)
        # "This component provides superior thermal coupling..."
        # מוודאים שמתחיל באות גדולה
        package_desc = package_desc[0].upper() + package_desc[1:] if package_desc else "It offers standard mounting capabilities"
        
        # גיוון בפתיחת המשפט השני
        s2_openers = ["This component", "The device", "It", "The specific package design"]
        sentence_2 = f"{random.choice(s2_openers)} {package_desc}"
        
        if not sentence_2.strip().endswith('.'):
             sentence_2 += "."

        # משפט 3: סגירה
        closer = random.choice(cls.CLOSERS)
        # לפעמים נחבר את הסגירה למשפט השני, ולפעמים היא תהיה נפרדת
        sentence_3 = closer[0].upper() + closer[1:]

        # === הרכבה סופית (וריאציות מבנה) ===
        structure_type = random.randint(1, 3)
        
        if structure_type == 1:
            # שלושה משפטים נפרדים (קלאסי)
            return f"{sentence_1} {sentence_2} {sentence_3}"
        
        elif structure_type == 2:
            # חיבור משפט 1 ו-3, ואז הפיזיקלי
            # "The X uses Y technology, making it ideal for Z. It features..."
            combined_1_3 = sentence_1.rstrip('.') + ", " + closer.replace(closer[0].upper(), closer[0].lower())
            return f"{combined_1_3} {sentence_2}"
            
        else:
            # משפט פיזיקלי באמצע עם מילת קישור
            # "The X uses Y technology. Furthermore, it features..., ensuring..."
            transition = random.choice(["Furthermore,", "Moreover,", "Additionally,", "In addition,"])
            return f"{sentence_1} {transition} {sentence_2} {sentence_3}"
            

class MarketingGenerator:
    # ... (קוד MarketingGenerator נשאר זהה לקוד המקורי שלך, ללא שינוי) ...
    MANUFACTURER_PREFIXES = ["Quantum", "Apex", "Vector", "Zenith", "Nova", "Flux", "Nexus", "Pulse", "Prime", "Ultra", "Mega", "Omni", "Precision", "Advanced", "Elite", "Summit", "Vertex", "Titan", "Sigma", "Delta", "Vortex"]
    MANUFACTURER_SUFFIXES = ["Silicon", "Dynamics", "Technologies", "Semiconductor", "Electronics", "Systems", "Micro", "Components", "Devices", "Solutions", "Labs", "Circuits", "Innovations", "Foundry", "Manufacturing", "Power"]
    PRODUCT_PREFIXES = ["Ultra", "Hyper", "Max", "Pro", "Elite", "Prime", "Precision", "Advanced", "High-Speed", "Low-Noise", "Compact", "Enhanced", "Smart", "Intelligent", "Adaptive", "Optimized", "Next-Gen"]
    APPLICATIONS = ["automotive power management", "industrial control systems", "telecommunications infrastructure", "medical instrumentation", "aerospace applications", "consumer electronics", "IoT devices", "battery management systems", "motor control", "LED lighting", "server power supplies", "renewable energy systems", "precision measurement equipment", "high-frequency switching", "portable devices", "ruggedized equipment"]
    PACKAGE_BENEFITS = {
        "thermal": ["provides superior thermal coupling to the PCB for efficient heat dissipation", "features enhanced thermal management capabilities with low junction-to-ambient resistance", "offers excellent heat spreading characteristics for high-power applications", "ensures optimal thermal performance in thermally constrained environments"],
        "space": ["enables high-density PCB layouts with its compact footprint", "maximizes board space utilization in space-critical designs", "allows for miniaturized system designs without compromising performance", "provides industry-leading power density in a small form factor"],
        "mechanical": ["delivers robust mechanical stability under high-vibration conditions", "ensures reliable performance in demanding mechanical environments", "features reinforced lead structure for enhanced durability", "provides superior mechanical strength for harsh operating conditions"],
        "reliability": ["guarantees long-term reliability with industry-proven construction", "meets stringent automotive qualification standards", "ensures consistent performance across wide temperature ranges", "provides exceptional reliability in mission-critical applications"]
    }
    DESCRIPTION_TEMPLATES = [
        "{product_name} from {manufacturer} represents cutting-edge {component_type} technology designed for {application}. {package_benefit_1}. {package_benefit_2}. This component delivers {performance_trait} with {key_feature}, making it ideal for demanding applications requiring {package_strength}.",
        
        "Designed by {manufacturer}, the {product_name} integrates advanced {technology} within a {package_description}. {package_benefit_1}, allowing for efficient operation in {environment}. {package_benefit_2}, ensuring {reliability_aspect} across the operating range.",
        
        "{manufacturer} introduces the {product_name}, a high-performance {component_type} engineered for {application}. {package_benefit_1} while {package_benefit_2}. The {package_form_factor} design emphasizes {design_focus}, delivering {performance_outcome} in space-critical applications.",
        
        "The {product_name} by {manufacturer} combines {technology} with proven reliability for {application}. {package_benefit_1}. The {package_mounting_type} construction {package_benefit_2}, making it suitable for {environment} where {package_strength} is essential."
    ]
    TECHNOLOGIES = ["trench MOSFET technology", "advanced planar architecture", "super-junction design", "precision analog circuitry", "low-noise amplification", "high-speed switching topology", "wide-bandgap semiconductor technology", "integrated protection features"]
    PERFORMANCE_TRAITS = ["exceptional efficiency", "industry-leading performance", "superior linearity", "minimal conduction losses", "ultra-low on-resistance", "excellent thermal stability", "high switching speeds", "low electromagnetic interference", "outstanding precision"]
    KEY_FEATURES = ["low-side current sensing", "integrated gate drive", "thermal shutdown protection", "ESD-hardened inputs", "wide supply voltage range", "true shutdown capability", "over-current protection", "adaptive dead-time control"]
    ENVIRONMENTS = ["automotive", "industrial", "high-temperature", "outdoor", "mission-critical", "space-constrained", "noise-sensitive", "high-reliability"]
    RELIABILITY_ASPECTS = ["long-term stability", "minimal drift", "consistent operation", "predictable behavior", "robust protection"]
    DESIGN_FOCUS = ["thermal efficiency", "power density", "signal integrity", "ease of integration", "cost optimization"]
    PERFORMANCE_OUTCOMES = ["reduced system complexity", "improved power efficiency", "enhanced signal fidelity", "superior thermal performance", "optimized size-to-performance ratio"]
    FEATURES_POOL = ["Low quiescent current for extended battery life", "Wide operating temperature range (-55°C to +150°C)", "Built-in overcurrent and thermal protection", "High precision voltage reference (±1% tolerance)", "Fast transient response (<10µs settling time)", "Low noise operation suitable for audio applications", "Integrated soft-start to reduce inrush current", "ESD protection up to 2kV (HBM)", "High PSRR (>60dB at 1kHz) for improved noise immunity", "Industry-standard pinout for easy migration", "RoHS compliant and halogen-free", "Automotive-grade qualification (AEC-Q100)"]
    SECTION_INTRO_TEMPLATES = {
        "abs_max": ["The following ratings are absolute maximum values that must not be exceeded to prevent permanent device damage.", "Stress beyond the limits specified may cause irreversible harm to the component.", "These ratings define the upper bounds of safe operation; continuous operation at maximum ratings is not recommended."],
        "electrical": ["The electrical specifications below represent typical performance under standard test conditions unless otherwise noted.", "These parameters characterize the component's behavior across its recommended operating range.", "Electrical characteristics are guaranteed over the specified temperature and voltage ranges."],
        "thermal": ["Thermal performance is critical for ensuring long-term reliability and optimal operation.", "The thermal parameters below define heat transfer characteristics from junction to ambient.", "Proper thermal management is essential; exceeding thermal limits may degrade performance or cause failure."],
        "dynamic": ["Dynamic characteristics define the component's transient and switching behavior.", "These parameters are measured under specific load and frequency conditions as detailed in the test setup.", "Switching performance may vary with layout parasitics and external component selection."],
        "mechanical": ["Mechanical specifications provide dimensional and packaging information for PCB layout and assembly.", "All dimensions are in millimeters unless otherwise specified; tolerances are per JEDEC standards.", "Package outlines conform to industry-standard footprints for compatibility with automated assembly."],
        "reliability": ["Reliability data represents performance under accelerated stress testing conditions.", "These metrics ensure the component meets industry standards for quality and longevity.", "Testing methodologies comply with JEDEC and military specifications where applicable."],
        "default": ["The specifications below define key operating parameters for this component.", "All values are measured under standard test conditions unless otherwise indicated.", "Performance is guaranteed across the specified operating range."]
    }

    def __init__(self, mode: str = 'fast', model: str = 'ollama'):
        """
        ✅ תיקון תאימות לאחור (Backward Compatibility Fix)
        
        Args:
            mode: 'fast' (template-based) or 'high_quality' (LLM-based with locked facts)
            model: 'ollama' or other LLM backend (only used in high_quality mode)
        """
        self.mode = mode
        self.model = model
        self.package_characterizer = PackageCharacterizer()

    def generate(self, context, package_specs: dict = None) -> Dict[str, Any]:
        """
        יוצר תוכן שיווקי - תומך בשני מצבים:
        1. fast: תבניות בלבד (ללא LLM)
        2. high_quality: LLM עם עובדות נעולות (מונע הזיות)
        """
        manufacturer = self._generate_manufacturer_name()
        product_name = self._generate_product_name(context)
        
        # יצירת תיאור - תלוי במצב
        if self.mode == 'high_quality' and package_specs:
            # מצב LLM - עם נעילת עובדות
            description = self._generate_description_with_llm(context, manufacturer, product_name, package_specs)
        else:
            # מצב מהיר - תבניות בלבד
            if package_specs:
                package_desc = self.package_characterizer.get_package_benefit(context.package, package_specs)
                description = MarketingSentenceBuilder.generate(context, manufacturer, product_name, package_desc)
            else:
                description = self._generate_description(context, manufacturer, product_name)
        
        # הגדלת מגוון ב-features
        features = random.sample(self.FEATURES_POOL, random.randint(4, 7))
        applications = random.sample(self.APPLICATIONS, random.randint(3, 5))
        
        # Section Intros (נוצרות לפי הצורך בזמן ה-Rendering)
        sections_present = ["ELECTRICAL_CHARACTERISTICS", "ABS_MAX", "THERMAL"]
        section_intros = self._generate_section_intros(sections_present)
        
        footnotes = self._generate_footnotes(context)
        
        return {
            "marketing_name": product_name,
            "manufacturer": manufacturer,
            "description": description,
            "section_intros": section_intros,
            "footnotes": footnotes,
            "key_features": features,
            "typical_applications": applications
        }

    def _generate_description_with_llm(self, context, manufacturer: str, product_name: str, package_specs: dict) -> str:
        """
        🆕 יצירת תיאור באמצעות LLM - עם נעילת עובדות (Locked Facts)
        
        הפרומפט מספק את כל העובדות הקשיחות (מידות, Rth, פינים)
        ומבקש מה-LLM רק לנסח אותן בצורה שיווקית ללא תוספות.
        """
        # חילוץ עובדות קשיחות מה-package_specs
        pkg_info = self.package_characterizer.get_package_info(context.package)
        limits = package_specs.get("limits", {})
        dims = package_specs.get("dimensions", [])
        
        # בניית מחרוזת עובדות
        facts = []
        facts.append(f"Package: {context.package}")
        facts.append(f"Form Factor: {pkg_info['form_factor']}")
        facts.append(f"Component Type: {context.component_type}")
        
        if dims and len(dims) >= 2:
            facts.append(f"Physical Dimensions: {dims[0]}×{dims[1]} mm")
        
        if "rth_jc" in limits and limits["rth_jc"]:
            facts.append(f"Junction-to-Case Thermal Resistance: {limits['rth_jc']} °C/W")
        
        if "rth_ja" in limits and limits["rth_ja"]:
            facts.append(f"Junction-to-Ambient Thermal Resistance: {limits['rth_ja']} °C/W")
        
        facts_text = "\n".join([f"- {fact}" for fact in facts])
        
        # בניית הפרומפט הנעול
        prompt = f"""You are a technical marketing writer for semiconductor datasheets.

STRICT RULES:
1. Use ONLY the facts provided below - do not add any numbers, dimensions, or specifications
2. Write a concise 2-3 sentence product description
3. Focus on applications and benefits, not on inventing technical specs
4. Do not mention specific pin counts, lead counts, or physical measurements unless explicitly provided

GIVEN FACTS:
{facts_text}

Product Name: {product_name}
Manufacturer: {manufacturer}

Write a professional marketing description following the rules above. Output only the description text, no preamble."""

        try:
            if self.model == 'ollama':
                # קריאה ל-Ollama API
                response = requests.post(
                    'http://localhost:11434/api/generate',
                    json={
                        'model': 'llama2',  # או כל מודל אחר שמותקן
                        'prompt': prompt,
                        'stream': False
                    },
                    timeout=30
                )
                
                if response.status_code == 200:
                    result = response.json()
                    description = result.get('response', '').strip()
                    
                    # ניקוי והסרת פרמבולים אפשריים
                    description = self._sanitize_text(description)
                    
                    # אימות שהתיאור לא ריק
                    if description and len(description) > 20:
                        return description
        
        except Exception as e:
            print(f"⚠️ LLM failed: {e}, falling back to templates")
        
        # Fallback: אם ה-LLM נכשל, נשתמש בתבניות
        package_desc = self.package_characterizer.get_package_benefit(context.package, package_specs)
        return MarketingSentenceBuilder.generate(context, manufacturer, product_name, package_desc)

    def _generate_description_package_aware(self, context, manufacturer: str, product_name: str, package_specs: dict) -> str:
        """🆕 יוצר תיאור המותאם למאפייני ה-package"""
        
        template = random.choice(self.DESCRIPTION_TEMPLATES)
        
        # קבלת מידע על package
        pkg_info = self.package_characterizer.get_package_info(context.package)
        
        # יצירת שני benefits שונים
        benefit_1 = self.package_characterizer.get_package_benefit(context.package, package_specs)
        
        # Benefit 2 - מקטגוריה משנית
        secondary_categories = ["thermal", "space", "mechanical", "reliability"]
        secondary_categories.remove(pkg_info["primary_benefit"])
        secondary_benefit = random.choice(secondary_categories)
        
        benefit_2_templates = PackageCharacterizer.PACKAGE_BENEFIT_TEMPLATES.get(secondary_benefit, {})
        benefit_2 = random.choice(benefit_2_templates.get("default", ["ensures reliable operation"]))
        
        # Capitalize first letter of benefit_2
        if benefit_2 and not benefit_2[0].isupper():
            benefit_2 = benefit_2[0].upper() + benefit_2[1:]
        
        # תיאורים ספציפיים לpackage
        package_descriptions = {
            "through_hole": f"{context.package} through-hole package",
            "surface_mount": f"industry-standard {context.package} surface-mount package",
            "specialized": f"{context.package} specialized package"
        }
        
        mounting_descriptions = {
            "screw_mount": "bolt-down",
            "smd_reflow": "surface-mount",
            "isolated_mount": "electrically isolated",
            "flange_bolt": "flange-mounted"
        }
        
        package_strengths = {
            "thermal": "superior thermal management",
            "space": "compact footprint and high power density",
            "mechanical": "mechanical robustness and vibration resistance",
            "reliability": "proven long-term reliability"
        }
        
        description = template.format(
            product_name=product_name,
            manufacturer=manufacturer,
            component_type=context.component_type,
            package_benefit_1=benefit_1,
            package_benefit_2=benefit_2,
            package_description=package_descriptions.get(pkg_info["form_factor"], f"{context.package} package"),
            package_form_factor=pkg_info["form_factor"].replace("_", "-"),
            package_mounting_type=mounting_descriptions.get(pkg_info["mounting"], "standard"),
            package_strength=package_strengths.get(pkg_info["primary_benefit"], "reliability"),
            application=random.choice(self.APPLICATIONS),
            technology=random.choice(self.TECHNOLOGIES),
            performance_trait=random.choice(self.PERFORMANCE_TRAITS),
            key_feature=random.choice(self.KEY_FEATURES),
            environment=random.choice(self.ENVIRONMENTS),
            reliability_aspect=random.choice(self.RELIABILITY_ASPECTS),
            design_focus=random.choice(self.DESIGN_FOCUS),
            performance_outcome=random.choice(self.PERFORMANCE_OUTCOMES)
        )
        
        return self._sanitize_text(description)
        
    def _generate_manufacturer_name(self) -> str:
        prefix = random.choice(self.MANUFACTURER_PREFIXES)
        suffix = random.choice(self.MANUFACTURER_SUFFIXES)
        return f"{prefix}{suffix}" if random.random() < 0.7 else f"{prefix} {suffix}"

    def _generate_product_name(self, context) -> str:
        prefix = random.choice(self.PRODUCT_PREFIXES)
        comp_type = context.component_type.upper()
        patterns = [f"{prefix}-{comp_type[:3]}{random.randint(100, 999)}", f"{prefix}{comp_type[:4]}-{random.choice(['TX', 'RX', 'MX', 'LX', 'HX'])}{random.randint(1, 99)}", f"{comp_type[:2]}{random.randint(1000, 9999)}{random.choice(['A', 'B', 'C', 'X', 'P'])}", f"{prefix} {comp_type} Series {random.randint(1000, 9999)}"]
        return random.choice(patterns)

    def _generate_description(self, context, manufacturer: str, product_name: str) -> str:
        """
        פונקציית יצירת תיאור סטנדרטית (ללא package_specs מפורטים).
        ממלאת את כל המשתנים הנדרשים לתבניות באמצעות נתונים בסיסיים.
        """
        template = random.choice(self.DESCRIPTION_TEMPLATES)
        
        # 1. שליפת מידע בסיסי על המארז (למניעת KeyError)
        pkg_info = self.package_characterizer.get_package_info(context.package)
        
        # 2. חישוב Benefits רנדומליים (כמו בקוד המקורי)
        benefit_categories = random.sample(list(self.PACKAGE_BENEFITS.keys()), 2)
        package_benefit_1 = random.choice(self.PACKAGE_BENEFITS[benefit_categories[0]])
        package_benefit_2 = random.choice(self.PACKAGE_BENEFITS[benefit_categories[1]])
        if not package_benefit_2[0].isupper(): 
            package_benefit_2 = package_benefit_2[0].upper() + package_benefit_2[1:]
        
        # 3. הגדרת המיפויים החסרים (הסיבה לקריסה הקודמת!)
        package_descriptions = {
            "through_hole": f"{context.package} through-hole package",
            "surface_mount": f"industry-standard {context.package} surface-mount package",
            "specialized": f"{context.package} specialized package"
        }
        mounting_descriptions = {
            "screw_mount": "bolt-down",
            "smd_reflow": "surface-mount",
            "isolated_mount": "electrically isolated",
            "flange_bolt": "flange-mounted"
        }
        package_strengths = {
            "thermal": "superior thermal management",
            "space": "compact footprint and high power density",
            "mechanical": "mechanical robustness and vibration resistance",
            "reliability": "proven long-term reliability"
        }
        
        # 4. הרכבת מילון הארגומנטים המלא
        args = {
            "product_name": product_name,
            "manufacturer": manufacturer,
            "component_type": context.component_type,
            "package": context.package,
            "application": random.choice(self.APPLICATIONS),
            "package_benefit_1": package_benefit_1,
            "package_benefit_2": package_benefit_2,
            "technology": random.choice(self.TECHNOLOGIES),
            "performance_trait": random.choice(self.PERFORMANCE_TRAITS),
            "key_feature": random.choice(self.KEY_FEATURES),
            "environment": random.choice(self.ENVIRONMENTS),
            "reliability_aspect": random.choice(self.RELIABILITY_ASPECTS),
            "design_focus": random.choice(self.DESIGN_FOCUS),
            "performance_outcome": random.choice(self.PERFORMANCE_OUTCOMES),
            
            # --- התיקון: מילוי השדות שהיו חסרים ---
            "package_description": package_descriptions.get(pkg_info["form_factor"], f"{context.package} package"),
            "package_form_factor": pkg_info["form_factor"].replace("_", "-"),
            "package_mounting_type": mounting_descriptions.get(pkg_info["mounting"], "standard"),
            "package_strength": package_strengths.get(pkg_info["primary_benefit"], "reliability")
        }
        
        # 5. יצירת הטקסט
        try:
            description = template.format(**args)
        except KeyError as e:
            # Fallback חירום למקרה שמשהו עדיין חסר
            print(f"⚠️ Template formatting error: {e}, using simple fallback.")
            description = f"The {product_name} is a reliable {context.component_type} in a {context.package} package."
        
        return self._sanitize_text(description)

    def _generate_section_intros(self, sections_present: List[str]) -> Dict[str, str]:
        section_intros = {}
        for section in sections_present:
            section_upper = section.upper()
            if any(keyword in section_upper for keyword in ['ABS', 'MAX', 'ABSOLUTE']): category = 'abs_max'
            elif any(keyword in section_upper for keyword in ['THERM', 'THERMAL']): category = 'thermal'
            elif any(keyword in section_upper for keyword in ['DYN', 'SWITCH', 'AC']): category = 'dynamic'
            elif any(keyword in section_upper for keyword in ['MECH', 'PKG']): category = 'mechanical'
            elif any(keyword in section_upper for keyword in ['REL', 'QUALITY']): category = 'reliability'
            elif any(keyword in section_upper for keyword in ['ELEC', 'DC']): category = 'electrical'
            else: category = 'default'
            section_intros[section] = random.choice(self.SECTION_INTRO_TEMPLATES[category])
        return section_intros

    def _generate_footnotes(self, context) -> List[str]:
        footnote_pool = ["All specifications measured at 25°C ambient temperature.", "Pulse width limited by safe operating area.", "Thermal resistance values based on standard JEDEC test board.", "Production tested parameters are 100% verified.", "Specifications subject to change without notice.", "RoHS compliant and halogen-free."]
        return random.sample(footnote_pool, random.randint(1, 3))

    def _parse_json(self, text: str, context) -> Dict[str, Any]:
        try:
            clean_text = text.replace("```json", "").replace("```", "").strip()
            return json.loads(clean_text)
        except:
            try:
                match = re.search(r'\{.*\}', text, re.DOTALL)
                if match: return json.loads(match.group())
            except: pass
        return self._get_fallback_data(context)

    def _get_fallback_data(self, context) -> Dict[str, Any]:
        if context is None: return { "marketing_name": "GEN-1000", "manufacturer": "Standard Semi", "description": "Generic comp.", "section_intros": {}, "footnotes": [] }
        print("⚠️ LLM failed, using template-based fallback")
        return self._generate_with_templates(context, [])

    def _sanitize_text(self, text: str) -> str:
        replacements = {'generic comp': 'specialized component', 'generic component': 'high-performance device', 'standard part': 'precision device', 'basic device': 'engineered solution', 'simple component': 'optimized component', 'null': 'N/A', 'undefined': 'specified', '###': '', 'ERROR': 'nominal', 'FAILED': 'tested'}
        result = text
        for old, new in replacements.items():
            result = result.replace(old, new)
            result = result.replace(old.upper(), new.upper())
            result = result.replace(old.lower(), new.lower())
        return result


# ==========================================
# 2. HTML Renderer (Presentation)
# ==========================================

class DatasheetHtmlRenderer(BaseDatasheetRenderer):
    """
    Renders the final HTML with features like:
    - Dynamic header structures (Classic/Modern/Minimalist)
    - Page layouts (Standard/Two-Col/Grid)
    - Visual Noise (Watermarks, Image Placeholders)
    
    ✅ FIX #2 APPLIED: HTML renderer now ONLY uses data from result.parameters
    ✅ FIX #3 APPLIED: Graph placement limited to ELECTRICAL_CHARACTERISTICS section
    """
    
    HEADER_STRUCTURES = {
        "classic": { "template": "<div class='header header-classic'><div class='header-meta'><span class='meta-id'>Document ID: {sample_id}</span><span class='meta-manufacturer'>{manufacturer}</span></div><h1 class='product-title'>{marketing_name}</h1><div class='product-description'><p><strong>Description:</strong> {description}</p></div><div class='technical-info'><span>Package: {package}</span> | <span>Technology: {archetype}</span></div></div>", "css_class": "header-classic" },
        "modern": { "template": "<div class='header header-modern'><div class='header-brand'><h2>{manufacturer}</h2></div><div class='header-content-centered'><h1>{marketing_name}</h1><p class='lead-description'>{description}</p><div class='spec-badges'><span class='badge'>{package}</span><span class='badge'>{archetype}</span><span class='badge-id'>{sample_id}</span></div></div></div>", "css_class": "header-modern" },
        "minimalist": { "template": "<div class='header header-minimalist'><div class='header-row'><div class='header-left'><small class='text-muted'>{manufacturer}</small><h1>{marketing_name}</h1></div><div class='header-right'><code>{sample_id}</code></div></div><p class='description-minimalist'>{description}</p><hr class='divider'><small class='tech-specs'>{package} • {archetype}</small></div>", "css_class": "header-minimalist" }
    }
    
    LAYOUT_CLASSES = ["layout-standard", "layout-two-column", "layout-grid"]
    WATERMARKS = ["DRAFT", "CONFIDENTIAL", "PRELIMINARY", "INTERNAL USE ONLY", None, None, None]

    TABLE_STRUCTURES = {
        "max_ratings_only": {"columns": ["Parameter", "symbol", "Value", "Unit", "Condition"]},
        "thermal_data": {"columns": ["Parameter", "symbol", "Typ", "Max", "Unit", "Condition"]},
        "compact_commercial": {"columns": ["Parameter", "Value", "Unit"]},
        "standard_full": {"columns": ["Parameter", "symbol", "Min", "Typ", "Max", "Unit", "Condition"]}
    }

    def _format_display_value(self, val: Any) -> str:
        """
        ✅ פונקציה מינימלית - רק HTML escaping, ללא שום עיבוד
        כל הערכים כבר עובדו ב-_format_row ונשמרו ב-JSON
        """
        if val is None or val == "" or val == "—": 
            return "—"
        
        # המרה פשוטה למחרוזת + HTML escaping
        return html.escape(str(val))
    
    def _get_enhanced_css(self, watermark: str = None) -> str:
        watermark_css = ""
        if watermark:
            watermark_css = f"body::before {{ content: '{watermark}'; position: fixed; top: 50%; left: 50%; transform: translate(-50%, -50%) rotate(-45deg); font-size: 120px; color: rgba(200, 200, 200, 0.15); font-weight: bold; z-index: 9999; pointer-events: none; }}"
        return f"body {{ background-color: white; font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; margin: 20px; line-height: 1.6; color: #333; }} {watermark_css} .layout-standard {{ max-width: 1200px; margin: 0 auto; }} .layout-two-column {{ max-width: 1400px; margin: 0 auto; }} .layout-two-column .sections-container {{ display: grid; grid-template-columns: 1fr 1fr; gap: 30px; margin-top: 20px; }} .layout-grid {{ max-width: 1600px; margin: 0 auto; }} .layout-grid .sections-container {{ display: grid; grid-template-columns: repeat(auto-fit, minmax(500px, 1fr)); gap: 25px; margin-top: 20px; }} .header {{ margin-bottom: 30px; padding: 20px; border-bottom: 2px solid #e0e0e0; }} .header-classic .product-title {{ font-size: 28px; margin: 10px 0; }} .header-modern {{ text-align: center; background: linear-gradient(135deg, #f5f7fa 0%, #c3cfe2 100%); border-radius: 8px; border: none; }} .header-modern h1 {{ font-size: 32px; margin: 15px 0; }} .header-modern .badge {{ background: white; padding: 6px 16px; border-radius: 20px; font-size: 12px; margin: 0 5px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); }} .header-minimalist {{ border-bottom: 1px solid #ddd; padding: 15px 0; }} .section {{ margin: 30px 0; background: white; padding: 20px; border-radius: 4px; box-shadow: 0 1px 3px rgba(0,0,0,0.1); }} .section-title {{ font-size: 20px; margin-bottom: 15px; border-bottom: 2px solid #3498db; padding-bottom: 8px; }} table {{ width: 100%; border-collapse: collapse; margin: 15px 0; font-size: 13px; }} th, td {{ padding: 10px; text-align: left; border: 1px solid #ddd; }} th {{ background-color: #f8f9fa; font-weight: 600; }} tr:nth-child(even) {{ background-color: #f9f9f9; }} .subheader td {{ background-color: #e8e8e8 !important; font-weight: bold; }} .features-apps {{ display: grid; grid-template-columns: 1fr 1fr; gap: 30px; margin: 30px 0; }} .features-apps ul {{ list-style: disc; padding-left: 20px; }} .image-placeholder {{ background: #f0f0f0; border: 2px dashed #ccc; padding: 40px; text-align: center; color: #999; margin: 20px 0; }} .footer-notes {{ margin-top: 40px; padding: 20px; background: #f8f9fa; border-radius: 4px; }}"
    def _render_features_and_apps(self, marketing_data: Dict) -> str:
        features = marketing_data.get("key_features", [])
        applications = marketing_data.get("typical_applications", [])
        if not features and not applications: return ""
        html_output = '<div class="features-apps">'
        if features:
            html_output += '<div class="features-section"><h3>Key Features</h3><ul>'
            for feature in features: html_output += f'<li>{html.escape(feature)}</li>'
            html_output += '</ul></div>'
        if applications:
            html_output += '<div class="apps-section"><h3>Typical Applications</h3><ul>'
            for app in applications: html_output += f'<li>{html.escape(app)}</li>'
            html_output += '</ul></div>'
        html_output += '</div>'
        return html_output
        
    def _get_random_image_placeholder(self, result_context) -> str:
        """
        ✅ FIX #3: Now accepts result.context to determine component type
        """
        # 30% סיכוי שלא יהיה גרף בכלל (כדי לשמור על גיוון בדאטה)
        if random.random() < 0.3:
            return ""

        # יצירת תמונת הגרף האקראית (Base64 HTML)
        try:
            component_type = result_context.component_type if result_context else None
            chart_html = RandomChartGenerator.generate_chart_image(component_type)
        except Exception as e:
            print(f"⚠️ Error generating chart: {e}")
            return ""
        
        # בחירת כותרת מתאימה לגרף
        caption = "Figure 1: Typical Performance Characteristics"
        
        # החזרת ה-HTML המלא עם התמונה והכותרת
        return f"""
        <div class="graph-container" style="margin: 25px auto; width: 80%; text-align: center; page-break-inside: avoid;">
            {chart_html}
            <div style="font-size: 13px; color: #555; margin-top: 8px; font-style: italic; font-family: sans-serif;">
                {caption}
            </div>
        </div>
        """

    def render(self, result: 'DatasheetResult', marketing_data: Dict) -> str:
        watermark = random.choice(self.WATERMARKS)
        layout_class = random.choice(self.LAYOUT_CLASSES)
        css_content = self._get_enhanced_css(watermark)
        
        layout_config = {
            "swap_symbol_position": random.random() < 0.3,
            "merge_min_typ_max": random.random() < 0.2,
            "use_alt_column_names": random.random() < 0.4,
            "use_subheaders": random.random() < 0.2
        }
        
        sections = {}
        for param in result.parameters:
            if param.section not in sections: sections[param.section] = []
            sections[param.section].append(param)
        
        body_content = self._render_header(result, marketing_data)
        body_content += self._render_features_and_apps(marketing_data)
        
        sections_html = '<div class="sections-container">'
        section_map = { "ABS_MAX": "max_ratings_only", "THERMAL": "thermal_data", "PACKAGE": "compact_commercial", "RELIABILITY": "standard_full" }

        for section_name, parameters in sections.items():
            struct_key = section_map.get(section_name, "standard_full")
            if "THERM" in section_name: struct_key = "thermal_data"
            elif "MAX" in section_name: struct_key = "max_ratings_only"
            elif "PKG" in section_name or "MECH" in section_name: struct_key = "compact_commercial"

            table_html = self._render_table(parameters, struct_key, layout_config)
            intro = marketing_data.get("section_intros", {}).get(section_name, "")
            
            # ✅ FIX #3: Only show graphs in ELECTRICAL_CHARACTERISTICS section
            graph_html = ""
            if "ELEC" in section_name.upper() or "CHAR" in section_name.upper():
                graph_html = self._get_random_image_placeholder(result.context)
            
            sections_html += f"<div class='section'><h2 class='section-title'>{html.escape(section_name.replace('_', ' ').title())}</h2><p><em>{html.escape(intro)}</em></p>{table_html}{graph_html}</div>"
        
        sections_html += '</div>'
        body_content += sections_html
        body_content += self._render_footer(marketing_data)

        return f"<!DOCTYPE html><html><head><meta charset='UTF-8'><style>{css_content}</style></head><body class='{layout_class}'>{body_content}</body></html>"

    def _render_header(self, result, marketing) -> str:
        header_type = random.choice(list(self.HEADER_STRUCTURES.keys()))
        config = self.HEADER_STRUCTURES[header_type]
        data = { "sample_id": html.escape(result.context.sample_id), "manufacturer": html.escape(marketing['manufacturer']), "marketing_name": html.escape(marketing['marketing_name']), "description": html.escape(marketing['description']), "package": html.escape(result.context.package), "archetype": html.escape(result.context.archetype) }
        return config["template"].format(**data)

    def _get_column_name_variant(self, base_name: str, use_alt: bool) -> str:
        if not use_alt: return base_name
        alternatives = { "Parameter": ["Characteristic", "Item", "Description"], "symbol": ["Sym.", "Symbol"], "Condition": ["Test Conditions", "Notes"], "Unit": ["Units"], "Min": ["Minimum", "Min."], "Typ": ["Typical", "Typ."], "Max": ["Maximum", "Max."], "Value": ["Rating", "Limit"] }
        return random.choice(alternatives.get(base_name, [base_name]))

    def _render_table(self, parameters: List[Any], structure_key: str, layout_config: Dict) -> str:
        """
        ✅ FIX #2 APPLIED: This method now ONLY uses data from the parameters list.
        No PackageCharacterizer calls, no random data generation, no "filling in" missing data.
        """
        structure = self.TABLE_STRUCTURES.get(structure_key, {"columns": ["Parameter", "symbol", "Min", "Typ", "Max", "Unit", "Condition"]})
        cols = list(structure["columns"])
        
        if layout_config["swap_symbol_position"] and "symbol" in cols and "Parameter" in cols:
            p, s = cols.index("Parameter"), cols.index("symbol")
            cols[p], cols[s] = cols[s], cols[p]
            
        if layout_config["merge_min_typ_max"] and all(c in cols for c in ["Min", "Typ", "Max"]):
            cols = [c for c in cols if c not in ["Min", "Typ", "Max"]]
            cols.insert(cols.index("Condition") if "Condition" in cols else len(cols), "Min/Typ/Max")

        display_cols = [(col, self._get_column_name_variant(col, layout_config["use_alt_column_names"])) for col in cols]
        
        html_output = "<table><thead><tr>"
        for _, name in display_cols: html_output += f"<th>{html.escape(name)}</th>"
        html_output += "</tr></thead><tbody>"
        
        prev_cat = None
        for param in parameters:
            if layout_config["use_subheaders"]:
                curr_cat = getattr(param, 'category', None) or getattr(param, 'section', None)
                if curr_cat and curr_cat != prev_cat:
                    html_output += f"<tr class='subheader'><td colspan='{len(cols)}'>{html.escape(curr_cat.replace('_', ' ').title())}</td></tr>"
                    prev_cat = curr_cat
            
            # ✅ Using the base class method to format the row
            row_data = self._format_row(param) 
            
            html_output += "<tr>"
            for logical_col, _ in display_cols:
                val = "—"
                if logical_col == "Parameter": val = row_data["Parameter"]
                elif logical_col == "symbol": val = row_data["symbol"]
                elif logical_col == "Condition": val = row_data["Condition"]
                elif logical_col == "Unit": val = row_data["Unit"]
                elif logical_col == "Min": val = row_data["Min"]
                elif logical_col == "Typ": val = row_data["Typ"]
                elif logical_col == "Max": val = row_data["Max"]
                elif logical_col == "Min/Typ/Max":
                    # ✅ ערכים כבר מעוצבים, רק צריך לחבר אותם
                    min_str = row_data['Min'] if row_data['Min'] else "—"
                    typ_str = row_data['Typ'] if row_data['Typ'] else "—"
                    max_str = row_data['Max'] if row_data['Max'] else "—"
                    val = f"{min_str} / {typ_str} / {max_str}"
                elif logical_col == "Value": 
                    val = row_data.get("Value") or row_data.get("Max") or row_data.get("Typ") or "—"
                
                # ✅ רק HTML escaping - ללא שום עיבוד נוסף של הערך
                display_val = html.escape(str(val)) if val and val != "—" else "—"
                
                html_output += f"<td>{display_val}</td>"
            html_output += "</tr>"
            
        return html_output + "</tbody></table>"

    def _render_footer(self, marketing):
        notes = "".join([f"<li>{html.escape(n)}</li>" for n in marketing.get('footnotes', [])])
        return f"<div class='footer-notes'><h3>Notes:</h3><ul>{notes}</ul></div>"

class ScanAugmentor:
    """
    מחלקה להוספת אפקטים של סריקה ובלייה לתמונות דפי נתונים.
    גרסה מתוקנת: מתמקדת רק בעיבוד תמונה (OpenCV) ללא תלות ב-Html2Image.
    """

    def __init__(self):
        # ✅ תיקון: הסרנו את האתחול של Html2Image מכאן.
        # המחלקה הזו מקבלת תמונה מוכנה ומלכלכת אותה.
        pass

    def apply_rotation(self, image, max_angle=2.0):
        """סיבוב קל המדמה הנחה עקומה בסורק"""
        if image is None: return None
        angle = random.uniform(-max_angle, max_angle)
        h, w = image.shape[:2]
        center = (w // 2, h // 2)
        
        # מטריצת הסיבוב
        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        # מילוי הרקע בלבן כדי שלא יהיו שוליים שחורים
        rotated = cv2.warpAffine(image, M, (w, h), borderValue=(255, 255, 255))
        return rotated

    def apply_blur(self, image):
        """טשטוש המדמה פוקוס לא מושלם בסורק"""
        if image is None: return None
        if random.random() < 0.5:
            # Gaussian Blur קל
            return cv2.GaussianBlur(image, (3, 3), 0)
        else:
            # טשטוש תנועה קל (Motion Blur)
            kernel_size = 3
            kernel = np.zeros((kernel_size, kernel_size))
            kernel[int((kernel_size-1)/2), :] = np.ones(kernel_size)
            kernel /= kernel_size
            return cv2.filter2D(image, -1, kernel)

    def apply_salt_and_pepper(self, image, prob=0.002):
        """רעש מלח-פלפל המדמה לכלוך על הזכוכית או אבק"""
        if image is None: return None
        output = np.copy(image)
        
        # רעש שחור (Pepper)
        num_pepper = np.ceil(prob * image.size * 0.5)
        coords = [np.random.randint(0, i - 1, int(num_pepper)) for i in image.shape]
        output[coords[0], coords[1], :] = 0
        
        # רעש לבן (Salt) - פחות קריטי על רקע לבן אבל קיים
        num_salt = np.ceil(prob * image.size * 0.5)
        coords = [np.random.randint(0, i - 1, int(num_salt)) for i in image.shape]
        output[coords[0], coords[1], :] = 255
        
        return output

    def apply_scan_artifacts(self, image):
        """הוספת הצללות או פסים של סורק (Scanner lines)"""
        if image is None: return None
        # הוספת "צל" הדרגתי (Vignette עדין)
        rows, cols = image.shape[:2]
        X_resultant_kernel = cv2.getGaussianKernel(cols, cols/2)
        Y_resultant_kernel = cv2.getGaussianKernel(rows, rows/2)
        kernel = Y_resultant_kernel * X_resultant_kernel.T
        mask = 255 * kernel / np.linalg.norm(kernel)
        
        # הפיכת המסכה שתהיה בהירה באמצע וכהה בקצוות
        mask = cv2.resize(mask, (cols, rows))
        mask = (mask - mask.min()) / (mask.max() - mask.min())
        mask = 0.8 + 0.2 * mask # שומר על הבהירות, רק מכהה קצוות בקטנה
        
        # החלה על כל הערוצים
        if len(image.shape) == 3:
            mask = mask[:, :, np.newaxis]
            
        return (image * mask).astype(np.uint8)
    
    def degrade_quality(self, image, quality=85):
        """שמירה כ-JPEG איכות נמוכה וטעינה מחדש (Compression Artifacts)"""
        if image is None: return None
        encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), quality]
        result, encimg = cv2.imencode('.jpg', image, encode_param)
        decimg = cv2.imdecode(encimg, 1)
        return decimg
        
class DatasheetJSONRenderer(BaseDatasheetRenderer):
    """
    מחלקה זו אחראית בלעדית על המרת הנתונים הגולמיים לפורמט JSON סופי.
    משתמשת בלוגיקת העזר של BaseDatasheetRenderer לטיפול ביחידות ופורמט.
    
    ✅ FIX #1 APPLIED: JSON output now uses formatted values from _format_row
    """

    def render(self, result: 'DatasheetResult', marketing_data: Dict = None) -> Dict[str, Any]:
        ground_truth = {}
        display_tables = {}
        seen_labels = set()

        for param in result.parameters:
            # ✅ FIX #1: First, perform the visual formatting (exactly like in HTML)
            # This function performs unit conversions (e.g., 2200 -> 2.2) and selects units (kΩ)
            formatted_row = self._format_row(param)
            
            # Extract the normalized values from the display for use in JSON
            # Note: We save strings ("2.2") not numbers to preserve exact format
            visual_min = formatted_row.get("Min")
            visual_typ = formatted_row.get("Typ")
            visual_max = formatted_row.get("Max")
            visual_unit = formatted_row.get("Unit")

            # ✅ FIX #1: Building Ground Truth that matches the image 100%
            ground_truth[param.key] = {
                "spec_type": self._get_enum_val(param.spec_type).lower(),
                "engineering_class": self._get_enum_val(param.engineering_class),
                "column_model": self._determine_column_model(param),
                "condition": param.condition,
                
                # The critical fix: saving unit and values as they are displayed
                "unit": visual_unit, 
                "min": visual_min,
                "typ": visual_typ,
                "max": visual_max,
                
                # Optional: saving the original value for debugging purposes
                "raw_value_typ": param.value_typ, 
                "raw_unit": param.unit
            }

            if param.section not in display_tables: display_tables[param.section] = []

            # No need to call _format_row again, we already have the formatted_row variable
            
            label_key = (param.section, formatted_row["Parameter"], param.condition)
            if label_key not in seen_labels:
                seen_labels.add(label_key)
                display_tables[param.section].append(formatted_row)

        return {
            "id": result.context.sample_id,
            "metadata": {
                "component_type": result.context.component_type,
                "package": result.context.package,
                "archetype": result.context.archetype,
                "process_corner": getattr(result.context, 'process_corner', "N/A")
            },
            "ground_truth": ground_truth,
            "display_data": display_tables,
            "marketing_generated": marketing_data if marketing_data else {}
        }

    def _determine_column_model(self, p) -> str:
        has_min = p.value_min is not None
        has_max = p.value_max is not None
        has_typ = p.value_typ is not None
        if has_min and has_max: return "MIN_MAX"
        if has_max and not has_min: return "MAX_ONLY"
        if has_min and not has_max: return "MIN_ONLY"
        if has_typ: return "TYP_ONLY"
        return "UNKNOWN"

    def _get_enum_val(self, val) -> str:
        if hasattr(val, 'name'): return val.name
        if hasattr(val, 'value'): return val.value
        return str(val)

class RandomChartGenerator:
    """
    מייצר גרפים הנדסיים גנריים (ללא קשר לנתונים) כדי למלא את המקום ויזואלית
    
    ✅ FIX #3 APPLIED: Now accepts component_type and forces Linear plots for Resistors
    """
    
    STYLES = ['seaborn-v0_8-darkgrid', 'bmh', 'ggplot', 'classic']
    LABELS = [
        ('Voltage (V)', 'Current (A)'),
        ('Frequency (Hz)', 'Gain (dB)'),
        ('Temperature (°C)', 'Power Dissipation (W)'),
        ('Time (µs)', 'Voltage (V)'),
        ('Load Current (mA)', 'Dropout Voltage (mV)')
    ]

    @staticmethod
    def generate_chart_image(component_type: str = None) -> str:
        """
        מייצר גרף ומחזיר אותו כ-HTML Base64 Image
        
        ✅ FIX #3: If component_type is RESISTOR, force linear IV curve
        """
        import base64
        
        # בחירת סגנון אקראי
        try:
            plt.style.use(random.choice(RandomChartGenerator.STYLES))
        except:
            plt.style.use('ggplot')
            
        # שינוי 1: הקטנת הפרופורציות (במקור היה 5, 3.5)
        # זה גורם לפונטים להיראות גדולים יותר יחסית לגרף, שזה טוב להקטנה
        fig, ax = plt.subplots(figsize=(4.5, 3), dpi=100)
        
        # יצירת דאטה פיקטיבי
        x = np.linspace(0, 10, 100)
        
        # ✅ FIX #3: Force linear chart for resistors
        if component_type and "RESISTOR" in component_type.upper():
            chart_type = 'linear'
        else:
            chart_type = random.choice(['linear', 'exp', 'log', 'saturation'])
        
        if chart_type == 'linear':
            y = random.uniform(0.1, 2) * x + random.uniform(0, 5)
        elif chart_type == 'exp':
            y = np.exp(x / random.uniform(2, 5))
        elif chart_type == 'log':
            y = np.log(x + 1) * random.uniform(1, 5)
        else: # saturation
            y = 10 * (1 - np.exp(-x))

        # הוספת רעש קל
        y = y + np.random.normal(0, 0.2, len(x))

        ax.plot(x, y, linewidth=2)
        
        # עיצוב
        labels = random.choice(RandomChartGenerator.LABELS)
        ax.set_xlabel(labels[0], fontsize=9)
        ax.set_ylabel(labels[1], fontsize=9)
        # הקטנת הכותרת
        ax.set_title(f"Typical Characteristics", fontsize=10)
        ax.grid(True, linestyle='--', alpha=0.7)
        
        # שמירה לזיכרון
        buf = io.BytesIO()
        plt.savefig(buf, format='png', bbox_inches='tight')
        plt.close(fig)
        
        # המרה ל-Base64
        data = base64.b64encode(buf.getvalue()).decode('utf-8')
        
        # שינוי 2: CSS מגביל רוחב (max-width) וממרכז (margin: 0 auto)
        # הוספתי div עוטף כדי להבטיח מרכוז
        return f"""
        <div style="text-align: center; width: 100%; margin: 10px 0;">
            <img src="data:image/png;base64,{data}" 
                 style="max-width: 60%; height: auto; border: 1px solid #ccc; box-shadow: 2px 2px 5px rgba(0,0,0,0.05);">
        </div>
        """

Overwriting synthetic_pipeline/generators.py


עכשיו יצירת מחלקת יורש עבור RESISTOR

In [3]:
%%writefile synthetic_pipeline/resistor_strategy.py

import random
import math
import uuid
from typing import Dict, Any, Optional, Tuple
from synthetic_pipeline.data_types import (
    GenerationContext,
    DatasheetResult,
    GeneratedParameter,
    SpecType,
    EngineeringClass,
)
from synthetic_pipeline.strategy_base import ComponentStrategy
# מניח שרשימות ה-E-Series נמצאות ב-utils, אם לא - צריך להחזיר אותן לקבועים
from synthetic_pipeline.strategy_utils import snap_to_e_series, apply_final_rounding

# ============================================================================
# CONSTANTS – PHYSICAL & INDUSTRIAL TRUTH
# ============================================================================

class ResistorConstants:
    """
    All constants here represent realistic manufacturer envelopes
    """
    # רשימות ערכים סטנדרטיים (אם חסר ב-utils, נגדיר כאן)
    E24_SERIES = [1.0, 1.1, 1.2, 1.3, 1.5, 1.6, 1.8, 2.0, 2.2, 2.4, 2.7, 3.0, 3.3, 3.6, 3.9, 4.3, 4.7, 5.1, 5.6, 6.2, 6.8, 7.5, 8.2, 9.1]

    PACKAGE_SPECS = {
        # ✅ תוקן: נתונים מדויקים לפי תקני IPC-7351 ו-IEC 60115
        # dims: L, W, H במילימטרים
        # weight: גרם (מבוסס על נפח × צפיפות קרמיקה)
        # P_max: וואט @ 70°C (derating curve)
        
        "01005": { 
            "dims": {"L": 0.4, "W": 0.2, "H": 0.13}, 
            "tol_mm": 0.02, 
            "weight_mg": 0.08,  # ✅ 0.08 מ"ג (80 מיקרוגרם)
            "limits": {"P_70C": 0.031, "V_max": 15, "V_over": 30, "I_max": 0.05}, 
            "R_range": {"min": 10, "max": 100_000} 
        },
        "0201":  { 
            "dims": {"L": 0.6, "W": 0.3, "H": 0.23}, 
            "tol_mm": 0.03, 
            "weight_mg": 0.2,   # ✅ 0.2 מ"ג
            "limits": {"P_70C": 0.05, "V_max": 25, "V_over": 50, "I_max": 0.1}, 
            "R_range": {"min": 10, "max": 200_000} 
        },
        "0402":  { 
            "dims": {"L": 1.0, "W": 0.5, "H": 0.35}, 
            "tol_mm": 0.05, 
            "weight_mg": 0.6,   # ✅ 0.6 מ"ג (המציאות!)
            "limits": {"P_70C": 0.063, "V_max": 50, "V_over": 100, "I_max": 0.15}, 
            "R_range": {"min": 1, "max": 1_000_000} 
        },
        "0603":  { 
            "dims": {"L": 1.6, "W": 0.8, "H": 0.45}, 
            "tol_mm": 0.1, 
            "weight_mg": 1.5,   # ✅ 1.5 מ"ג
            "limits": {"P_70C": 0.1, "V_max": 75, "V_over": 150, "I_max": 0.2}, 
            "R_range": {"min": 0.1, "max": 2_000_000} 
        },
        "0805":  { 
            "dims": {"L": 2.0, "W": 1.25, "H": 0.5}, 
            "tol_mm": 0.1, 
            "weight_mg": 3.2,   # ✅ 3.2 מ"ג
            "limits": {"P_70C": 0.125, "V_max": 150, "V_over": 300, "I_max": 0.3}, 
            "R_range": {"min": 0.05, "max": 10_000_000} 
        },
        "1206":  { 
            "dims": {"L": 3.2, "W": 1.6, "H": 0.55}, 
            "tol_mm": 0.2, 
            "weight_mg": 7.0,   # ✅ 7 מ"ג
            "limits": {"P_70C": 0.25, "V_max": 200, "V_over": 400, "I_max": 0.5}, 
            "R_range": {"min": 0.01, "max": 10_000_000} 
        },
        "1210":  { 
            "dims": {"L": 3.2, "W": 2.5, "H": 0.55}, 
            "tol_mm": 0.2, 
            "weight_mg": 11.0,  # ✅ 11 מ"ג
            "limits": {"P_70C": 0.5, "V_max": 200, "V_over": 400, "I_max": 1.0}, 
            "R_range": {"min": 0.01, "max": 10_000_000} 
        },
        "2512":  { 
            "dims": {"L": 6.4, "W": 3.2, "H": 0.55}, 
            "tol_mm": 0.2, 
            "weight_mg": 28.0,  # ✅ 28 מ"ג
            "limits": {"P_70C": 1.0, "V_max": 200, "V_over": 400, "I_max": 2.0}, 
            "R_range": {"min": 0.001, "max": 1_000_000} 
        },
        # THT Packages
        "Axial_1/4W": { 
            "dims": {"L": 6.5, "W": 2.3, "H": 2.3}, 
            "tol_mm": 0.3, 
            "weight_mg": 120.0,  # ✅ 120 מ"ג (גוף קרמי + חוטים)
            "limits": {"P_70C": 0.25, "V_max": 250, "V_over": 500, "I_max": 0.5}, 
            "R_range": {"min": 0.1, "max": 10_000_000} 
        },
    }

    # ✅ תוקן: Tolerance mapping מדויק יותר
    TOLERANCE_MAP = {
        "Standard_SMD": 5.0,           # ±5% (E24)
        "Precision_Thin_Film": 0.1,    # ±0.1% (E96/E192)
        "Thick_Film": 1.0,             # ±1% (E96)
        "Current_Sense": 1.0,          # ±1% (Low ohm, tight tolerance)
        "JUMPER": 0.0,                 # ✅ Zero-ohm has no tolerance
        "Standard_THT": 5.0,           # ±5% (E24)
        "High_Power": 5.0,             # ±5% (Wire-wound)
        "Metal_Film": 1.0,             # ±1% (E96)
    }

    # ✅ תוקן: TCR (Temperature Coefficient of Resistance) in ppm/°C
    TCR_MAP = {
        "Standard_SMD": 100,           # ±100 ppm/°C (Thick Film)
        "Precision_Thin_Film": 25,     # ±25 ppm/°C (Thin Film)
        "Thick_Film": 100,             # ±100 ppm/°C
        "Current_Sense": 50,           # ±50 ppm/°C (Metal alloy)
        "JUMPER": 0,                   # No TCR for jumpers
        "Standard_THT": 250,           # ±250 ppm/°C (Carbon Film)
        "High_Power": 50,              # ±50 ppm/°C (Wire-wound)
        "Metal_Film": 50,              # ±50 ppm/°C
    }

    # ✅ חדש: Maximum Operating Temperature
    MAX_TEMP_MAP = {
        "Standard_SMD": 125,
        "Precision_Thin_Film": 125,
        "Thick_Film": 125,
        "Current_Sense": 170,         # Higher for automotive
        "JUMPER": 125,
        "Standard_THT": 155,          # THT can handle more
        "High_Power": 275,            # Wire-wound high temp
        "Metal_Film": 155,
    }

# ============================================================================
# STRATEGY
# ============================================================================

class ResistorStrategy(ComponentStrategy):
    """
    Hermetic physics-based resistor generation.
    ✅ Improved Version: Accurate physics, realistic values, industry standards.
    """

    def create_context(self, schema: Dict, requested_corner: Optional[str]) -> GenerationContext:
        archetypes = schema.get("archetypes", ["Standard_SMD"])
        archetype = random.choice(archetypes)

        packages = list(ResistorConstants.PACKAGE_SPECS.keys())
        if "valid_packages" in schema:
            packages = [p for p in packages if p in schema["valid_packages"]]

        package = random.choice(packages)
        tolerance = ResistorConstants.TOLERANCE_MAP.get(archetype, 5.0)

        return GenerationContext(
            sample_id=str(uuid.uuid4())[:8],
            component_type="RESISTOR",
            package=package,
            archetype=archetype,
            tolerance=tolerance,
            process_corner=requested_corner or "TYPICAL",
            extras={}
        )

    # ----------------------------------------------------------------------
    # SINGLE SOURCE OF TRUTH – RESISTANCE
    # ----------------------------------------------------------------------
    def _get_or_create_resistance(self, context: GenerationContext) -> float:
        """
        Thread-safe resistance generation with E-series snapping.
        ✅ Improved: Better handling of jumpers and current sense resistors.
        """
        with context._lock:
            if "R" in context.extras:
                return context.extras["R"]

            pkg = ResistorConstants.PACKAGE_SPECS[context.package]

            # ✅ Zero-ohm jumper handling
            if context.archetype == "JUMPER":
                context.extras["R"] = 0.0
                return 0.0

            r_min = pkg["R_range"]["min"]
            r_max = pkg["R_range"]["max"]

            # ✅ Current sense resistors: favor low values (0.001Ω - 0.1Ω)
            if context.archetype == "Current_Sense":
                r_min = max(r_min, 0.001)
                r_max = min(r_max, 1.0)  # Limit to 1Ω max

            # Log-uniform distribution for realistic inventory spread
            exponent = random.uniform(math.log10(r_min), math.log10(r_max))
            raw = 10 ** exponent

            # ✅ E-series selection based on tolerance
            if context.tolerance <= 0.5:
                series = "E192"  # ±0.1% - ±0.5%
            elif context.tolerance <= 1.0:
                series = "E96"   # ±1%
            else:
                series = "E24"   # ±5%

            R = snap_to_e_series(raw, series)

            context.extras["R"] = R
            return R

    # ----------------------------------------------------------------------
    # REQUIRED IMPLEMENTATION: Create Custom Parameter
    # ----------------------------------------------------------------------
    def create_custom_parameter(self, key: str, context: GenerationContext, param_def: Dict) -> Optional[GeneratedParameter]:
        """
        ✅ Improved: Added more parameters and better handling.
        """
        if key == "resistance":
            # 1. Get the authoritative resistance value
            typ_v = self._get_or_create_resistance(context)
            
            # 2. Calculate min/max based on tolerance
            tol_dec = context.tolerance / 100.0
            min_v = typ_v * (1 - tol_dec) if typ_v > 0 else 0
            max_v = typ_v * (1 + tol_dec) if typ_v > 0 else 0

            # 3. Rounding for display
            def _clean(val): return float(f"{val:.4g}")
            
            # ✅ Special handling for jumpers
            if context.archetype == "JUMPER":
                return GeneratedParameter(
                    key="resistance",
                    label="Resistance (Jumper)",
                    symbol="R",
                    section="ELEC_CHAR",
                    value_typ=0.0,
                    value_max=0.05,  # Max 50mΩ for jumper
                    unit="Ω",
                    condition="Zero-ohm link",
                    spec_type=SpecType.NOMINAL,
                    engineering_class=EngineeringClass.PERFORMANCE
                )
            
            return GeneratedParameter(
                key="resistance",
                label="Resistance",
                symbol="R",
                section="ELEC_CHAR",
                value_min=_clean(min_v),
                value_typ=typ_v,
                value_max=_clean(max_v),
                unit="Ω",
                condition=f"T=25°C",
                spec_type=SpecType.NOMINAL,
                engineering_class=EngineeringClass.PERFORMANCE
            )

        if key == "tolerance":
            return GeneratedParameter(
                key="tolerance", 
                label="Resistance Tolerance", 
                symbol="Tol", 
                section="ELEC_CHAR",
                value_typ=context.tolerance, 
                unit="%",
                spec_type=SpecType.NOMINAL, 
                engineering_class=EngineeringClass.PERFORMANCE
            )

        if key == "max_operating_temp":
            # ✅ חדש: טמפרטורת הפעלה מקסימלית
            max_temp = ResistorConstants.MAX_TEMP_MAP.get(context.archetype, 125)
            return GeneratedParameter(
                key="max_operating_temp",
                label="Maximum Operating Temperature",
                symbol="T_max",
                section="ABS_MAX",
                value_typ=max_temp,
                unit="°C",
                spec_type=SpecType.MAX_RATING,
                engineering_class=EngineeringClass.RELIABILITY
            )
        
        return None

    # ----------------------------------------------------------------------
    # PHYSICS ENGINE
    # ----------------------------------------------------------------------
    def calculate_base_value(self, key: str, limits: Any, context: GenerationContext) -> Optional[float]:
        """
        ✅ Fixed: Corrected weight calculation (was off by 1000×)
        ✅ Improved: Added derating and realistic voltage limits
        """
        pkg = ResistorConstants.PACKAGE_SPECS[context.package]
        dims = pkg["dims"]
        lim = pkg["limits"]
        tol = pkg["tol_mm"]

        # ---- MECHANICAL ----
        if key == "length": 
            return dims["L"] + random.uniform(-tol, tol)
        
        if key == "width": 
            return dims["W"] + random.uniform(-tol, tol)
        
        if key == "height": 
            return dims["H"] + random.uniform(-tol / 2, tol / 2)
        
        if key == "weight":
            # ✅ CRITICAL FIX: Use pre-calculated weight from specs
            # Previous bug: calculated weight was off by 1000× (used wrong density units)
            base_weight_mg = pkg["weight_mg"]
            
            # Add ±5% manufacturing variation
            return base_weight_mg * random.uniform(0.95, 1.05)

        # ---- ELECTRICAL ----
        # Always get R first (Thread-safe)
        R = self._get_or_create_resistance(context)

        if key in ("rated_power", "power_rating"):
            # ✅ Power rating @ 70°C (standard derating point)
            return lim["P_70C"]

        if key == "max_power_25C":
            # ✅ חדש: הספק מקסימלי @ 25°C (before derating)
            # Typically 1.5-2× the 70°C rating
            return lim["P_70C"] * 1.6

        if key in ("rated_voltage", "max_working_voltage"):
            # ✅ Voltage limited by BOTH thermal (√PR) and package max
            # V = √(P×R), but cannot exceed V_max
            if R <= 0.001:  # Jumper or very low resistance
                return lim["V_max"] * 0.1  # Jumpers rated very low voltage
            
            v_thermal = math.sqrt(lim["P_70C"] * R)
            return min(v_thermal, lim["V_max"])

        if key == "max_overload_voltage":
            # ✅ Overload voltage (short-term, non-operating)
            # Typically 2-2.5× working voltage, limited by package
            v_work = self.calculate_base_value("rated_voltage", limits, context)
            if v_work is None:
                v_work = lim["V_max"]
            return min(v_work * 2.5, lim["V_over"])

        if key in ("rated_current", "max_current"):
            # ✅ I = √(P/R), limited by package max
            if R <= 0.001:  
                return lim["I_max"]  # Jumper: use package limit
            
            i_thermal = math.sqrt(lim["P_70C"] / R)
            return min(i_thermal, lim["I_max"])

        if key == "tcr":
            # Temperature Coefficient of Resistance (ppm/°C)
            return ResistorConstants.TCR_MAP.get(context.archetype, 100)

        if key == "max_element_temp":
            # ✅ חדש: טמפרטורת אלמנט מקסימלית (בדרך כלל +20-30°C ממקסימום הפעלה)
            max_op = ResistorConstants.MAX_TEMP_MAP.get(context.archetype, 125)
            return max_op + 25  # Element can be hotter than ambient

        return None

    # ----------------------------------------------------------------------
    # FINAL CONSISTENCY GUARDS
    # ----------------------------------------------------------------------
    # ----------------------------------------------------------------------
    # FINAL CONSISTENCY GUARDS
    # ----------------------------------------------------------------------
    def apply_correlations(self, result: DatasheetResult) -> None:
        """
        ✅ Improved: Better physics validation and warnings
        """
        # המרה למילון כדי שנוכל לשלוף לפי מפתח
        params_map = {p.key: p for p in result.parameters}
        
        try:
            # --- התיקון: שימוש ב-params_map במקום ב-result ---
            
            # בדיקה אם הפרמטר קיים לפני שניגשים ל-value_typ
            r_param = params_map.get("resistance")
            p_param = params_map.get("rated_power")
            
            if not r_param or not p_param:
                return

            R = r_param.value_typ
            P = p_param.value_typ
            
            # Get voltage and current
            V_param = params_map.get("rated_voltage") # תיקון כאן
            I_param = params_map.get("rated_current") # תיקון כאן
            
            if V_param is None or I_param is None:
                return
            
            V = V_param.value_typ
            I = I_param.value_typ
            
            # ✅ Physics Check (Skip for Jumpers)
            if R > 0.01:  # Skip very low resistances
                # Ohm's law: V = I × R
                expected_I = V / R if R > 0 else 0
                
                # הגנה מפני חלוקה ב-0 או מספרים אפסיים
                denom_i = max(expected_I, 1e-9)
                i_error = abs(expected_I - I) / denom_i
                
                if i_error > 0.05:  # 5% tolerance
                    print(f"⚠️  Physics mismatch in {result.context.sample_id}:")
                    print(f"    R={R}Ω, V={V}V, I={I}A")
                    print(f"    Expected I={expected_I:.3f}A (error: {i_error*100:.1f}%)")
                
                # Power check: P = V × I = V² / R = I² × R
                expected_P_from_V = V * V / R
                expected_P_from_I = I * I * R
                
                denom_p = max(P, 1e-9)
                p_error = abs(expected_P_from_V - P) / denom_p
                
                if p_error > 0.1:  # 10% tolerance
                    print(f"⚠️  Power rating issue in {result.context.sample_id}:")
                    print(f"    P_rated={P}W, P_from_V²/R={expected_P_from_V:.3f}W")
                    print(f"    (This is expected due to derating)")
            
            # ✅ Current sense resistor validation
            if result.context.archetype == "Current_Sense":
                if R > 0.1:
                    print(f"⚠️  Current sense resistor too high: {R}Ω (should be <0.1Ω)")
            
        except Exception as e:
            print(f"⚠️  Correlation check failed: {e}")

Overwriting synthetic_pipeline/resistor_strategy.py


יצירת מחלקה יורשת לרכיב CAPACITOR


In [15]:
%%writefile synthetic_pipeline/capacitor_strategy.py

import random
import math
import uuid
import re
from typing import Dict, Any, Optional, List, Tuple

# ==========================================
# 1. PIPELINE IMPORTS
# ==========================================
from synthetic_pipeline.data_types import (
    GenerationContext, 
    DatasheetResult, 
    GeneratedParameter,
    SpecType, 
    EngineeringClass,
)

from synthetic_pipeline.strategy_utils import snap_to_e_series
from synthetic_pipeline.strategy_base import ComponentStrategy

# ============================================================================
# CONSTANTS & PHYSICS SPECS
# ============================================================================
class CapacitorConstants:
    """Constants and lookup tables for capacitor generation"""
    
    # ------------------------------------------------------------------------
    # MASTER PACKAGE SPECIFICATIONS
    # dims: [Length, Width, Height, Weight(g)]
    # limits: {max_v: Volt, cv: uF*V product}
    # esl: Equivalent Series Inductance (nH) - Critical for SRF calculation
    # ------------------------------------------------------------------------
    PACKAGE_SPECS = {
        # --- CERAMIC MLCC ---
        "0201": { "dims": [0.6, 0.3, 0.3, 0.0003], "limits": { "max_v": 25,  "cv": 100 },  "esl": 0.2 },
        "0402": { "dims": [1.0, 0.5, 0.5, 0.001],  "limits": { "max_v": 50,  "cv": 500 },  "esl": 0.4 },
        "0603": { "dims": [1.6, 0.8, 0.8, 0.002],  "limits": { "max_v": 100, "cv": 2200 }, "esl": 0.6 },
        "0805": { "dims": [2.0, 1.25, 1.25, 0.005],"limits": { "max_v": 100, "cv": 10000 },"esl": 0.8 },
        "1206": { "dims": [3.2, 1.6, 1.6, 0.010],  "limits": { "max_v": 200, "cv": 22000 },"esl": 1.0 },
        "1210": { "dims": [3.2, 2.5, 2.5, 0.015],  "limits": { "max_v": 250, "cv": 47000 },"esl": 1.2 },
        
        # --- ELECTROLYTIC / POLYMER (RADIAL) ---
        "Radial_5mm":   { "dims": [5.0, 5.0, 11.0, 0.4], "limits": { "max_v": 50, "cv": 22000 },  "esl": 3.0 },
        "Radial_6.3mm": { "dims": [6.3, 6.3, 11.0, 0.6], "limits": { "max_v": 63, "cv": 50000 },  "esl": 4.0 },
        "Radial_8mm":   { "dims": [8.0, 8.0, 12.0, 1.2], "limits": { "max_v": 100,"cv": 100000 }, "esl": 5.0 },
        "Radial_10mm":  { "dims": [10.0, 10.0, 16.0, 2.0],"limits": { "max_v": 100,"cv": 200000 }, "esl": 6.0 },

        # --- TANTALUM / SMD MOLDED ---
        "Case_A": { "dims": [3.2, 1.6, 1.6, 0.02], "limits": { "max_v": 25, "cv": 2200 }, "esl": 1.2 },
        "Case_B": { "dims": [3.5, 2.8, 1.9, 0.05], "limits": { "max_v": 35, "cv": 4700 }, "esl": 1.5 },
        "Case_C": { "dims": [6.0, 3.2, 2.6, 0.10], "limits": { "max_v": 50, "cv": 10000 },"esl": 1.8 },
        "Case_D": { "dims": [7.3, 4.3, 2.9, 0.15], "limits": { "max_v": 50, "cv": 22000 },"esl": 2.2 },
        
        # --- SUPERCAPACITOR ---
        "Coin_Type": { "dims": [19.0, 19.0, 5.0, 2.5], "limits": { "max_v": 5.5, "cv": 20000000 }, "esl": 10.0 },
    }
    
    # Dissipation Factor (DF) - The chemistry factor for ESR calculation
    DF_MAP = {
        "Ceramic_MLCC": 0.02,        # Low loss
        "Ceramic_C0G": 0.001,        # Ultra low loss
        "Electrolytic_Alum": 0.15,   # Higher loss
        "Electrolytic_Polymer": 0.05,# Low ESR
        "Tantalum_Solid": 0.06,
        "Film_Polypropylene": 0.001,
        "Supercapacitor": 1.0        # High internal resistance relative to cap
    }
    
    TOLERANCE_MAP = {
        "Ceramic_MLCC": 0.10, "Electrolytic_Alum": 0.20, "Tantalum_Solid": 0.10, 
        "Film_Polypropylene": 0.05, "Supercapacitor": 0.30
    }

# ============================================================================
# STRATEGY CLASS
# ============================================================================
class CapacitorStrategy(ComponentStrategy):
    """
    Hermetic Capacitor Strategy.
    Physics Chain: Package -> Limits (CV) -> Voltage -> Capacitance -> ESR/SRF (Derived)
    """

    def __init__(self):
        self.constants = CapacitorConstants()

    # ------------------------------------------------------------------------
    # 1. CREATE CONTEXT (THE HERMETIC SELECTION)
    # ------------------------------------------------------------------------
    def create_context(self, schema: Dict, requested_corner: Optional[str]) -> GenerationContext:
        archetype = random.choice(schema.get('archetypes', ["Ceramic_MLCC"]))
        
        # Filter valid packages
        valid_packages = list(self.constants.PACKAGE_SPECS.keys())
        if "valid_packages" in schema:
             valid_packages = [p for p in valid_packages if p in schema["valid_packages"]]
        
        target_package = random.choice(valid_packages)
        tolerance = self.constants.TOLERANCE_MAP.get(archetype, 0.20)
        
        # --- CRITICAL: Link Voltage & Capacitance Physically ---
        v_rated, c_val = self._generate_physics_compliant_values(archetype, target_package, schema)
        
        extras = {
            'voltage_rating': v_rated,
            'capacitance_value': c_val,
            'is_polar': archetype in ['Electrolytic_Alum', 'Tantalum_Solid', 'Supercapacitor']
        }

        return GenerationContext(
            sample_id=str(uuid.uuid4())[:8],
            component_type="CAPACITOR",
            package=target_package,
            archetype=archetype,
            tolerance=tolerance,
            process_corner=requested_corner or "TYPICAL",
            extras=extras
        )

    # ------------------------------------------------------------------------
    # 2. HELPER: CV PRODUCT LOGIC
    # ------------------------------------------------------------------------
    def _generate_physics_compliant_values(self, archetype: str, package: str, schema: Dict) -> Tuple[float, float]:
        """
        Generates Voltage and Capacitance that FIT inside the chosen package.
        Formula: Capacitance * Voltage <= Max_CV_Product_of_Package
        """
        pkg_data = self.constants.PACKAGE_SPECS.get(package)
        if not pkg_data: return 50.0, 1e-9 # Fallback

        limits = pkg_data['limits']
        max_v_pkg = limits['max_v']
        max_cv_si = limits['cv'] * 1e-6 # Convert uF*V to F*V

        # 1. Pick a Voltage
        std_volts = [4.0, 6.3, 10, 16, 25, 35, 50, 63, 100, 200, 250, 400, 630]
        valid_std = [v for v in std_volts if v <= max_v_pkg]
        v_rated = random.choice(valid_std) if valid_std else max_v_pkg

        # 2. Calculate Max Capacitance allowed
        max_cap_si = max_cv_si / v_rated

        # 3. Pick Capacitance (Log-Uniform)
        min_cap_si = 1e-12 
        if "Super" in archetype: min_cap_si = 0.1
        
        if max_cap_si < min_cap_si: max_cap_si = min_cap_si * 1.1

        log_min = math.log10(min_cap_si)
        log_max = math.log10(max_cap_si)
        c_final = 10 ** random.uniform(log_min, log_max)

        # Snap to E-series
        c_final = snap_to_e_series(c_final, "E12")
        
        return v_rated, c_final

    # ------------------------------------------------------------------------
    # 3. CREATE PARAMETERS
    # ------------------------------------------------------------------------
    def create_custom_parameter(self, key: str, context: GenerationContext, param_def: Dict) -> Optional[GeneratedParameter]:
        
        if key == 'capacitance':
            c_val = context.extras['capacitance_value']
            tol = context.tolerance
            disp_val, disp_unit = self._auto_scale(c_val, 'F')
            
            return GeneratedParameter(
                key='capacitance', label="Capacitance", symbol="C", section="ELEC_CHAR",
                value_min=self._round(disp_val * (1-tol)),
                value_typ=self._round(disp_val),
                value_max=self._round(disp_val * (1+tol)),
                unit=disp_unit,
                condition="1kHz, 25°C",
                spec_type=SpecType.NOMINAL, engineering_class=EngineeringClass.PERFORMANCE
            )

        if key == 'rated_voltage':
            v_val = context.extras['voltage_rating']
            return GeneratedParameter(
                key='rated_voltage', label="Rated Voltage", symbol="V_R", section="ABS_MAX",
                value_typ=v_val, unit="V",
                spec_type=SpecType.MAX_RATING, engineering_class=EngineeringClass.SAFETY_LIMIT
            )

        return None

    # ------------------------------------------------------------------------
    # 4. PHYSICS ENGINE: BASE VALUES (DIMS & SPECS)
    # ------------------------------------------------------------------------
    def calculate_base_value(self, key: str, limits: Any, context: GenerationContext) -> Any:
        pkg_data = self.constants.PACKAGE_SPECS.get(context.package)
        if not pkg_data: return None
        dims = pkg_data['dims']
        
        if key == 'length': return dims[0]
        if key == 'width': return dims[1]
        if key == 'height': return dims[2]
        if key == 'weight': return dims[3]
        if key == 'esl': return pkg_data.get('esl', 0.5)
        if key == 'dissipation_factor': return self.constants.DF_MAP.get(context.archetype, 0.05)

        return None

    # ------------------------------------------------------------------------
    # 5. DERIVED PHYSICS (CORRELATIONS) - THE OPTIMIZED LOGIC
    # ------------------------------------------------------------------------
    def apply_correlations(self, result: DatasheetResult) -> None:
        """
        Logic:
        - ESR derived from DF and Freq.
        - SRF derived from ESL and C.
        - Ripple Current derived from Volume (Thermal Limit) and ESR.
        """
        ctx = result.context
        
        # Safety & Data Extraction
        if "capacitance_value" not in ctx.extras: return
        C = ctx.extras["capacitance_value"]
        if C <= 0: return

        # Fetch constants safely
        pkg_data = self.constants.PACKAGE_SPECS.get(ctx.package, {})
        ESL = pkg_data.get("esl", 1.0) * 1e-9  # nH to H
        DF = self.constants.DF_MAP.get(ctx.archetype, 0.05)

        for p in result.parameters:

            # --- ESR Calculation ---
            if p.key == "esr":
                # Ceramics are tested at higher freq typically
                is_ceramic = "Ceramic" in ctx.archetype
                freq = 100e3 if is_ceramic else 1e3 
                
                # Formula: ESR = tan(delta) / (2*pi*f*C)
                esr_ohm = DF / (2 * math.pi * freq * C)
                
                # Add parasitic resistance
                parasitic = 0.005 if is_ceramic else 0.05
                esr_ohm += parasitic

                # Auto-scale Unit
                if esr_ohm < 1.0:
                    p.value_typ = self._round(esr_ohm * 1000)
                    p.unit = "mΩ"
                else:
                    p.value_typ = self._round(esr_ohm)
                    p.unit = "Ω"
                
                p.condition = f"{int(freq/1000)}kHz"

            # --- SRF Calculation (Resonance) ---
            if p.key == "self_resonant_freq":
                # Formula: f = 1 / (2*pi*sqrt(LC))
                # 
                if ESL > 0:
                    f_res = 1.0 / (2 * math.pi * math.sqrt(ESL * C))
                    p.value_typ = round(f_res / 1e6, 1) # MHz
                    p.unit = "MHz"

            # --- Ripple Current Calculation ---
            if p.key == "ripple_current":
                # 1. Recalculate ESR at switching freq (100kHz)
                esr_at_ripple_freq = DF / (2 * math.pi * 100e3 * C)
                if esr_at_ripple_freq < 0.001: esr_at_ripple_freq = 0.001 # prevent zero div

                # 2. Estimate Max Power Dissipation based on Volume
                dims = pkg_data.get("dims", [1, 1, 1])
                vol_mm3 = dims[0] * dims[1] * dims[2]
                
                # Thermal limit estimation: ~10mW per mm^3
                p_max_watts = vol_mm3 * 0.01 
                
                # 3. Calculate Current limit: I = sqrt(P / R)
                i_ripple = math.sqrt(p_max_watts / esr_at_ripple_freq)
                p.value_typ = int(i_ripple * 1000) # mA
                p.unit = "mA"

    # ------------------------------------------------------------------------
    # UTILS
    # ------------------------------------------------------------------------
    def _auto_scale(self, val, unit):
        if val < 1e-9: return val * 1e12, "pF"
        if val < 1e-6: return val * 1e9, "nF"
        if val < 1e-3: return val * 1e6, "µF"
        if val < 1:    return val * 1e3, "mF"
        return val, unit

    def _round(self, val):
        if val is None: return None
        if val < 10: return round(val, 2)
        return int(val)

Overwriting synthetic_pipeline/capacitor_strategy.py


יצירת מחלקה יורשת עבור DIODE

In [4]:
%%writefile synthetic_pipeline/diode_strategy.py

import random
import math
import uuid
from typing import Dict, Any, Optional

# ==========================================
# PIPELINE IMPORTS
# ==========================================
from synthetic_pipeline.data_types import (
    GenerationContext,
    DatasheetResult,
    GeneratedParameter,
    SpecType,
)

from synthetic_pipeline.strategy_base import ComponentStrategy


# ============================================================================
# CONSTANTS CLASS
# ============================================================================
class DiodeConstants:
    """Constants and lookup tables for diode generation"""

    PACKAGE_SPECS = {
        # SMD Packages - ערכים מדויקים לפי תקני IPC
        "SOD-323": { "dims": [1.7, 1.25, 0.95, 0.004], "limits": { "max_p": 0.2, "max_i": 0.25, "rth_ja": 650 } },
        "SOD-123": { "dims": [2.7, 1.6, 1.15, 0.010], "limits": { "max_p": 0.4, "max_i": 1.0,  "rth_ja": 350 } },
        "SOT-23":  { "dims": [2.9, 1.3, 1.0, 0.008],  "limits": { "max_p": 0.35,"max_i": 0.6,  "rth_ja": 400 } },

        # Through-Hole Packages
        "DO-35":   { "dims": [3.8, 1.7, 1.7, 0.13],   "limits": { "max_p": 0.5, "max_i": 0.5,  "rth_ja": 250 } },
        "DO-41":   { "dims": [5.2, 2.7, 2.7, 0.35],   "limits": { "max_p": 1.0, "max_i": 1.5,  "rth_ja": 100 } },
        "DO-201":  { "dims": [9.5, 5.3, 5.3, 1.20],   "limits": { "max_p": 5.0, "max_i": 5.0,  "rth_ja": 40  } },

        # Power Packages
        "TO-220":  { "dims": [10.0, 4.5, 15.0, 2.0],  "limits": { "max_p": 50,  "max_i": 20,   "rth_ja": 60  } },
        "TO-247":  { "dims": [16.0, 5.0, 21.0, 6.0],  "limits": { "max_p": 150, "max_i": 60,   "rth_ja": 40  } },

        # Surface Mount Power
        "SMA":     { "dims": [4.6, 2.6, 2.1, 0.06],   "limits": { "max_p": 1.0, "max_i": 1.5,  "rth_ja": 120 } },
        "SMB":     { "dims": [5.4, 3.6, 2.3, 0.10],   "limits": { "max_p": 3.0, "max_i": 3.0,  "rth_ja": 90  } },
        "SMC":     { "dims": [7.9, 5.9, 2.3, 0.21],   "limits": { "max_p": 5.0, "max_i": 5.0,  "rth_ja": 75  } }
    }

    ARCHETYPE_PHYSICS = {
        # ✅ תוקן: ערכי Vf ו-Ir מבוססים על פיזיקה אמיתית של חצי-מוליכים
        "Power_Rectifier":     { "vf_base": 0.95, "ir_factor": 1.0,   "trr_base": 2000, "is_schottky": False, "is_zener": False },
        "Schottky_Barrier":    { "vf_base": 0.45, "ir_factor": 100.0, "trr_base": 0,    "is_schottky": True,  "is_zener": False },
        "Fast_Recovery":       { "vf_base": 1.1,  "ir_factor": 5.0,   "trr_base": 150,  "is_schottky": False, "is_zener": False },
        "Ultra_Fast_Recovery": { "vf_base": 1.3,  "ir_factor": 10.0,  "trr_base": 50,   "is_schottky": False, "is_zener": False },
        "Zener":               { "vf_base": 1.0,  "ir_factor": 2.0,   "trr_base": None, "is_schottky": False, "is_zener": True  },
        "TVS_Transient":       { "vf_base": 3.3,  "ir_factor": 5.0,   "trr_base": None, "is_schottky": False, "is_zener": False },
        "Signal_Small_Signal": { "vf_base": 0.7,  "ir_factor": 0.1,   "trr_base": 4,    "is_schottky": False, "is_zener": False },
        "Switching":           { "vf_base": 0.85, "ir_factor": 0.5,   "trr_base": 4,    "is_schottky": False, "is_zener": False },
        "PIN_Diode":           { "vf_base": 0.95, "ir_factor": 0.01,  "trr_base": 1000, "is_schottky": False, "is_zener": False },
        "Avalanche":           { "vf_base": 0.9,  "ir_factor": 1.0,   "trr_base": 1500, "is_schottky": False, "is_zener": False }
    }

    # ✅ חדש: ערכי Zener סטנדרטיים לפי סדרת E24
    STANDARD_ZENER_VOLTAGES = [
        2.4, 2.7, 3.0, 3.3, 3.6, 3.9, 4.3, 4.7, 5.1, 5.6, 6.2, 6.8, 7.5, 
        8.2, 9.1, 10, 11, 12, 13, 15, 16, 18, 20, 22, 24, 27, 30, 33, 36, 
        39, 43, 47, 51, 56, 62, 68, 75, 82, 91, 100
    ]

    # ✅ חדש: ערכי מתח נדחפים סטנדרטיים
    STANDARD_REVERSE_VOLTAGES = [
        20, 30, 40, 50, 60, 75, 100, 150, 200, 300, 400, 600, 800, 1000, 
        1200, 1400, 1600, 2000
    ]

    # ✅ חדש: מקדמי טמפרטורה לזרם דליפה (Ir @ 125°C / Ir @ 25°C)
    TEMP_COEFFICIENT_IR = {
        "Power_Rectifier": 50,      # Si PN junction: x50 @ 125°C
        "Schottky_Barrier": 200,    # Schottky: x100-300 @ 125°C
        "Fast_Recovery": 40,
        "Ultra_Fast_Recovery": 35,
        "Zener": 30,
        "TVS_Transient": 25,
        "Signal_Small_Signal": 60,
        "Switching": 50,
        "PIN_Diode": 80,           # PIN very sensitive
        "Avalanche": 45
    }


# ============================================================================
# STRATEGY
# ============================================================================
class DiodeStrategy(ComponentStrategy):

    def __init__(self):
        self.constants = DiodeConstants()

    # ------------------------------------------------------------------
    # CONTEXT
    # ------------------------------------------------------------------
    def create_context(self, schema: Dict, requested_corner: Optional[str]) -> GenerationContext:
        archetype = random.choice(schema.get("archetypes", ["Power_Rectifier"]))

        valid_packages = list(self.constants.PACKAGE_SPECS.keys())
        if "valid_packages" in schema:
            valid_packages = [p for p in valid_packages if p in schema["valid_packages"]]

        package = random.choice(valid_packages)
        ratings = self._generate_physics_compliant_ratings(archetype, package)

        return GenerationContext(
            sample_id=str(uuid.uuid4())[:8],
            component_type="DIODE",
            package=package,
            archetype=archetype,
            tolerance=0.05,
            process_corner=requested_corner or "TYPICAL",
            extras={
                "rated_current": ratings["if_rated"],
                "rated_voltage": ratings["vr_rated"],
                "power_rating": ratings["p_max"],
                "physics_meta": self.constants.ARCHETYPE_PHYSICS[archetype],
            }
        )

    # ------------------------------------------------------------------
    # RATINGS ENGINE (✅ שופר)
    # ------------------------------------------------------------------
    def _generate_physics_compliant_ratings(self, archetype: str, package: str) -> Dict[str, float]:
        pkg = self.constants.PACKAGE_SPECS[package]["limits"]
        meta = self.constants.ARCHETYPE_PHYSICS[archetype]

        # ✅ תוקן: חישוב תרמי מדויק יותר
        # P = Vf × If → If_max = P_max / Vf
        vf_typical = meta["vf_base"]
        thermal_limit_current = pkg["max_p"] / vf_typical
        
        # גם מגבלת זרם של האריזה
        max_i = min(pkg["max_i"], thermal_limit_current * 0.8)  # Safety margin
        
        # ✅ תוקן: ערכי זרם סטנדרטיים E6/E12
        std_i = [0.05, 0.1, 0.15, 0.2, 0.3, 0.5, 0.75, 1.0, 1.5, 2.0, 3.0, 5.0, 10.0, 15.0, 20.0, 30.0, 60.0]
        
        valid_i = [i for i in std_i if i <= max_i]
        if not valid_i: 
            valid_i = [round(max_i * 0.7, 2)]  # Derating
        
        if_rated = random.choice(valid_i)

        # ✅ תוקן: בחירת מתחים סטנדרטיים
        if meta["is_zener"]:
            # Zener מוגבל לפי הספק
            max_zener = min(100, pkg["max_p"] * 20)  # Rough estimate
            vr = random.choice([v for v in self.constants.STANDARD_ZENER_VOLTAGES if v <= max_zener])
        else:
            # Schottky מוגבל ל-200V בדרך כלל
            max_v = 200 if meta["is_schottky"] else 2000
            vr = random.choice([v for v in self.constants.STANDARD_REVERSE_VOLTAGES if v <= max_v])

        return {"if_rated": if_rated, "vr_rated": vr, "p_max": pkg["max_p"]}

    # ------------------------------------------------------------------
    # PARAMETERS
    # ------------------------------------------------------------------
    def create_custom_parameter(self, key: str, context: GenerationContext, param_def: Dict):

        meta = context.extras["physics_meta"]
        if_rated = context.extras["rated_current"]
        vr_rated = context.extras["rated_voltage"]

        if key == "forward_current":
            return GeneratedParameter(
                key=key, label="Average Forward Current", symbol="I_F(AV)",
                section="ABS_MAX", value_typ=if_rated, unit="A",
                spec_type=SpecType.MAX_RATING
            )

        if key in ("reverse_voltage", "zener_voltage"):
            if ("zener" in key) != meta["is_zener"]:
                return None
            return GeneratedParameter(
                key=key,
                label="Rated Voltage",
                symbol="V_Z" if meta["is_zener"] else "V_RRM",
                section="ABS_MAX",
                value_typ=vr_rated,
                unit="V",
                spec_type=SpecType.MAX_RATING
            )

        if key == "forward_voltage":
            # ✅ תוקן: Vf עולה עם זרם (לוגריתמית)
            # Vf @ If = Vf_base + n*Vt*ln(If/If_ref)
            # n ≈ 1-2 (ideality factor), Vt ≈ 26mV @ 25°C
            vf_base = meta["vf_base"]
            current_factor = math.log10(max(if_rated, 0.1)) * 0.05  # Simplified
            vf_at_rated = vf_base + current_factor
            
            # Variation ±5%
            vf = vf_at_rated * random.uniform(0.97, 1.03)
            
            return GeneratedParameter(
                key=key, label="Forward Voltage", symbol="V_F",
                section="ELEC_CHAR",
                value_typ=round(vf, 2),
                value_max=round(vf * 1.15, 2),  # Spec tolerance
                unit="V",
                condition=f"I_F={if_rated}A, T_A=25°C"
            )

        if key == "reverse_current":
            return GeneratedParameter(
                key=key, label="Reverse Leakage Current", symbol="I_R",
                section="ELEC_CHAR",
                value_max=0.0,  # filled in correlations
                unit="µA"
            )

        if key == "reverse_current_hot":
            # ✅ חדש: זרם דליפה בטמפרטורה גבוהה
            return GeneratedParameter(
                key=key, label="Reverse Leakage Current (Hot)", symbol="I_R",
                section="ELEC_CHAR",
                value_max=0.0,  # filled in correlations
                unit="µA"
            )

        if key == "surge_current":
            # ✅ חדש: זרם הלם (קריטי לדיודות כוח)
            return GeneratedParameter(
                key=key, label="Non-Repetitive Peak Forward Surge Current", symbol="I_FSM",
                section="ABS_MAX",
                value_typ=0.0,  # filled in correlations
                unit="A",
                spec_type=SpecType.MAX_RATING
            )

        if key == "zener_impedance":
            # ✅ חדש: עכבת זנר (קריטי עבור Zener diodes)
            if not meta["is_zener"]:
                return None
            return GeneratedParameter(
                key=key, label="Zener Impedance", symbol="Z_z",
                section="ELEC_CHAR",
                value_typ=0.0,  # filled in correlations
                unit="Ω"
            )

        if key == "zener_test_current":
            # ✅ חדש: זרם בדיקה לזנר
            if not meta["is_zener"]:
                return None
            return GeneratedParameter(
                key=key, label="Zener Test Current", symbol="I_ZT",
                section="ELEC_CHAR",
                value_typ=0.0,  # filled in correlations
                unit="mA"
            )

        if key == "junction_capacitance":
            return GeneratedParameter(
                key=key, label="Junction Capacitance", symbol="C_j",
                section="ELEC_CHAR",
                value_typ=0.0,
                unit="pF"
            )

        if key == "reverse_recovery_time":
            if meta["is_schottky"]:
                return None
            return GeneratedParameter(
                key=key, label="Reverse Recovery Time", symbol="t_rr",
                section="ELEC_CHAR",
                value_typ=0,
                unit="ns"
            )

        return None

    # ------------------------------------------------------------------
    # DIMS
    # ------------------------------------------------------------------
    def calculate_base_value(self, key: str, limits: Any, context: GenerationContext):
        dims = self.constants.PACKAGE_SPECS[context.package]["dims"]
        noise = random.uniform(0.98, 1.02)  # Manufacturing tolerance
        
        mapping = {"length": dims[0], "width": dims[1], "height": dims[2], "weight": dims[3]}
        if key in mapping:
            return mapping[key] * noise
        return None

    # ------------------------------------------------------------------
    # CORRELATIONS (✅ שופר משמעותית)
    # ------------------------------------------------------------------
    def apply_correlations(self, result: DatasheetResult):
        ctx = result.context
        meta = ctx.extras.get("physics_meta", {})
        if_rated = ctx.extras.get("rated_current")
        vr_rated = ctx.extras.get("rated_voltage")
        p_max = ctx.extras.get("power_rating")
        
        if if_rated is None or vr_rated is None:
            print(f"   ⚠️  Skipping correlations for {ctx.sample_id}: missing rated_current or rated_voltage")
            return
        
        for p in result.parameters:
            if p.key == "reverse_current":
                # ✅ פיזיקה מדויקת של זרם דליפה @ 25°C
                ir_factor = meta.get("ir_factor", 1.0)
                
                if ir_factor is not None:
                    area_factor = math.sqrt(if_rated)
                    
                    if meta.get("is_schottky"):
                        base_leakage_uA = 5.0 * area_factor * ir_factor
                    else:
                        base_leakage_uA = 0.1 * area_factor * ir_factor
                    
                    # Auto-scale units
                    if base_leakage_uA < 1.0:
                        p.value_max = round(base_leakage_uA * 1000, 1)
                        p.unit = "nA"
                    elif base_leakage_uA >= 1000:
                        p.value_max = round(base_leakage_uA / 1000, 3)
                        p.unit = "mA"
                    else:
                        p.value_max = round(base_leakage_uA, 2)
                        p.unit = "µA"
                        
                    p.condition = f"V_R={vr_rated}V, T_A=25°C"
            
            elif p.key == "reverse_current_hot":
                # ✅ חדש: זרם דליפה @ 125°C (קריטי לשוטקי!)
                ir_factor = meta.get("ir_factor", 1.0)
                archetype = ctx.archetype
                temp_mult = self.constants.TEMP_COEFFICIENT_IR.get(archetype, 50)
                
                if ir_factor is not None:
                    area_factor = math.sqrt(if_rated)
                    
                    if meta.get("is_schottky"):
                        base_leakage_uA = 5.0 * area_factor * ir_factor
                    else:
                        base_leakage_uA = 0.1 * area_factor * ir_factor
                    
                    # Apply temperature multiplication
                    hot_leakage_uA = base_leakage_uA * temp_mult
                    
                    # Auto-scale units
                    if hot_leakage_uA < 1.0:
                        p.value_max = round(hot_leakage_uA * 1000, 1)
                        p.unit = "nA"
                    elif hot_leakage_uA >= 1000:
                        p.value_max = round(hot_leakage_uA / 1000, 3)
                        p.unit = "mA"
                    else:
                        p.value_max = round(hot_leakage_uA, 2)
                        p.unit = "µA"
                        
                    p.condition = f"V_R={vr_rated}V, T_A=125°C"
            
            elif p.key == "surge_current":
                # ✅ חדש: זרם הלם I_FSM ≈ 10-30× If_rated
                # Smaller diodes: higher ratio, Power diodes: lower ratio
                if if_rated < 1.0:
                    surge_ratio = random.uniform(25, 35)  # Signal/small
                elif if_rated < 5.0:
                    surge_ratio = random.uniform(15, 25)  # Medium
                else:
                    surge_ratio = random.uniform(10, 15)  # Power
                
                i_fsm = if_rated * surge_ratio
                p.value_typ = round(i_fsm, 1)
                p.condition = "t=8.3ms (1/2 cycle @ 60Hz), T_A=25°C"
            
            elif p.key == "zener_impedance":
                # ✅ חדש: עכבת זנר Z_z @ I_ZT
                # Z_z inversely proportional to power rating
                # Low voltage zeners: higher impedance
                if p_max is not None and vr_rated is not None:
                    # Empirical formula from datasheets:
                    # Z_z ≈ Vz / (10 × P_max) for medium power zeners
                    base_z = vr_rated / (10 * p_max)
                    
                    # Low voltage zeners have higher Z_z
                    if vr_rated < 5.0:
                        base_z *= 3.0
                    elif vr_rated < 10.0:
                        base_z *= 1.5
                    
                    # Add variation ±30%
                    zz = base_z * random.uniform(0.7, 1.3)
                    
                    # Auto-scale units
                    if zz < 1.0:
                        p.value_typ = round(zz * 1000, 1)
                        p.unit = "mΩ"
                    elif zz >= 1000:
                        p.value_typ = round(zz / 1000, 2)
                        p.unit = "kΩ"
                    else:
                        p.value_typ = round(zz, 1)
                        p.unit = "Ω"
                    
                    p.condition = f"I_ZT (see below)"
            
            elif p.key == "zener_test_current":
                # ✅ חדש: I_ZT - זרם הבדיקה של הזנר
                # Typically I_ZT ≈ 0.1 × P_max / V_z (10% of max power)
                if p_max is not None and vr_rated is not None:
                    izt_mA = (0.1 * p_max / vr_rated) * 1000  # Convert to mA
                    
                    # Round to standard values
                    standard_izt = [0.25, 0.5, 1, 2, 5, 10, 20, 50, 100, 200, 500]
                    p.value_typ = min(standard_izt, key=lambda x: abs(x - izt_mA))
                    p.condition = f"V_Z={vr_rated}V ±5%"
            
            elif p.key == "junction_capacitance":
                # ✅ Cj ∝ Area / sqrt(Vr)
                if if_rated is not None:
                    cj_base = 20 * math.sqrt(if_rated)
                    
                    vr_test = 4.0
                    vbi = 0.7
                    cj_at_vr = cj_base / math.sqrt(1 + vr_test / vbi)
                    
                    p.value_typ = round(cj_at_vr, 1)
                    p.condition = "V_R=4V, f=1MHz"
            
            elif p.key == "reverse_recovery_time":
                trr_base = meta.get("trr_base")
                
                if trr_base is not None and if_rated is not None:
                    size_factor = math.sqrt(if_rated) if if_rated > 0.1 else 1.0
                    trr_calculated = trr_base * size_factor
                    
                    trr_final = trr_calculated * random.uniform(0.8, 1.2)
                    
                    p.value_typ = int(trr_final)
                    p.condition = f"I_F={if_rated}A, di/dt=100A/µs"

Overwriting synthetic_pipeline/diode_strategy.py


מחלקה יורשת עבור MOSFET


In [17]:
%%writefile synthetic_pipeline/mosfet_strategy.py

import random
import math
import uuid
import re
from typing import Dict, Any, Optional, List, Tuple
from dataclasses import dataclass

# ==========================================
# 1. PIPELINE IMPORTS
# ==========================================
from synthetic_pipeline.data_types import (
    GenerationContext, 
    DatasheetResult, 
    GeneratedParameter,
    SpecType, 
    EngineeringClass,
)

from synthetic_pipeline.strategy_base import ComponentStrategy


# ============================================================================
# CONSTANTS CLASS - SINGLE SOURCE OF TRUTH
# ============================================================================
class MosfetConstants:
    """Constants and lookup tables for MOSFET generation"""
    
    # ========================================================================
    # MASTER PACKAGE SPECIFICATIONS - SINGLE SOURCE OF TRUTH
    # ========================================================================
    PACKAGE_SPECS = {
        # ====================================================================
        # HV_Power_THT (Through-Hole)
        # ====================================================================
        "TO-220": {
            "dimensions": [10.0, 4.5, 9.0, 2.0],
            "limits": { "max_power_watts": 150, "max_current_a": 70, "max_voltage_v": 1000, "rth_jc": 1.0, "rth_ja": 62.5 }
        },
        "TO-220AB": {
            "dimensions": [10.0, 4.5, 9.0, 2.0],
            "limits": { "max_power_watts": 150, "max_current_a": 70, "max_voltage_v": 1000, "rth_jc": 1.0, "rth_ja": 62.5 }
        },
        "TO-247": {
            "dimensions": [16.0, 5.0, 21.0, 6.0],
            "limits": { "max_power_watts": 300, "max_current_a": 100, "max_voltage_v": 1200, "rth_jc": 0.5, "rth_ja": 40.0 }
        },
        "TO-247-3": {
            "dimensions": [16.0, 5.0, 21.0, 6.0],
            "limits": { "max_power_watts": 300, "max_current_a": 100, "max_voltage_v": 1700, "rth_jc": 0.5, "rth_ja": 40.0 }
        },
        "TO-247-4": {
            "dimensions": [16.0, 5.0, 21.0, 6.0],
            "limits": { "max_power_watts": 300, "max_current_a": 100, "max_voltage_v": 1700, "rth_jc": 0.5, "rth_ja": 40.0 }
        },
        "TO-3P": {
            "dimensions": [15.6, 4.8, 19.9, 5.5],
            "limits": { "max_power_watts": 250, "max_current_a": 80, "max_voltage_v": 1000, "rth_jc": 0.7, "rth_ja": 40.0 }
        },
        "TO-264": {
            "dimensions": [20.0, 5.0, 26.0, 10.0],
            "limits": { "max_power_watts": 500, "max_current_a": 150, "max_voltage_v": 1500, "rth_jc": 0.2, "rth_ja": 30.0 }
        },
        "ITO-220": {
            "dimensions": [10.0, 4.5, 9.0, 1.8],
            "limits": { "max_power_watts": 50, "max_current_a": 30, "max_voltage_v": 1500, "rth_jc": 3.0, "rth_ja": 65.0 }
        },
        "TO-220FP": {
            "dimensions": [10.0, 4.5, 9.0, 1.8],
            "limits": { "max_power_watts": 40, "max_current_a": 30, "max_voltage_v": 1500, "rth_jc": 4.0, "rth_ja": 65.0 }
        },

        # ====================================================================
        # MV_Power_SMD (Surface Mount Power)
        # ====================================================================
        "DPAK(TO-252)": {
            "dimensions": [6.6, 6.1, 2.3, 0.35],
            "limits": { "max_power_watts": 60, "max_current_a": 30, "max_voltage_v": 200, "rth_jc": 1.5, "rth_ja": 50.0 }
        },
        "TO-252": {
            "dimensions": [6.6, 6.1, 2.3, 0.35],
            "limits": { "max_power_watts": 60, "max_current_a": 30, "max_voltage_v": 200, "rth_jc": 1.5, "rth_ja": 50.0 }
        },
        "D2PAK(TO-263)": {
            "dimensions": [10.0, 9.0, 4.5, 1.5],
            "limits": { "max_power_watts": 150, "max_current_a": 70, "max_voltage_v": 600, "rth_jc": 0.8, "rth_ja": 40.0 }
        },
        "TO-263-7": {
            "dimensions": [10.0, 9.0, 4.5, 1.5],
            "limits": { "max_power_watts": 180, "max_current_a": 80, "max_voltage_v": 1200, "rth_jc": 0.7, "rth_ja": 40.0 }
        },
        "D2PAK-7L": {
            "dimensions": [10.0, 9.0, 4.5, 1.5],
            "limits": { "max_power_watts": 180, "max_current_a": 80, "max_voltage_v": 1200, "rth_jc": 0.7, "rth_ja": 40.0 }
        },
        "PowerPAK_SO-8": {
            "dimensions": [6.15, 5.15, 1.0, 0.1],
            "limits": { "max_power_watts": 50, "max_current_a": 40, "max_voltage_v": 150, "rth_jc": 1.5, "rth_ja": 50.0 }
        },
        "PQFN_5x6": {
            "dimensions": [5.0, 6.0, 0.9, 0.08],
            "limits": { "max_power_watts": 30, "max_current_a": 30, "max_voltage_v": 150, "rth_jc": 2.0, "rth_ja": 50.0 }
        },
        "DFN_5x6": {
            "dimensions": [5.0, 6.0, 0.9, 0.08],
            "limits": { "max_power_watts": 30, "max_current_a": 30, "max_voltage_v": 150, "rth_jc": 2.0, "rth_ja": 50.0 }
        },
        "DFN_8x8": {
            "dimensions": [8.0, 8.0, 0.9, 0.15],
            "limits": { "max_power_watts": 80, "max_current_a": 60, "max_voltage_v": 650, "rth_jc": 1.0, "rth_ja": 40.0 }
        },
        "TOLL": {
            "dimensions": [10.0, 11.7, 2.3, 0.6],
            "limits": { "max_power_watts": 300, "max_current_a": 120, "max_voltage_v": 200, "rth_jc": 0.4, "rth_ja": 40.0 }
        },
        "DirectFET": {
            "dimensions": [6.3, 4.9, 0.7, 0.1],
            "limits": { "max_power_watts": 40, "max_current_a": 35, "max_voltage_v": 100, "rth_jc": 1.0, "rth_ja": 45.0 }
        },

        # ====================================================================
        # LV_Logic_Level (Small Signal)
        # ====================================================================
        "SOT-23": {
            "dimensions": [2.9, 1.3, 1.0, 0.008],
            "limits": { "max_power_watts": 0.35, "max_current_a": 3.0, "max_voltage_v": 60, "rth_jc": 100, "rth_ja": 350 }
        },
        "SOT-23-3": {
            "dimensions": [2.9, 1.3, 1.0, 0.008],
            "limits": { "max_power_watts": 0.35, "max_current_a": 3.0, "max_voltage_v": 60, "rth_jc": 100, "rth_ja": 350 }
        },
        "SOT-23-6": {
            "dimensions": [2.9, 1.6, 1.1, 0.015],
            "limits": { "max_power_watts": 0.3, "max_current_a": 2.0, "max_voltage_v": 60, "rth_jc": 110, "rth_ja": 300 }
        },
        "SOT-323": {
            "dimensions": [2.0, 1.25, 0.95, 0.005],
            "limits": { "max_power_watts": 0.2, "max_current_a": 1.0, "max_voltage_v": 50, "rth_jc": 150, "rth_ja": 500 }
        },
        "SC-70": {
            "dimensions": [2.0, 1.25, 0.95, 0.005],
            "limits": { "max_power_watts": 0.2, "max_current_a": 1.0, "max_voltage_v": 50, "rth_jc": 150, "rth_ja": 500 }
        },
        "SC-70-6": {
            "dimensions": [2.0, 1.25, 0.95, 0.006],
            "limits": { "max_power_watts": 0.15, "max_current_a": 0.8, "max_voltage_v": 50, "rth_jc": 160, "rth_ja": 550 }
        },
        "SOT-363": {
            "dimensions": [2.0, 1.25, 0.95, 0.006],
            "limits": { "max_power_watts": 0.15, "max_current_a": 0.8, "max_voltage_v": 50, "rth_jc": 160, "rth_ja": 550 }
        },
        "SOT-223": {
            "dimensions": [6.5, 3.5, 1.6, 0.12],
            "limits": { "max_power_watts": 1.5, "max_current_a": 5.0, "max_voltage_v": 100, "rth_jc": 15, "rth_ja": 60 }
        },
        "SOT-89": {
            "dimensions": [4.5, 2.5, 1.5, 0.05],
            "limits": { "max_power_watts": 1.0, "max_current_a": 4.0, "max_voltage_v": 100, "rth_jc": 20, "rth_ja": 100 }
        },
        "TSOP-6": {
            "dimensions": [2.9, 1.6, 1.0, 0.015],
            "limits": { "max_power_watts": 1.0, "max_current_a": 3.0, "max_voltage_v": 60, "rth_jc": 50, "rth_ja": 150 }
        },
        "DFN_2x2": {
            "dimensions": [2.0, 2.0, 0.6, 0.01],
            "limits": { "max_power_watts": 0.8, "max_current_a": 3.0, "max_voltage_v": 30, "rth_jc": 40, "rth_ja": 100 }
        },
        "DFN_3x3": {
            "dimensions": [3.3, 3.3, 0.9, 0.04],
            "limits": { "max_power_watts": 1.5, "max_current_a": 10, "max_voltage_v": 60, "rth_jc": 15, "rth_ja": 60 }
        },
        "SON": {
            "dimensions": [3.0, 3.0, 0.8, 0.03],
            "limits": { "max_power_watts": 1.2, "max_current_a": 8, "max_voltage_v": 40, "rth_jc": 20, "rth_ja": 70 }
        },

        # ====================================================================
        # RF_Power
        # ====================================================================
        "Flange_Mount": {
            "dimensions": [20.0, 6.0, 4.0, 5.0],
            "limits": { "max_power_watts": 300, "max_current_a": 50, "max_voltage_v": 100, "rth_jc": 0.3, "rth_ja": 30.0 }
        },
        "NI-780": {
            "dimensions": [20.0, 10.0, 4.0, 3.0],
            "limits": { "max_power_watts": 200, "max_current_a": 40, "max_voltage_v": 120, "rth_jc": 0.4, "rth_ja": 35.0 }
        },
        "NI-1230": {
            "dimensions": [32.0, 12.0, 5.0, 8.0],
            "limits": { "max_power_watts": 500, "max_current_a": 80, "max_voltage_v": 150, "rth_jc": 0.2, "rth_ja": 25.0 }
        }
    }
    
    # Physics Scaling Factors (Voltage -> Resistance penalty)
    RDS_VOLTAGE_SCALING = {
        "HV_Power_THT": 2.5, "MV_Power_SMD": 2.0, "LV_Logic_Level": 1.2,
        "SiC_High_Voltage": 2.2, "GaN_High_Frequency": 1.5, "RF_Power": 2.0,
        "Dual_N_Channel": 1.5, "Dual_P_Channel": 1.8,
        "Complementary_Pair": 1.6, "Depletion_Mode": 2.0
    }
    
    # Gate Threshold Defaults
    VGS_TH_DEFAULTS = {
        "HV_Power_THT": 3.0, "MV_Power_SMD": 2.0, "LV_Logic_Level": 1.5,
        "SiC_High_Voltage": 3.0, "GaN_High_Frequency": 1.4,
        "Dual_P_Channel": -2.0, "Depletion_Mode": -2.0
    }

    TECH_FOM = {
        "Low_Voltage_Trench": 0.5,   # Great for <100V
        "Planar_HV": 8.0,            # Old tech, high Rds for HV
        "SuperJunction_HV": 2.5,     # Modern HV tech
        "P_Channel_Standard": 2.0,   # P-Channel always has higher Rds than N-Channel
        "SiC_Standard": 1.5,         # Very low Rds for HV
        "GaN_Standard": 1.0          # Extremely efficient
    }
    
    # 🆕 Gate Capacitance Density (pF/mm²)
    CAP_DENSITY = {
        "Low_Voltage_Trench": 400,    # High density (thin oxide)
        "Planar_HV": 100,             # Low density (thick oxide)
        "SuperJunction_HV": 150,
        "SiC_Standard": 80,           # Wide bandgap = thicker oxide
        "GaN_Standard": 60,
        "P_Channel_Standard": 200
    }


# ============================================================================
# STRATEGY CLASS
# ============================================================================
class MosfetStrategy(ComponentStrategy):
    """Hermetic MOSFET Strategy - FIXED VERSION"""

    def __init__(self):
        self.constants = MosfetConstants()

    # ----------------------------------------------------------------------
    # CONTEXT - The Lock-in Phase
    # ----------------------------------------------------------------------
    def create_context(self, schema: Dict, requested_corner: Optional[str]) -> GenerationContext:
        archetype = random.choice(schema.get("archetypes", ["MV_Power_SMD"]))
        
        # 1. Filter Packages
        valid_packages = list(self.constants.PACKAGE_SPECS.keys())
        if "valid_packages" in schema:
            valid_packages = [p for p in valid_packages if p in schema["valid_packages"]]
        
        # Ensure small archetypes don't get huge packages
        if "LV" in archetype or "Logic" in archetype:
            valid_packages = [p for p in valid_packages if "TO-247" not in p]
            
        # Fallback
        if not valid_packages:
            valid_packages = ["SOT-23"]

        package = random.choice(valid_packages)

        # 2. Physics Lock: Generate Voltage & Resistance Base
        phys_data = self._generate_physics_values(archetype, package)

        extras = {
            "vdss": phys_data["vdss"],
            "id_rated": phys_data["id_rated"],
            "rds_on_target": phys_data["rds_on"],
            "is_p_channel": "P_Channel" in archetype,
            "tech_type": phys_data["tech_type"],
            "ciss_est": phys_data["ciss_est"],  # 🆕 Pre-calculate for reuse
            "rg_internal": phys_data["rg_internal"]  # 🆕 Internal gate resistance
        }

        return GenerationContext(
            sample_id=f"MOS_{uuid.uuid4().hex[:8]}",
            component_type="MOSFET",
            package=package,
            archetype=archetype,
            tolerance=0.1,
            process_corner=requested_corner or "TYPICAL",
            extras=extras,
        )

    # ----------------------------------------------------------------------
    # PHYSICS ENGINE (IMPROVED)
    # ----------------------------------------------------------------------
    def _generate_physics_values(self, archetype: str, package: str) -> Dict[str, Any]:
        pkg_limits = self.constants.PACKAGE_SPECS[package]["limits"]
        
        # A. Choose Voltage (Vdss)
        is_hv = "HV" in archetype or pkg_limits["max_voltage_v"] > 200
        possible_volts = [20, 30, 40, 60, 100, 200, 400, 600, 650, 800, 1000, 1200, 1700]
        
        valid_volts = [v for v in possible_volts if v <= pkg_limits["max_voltage_v"]]
        
        if is_hv: 
            valid_volts = [v for v in valid_volts if v >= 200]
        else: 
            valid_volts = [v for v in valid_volts if v < 200]
        
        if not valid_volts: 
            valid_volts = [pkg_limits["max_voltage_v"]] 
        vdss = random.choice(valid_volts)

        # B. Choose Technology & Calculate Rds(on)
        is_p_channel = "P_Channel" in archetype
        if is_p_channel: 
            tech = "P_Channel_Standard"
        elif "SiC" in archetype:
            tech = "SiC_Standard"
        elif "GaN" in archetype:
            tech = "GaN_Standard"
        elif vdss <= 100: 
            tech = "Low_Voltage_Trench"
        elif vdss >= 500: 
            tech = "SuperJunction_HV"
        else: 
            tech = "Planar_HV"
            
        tech_fom = self.constants.TECH_FOM[tech]
        
        # Estimate Die Size available in package
        die_size_factor = pkg_limits["max_power_watts"] / 50.0 
        
        # Physics: Rds * Area ~ V^2.0
        base_rds = tech_fom * (vdss / 50.0)**2.0 * 0.01 
        
        rds_on = base_rds / max(die_size_factor, 0.05)
        
        # C. Calculate Max Current (Id) with THERMAL VALIDATION
        # Thermal current limit
        i_thermal = math.sqrt(pkg_limits["max_power_watts"] / rds_on)
        
        # 🔧 VALIDATION: Check if junction temp stays below limit
        p_actual = (i_thermal ** 2) * rds_on
        t_rise = p_actual * pkg_limits["rth_ja"]
        tj_estimated = 25 + t_rise
        
        # If junction temp exceeds 150°C, reduce current
        if tj_estimated > 150:
            i_thermal_limited = math.sqrt((150 - 25) / (rds_on * pkg_limits["rth_ja"]))
            i_thermal = min(i_thermal, i_thermal_limited)
        
        id_rated = min(i_thermal, pkg_limits["max_current_a"])
        id_rated = float(f"{id_rated:.1f}")
        
        # D. 🆕 Calculate Gate Capacitance (Ciss)
        # Physics: Ciss ∝ Die_Area
        # Die area estimated from Id_rated and Rds
        die_area_mm2 = id_rated * rds_on * 100  # Rough estimate
        
        cap_density = self.constants.CAP_DENSITY.get(tech, 200)
        ciss_est = die_area_mm2 * cap_density
        ciss_est = min(max(ciss_est, 50), 50000)  # Realistic range 50pF - 50nF
        
        # E. 🆕 Internal Gate Resistance (affects switching speed)
        rg_internal = 1.0 + vdss / 200  # Ω, higher voltage = higher Rg
        
        return {
            "vdss": vdss,
            "id_rated": id_rated,
            "rds_on": rds_on,
            "tech_type": tech,
            "ciss_est": ciss_est,
            "rg_internal": rg_internal
        }

    # ----------------------------------------------------------------------
    # PARAMETER GENERATION (ENHANCED)
    # ----------------------------------------------------------------------
    def create_custom_parameter(self, key: str, context: GenerationContext, param_def: Dict):
        extras = context.extras
        vdss = extras["vdss"]
        id_rated = extras["id_rated"]
        rds_on = extras["rds_on_target"]
        ciss_est = extras["ciss_est"]
        rg_int = extras["rg_internal"]
        
        if key == "vdss":
            return self._abs_max("vdss", vdss, "V", "V_DSS", context)

        if key == "id_cont":
            return self._abs_max("id_cont", id_rated, "A", "I_D", context)

        if key == "rds_on":
            val_typ = rds_on * random.uniform(0.9, 1.0)
            val_max = rds_on * 1.3
            
            disp_val, disp_unit = self._auto_scale(val_max, "Ω")
            return GeneratedParameter(
                key="rds_on", label="Drain-Source On Resistance", symbol="R_DS(on)",
                section="ELEC_CHAR",
                value_typ=round(self._auto_scale(val_typ, "Ω")[0], 2),
                value_max=round(disp_val, 2),
                unit=disp_unit,
                condition=f"V_GS=10V, I_D={round(id_rated/2, 1)}A",
                spec_type=SpecType.MAX_RATING, engineering_class=EngineeringClass.PERFORMANCE
            )

        if key == "vgs_th":
            base = self.constants.VGS_TH_DEFAULTS.get(context.archetype, 2.0)
            if extras["is_p_channel"]: 
                base = -abs(base)
            
            # 🔧 FIXED: Realistic variation ±0.5V (not percentage)
            return GeneratedParameter(
                key="vgs_th", label="Gate Threshold Voltage", symbol="V_GS(th)",
                section="ELEC_CHAR",
                value_min=round(base - 0.5, 1),
                value_typ=round(base, 1),
                value_max=round(base + 0.5, 1),
                unit="V", condition="V_DS=V_GS, I_D=250µA",
                spec_type=SpecType.NOMINAL
            )
        
        # 🆕 Gate Capacitances
        if key == "ciss":
            return GeneratedParameter(
                key="ciss", label="Input Capacitance", symbol="C_iss",
                section="ELEC_CHAR",
                value_typ=int(ciss_est),
                unit="pF",
                condition="V_DS=25V, V_GS=0V, f=1MHz",
                spec_type=SpecType.NOMINAL
            )
        
        if key == "coss":
            # Coss (output cap) typically 30-50% of Ciss
            coss = ciss_est * random.uniform(0.3, 0.5)
            return GeneratedParameter(
                key="coss", label="Output Capacitance", symbol="C_oss",
                section="ELEC_CHAR",
                value_typ=int(coss),
                unit="pF",
                condition="V_DS=25V, V_GS=0V, f=1MHz",
                spec_type=SpecType.NOMINAL
            )
        
        if key == "crss":
            # Crss (reverse transfer, Miller cap) typically 5-15% of Ciss
            crss = ciss_est * random.uniform(0.05, 0.15)
            return GeneratedParameter(
                key="crss", label="Reverse Transfer Capacitance", symbol="C_rss",
                section="ELEC_CHAR",
                value_typ=int(crss),
                unit="pF",
                condition="V_DS=25V, V_GS=0V, f=1MHz",
                spec_type=SpecType.NOMINAL
            )
        
        if key == "qg":
            # 🔧 FIXED: Proper Gate Charge Physics
            # Qg = Ciss × ΔVgs + Cgd × ΔVds (Miller charge dominant)
            # For typical gate drive: 0→10V with Vds=Vdss/2
            ciss_nf = ciss_est / 1000  # Convert to nF
            cgd_nf = (ciss_est * 0.1) / 1000  # Cgd ≈ 10% of Ciss
            
            qg_total = ciss_nf * 10 + cgd_nf * (vdss / 2)
            
            return GeneratedParameter(
                key="qg", label="Total Gate Charge", symbol="Q_g",
                section="ELEC_CHAR",
                value_typ=round(qg_total, 1),
                unit="nC",
                condition=f"V_DS={vdss/2:.0f}V, V_GS=10V, I_D={round(id_rated/2, 1)}A",
                spec_type=SpecType.NOMINAL
            )
        
        # 🆕 Switching Times
        if key == "td_on":
            # Turn-on delay: td(on) ∝ Rg_internal × Ciss
            td = rg_int * (ciss_est / 1000) * 2  # ns
            return self._typ_max_obj("td_on", round(td, 1), "ns", "t_d(on)")
        
        if key == "tr":
            # Rise time: tr ∝ Rg × (Ciss + Cgd)
            tr = rg_int * (ciss_est / 1000) * 1.5
            return self._typ_max_obj("tr", round(tr, 1), "ns", "t_r")
        
        if key == "td_off":
            # Turn-off delay (longer than td_on)
            td_off = rg_int * (ciss_est / 1000) * 3
            return self._typ_max_obj("td_off", round(td_off, 1), "ns", "t_d(off)")
        
        if key == "tf":
            # Fall time
            tf = rg_int * (ciss_est / 1000) * 2
            return self._typ_max_obj("tf", round(tf, 1), "ns", "t_f")
        
        # 🆕 Body Diode Parameters
        if key == "vsd":
            # Body diode forward voltage
            vsd = 0.7 if vdss <= 100 else (1.0 if vdss <= 600 else 1.2)
            return GeneratedParameter(
                key="vsd", label="Source-Drain Diode Voltage", symbol="V_SD",
                section="ELEC_CHAR",
                value_typ=vsd, value_max=round(vsd * 1.2, 1),
                unit="V", condition=f"I_S={round(id_rated/2, 1)}A",
                spec_type=SpecType.NOMINAL
            )
        
        if key == "trr":
            # Reverse recovery time (HV devices slower)
            trr_base = 50 if vdss <= 100 else (200 if vdss <= 600 else 500)
            return self._typ_max_obj("trr", trr_base, "ns", "t_rr")
        
        if key == "qrr":
            # Reverse recovery charge: Qrr ∝ trr × Id
            trr_base = 50 if vdss <= 100 else (200 if vdss <= 600 else 500)
            qrr = (trr_base * 1e-9) * (id_rated / 2) * 1e9  # nC
            return self._typ_max_obj("qrr", round(qrr, 1), "nC", "Q_rr")
        
        # 🆕 Temperature Coefficient
        if key == "rds_on_temp_coeff":
            # Positive temp coefficient: SiC lower than Silicon
            tc = 0.3 if "SiC" in extras["tech_type"] else 0.6
            
            return GeneratedParameter(
                key="rds_on_temp_coeff",
                label="Rds(on) Temperature Coefficient",
                symbol="α_Rds",
                section="ELEC_CHAR",
                value_typ=tc,
                unit="%/°C",
                condition="25°C to 175°C",
                spec_type=SpecType.NOMINAL
            )

        return None

    # ----------------------------------------------------------------------
    # BASE VALUES (ENHANCED)
    # ----------------------------------------------------------------------
    def calculate_base_value(self, key: str, limits: Any, context: GenerationContext) -> Any:
        pkg_spec = self.constants.PACKAGE_SPECS.get(context.package)
        if not pkg_spec: 
            return None

        dims = pkg_spec["dimensions"]
        pkg_limits = pkg_spec["limits"]
        extras = context.extras
        
        if key == "length": return dims[0]
        if key == "width": return dims[1]
        if key == "height": return dims[2]
        if key == "weight": return dims[3] if len(dims) > 3 else 0.1

        if key == "package_code": 
            return context.package
        
        if key == "power_dissipation":
            # 🔧 IMPROVED: Realistic power dissipation calculation
            
            # A. Conduction loss: P_cond = Id² × Rds(on)
            p_cond = (extras["id_rated"] ** 2) * extras["rds_on_target"]
            
            # B. Switching loss (assume 100kHz, hard switching)
            # P_sw = 0.5 × Vds × Id × (tr + tf) × f_sw
            ciss_est = extras["ciss_est"]
            rg_int = extras["rg_internal"]
            t_sw_total = (ciss_est / 1000) * rg_int * 3.5  # ns, tr+tf estimate
            f_sw = 100e3  # 100 kHz typical
            p_sw = 0.5 * extras["vdss"] * extras["id_rated"] * (t_sw_total * 1e-9) * f_sw
            
            p_total = p_cond + p_sw
            
            # Can't exceed package thermal limit
            pkg_limit = pkg_limits["max_power_watts"]
            return min(round(p_total, 2), pkg_limit)
        
        if key == "thermal_resistance_ja": 
            return pkg_limits["rth_ja"]
        
        if key == "thermal_resistance_jc": 
            return pkg_limits["rth_jc"]
        
        if key == "tj_max":
            # Junction temperature max
            tj = 175 if "SiC" in extras["tech_type"] else 150
            return tj

        return None

    # ----------------------------------------------------------------------
    # CORRELATIONS (SIMPLIFIED - already calculated in context)
    # ----------------------------------------------------------------------
    def apply_correlations(self, result: DatasheetResult):
        """
        Apply final correlations and adjustments to parameters.
        Most physics is already handled in create_custom_parameter,
        but this allows for cross-parameter validation.
        """
        ctx = result.context
        
        # Round all numeric values to reasonable precision
        for p in result.parameters:
            if p.value_typ is not None and isinstance(p.value_typ, float):
                p.value_typ = round(p.value_typ, 3)
            if p.value_min is not None and isinstance(p.value_min, float):
                p.value_min = round(p.value_min, 3)
            if p.value_max is not None and isinstance(p.value_max, float):
                p.value_max = round(p.value_max, 3)

    # ----------------------------------------------------------------------
    # HELPERS
    # ----------------------------------------------------------------------
    def _abs_max(self, key, val, unit, symbol, ctx):
        cond = "T_C=25°C"
        return GeneratedParameter(
            key=key, label="Absolute Maximum Rating", symbol=symbol,
            section="ABS_MAX", value_max=val, unit=unit,
            condition=cond, spec_type=SpecType.MAX_RATING,
            engineering_class=EngineeringClass.SAFETY_LIMIT
        )
    
    def _typ_max_obj(self, key, val_typ, unit, symbol):
        return GeneratedParameter(
            key=key, label=symbol, symbol=symbol,
            section="ELEC_CHAR",
            value_typ=val_typ, value_max=round(val_typ * 1.5, 2),
            unit=unit,
            condition="",
            spec_type=SpecType.NOMINAL
        )

    def _auto_scale(self, value, unit):
        if value >= 1: return value, unit
        if value >= 1e-3: return value * 1e3, "m" + unit
        return value * 1e6, "µ" + unit

Overwriting synthetic_pipeline/mosfet_strategy.py


מחלקה יורשת עבור Voltage Regulator

In [18]:
%%writefile synthetic_pipeline/voltage_Regulator_strategy.py
import random
import math
import uuid
import re
from typing import Dict, Any, Optional, List, Tuple
from dataclasses import dataclass

# ==========================================
# 1. PIPELINE IMPORTS
# ==========================================
from synthetic_pipeline.data_types import (
    GenerationContext, 
    DatasheetResult, 
    GeneratedParameter,
    SpecType, 
    EngineeringClass,
)

from synthetic_pipeline.strategy_base import ComponentStrategy

# ============================================================================
# CONSTANTS CLASS
# ============================================================================
class VoltageRegulatorConstants:
    """Constants and lookup tables for voltage regulator generation"""
    
    # ========================================================================
    # MASTER PACKAGE SPECIFICATIONS - SINGLE SOURCE OF TRUTH
    # ========================================================================
    PACKAGE_SPECS = {
        # --- SMALL SIGNAL ---
        "SOT-23-3": {
            "dimensions": [2.9, 1.3, 1.0, 0.015], 
            "limits": { "max_power_watts": 0.4, "max_current_a": 0.5, "max_voltage_v": 30, "rth_ja": 357, "rth_jc": 100 }
        },
        "SOT-23-5": {
            "dimensions": [2.9, 1.6, 1.0, 0.02],
            "limits": { "max_power_watts": 0.5, "max_current_a": 0.6, "max_voltage_v": 30, "rth_ja": 280, "rth_jc": 90 }
        },
        "SOT-223": {
            "dimensions": [6.5, 3.5, 1.6, 0.1],
            "limits": { "max_power_watts": 1.5, "max_current_a": 1.5, "max_voltage_v": 45, "rth_ja": 130, "rth_jc": 30 }
        },
        
        # --- POWER SMD ---
        "DPAK": {
            "dimensions": [6.6, 6.1, 2.3, 0.3],
            "limits": { "max_power_watts": 2.5, "max_current_a": 5.0, "max_voltage_v": 60, "rth_ja": 80, "rth_jc": 5 }
        },
        "D2PAK": {
            "dimensions": [10.2, 9.0, 4.5, 1.5],
            "limits": { "max_power_watts": 5.0, "max_current_a": 8.0, "max_voltage_v": 60, "rth_ja": 50, "rth_jc": 3 }
        },
        
        # --- ADVANCED / ULTRA SMALL ---
        "DFN_1x1": {
            "dimensions": [1.0, 1.0, 0.4, 0.002],
            "limits": { "max_power_watts": 0.3, "max_current_a": 0.3, "max_voltage_v": 5.5, "rth_ja": 200, "rth_jc": 50 }
        },
        "DFN_2x2": {
            "dimensions": [2.0, 2.0, 0.75, 0.015],
            "limits": { "max_power_watts": 0.8, "max_current_a": 1.0, "max_voltage_v": 25, "rth_ja": 150, "rth_jc": 20 }
        },
        "DFN_3x3": {
            "dimensions": [3.0, 3.0, 0.75, 0.03],
            "limits": { "max_power_watts": 1.5, "max_current_a": 3.0, "max_voltage_v": 30, "rth_ja": 60, "rth_jc": 8 }
        },
        
        # --- THROUGH HOLE ---
        "TO-220": {
            "dimensions": [10.0, 4.5, 9.0, 2.0],
            "limits": { "max_power_watts": 20.0, "max_current_a": 10.0, "max_voltage_v": 60, "rth_ja": 60, "rth_jc": 3 }
        },
        "TO-263": {
            "dimensions": [10.2, 9.0, 4.5, 1.0],
            "limits": { "max_power_watts": 5.0, "max_current_a": 5.0, "max_voltage_v": 60, "rth_ja": 50, "rth_jc": 5 }
        }
    }
    
    TOLERANCE_MAP = {
        "LDO_Low_Voltage_CMOS": 0.02,
        "LDO_High_Voltage_Bipolar": 0.04
    }
    
    # SI Units (Amperes)
    IQ_BASE = {
        "LDO_Low_Voltage_CMOS": 5e-6,      # 5 µA
        "LDO_High_Voltage_Bipolar": 1e-3   # 1 mA
    }
    
    DROPOUT_CURRENT_FACTOR = 0.4 # Vdo increases with current
    
    # Ohm * Ampere product (to estimate pass element resistance)
    RDS_SCALING = {
        "LDO_Low_Voltage_CMOS": 0.2, 
        "LDO_High_Voltage_Bipolar": 0.5
    }
    
    # Loop bandwidth vs quiescent current ratio (Hz/A)
    BW_IQ_RATIO = 50e6  # 50 MHz/A -> 1mA Iq = 50kHz BW
    
    # Output capacitor-bandwidth product (F·Hz)
    COUT_BW_PRODUCT = 1e-4  # 100 µF·kHz

    R_PASS_FACTOR = {
        # CMOS LDOs (PMOS Pass Element)
        "LDO_Low_Voltage_CMOS": 0.25,      # ~250mV drop at 1A
        "LDO_Ultra_Low_Drop": 0.15,        # ~150mV drop at 1A
        "LDO_CMOS_Tiny": 0.5,              # Higher resistance
        
        # Bipolar LDOs (PNP Pass Element)
        "LDO_High_Voltage_Bipolar": 0.6,   # ~600mV drop at 1A
        "LDO_Automotive_Bipolar": 0.8,
        
        # Standard Linear Regulators (NPN Darlington)
        "Standard_Linear_78xx": 2.0,       # ~2V drop at 1A
        "High_Current_Linear": 1.2,
        
        # Specialized
        "LDO_Ultra_Low_Noise": 0.35,
        "LDO_Wide_Input": 0.5
    }


# ============================================================================
# STRATEGY CLASS
# ============================================================================
class VoltageRegulatorStrategy(ComponentStrategy):
    """Hermetic Voltage Regulator Strategy - FIXED VERSION"""

    def __init__(self):
        self.C = VoltageRegulatorConstants()

    # ------------------------------------------------------------------
    # 1. CONTEXT - The Lock-in Phase (FIXED)
    # ------------------------------------------------------------------
    def create_context(self, schema: Dict, requested_corner: Optional[str]) -> GenerationContext:
        archetype = random.choice(schema.get("archetypes", ["LDO_Low_Voltage_CMOS"]))
        
        # Filter valid packages based on archetype hints
        valid_packages = list(self.C.PACKAGE_SPECS.keys())
        if "Power" in archetype or "Bipolar" in archetype:
            valid_packages = [p for p in valid_packages if "TO" in p or "DPAK" in p or "SOT-223" in p]
        
        package = random.choice(valid_packages)

        # Hermetic Lock: Determine Vout and Iout constraints NOW
        primary = self._generate_primary(schema, archetype, package)

        # 🔧 FIX: Calculate vdrop and iq IMMEDIATELY in context
        r_pass = self.C.R_PASS_FACTOR.get(archetype, 0.4)
        vdrop = primary["iout"] * r_pass
        
        base_iq = self.C.IQ_BASE.get(archetype, 50e-6)
        iq = base_iq * random.uniform(0.8, 1.3)

        extras = dict(
            vout=primary["vout"],
            iout=primary["iout"],
            vdrop=vdrop,      # ✅ NOW CALCULATED
            iq=iq,            # ✅ NOW CALCULATED
            loop_bw=None,     # Will be calculated in apply_correlations
            is_cmos="CMOS" in archetype,
        )

        return GenerationContext(
            sample_id=f"LDO_{uuid.uuid4().hex[:6]}",
            component_type="VOLTAGE_REGULATOR",
            package=package,
            archetype=archetype,
            tolerance=0.02 if "CMOS" in archetype else 0.04,
            process_corner=requested_corner or "TYPICAL",
            extras=extras,
        )

    # ------------------------------------------------------------------
    # 2. PRIMARY LOCK (IMPROVED PHYSICS)
    # ------------------------------------------------------------------
    def _generate_primary(self, schema, archetype, package):
        pkg_limits = self.C.PACKAGE_SPECS[package]["limits"]

        # Extract options from schema limits or use defaults
        vouts = self._extract_numeric_list(self._get_schema_limits(schema, "output_voltage", archetype))
        iouts = self._extract_numeric_list(self._get_schema_limits(schema, "iout_max", archetype))

        # Try to find a valid combination within package physics limits
        for _ in range(100):  # More iterations
            vout = random.choice(vouts or [1.2, 1.8, 2.5, 3.3, 5.0])
            iout = random.choice(iouts or [0.1, 0.3, 0.5])

            # Estimate dropout for this archetype
            r_pass = self.C.R_PASS_FACTOR.get(archetype, 0.4)
            vdrop_est = iout * r_pass
            
            # Worst-case Vin (max input from package rating)
            vin_max = min(pkg_limits["max_voltage_v"], vout + 10)
            
            # Worst-case power dissipation
            p_worst = (vin_max - vout) * iout + vout * 1e-3  # Assume 1mA Iq
            
            # Thermal check: P < (Tj_max - Ta) / Rth_ja
            t_junction_max = 125.0
            t_ambient = 85.0  # Worst case ambient
            p_thermal_limit = (t_junction_max - t_ambient) / pkg_limits["rth_ja"]
            
            # Physics Check - ALL CONDITIONS
            if (iout <= pkg_limits["max_current_a"] and 
                p_worst <= pkg_limits["max_power_watts"] and
                p_worst <= p_thermal_limit and
                vout <= pkg_limits["max_voltage_v"]):
                return dict(vout=vout, iout=iout)

        # Aggressive fallback if solver fails
        return dict(
            vout=min(vouts or [3.3]), 
            iout=pkg_limits["max_current_a"] * 0.3
        )

    # ------------------------------------------------------------------
    # 3. PARAMETERS GENERATION (SIMPLIFIED - values pre-calculated)
    # ------------------------------------------------------------------
    def create_custom_parameter(self, key, ctx, pd):

        if key == "output_voltage":
            v = ctx.extras["vout"]
            return self._nominal(pd, v, "V", ctx)

        if key == "iout_max":
            i = ctx.extras["iout"]
            val_disp, unit = self._auto_scale(i, "A")
            return GeneratedParameter(
                key=key,
                label=pd["llm_context"]["formal_name"],
                symbol=pd["symbol"],
                section="ELEC_CHAR",
                value_typ=val_disp,
                value_max=val_disp,
                unit=unit,
                condition=pd["scenarios"][0]["condition"],
                spec_type=SpecType.MAX_RATING,
                engineering_class=EngineeringClass.PERFORMANCE,
            )

        if key == "quiescent_current":
            # ✅ Already calculated in context
            val_disp, unit = self._auto_scale(ctx.extras["iq"], "A")
            return self._typ_max_obj(pd, val_disp, unit)

        if key == "dropout_voltage":
            # ✅ Already calculated in context
            val_disp, unit = self._auto_scale(ctx.extras["vdrop"], "V")
            return self._typ_max_obj(pd, val_disp, unit)
        
        # 🆕 NEW PARAMETERS
        if key == "line_regulation":
            # ΔVout / ΔVin (better with higher loop gain ~ Iq)
            reg_pct = 0.05 / (ctx.extras["iq"] * 1e6 + 1)  # Prevent div by zero
            return self._typ_max_obj(pd, round(reg_pct, 4), "%/V")
        
        if key == "load_regulation":
            # ΔVout / ΔIout (mV/mA) - output resistance effect
            rout = ctx.extras["vdrop"] / (ctx.extras["iout"] + 1e-6) * 0.01
            reg_mv_ma = rout * 1000
            return self._typ_max_obj(pd, round(reg_mv_ma, 2), "mV/mA")

        return None

    # ------------------------------------------------------------------
    # 4. CORRELATIONS (IMPROVED PHYSICS)
    # ------------------------------------------------------------------
    def apply_correlations(self, result: DatasheetResult):
        ctx = result.context

        # Physics: Bandwidth is proportional to quiescent current (g_m stage)
        if ctx.extras["loop_bw"] is None and ctx.extras["iq"]:
            ctx.extras["loop_bw"] = ctx.extras["iq"] * self.C.BW_IQ_RATIO

        for p in result.parameters:
            # 🔧 IMPROVED: Noise Correlation with architecture dependency
            if p.key == "output_noise" and p.value_typ and ctx.extras["loop_bw"]:
                # Base noise density depends on architecture
                noise_density_base = {
                    "LDO_Low_Voltage_CMOS": 20,      # µV/√Hz
                    "LDO_High_Voltage_Bipolar": 60,
                    "LDO_Ultra_Low_Noise": 10,
                }.get(ctx.archetype, 40)
                
                # Integrated noise: Vnoise_rms = noise_density * sqrt(BW)
                bw_effective = ctx.extras["loop_bw"]
                integrated_noise = noise_density_base * math.sqrt(bw_effective / 1e3)  # Normalize
                
                p.value_typ = round(integrated_noise, 1)
            
            # 🔧 IMPROVED: PSRR with frequency rolloff
            if p.key == "psrr" and ctx.extras["loop_bw"]:
                # Extract test frequency from condition
                freq_match = re.search(r'(\d+)\s*(Hz|kHz|MHz)', p.condition or "")
                if freq_match:
                    freq_val = float(freq_match.group(1))
                    freq_unit = freq_match.group(2)
                    test_freq = freq_val * {"Hz": 1, "kHz": 1e3, "MHz": 1e6}[freq_unit]
                    
                    f_pole = ctx.extras["loop_bw"] / 10
                    
                    if test_freq > f_pole:
                        # --- התיקון שלך נכנס כאן ---
                        base_val = p.value_typ
                        
                        # טיפול במקרה שהערך הגיע מהסכמה כמילון
                        if isinstance(base_val, dict):
                            # לוקחים את הערך המינימלי כנקודת מוצא
                            base_val = min(base_val.values()) if base_val else 60.0
                        
                        # ביצוע החישוב המתמטי רק אם יש לנו מספר
                        if isinstance(base_val, (int, float)):
                            rolloff_db = 20 * math.log10(test_freq / f_pole)
                            p.value_typ = max(base_val - rolloff_db, 20)  # Floor at 20dB

            self._round(p)

    # ------------------------------------------------------------------
    # 5. BASE VALUES & HELPERS (FIXED)
    # ------------------------------------------------------------------
    def calculate_base_value(self, key, limits, ctx):
        pkg_data = self.C.PACKAGE_SPECS[ctx.package]
        pkg_limits = pkg_data["limits"]

        if key == "package_code": return ctx.package
        if key == "dimensions_lxwxh": return pkg_data["dimensions"]

        if key == "power_dissipation":
            # 🔧 FIXED: Actual power dissipation at max load
            # Assume typical Vin = Vout + 2*Vdrop (reasonable operating point)
            vin_typ = ctx.extras["vout"] + 2 * ctx.extras["vdrop"]
            p_load = (vin_typ - ctx.extras["vout"]) * ctx.extras["iout"]
            p_quiescent = ctx.extras["vout"] * ctx.extras["iq"]
            p_actual = p_load + p_quiescent
            
            return round(p_actual * 1.2, 2)  # 20% margin

        if key == "thermal_resistance_ja": 
            return pkg_limits["rth_ja"]

        if key == "minimum_input_voltage":
            # 🔧 FIXED: Vin_min must provide AT LEAST the specified dropout
            # Add 10% margin to ensure spec compliance
            return ctx.extras["vout"] + ctx.extras["vdrop"] * 1.1

        if key == "output_voltage": return ctx.extras["vout"]
        if key == "iout_max": return ctx.extras["iout"]

        return self._pick_generic(limits)

    # ------------------------------------------------------------------
    # UTILS
    # ------------------------------------------------------------------
    def _nominal(self, pd, val, unit, ctx):
        tol = ctx.tolerance
        return GeneratedParameter(
            key=pd["key"], label=pd["llm_context"]["formal_name"], symbol=pd["symbol"],
            section="ELEC_CHAR",
            value_min=val * (1 - tol), value_typ=val, value_max=val * (1 + tol),
            unit=unit, condition=pd["scenarios"][0]["condition"],
            spec_type=SpecType.NOMINAL, engineering_class=EngineeringClass.PERFORMANCE,
        )

    def _typ_max_obj(self, pd, val, unit):
        return GeneratedParameter(
            key=pd["key"], label=pd["llm_context"]["formal_name"], symbol=pd["symbol"],
            section="ELEC_CHAR",
            value_typ=val, value_max=val * 1.5,
            unit=unit, condition=pd["scenarios"][0]["condition"],
            spec_type=SpecType.NOMINAL, engineering_class=EngineeringClass.PERFORMANCE,
        )

    def _auto_scale(self, val, unit):
        if val is None: return 0, unit
        if val < 1e-6: return val * 1e9, "n" + unit
        if val < 1e-3: return val * 1e6, "µ" + unit
        if val < 1: return val * 1e3, "m" + unit
        return val, unit

    def _round(self, p):
        for f in ("value_typ", "value_min", "value_max"):
            v = getattr(p, f)
            if isinstance(v, float): setattr(p, f, round(v, 3))

    def _get_schema_limits(self, schema, key, archetype):
        p = next((x for x in schema.get("ELEC_CHAR", []) if x["key"] == key), None)
        if not p: return []
        return p["scenarios"][0]["limits"].get(archetype, [])

    def _extract_numeric_list(self, limits):
        out = []
        for x in limits if isinstance(limits, list) else [limits]:
            if isinstance(x, (int, float)): out.append(float(x))
            elif isinstance(x, str):
                m = re.search(r"[\d.]+", x)
                if m: out.append(float(m.group()))
        return out

    def _pick_generic(self, limits):
        if isinstance(limits, list) and limits: return random.choice(limits)
        return limits

Overwriting synthetic_pipeline/voltage_Regulator_strategy.py


מחלקה יורשת עבור Inductor

In [19]:
%%writefile synthetic_pipeline/inductor_strategy.py
import random
import math
import uuid
import re
from typing import Dict, Any, Optional, List, Tuple
from dataclasses import dataclass

# ==========================================
# 1. PIPELINE IMPORTS
# ==========================================
from synthetic_pipeline.data_types import (
    GenerationContext, 
    DatasheetResult, 
    GeneratedParameter,
    SpecType, 
    EngineeringClass,
)

from synthetic_pipeline.strategy_utils import snap_to_e_series
from synthetic_pipeline.strategy_base import ComponentStrategy

# ============================================================================
# CONSTANTS CLASS - SINGLE SOURCE OF TRUTH
# ============================================================================
class InductorConstants:
    """Constants and lookup tables for inductor generation"""
    
    # ========================================================================
    # MASTER PACKAGE SPECIFICATIONS - SINGLE SOURCE OF TRUTH
    # ========================================================================
    PACKAGE_SPECS = {
        # --- ULTRA-SMALL SMD (RF / Signal) ---
        "01005": {
            "dimensions": [0.4, 0.2, 0.13, 0.0001],  # L, W, H (mm), Weight (g)
            "limits": { "max_current_a": 0.1, "max_inductance_h": 47e-9, "max_power_watts": 0.03, "dcr_max_ohm": 0.8, "footprint_area_mm2": 0.08, "core_volume_mm3": 0.01 }
        },
        "0201": {
            "dimensions": [0.6, 0.3, 0.23, 0.0002],
            "limits": { "max_current_a": 0.2, "max_inductance_h": 100e-9, "max_power_watts": 0.05, "dcr_max_ohm": 0.5, "footprint_area_mm2": 0.18, "core_volume_mm3": 0.04 }
        },
        "0402": {
            "dimensions": [1.0, 0.5, 0.5, 0.001],
            "limits": { "max_current_a": 0.5, "max_inductance_h": 220e-9, "max_power_watts": 0.1, "dcr_max_ohm": 0.3, "footprint_area_mm2": 0.5, "core_volume_mm3": 0.25 }
        },
        "0603": {
            "dimensions": [1.6, 0.8, 0.8, 0.003],
            "limits": { "max_current_a": 1.0, "max_inductance_h": 1e-6, "max_power_watts": 0.15, "dcr_max_ohm": 0.2, "footprint_area_mm2": 1.28, "core_volume_mm3": 1.15 }
        },
        "0805": {
            "dimensions": [2.0, 1.25, 1.25, 0.008],
            "limits": { "max_current_a": 2.0, "max_inductance_h": 4.7e-6, "max_power_watts": 0.25, "dcr_max_ohm": 0.15, "footprint_area_mm2": 2.5, "core_volume_mm3": 3.1 }
        },
        
        # --- STANDARD SMD (Power / General) ---
        "1008": {
            "dimensions": [2.5, 2.0, 2.0, 0.015],
            "limits": { "max_current_a": 2.5, "max_inductance_h": 10e-6, "max_power_watts": 0.4, "dcr_max_ohm": 0.12, "footprint_area_mm2": 5.0, "core_volume_mm3": 10.0 }
        },
        "1206": {
            "dimensions": [3.2, 1.6, 1.6, 0.02],
            "limits": { "max_current_a": 3.0, "max_inductance_h": 22e-6, "max_power_watts": 0.5, "dcr_max_ohm": 0.1, "footprint_area_mm2": 5.12, "core_volume_mm3": 8.2 }
        },
        "1210": {
            "dimensions": [3.2, 2.5, 2.5, 0.04],
            "limits": { "max_current_a": 4.0, "max_inductance_h": 47e-6, "max_power_watts": 0.8, "dcr_max_ohm": 0.08, "footprint_area_mm2": 8.0, "core_volume_mm3": 20.0 }
        },
        "1812": {
            "dimensions": [4.5, 3.2, 3.2, 0.08],
            "limits": { "max_current_a": 6.0, "max_inductance_h": 100e-6, "max_power_watts": 1.2, "dcr_max_ohm": 0.06, "footprint_area_mm2": 14.4, "core_volume_mm3": 46.0 }
        },
        
        # --- POWER INDUCTOR PACKAGES ---
        "2520": {
            "dimensions": [2.5, 2.0, 1.0, 0.025],
            "limits": { "max_current_a": 5.0, "max_inductance_h": 22e-6, "max_power_watts": 1.5, "dcr_max_ohm": 0.05, "footprint_area_mm2": 5.0, "core_volume_mm3": 5.0 }
        },
        "3225": {
            "dimensions": [3.2, 2.5, 2.0, 0.045],
            "limits": { "max_current_a": 6.5, "max_inductance_h": 47e-6, "max_power_watts": 2.0, "dcr_max_ohm": 0.04, "footprint_area_mm2": 8.0, "core_volume_mm3": 16.0 }
        },
        "4040": {
            "dimensions": [4.0, 4.0, 3.0, 0.15],
            "limits": { "max_current_a": 8.0, "max_inductance_h": 100e-6, "max_power_watts": 3.0, "dcr_max_ohm": 0.03, "footprint_area_mm2": 16.0, "core_volume_mm3": 48.0 }
        },
        "5050": {
            "dimensions": [5.0, 5.0, 4.0, 0.3],
            "limits": { "max_current_a": 10.0, "max_inductance_h": 220e-6, "max_power_watts": 5.0, "dcr_max_ohm": 0.02, "footprint_area_mm2": 25.0, "core_volume_mm3": 100.0 }
        },
        "6060": {
            "dimensions": [6.0, 6.0, 5.0, 0.6],
            "limits": { "max_current_a": 15.0, "max_inductance_h": 470e-6, "max_power_watts": 8.0, "dcr_max_ohm": 0.015, "footprint_area_mm2": 36.0, "core_volume_mm3": 180.0 }
        },
        
        # --- THROUGH-HOLE / SPECIALTY ---
        "Radial": {
            "dimensions": [8.0, 8.0, 10.0, 1.5],
            "limits": { "max_current_a": 12.0, "max_inductance_h": 10e-3, "max_power_watts": 5.0, "dcr_max_ohm": 0.05, "footprint_area_mm2": 64.0, "core_volume_mm3": 640.0 }
        },
        "Axial": {
            "dimensions": [12.0, 4.0, 4.0, 0.8],
            "limits": { "max_current_a": 5.0, "max_inductance_h": 1e-3, "max_power_watts": 2.0, "dcr_max_ohm": 0.1, "footprint_area_mm2": 48.0, "core_volume_mm3": 192.0 }
        },
        "Toroid": {
            "dimensions": [20.0, 20.0, 10.0, 10.0],
            "limits": { "max_current_a": 30.0, "max_inductance_h": 100e-3, "max_power_watts": 25.0, "dcr_max_ohm": 0.005, "footprint_area_mm2": 400.0, "core_volume_mm3": 4000.0 }
        }
    }

    ARCHETYPES = ["Multilayer_Ceramic", "Wirewound_Ferrite", "Power_Shielded"]
    
    # Tolerance mapping
    TOLERANCE_MAP = {
        "Power_Ferrite": 0.20,
        "RF_Ceramic": 0.05,
        "Common_Mode_Choke": 0.25
    }
    
    # Core material properties
    CORE_MATERIALS = {
        "Power_Ferrite": {
            "permeability": 1500,
            "saturation_t": 0.4,  # Tesla (SI)
            "curie_temp_c": 220,
            "has_saturation": True
        },
        "RF_Ceramic": {
            "permeability": 1,  # Air core
            "saturation_t": None,
            "curie_temp_c": 800,
            "has_saturation": False
        },
        "Common_Mode_Choke": {
            "permeability": 5000,
            "saturation_t": 0.35,
            "curie_temp_c": 150,
            "has_saturation": True
        }
    }
    
    # Physical constants (SI units)
    COPPER_RESISTIVITY_OHM_M = 1.72e-8  # Ω·m at 20°C
    MU_0 = 4 * math.pi * 1e-7  # H/m
    WIRE_FILL_FACTOR = 0.35
    
    # Parasitic capacitance base (Farads - SI)
    SRF_BASE_CAPACITANCE_F = {
        "Power_Ferrite": 3e-12,  # 3 pF
        "RF_Ceramic": 0.3e-12,   # 0.3 pF
        "Common_Mode_Choke": 8e-12
    }
    
    # Q factor baseline
    Q_FACTOR_BASE = {
        "Power_Ferrite": 30,
        "RF_Ceramic": 60,
        "Common_Mode_Choke": 20
    }
    
    # Thermal resistance baseline (°C/W)
    THERMAL_RESISTANCE_BASE = {
        "Power_Ferrite": 100,
        "RF_Ceramic": 150,
        "Common_Mode_Choke": 80
    }


# ============================================================================
# STRATEGY CLASS
# ============================================================================
class InductorStrategy(ComponentStrategy):
    """Physics-validated synthetic inductor datasheet generator"""
    
    def __init__(self):
        self.constants = InductorConstants()
    
    # ------------------------------------------------------------------------
    # 1. CREATE CONTEXT (HIERARCHICAL SELECTION)
    # ------------------------------------------------------------------------
    def create_context(self, schema: Dict, requested_corner: Optional[str]) -> GenerationContext:
        """Select archetype and package, precompute primary values"""
        
        # Step 1: Select archetype
        archetypes = schema.get('archetypes', ["Power_Ferrite"])
        archetype = random.choice(archetypes)
        
        # Step 2: Get valid packages for archetype
        package_param = next((p for p in schema['PACKAGE'] if p['key']=='package_code'), None)
        valid_packages = ["0603"] # Fallback
        
        if package_param:
            raw_limits = package_param['scenarios'][0]['limits']
            if isinstance(raw_limits, dict):
                valid_packages = raw_limits.get(archetype, [])
            
        # Ensure packages exist in our physics constants
        valid_packages = [p for p in valid_packages if p in self.constants.PACKAGE_SPECS]
        if not valid_packages: 
            valid_packages = ["0603"] 
        
        # Step 3: Select package
        target_package = random.choice(valid_packages)
        tolerance = self.constants.TOLERANCE_MAP.get(archetype, 0.20)
        
        # Step 4: Generate primary values (L)
        # We calculate Inductance NOW to lock the physics.
        primary_values = self._generate_primary_values(schema, archetype, target_package)
        
        # Build extras
        extras = {
            'requested_corner': requested_corner,
            'inductance_h': primary_values['inductance_h'],
            # שאר הערכים יחושבו "על המקום" (Just-in-Time) בפעם הראשונה שיידרשו
            'physics_calculated': False, 
            'rated_current_a': None,
            'saturation_current_a': None,
            'dcr_ohm': None,
            'srf_hz': None,
            'q_factor': None
        }
        
        return GenerationContext(
            sample_id=f"IND_{archetype}_{target_package}_{random.randint(1000,9999)}",
            component_type="INDUCTOR",
            package=target_package,
            archetype=archetype,
            tolerance=tolerance,
            process_corner=requested_corner or "TYPICAL",
            extras=extras
        )
    
    # ------------------------------------------------------------------------
    # 2. PHYSICS ENGINE (CALCULATED ON DEMAND)
    # ------------------------------------------------------------------------
    def _ensure_physics(self, context: GenerationContext):
        """
        Runs the heavy physics math.
        Called automatically before accessing dependent parameters.
        """
        if context.extras.get('physics_calculated'):
            return

        l_si = context.extras.get('inductance_h')
        if not l_si or l_si <= 0: return
        
        archetype = context.archetype
        pkg_spec = self.constants.PACKAGE_SPECS[context.package]
        material = self.constants.CORE_MATERIALS.get(archetype, self.constants.CORE_MATERIALS["Power_Ferrite"])
        
        # --- Core geometry ---
        core_area_m2 = pkg_spec['limits']['footprint_area_mm2'] * 1e-6
        mu_r = material['permeability']
        
        # --- Number of turns (N) ---
        # L = (mu_0 * mu_r * A * N^2) / l_path
        # Approximation: l_path ~ 4 * sqrt(Area) (Toroid-like loop)
        l_path_m = math.sqrt(core_area_m2) * 4.0
        
        if archetype == "RF_Ceramic":
            # Air core (mu_r = 1 effectively for calculation structure)
            k = self.constants.MU_0 * core_area_m2 / l_path_m
        else:
            k = self.constants.MU_0 * mu_r * core_area_m2 / l_path_m
            
        n_turns = math.sqrt(l_si / k)
        n_turns = max(n_turns, 1.0)
        
        # --- Wire Geometry ---
        # Window area ~ 40% of core footprint (simplification)
        window_area_m2 = core_area_m2 * 0.4
        wire_area_m2 = (window_area_m2 * self.constants.WIRE_FILL_FACTOR) / n_turns
        wire_diameter_m = 2 * math.sqrt(wire_area_m2 / math.pi)
        
        # Physical clamp (wire can't be microscopic or huge)
        wire_diameter_m = max(10e-6, min(wire_diameter_m, 2e-3))
        
        # Total wire length
        mean_turn_length_m = math.sqrt(core_area_m2) * 4
        total_wire_length_m = n_turns * mean_turn_length_m
        
        # --- DCR Calculation ---
        # R = rho * L / A
        wire_cross_section_m2 = math.pi * (wire_diameter_m/2)**2
        dcr_calc = (self.constants.COPPER_RESISTIVITY_OHM_M * total_wire_length_m) / wire_cross_section_m2
        
        # Adjustments (Skin effect, termination, imperfections)
        dcr_final = dcr_calc * 1.3 + 0.005 
        dcr_final = min(dcr_final, pkg_spec['limits']['dcr_max_ohm']) # Clamp to package limit
        
        context.extras['dcr_ohm'] = dcr_final
        
        # --- Saturation Current ---
        i_sat = None
        if material['has_saturation']:
            # B = mu * N * I / l_path  => I_sat = B_sat * l_path / (mu * N)
            i_sat_calc = (material['saturation_t'] * l_path_m) / (self.constants.MU_0 * mu_r * n_turns)
            i_sat = i_sat_calc * 0.7 # Safety margin
            context.extras['saturation_current_a'] = i_sat
        
        # --- Rated Current (Thermal) ---
        # P = I^2 * R. Max power defined by package.
        max_p = pkg_spec['limits']['max_power_watts']
        i_thermal = math.sqrt(max_p / dcr_final)
        
        # Combined Rated Current
        pkg_max_i = pkg_spec['limits']['max_current_a']
        i_rated = min(i_thermal, pkg_max_i)
        if i_sat:
            i_rated = min(i_rated, i_sat)
            
        context.extras['rated_current_a'] = i_rated
        
        # --- SRF Calculation ---
        # C_parasitic scales with size and turns
        c_base = self.constants.SRF_BASE_CAPACITANCE_F.get(archetype, 1e-12)
        c_parasitic = c_base * math.sqrt(n_turns) + 0.1e-12
        f_srf = 1.0 / (2 * math.pi * math.sqrt(l_si * c_parasitic))
        context.extras['srf_hz'] = f_srf

        # --- Q Factor ---
        # Approximation around 100MHz or SRF/10
        q_base = self.constants.Q_FACTOR_BASE.get(archetype, 30)
        context.extras['q_factor'] = q_base

        context.extras['physics_calculated'] = True

    # ------------------------------------------------------------------------
    # 3. CREATE CUSTOM PARAMETER (THE INTERFACE)
    # ------------------------------------------------------------------------
    def create_custom_parameter(self, key: str, context: GenerationContext, param_def: Dict) -> Optional[GeneratedParameter]:
        """
        Creates the datasheet parameters using the calculated physics.
        """
        # Ensure we have data
        self._ensure_physics(context)
        extras = context.extras
        
        # --- Inductance ---
        if key == 'inductance':
            val_si = extras['inductance_h']
            disp_val, unit = self._auto_scale(val_si, 'H')
            tol = context.tolerance
            
            return GeneratedParameter(
                key=key, label=param_def['llm_context']['formal_name'], symbol="L",
                section="ELEC_CHAR",
                value_min=disp_val * (1 - tol),
                value_typ=disp_val,
                value_max=disp_val * (1 + tol),
                unit=unit,
                condition="1kHz/100kHz", # Can be dynamic based on value
                spec_type=SpecType.NOMINAL,
                engineering_class=EngineeringClass.PERFORMANCE
            )

        # --- Rated Current ---
        if key == 'rated_current':
            i_r = extras.get('rated_current_a', 0.1)
            disp_val, unit = self._auto_scale(i_r, 'A')
            
            return GeneratedParameter(
                key=key, label="Rated Current", symbol="I_R",
                section="ABS_MAX",
                value_typ=round(disp_val, 2),
                unit=unit,
                spec_type=SpecType.MAX_RATING,
                engineering_class=EngineeringClass.PERFORMANCE
            )

        # --- DC Resistance ---
        if key == 'dc_resistance':
            dcr = extras.get('dcr_ohm', 0.1)
            disp_val, unit = self._auto_scale(dcr, 'Ω')
            
            return GeneratedParameter(
                key=key, label="DC Resistance", symbol="DCR",
                section="ELEC_CHAR",
                value_max=round(disp_val, 3), # DCR is usually a MAX spec
                unit=unit,
                spec_type=SpecType.MAX_LIMIT
            )

        # --- Saturation Current ---
        if key == 'saturation_current':
            i_sat = extras.get('saturation_current_a')
            if i_sat is None: return None # Not all inductors specify this
            
            disp_val, unit = self._auto_scale(i_sat, 'A')
            return GeneratedParameter(
                key=key, label="Saturation Current", symbol="I_sat",
                section="ELEC_CHAR",
                value_typ=round(disp_val, 2),
                unit=unit,
                condition="L drop 30%",
                spec_type=SpecType.TYPICAL
            )
            
        # --- SRF ---
        if key == 'self_resonant_frequency':
            srf = extras.get('srf_hz', 1e6)
            disp_val, unit = self._auto_scale(srf, 'Hz')
            
            return GeneratedParameter(
                key=key, label="Self Resonant Frequency", symbol="SRF",
                section="ELEC_CHAR",
                value_min=round(disp_val * 0.8, 1),
                unit=unit,
                spec_type=SpecType.MIN_LIMIT
            )

        # --- Quality Factor ---
        if key == 'quality_factor':
            q = extras.get('q_factor', 30)
            return GeneratedParameter(
                key=key, label="Quality Factor", symbol="Q",
                section="ELEC_CHAR",
                value_min=int(q * 0.7),
                value_typ=int(q),
                unit="",
                spec_type=SpecType.TYPICAL
            )

        return None

    # ------------------------------------------------------------------------
    # 4. BASE VALUES & HELPERS
    # ------------------------------------------------------------------------
    def calculate_base_value(self, key: str, limits: Any, context: GenerationContext) -> Any:
        pkg_spec = self.constants.PACKAGE_SPECS.get(context.package)
        if not pkg_spec: return None
        
        dims = pkg_spec['dimensions']
        if key == 'length': return dims[0]
        if key == 'width': return dims[1]
        if key == 'height': return dims[2]
        if key == 'weight': return dims[3]
        if key == 'package_code': return context.package
        
        return None

    def _generate_primary_values(self, schema: Dict, archetype: str, package: str) -> Dict[str,float]:
        pkg_spec = self.constants.PACKAGE_SPECS[package]
        max_l = pkg_spec['limits']['max_inductance_h']
        
        # Try to pull from schema
        l_param = next((p for p in schema.get('ELEC_CHAR', []) if p['key']=='inductance'), None)
        valid_l = []
        
        if l_param:
            l_options = self._extract_numeric_list(l_param['scenarios'][0]['limits'].get(archetype, []))
            # Convert values (heuristic: if > 1 it's likely uH)
            l_si = [l * 1e-6 if l > 0.1 else l for l in l_options]
            valid_l = [l for l in l_si if l <= max_l]
            
        if not valid_l:
            # Physics Fallback: Log-Uniform choice
            min_l = 1e-9
            val = 10 ** random.uniform(math.log10(min_l), math.log10(max_l))
            valid_l = [val]
            
        return {'inductance_h': random.choice(valid_l)}

    def _extract_numeric_list(self, limits: Any) -> List[float]:
        if isinstance(limits, list):
            vals = []
            for x in limits:
                if isinstance(x, (int, float)): vals.append(float(x))
                elif isinstance(x, str):
                    try:
                        m = re.search(r'[\d.]+', x)
                        if m: vals.append(float(m.group()))
                    except: pass
            return vals
        if isinstance(limits, (int, float)): return [float(limits)]
        return []

    def _auto_scale(self, value: float, base_unit: str) -> Tuple[float, str]:
        if value is None or value == 0: return 0, base_unit
        
        # Standard Engineering Prefixes
        prefixes = [(1e-9,'n'), (1e-6,'µ'), (1e-3,'m'), (1,''), (1e3,'k'), (1e6,'M'), (1e9,'G')]
        
        val_abs = abs(value)
        best_scale = 1.0
        best_prefix = ''
        
        for scale, prefix in prefixes:
            if val_abs < scale * 1000: # Try to keep number < 1000
                best_scale = scale
                best_prefix = prefix
                break
        
        # Correction for Ohms (prefer mOhm only for small values)
        if base_unit == 'Ω' and value < 1.0:
            return value * 1000, 'mΩ'
        if base_unit == 'Ω' and value >= 1.0:
            return value, 'Ω'
            
        return value / best_scale, f"{best_prefix}{base_unit}"

    def apply_correlations(self, result: DatasheetResult) -> None:
        pass

Overwriting synthetic_pipeline/inductor_strategy.py


מחלקה יורשת עבור רכיב BJT

In [20]:
%%writefile synthetic_pipeline/bjt_strategy.py
import random
import math
import uuid
import re
from typing import Dict, Any, Optional, List, Tuple
from dataclasses import dataclass

# ==========================================
# 1. PIPELINE IMPORTS
# ==========================================
from synthetic_pipeline.data_types import (
    GenerationContext, 
    DatasheetResult, 
    GeneratedParameter,
    SpecType, 
    EngineeringClass,
)

from synthetic_pipeline.strategy_base import ComponentStrategy


# ============================================================================
# CONSTANTS CLASS - Physics-Aware with SI Units
# ============================================================================
class BJTConstants:
    """Constants and lookup tables for BJT generation with accurate physics"""
    
    # Process Corners for variation
    # Allows simulation of Worst Case vs Best Case scenarios
    CORNERS = {
        "TYPICAL":  {"hfe": 1.0, "vce_sat": 1.0, "vceo": 1.0},
        "MIN_SPEC": {"hfe": 0.7, "vce_sat": 1.2, "vceo": 0.9}, 
        "MAX_SPEC": {"hfe": 1.3, "vce_sat": 0.8, "vceo": 1.1},
        # Fallback support
        "WORST_CASE": {"hfe": 0.7, "vce_sat": 1.2, "vceo": 0.9},
        "BEST_CASE":  {"hfe": 1.3, "vce_sat": 0.8, "vceo": 1.1}
    }
    
    # Physics: Early Voltage scales with breakdown voltage
    EARLY_VOLTAGE_RATIO = {"Small_Signal": 3.0, "Power_BJT": 1.5}
    
    # Physics: hFE Curve Parameters
    HFE_PEAK_IC_RATIO = 0.3 # Peak gain happens at 30% of rated current usually
    KIRK_EFFECT_KNEE_RATIO = 0.8 # High current rolloff starts here
    HFE_LOW_CURRENT_ROLLOFF = 0.5 # Low current rolloff factor
    
    # Parasitic Capacitance of Package (Farads)
    PKG_PARASITIC_C = {
        "SOT-23": 0.5e-12, "TO-92": 1.0e-12, "TO-220": 2.0e-12, "DPAK": 1.5e-12
    }

    # Standard Ratings (SI Units: Amperes, Volts)
    VCEO_STANDARD = {
        "Small_Signal": [20, 30, 40, 50, 60, 80],
        "Power_BJT": [60, 80, 100, 120, 150, 200, 400, 600]
    }
    
    # Package Specifications - SOURCE OF TRUTH (SI Units)
    # pd_pkg_limit: Limit of the case itself
    # rth_ja: Thermal resistance Junction-to-Ambient (C/W)
    # ic_limit: Max current for bond wires/leadframe
    PACKAGE_SPECS = {
        "SC-70": {
            "pd_pkg_limit": 0.15, "rth_jc": 200, "rth_ja": 500, 
            "ic_limit": 0.2, "archetype": "Small_Signal",
            "dims": [2.0, 1.25, 0.95, 0.005] # L, W, H, Weight
        },
        "SOT-23": {
            "pd_pkg_limit": 0.35, "rth_jc": 150, "rth_ja": 350, 
            "ic_limit": 0.8, "archetype": "Small_Signal",
            "dims": [2.9, 1.3, 1.0, 0.008]
        },
        "TO-92": {
            "pd_pkg_limit": 0.625, "rth_jc": 83, "rth_ja": 200, 
            "ic_limit": 1.0, "archetype": "Small_Signal",
            "dims": [4.8, 3.7, 4.5, 0.19]
        },
        "SOT-223": {
            "pd_pkg_limit": 1.5, "rth_jc": 25, "rth_ja": 60, 
            "ic_limit": 3.0, "archetype": "Power_BJT",
            "dims": [6.5, 3.5, 1.6, 0.13]
        },
        "DPAK": {
            "pd_pkg_limit": 2.5, "rth_jc": 15, "rth_ja": 50, 
            "ic_limit": 5.0, "archetype": "Power_BJT",
            "dims": [6.6, 6.1, 2.3, 0.35]
        },
        "TO-220": {
            "pd_pkg_limit": 20.0, "rth_jc": 2.5, "rth_ja": 62.5, 
            "ic_limit": 15.0, "archetype": "Power_BJT",
            "dims": [10.0, 4.5, 9.0, 2.0]
        },
        "TO-247": {
            "pd_pkg_limit": 50.0, "rth_jc": 1.0, "rth_ja": 40, 
            "ic_limit": 30.0, "archetype": "Power_BJT",
            "dims": [16.0, 5.5, 4.8, 6.0]
        },
        "TO-3": {
            "pd_pkg_limit": 5.0, "rth_jc": 0.8, "rth_ja": 35, 
            "ic_limit": 30.0, "archetype": "Power_BJT", 
            "dims": [24.0, 20.0, 4.8, 15.0]
        }
    }

    # Physics: VCE(sat) Intrinsic Drop
    VCE_SAT_INTERNAL = {"Small_Signal": 0.05, "Power_BJT": 0.2}
    
    # Physics: Capacitance (Farads)
    COB_AREA_BASE = {"Small_Signal": 4.0e-12, "Power_BJT": 50.0e-12}
    CIB_CURRENT_SCALING = 0.5
    CIB_HFE_FACTOR = 0.3
    
    # Physics: fT (Hz)
    FT_BASE = {"Small_Signal": 300.0e6, "Power_BJT": 50.0e6}
# ============================================================================
# STRATEGY CLASS - Physics-Aware with SI Units
# ============================================================================
class BJTStrategy(ComponentStrategy):
    """Strategy for generating synthetic BJT datasheets with accurate physics"""
    
    def __init__(self):
        self.constants = BJTConstants()
    
    # ------------------------------------------------------------------------
    # 1. CREATE CONTEXT
    # ------------------------------------------------------------------------
    def create_context(self, schema: Dict, requested_corner: Optional[str]) -> GenerationContext:
        archetype = random.choice(schema.get('archetypes', ["Small_Signal"]))
        
        # Filter packages by archetype
        valid_packages = [p for p, data in self.constants.PACKAGE_SPECS.items() 
                          if data["archetype"] == archetype]
        if not valid_packages: valid_packages = ["SOT-23"] # Fallback
        
        package = random.choice(valid_packages)
        pkg_data = self.constants.PACKAGE_SPECS[package]
        
        corner = requested_corner if requested_corner in self.constants.CORNERS else "TYPICAL"
        
        # --- PHYSICS LOCK-IN ---
        # Determine VCEO and IC_MAX based on package limits + random selection
        primary = self._generate_primary_ratings(archetype, pkg_data)
        
        extras = {
            'requested_corner': requested_corner,
            'process_corner': corner,
            'vceo': primary['vceo'],
            'ic_max': primary['ic_max'],
            'pd_max': primary['pd_max'],
            'hfe_peak': primary['hfe_peak'],
            'ft_typ': primary['ft_typ'],
            'pkg_data': pkg_data,
            # Physics params calculated later
            'rc_bulk': None, 
            'early_voltage': None
        }
        
        return GenerationContext(
            sample_id=f"BJT_{archetype}_{package}_{uuid.uuid4().hex[:4]}",
            component_type="BJT",
            package=package,
            archetype=archetype,
            tolerance=0.1,
            process_corner=corner,
            extras=extras
        )
    
    # ------------------------------------------------------------------------
    # 2. GENERATE PRIMARY RATINGS (The Physics Engine)
    # ------------------------------------------------------------------------
    def _generate_primary_ratings(self, archetype: str, pkg_data: Dict) -> Dict[str, float]:
        
        # A. VCEO (Breakdown Voltage)
        vceo_opts = self.constants.VCEO_STANDARD[archetype]
        vceo = float(random.choice(vceo_opts))
        
        # B. IC MAX (Current Rating)
        # Must act as a constraint on the package limit
        pkg_i_lim = pkg_data['ic_limit']
        # Generate a value logarithmically up to the package limit
        ic_target = 10 ** random.uniform(math.log10(pkg_i_lim * 0.1), math.log10(pkg_i_lim))
        # Snap to nice numbers
        ic_max = self._snap_to_grid(ic_target)
        
        # C. PD MAX (Power Dissipation) - THERMAL PHYSICS
        # Pd = (Tj_max - Ta) / Rth_ja
        tj_max = 150.0
        ta = 25.0
        pd_thermal = (tj_max - ta) / pkg_data['rth_ja']
        # The rating is the minimum of Package Limit and Thermal Limit
        pd_max = min(pkg_data['pd_pkg_limit'], pd_thermal)
        
        # D. Gain (hFE)
        hfe_opts = [100, 150, 200, 300] if archetype == "Small_Signal" else [40, 60, 80, 100]
        hfe_peak = float(random.choice(hfe_opts))
        
        # E. fT (Speed)
        # Higher voltage/power usually means slower (larger base width/parasitics)
        ft_base = self.constants.FT_BASE[archetype]
        ft_typ = ft_base * (50.0 / vceo) ** 0.3  # Slight penalty for HV
        
        return {
            'vceo': vceo,
            'ic_max': ic_max,
            'pd_max': pd_max,
            'hfe_peak': hfe_peak,
            'ft_typ': ft_typ
        }

    # ------------------------------------------------------------------------
    # 3. CREATE CUSTOM PARAMETER
    # ------------------------------------------------------------------------
    def create_custom_parameter(self, key: str, context: GenerationContext, param_def: Dict) -> Optional[GeneratedParameter]:
        extras = context.extras
        corner_mult = self.constants.CORNERS[extras['process_corner']]
        
        # --- VCEO ---
        if key == 'vceo':
            val = extras['vceo'] * corner_mult['vceo']
            return self._build_param(key, val, "V", param_def, is_max=True)

        # --- IC MAX ---
        if key == 'ic_max':
            val_si = extras['ic_max']
            disp_val, unit = self._auto_scale(val_si, 'A')
            return self._build_param(key, disp_val, unit, param_def, is_max=True)

        # --- POWER DISSIPATION ---
        if key == 'power_dissipation':
            # Already calculated thermally
            val_si = extras['pd_max']
            disp_val, unit = self._auto_scale(val_si, 'W')
            return self._build_param(key, disp_val, unit, param_def, is_max=True)

        # --- hFE (DC Current Gain) ---
        if key == 'hfe':
            # Logic for test condition
            ic_test_si = self._get_safe_test_current(param_def, extras['ic_max'])
            
            # Physics Calculation
            hfe_val = self._calculate_hfe_physics(ic_test_si, context)
            hfe_val *= corner_mult['hfe']
            
            # Construct Display
            ic_disp, i_unit = self._auto_scale(ic_test_si, 'A')
            cond = f"VCE=5V, IC={ic_disp:.1f}{i_unit}"
            
            return GeneratedParameter(
                key='hfe', label="DC Current Gain", symbol="hFE",
                section='ELEC_CHAR', value_min=int(hfe_val*0.6), value_typ=int(hfe_val),
                value_max=int(hfe_val*1.5), unit="", condition=cond,
                spec_type=SpecType.NOMINAL, engineering_class=EngineeringClass.PERFORMANCE
            )

        # --- VCE(sat) ---
        if key == 'vce_sat':
            ic_test_si = self._get_safe_test_current(param_def, extras['ic_max'])
            ib_test_si = ic_test_si / 10.0 # Forced Beta = 10 standard
            
            # Physics Calculation
            v_sat = self._calculate_vce_sat_physics(ic_test_si, context)
            v_sat *= corner_mult['vce_sat']
            
            v_disp, v_unit = self._auto_scale(v_sat, 'V')
            ic_disp, i_unit = self._auto_scale(ic_test_si, 'A')
            ib_disp, ib_unit = self._auto_scale(ib_test_si, 'A')
            
            return GeneratedParameter(
                key='vce_sat', label="Collector-Emitter Saturation Voltage", symbol="VCE(sat)",
                section='ELEC_CHAR', value_typ=v_disp, value_max=v_disp*1.3,
                unit=v_unit, condition=f"IC={ic_disp:.1f}{i_unit}, IB={ib_disp:.1f}{ib_unit}",
                spec_type=SpecType.MAX_LIMIT, engineering_class=EngineeringClass.PERFORMANCE
            )

        # --- Output Capacitance (Cob) ---
        if key == 'cob':
            # Physics: Area based + Package Parasitic
            # Area scales roughly with Ic_max
            area_factor = math.sqrt(extras['ic_max'] / 0.1) 
            base_cob = self.constants.COB_AREA_BASE[context.archetype]
            pkg_c = self.constants.PKG_PARASITIC_C.get(context.package, 1e-12)
            
            # Cob = C_junction + C_pkg. Junction C decreases with Voltage (V^-0.5)
            # Standard test is VCB=10V
            v_test = 10.0
            cob_junc = (base_cob * area_factor) / math.sqrt(v_test)
            cob_total = cob_junc + pkg_c
            
            val_disp, unit = self._auto_scale(cob_total, 'F')
            return GeneratedParameter(
                key='cob', label="Output Capacitance", symbol="Cob",
                section='ELEC_CHAR', value_typ=val_disp, value_max=val_disp*1.3,
                unit=unit, condition="VCB=10V, f=1MHz",
                spec_type=SpecType.MAX_LIMIT
            )

        # --- fT (Transition Frequency) ---
        if key == 'ft':
            val_disp, unit = self._auto_scale(extras['ft_typ'], 'Hz')
            return GeneratedParameter(
                key='ft', label="Transition Frequency", symbol="fT",
                section='ELEC_CHAR', value_typ=val_disp, unit=unit,
                condition="VCE=10V, f=100MHz", spec_type=SpecType.TYPICAL
            )

        return None

    # ------------------------------------------------------------------------
    # 4. PHYSICS HELPERS
    # ------------------------------------------------------------------------
    def _calculate_hfe_physics(self, ic: float, ctx: GenerationContext) -> float:
        """
        Calculates hFE at a specific current using a roll-off model.
        
        """
        extras = ctx.extras
        peak_hfe = extras['hfe_peak']
        ic_max = extras['ic_max']
        
        # Normalized current
        ic_norm = ic / ic_max
        
        # 1. Low Current Roll-off (Recombination)
        # Below 1% of rated current, gain drops
        low_current_factor = 1.0
        if ic_norm < 0.01:
            low_current_factor = (ic_norm / 0.01) ** 0.5
            
        # 2. High Current Roll-off (Kirk Effect / High Injection)
        # Above 50% of rated current, gain drops
        high_current_factor = 1.0
        if ic_norm > 0.5:
            high_current_factor = 1.0 / (1.0 + 2.0 * (ic_norm - 0.5)**2)
            
        return peak_hfe * low_current_factor * high_current_factor

    def _calculate_vce_sat_physics(self, ic: float, ctx: GenerationContext) -> float:
        """
        Calculates VCE(sat) = V_offset + I * R_bulk
        """
        extras = ctx.extras
        
        # Lazy load Rc_bulk if not set
        if extras['rc_bulk'] is None:
            # We want Vce(sat) to be approx 0.3V - 0.5V at max current
            target_v = 0.4 # Volts
            v_int = self.constants.VCE_SAT_INTERNAL[ctx.archetype]
            extras['rc_bulk'] = (target_v - v_int) / extras['ic_max']
            
        r_bulk = extras['rc_bulk']
        v_int = self.constants.VCE_SAT_INTERNAL[ctx.archetype]
        
        # Temperature effect (Simulation): Hotter = Higher Resistance
        if ctx.process_corner == "MAX_SPEC" or "HOT" in str(ctx.extras.get('requested_corner')):
             r_bulk *= 1.4 # Silicon resistance increases with temp
             
        return v_int + (ic * r_bulk)

    def _get_safe_test_current(self, param_def: Dict, ic_max: float) -> float:
        """Extracts a test current from schema condition, bounded by physics"""
        cond = param_def['scenarios'][0].get('condition', '')
        
        # Try to parse "IC=..."
        match = re.search(r'IC\s*=\s*([\d.]+)\s*([mµu]?)A', cond, re.IGNORECASE)
        if match:
            val = float(match.group(1))
            prefix = match.group(2).lower()
            if prefix == 'm': val *= 1e-3
            elif prefix in ['µ', 'u']: val *= 1e-6
            return val
            
        # Default logic if no condition found
        # Return 10% of max current, but lower bound 1mA
        return max(ic_max * 0.1, 1e-3)

    # ------------------------------------------------------------------------
    # 5. UTILS
    # ------------------------------------------------------------------------
    def _auto_scale(self, value: float, base_unit: str) -> Tuple[float, str]:
        if value is None or value == 0: return 0, base_unit
        
        # Special case for Ohms
        if base_unit == 'Ω':
            if value < 1.0: return value*1000, 'mΩ'
            return value, 'Ω'

        prefixes = [(1e9,'G'), (1e6,'M'), (1e3,'k'), (1,''), (1e-3,'m'), (1e-6,'µ'), (1e-9,'n'), (1e-12,'p')]
        
        for scale, prefix in prefixes:
            if abs(value) >= scale:
                # formatting to avoid 0.005 kA -> 5 A logic loop if not careful, 
                # but simple greedy works for standard BOM
                if abs(value) < scale * 1000: 
                    return value / scale, f"{prefix}{base_unit}"
        
        return value, base_unit

    def _build_param(self, key, val, unit, pd, is_max=False):
        return GeneratedParameter(
            key=key, label=pd['llm_context']['formal_name'], symbol=pd['symbol'],
            section='ABS_MAX' if is_max else 'ELEC_CHAR',
            value_typ=val if not is_max else None, value_max=val if is_max else None,
            unit=unit, condition=pd['scenarios'][0]['condition'],
            spec_type=SpecType.MAX_RATING if is_max else SpecType.NOMINAL
        )

    def _snap_to_grid(self, val):
        # Helper to make numbers look "engineered" (e.g. 0.15 instead of 0.14832)
        if val < 1: return round(val, 3)
        if val < 10: return round(val, 2)
        return int(val)

    # Required abstract method implementation
    def calculate_base_value(self, key: str, limits: Any, context: GenerationContext) -> Any:
        # Pass-through for package dims if needed by pipeline
        if key == 'package_code': return context.package
        if key in ['width', 'length', 'height']:
            idx = {'length':0, 'width':1, 'height':2}[key]
            return context.extras['pkg_data']['dims'][idx]
        return None

    def apply_correlations(self, result: DatasheetResult) -> None:
        pass

Overwriting synthetic_pipeline/bjt_strategy.py


מחלקה יורשת לרכיב OPAMP

In [10]:
%%writefile synthetic_pipeline/opamp_strategy.py
import random
import math
import uuid
import re
from typing import Dict, Any, Optional, List, Tuple
from dataclasses import dataclass

# ==========================================
# 1. PIPELINE IMPORTS
# ==========================================
from synthetic_pipeline.data_types import (
    GenerationContext, 
    DatasheetResult, 
    GeneratedParameter,
    SpecType, 
    EngineeringClass,
)

from synthetic_pipeline.strategy_base import ComponentStrategy



# ============================================================================
# CONSTANTS CLASS - Physics-Aware with SI Units
# ============================================================================
class OpampConstants:
    """Constants and lookup tables for OPAMP generation with accurate physics"""
    
    # Process Corners for variation
    # Allows simulation of Worst Case vs Best Case scenarios
    CORNERS = {
        "TYPICAL":    {"bw": 1.0, "iq": 1.0, "vos": 1.0, "ios": 1.0, "sr": 1.0},
        "WORST_CASE": {"bw": 0.8, "iq": 1.2, "vos": 1.5, "ios": 1.5, "sr": 0.8}, # Slower, hungrier, less precise
        "BEST_CASE":  {"bw": 1.2, "iq": 0.8, "vos": 0.6, "ios": 0.6, "sr": 1.2}  # Faster, efficient, precise
    }

    # Archetype Physics Metadata
    # tech: 'Bipolar' or 'CMOS' determines Gm vs Id relationship
    # sr_factor: Base multiplier for Slew Rate physics
    # noise_k: Coefficient for noise density calculation
    # bias_base: Base input bias current
    PHYSICS_META = {
        "General_Purpose":      {"tech": "Bipolar", "sr_factor": 1.0, "noise_k": 20e-9, "bias_base": 20e-9, "max_gbw": 20e6, "is_cmos": False},
        "Precision_Zero_Drift": {"tech": "CMOS",    "sr_factor": 0.5, "noise_k": 40e-9, "bias_base": 10e-12,"max_gbw": 10e6, "is_cmos": True},
        "High_Speed":           {"tech": "Bipolar", "sr_factor": 5.0, "noise_k": 3e-9,  "bias_base": 2e-6,  "max_gbw": 500e6, "is_cmos": False},
        "Low_Power":            {"tech": "CMOS",    "sr_factor": 0.8, "noise_k": 80e-9, "bias_base": 1e-12, "max_gbw": 5e6, "is_cmos": True},
        "JFET_Input":           {"tech": "JFET",    "sr_factor": 3.0, "noise_k": 15e-9, "bias_base": 50e-12,"max_gbw": 50e6, "is_cmos": False}
    }

    # Supply Voltage Ranges (Min, Max)
    SUPPLY_RANGES = {
        "General_Purpose": [(3.0, 32.0), (4.5, 36.0)],
        "Precision_Zero_Drift": [(1.8, 5.5), (2.7, 16.0)],
        "High_Speed": [(2.5, 5.5), (4.5, 12.0), (8.0, 24.0)],
        "Low_Power": [(1.8, 5.5), (2.2, 16.0)],
        "JFET_Input": [(4.5, 36.0)]
    }

    # Package Thermal Limits (Watts) & Rth_ja (C/W)
    # pd_max: Limit of the case itself
    PACKAGE_SPECS = {
        "SOT-23-5": {"pd_max": 0.25, "rth_ja": 250, "dims": [2.9, 1.6, 1.1]},
        "SC-70":    {"pd_max": 0.15, "rth_ja": 300, "dims": [2.0, 1.25, 0.95]},
        "SOIC-8":   {"pd_max": 0.65, "rth_ja": 120, "dims": [4.9, 3.9, 1.5]},
        "TSSOP-8":  {"pd_max": 0.45, "rth_ja": 160, "dims": [3.0, 4.4, 1.1]},
        "DIP-8":    {"pd_max": 1.00, "rth_ja": 90,  "dims": [9.3, 6.4, 3.3]},
        "QFN-16":   {"pd_max": 1.50, "rth_ja": 40,  "dims": [3.0, 3.0, 0.85]}
    }
# ============================================================================
# STRATEGY CLASS
# ============================================================================
class OpampStrategy(ComponentStrategy):
    """
    Hermetic OPAMP Strategy with rigorous Physics Engine.
    Enforces correlations between Iq, GBW, SR, Noise, and Package Limits.
    """

    def __init__(self):
        self.constants = OpampConstants()

    # ------------------------------------------------------------------------
    # 1. CREATE CONTEXT
    # ------------------------------------------------------------------------
    def create_context(self, schema: Dict, requested_corner: Optional[str]) -> GenerationContext:
        archetype = random.choice(schema.get('archetypes', ['General_Purpose']))
        
        # Package Selection with Validity Check
        valid_pkgs = list(self.constants.PACKAGE_SPECS.keys())
        if archetype == "High_Speed":
            # High speed needs low inductance, prefer SMT
            valid_pkgs = [p for p in valid_pkgs if "DIP" not in p]
        elif archetype == "Precision_Zero_Drift":
            # Precision often comes in standard pinouts
            valid_pkgs = [p for p in valid_pkgs if "SOIC" in p or "SOT" in p]
            
        package = random.choice(valid_pkgs)
        corner = requested_corner if requested_corner in self.constants.CORNERS else "TYPICAL"

        # --- HERMETIC PHYSICS GENERATION ---
        # Calculate all dependent values BEFORE creating parameters
        phys_data = self._generate_physics_state(archetype, package, corner)
        
        extras = {
            'requested_corner': requested_corner,
            'process_corner': corner,
            'package': package,
            'pkg_specs': self.constants.PACKAGE_SPECS[package],
            
            # Locked Values (SI Units)
            'supply_min': phys_data['supply_min'],
            'supply_max': phys_data['supply_max'],
            'iq_typ': phys_data['iq_typ'],
            'gbw_typ': phys_data['gbw_typ'],
            'sr_typ': phys_data['sr_typ'],
            'vos_typ': phys_data['vos_typ'],
            'bias_typ': phys_data['bias_typ'],
            'ios_typ': phys_data['ios_typ'],
            'en_typ': phys_data['en_typ'],
            'cmrr_typ': phys_data['cmrr_typ'],
            'psrr_typ': phys_data['psrr_typ'],
            
            # Metadata
            'tech': self.constants.PHYSICS_META[archetype]['tech']
        }

        return GenerationContext(
            sample_id=f"OPAMP_{archetype[:3].upper()}_{uuid.uuid4().hex[:4]}",
            component_type="OPAMP",
            package=package,
            archetype=archetype,
            tolerance=0.1,
            process_corner=corner,
            extras=extras
        )

    # ------------------------------------------------------------------------
    # 2. PHYSICS ENGINE (The Core Logic)
    # ------------------------------------------------------------------------
    def _generate_physics_state(self, archetype: str, package: str, corner: str) -> Dict[str, float]:
        """
        Calculates a consistent set of parameters based on physics constraints.
        """
        meta = self.constants.PHYSICS_META[archetype]
        pkg_data = self.constants.PACKAGE_SPECS[package]
        c_data = self.constants.CORNERS[corner]
        
        # 1. Supply Voltage Selection
        # High speed often means lower voltage due to breakdown/speed tradeoffs
        v_range = random.choice(self.constants.SUPPLY_RANGES[archetype])
        v_min, v_max = v_range
        
        # 2. Quiescent Current (Iq) - The Driver
        # Iq drives GBW and SR. Constrained by Package Power.
        if archetype == "Low_Power":
            iq_target = random.uniform(0.5e-6, 50e-6)
        elif archetype == "High_Speed":
            iq_target = random.uniform(1e-3, 20e-3)
        else:
            iq_target = random.uniform(100e-6, 3e-3)
            
        # Apply Corner variation to Iq
        iq_target *= c_data['iq']
            
        # Thermal Safety Check: P_d = V_max * Iq
        # Limit Iq so P_d < 50% of Package Max (Headroom for load current)
        max_iq_thermal = (pkg_data['pd_max'] * 0.5) / v_max
        iq_target = min(iq_target, max_iq_thermal)
        
        # 3. GBW Calculation (Technology Dependent)
        # Bipolar: gm ~ Ic/Vt -> GBW linear with Iq
        # CMOS/JFET: gm ~ sqrt(Id) -> GBW sqrt with Iq
        # C_comp assumed roughly constant (~5-10pF internal)
        if meta['tech'] == "Bipolar":
            # Efficiency: ~10-20 MHz/mA for good bipolar
            gbw_efficiency = random.uniform(10e9, 20e9) 
            gbw = iq_target * gbw_efficiency
        else: # CMOS/JFET
            # Efficiency: ~2-5 MHz/sqrt(mA) 
            gbw_efficiency = random.uniform(100e6, 200e6) 
            gbw = math.sqrt(iq_target) * gbw_efficiency
            
        # Clamp GBW to realistic max per archetype
        gbw = min(gbw, meta['max_gbw'])
        gbw *= c_data['bw'] # Corner effect
        
        # 4. Slew Rate (SR)
        # SR = I_tail / C_c. Since GBW = gm / C_c, and gm ~ I (Bip) or sqrt(I) (MOS),
        # SR tracks Iq strongly.
        # Physics model: SR [V/s] approx = k * Iq
        sr_k = 1e9 if meta['tech'] == "Bipolar" else 5e8 # V/s per Ampere rough scaling
        if archetype == "High_Speed": sr_k *= 5.0 # Current feedback / slew enhanced
        
        sr_val_si = iq_target * sr_k * c_data['sr']
        
        # Slew Rate Saturation Check: SR cannot exceed physics of supply rails vs freq
        # This prevents 500V/us in a 5V part which implies unrealistic bandwidth
        # No strict limit, but sanity clamp:
        sr_val_si = max(sr_val_si, 0.01e6) # Min 0.01 V/us
        
        # 5. Noise (en)
        # Thermal noise dominant: Vn ~ sqrt(4kTR) ~ 1/sqrt(gm)
        # So Vn scales with 1/sqrt(Iq) generally for both techs (simplified)
        en_val = meta['noise_k'] * math.sqrt(1e-3 / max(iq_target, 1e-9))
        
        # 6. Offset (Vos) & Bias (Ib)
        # Vos: Random process mismatch. Better in Bipolar, worse in CMOS (unless Chopper)
        vos_base = 50e-6 if "Precision" in archetype else 1e-3
        vos = vos_base * random.uniform(0.5, 2.0) * c_data['vos']
        
        # Ib: Technology definition
        # Bipolar: Ib ~ Iq / Beta. Beta ~ 100-200.
        # CMOS/JFET: Leakage driven.
        if meta['tech'] == "Bipolar":
            beta = random.uniform(100, 300)
            ib = iq_target / beta
        else:
            ib = meta['bias_base'] * random.uniform(0.5, 5.0) # Leakage variation
            
        ib *= c_data['ios']
        ios = ib * 0.1 # Offset current typically 10% of bias
        
        # 7. CMRR / PSRR
        # Linked to Open Loop Gain (Aol). Higher GBW usually implies lower DC gain in simple stages,
        # but modern topologies compensate.
        # High precision = High CMRR.
        cmrr_base = 120 if "Precision" in archetype else 90
        cmrr = cmrr_base + random.uniform(-10, 10)
        psrr = cmrr - random.uniform(0, 20)
        
        return {
            'supply_min': v_min,
            'supply_max': v_max,
            'iq_typ': iq_target,
            'gbw_typ': gbw,
            'sr_typ': sr_val_si,
            'vos_typ': vos,
            'bias_typ': ib,
            'ios_typ': ios,
            'en_typ': en_val,
            'cmrr_typ': cmrr,
            'psrr_typ': psrr
        }

    # ------------------------------------------------------------------------
    # 3. PARAMETER CREATION (Display Layer)
    # ------------------------------------------------------------------------
    def create_custom_parameter(self, key: str, context: GenerationContext, param_def: Dict) -> Optional[GeneratedParameter]:
        extras = context.extras
        
        if key == 'supply_voltage_range':
            return GeneratedParameter(
                key=key, label="Supply Voltage Range", symbol="Vs", section="REC_OP_COND",
                value_min=extras['supply_min'], value_max=extras['supply_max'], unit="V",
                condition="Single Supply", spec_type=SpecType.OPERATIONAL_RANGE,
                engineering_class=EngineeringClass.OPERATING_CONDITION
            )

        if key == 'supply_current':
            val = extras['iq_typ']
            disp, unit = self._auto_scale(val, 'A')
            return GeneratedParameter(
                key=key, label="Quiescent Current", symbol="Iq", section="ELEC_CHAR",
                value_typ=self._round(disp, 2), value_max=self._round(disp*1.2, 2), unit=unit,
                condition="No Load", spec_type=SpecType.TYPICAL,
                engineering_class=EngineeringClass.PERFORMANCE
            )

        if key == 'gbw':
            val = extras['gbw_typ']
            disp, unit = self._auto_scale(val, 'Hz')
            return GeneratedParameter(
                key=key, label="Gain Bandwidth Product", symbol="GBW", section="ELEC_CHAR",
                value_typ=self._round(disp, 1), unit=unit, condition="RL=10k",
                spec_type=SpecType.TYPICAL, engineering_class=EngineeringClass.PERFORMANCE
            )

        if key == 'slew_rate':
            # Display in V/µs
            val_us = extras['sr_typ'] / 1e6
            # Intelligent rounding
            disp = self._round(val_us, 2) if val_us < 10 else self._round(val_us, 0)
            return GeneratedParameter(
                key=key, label="Slew Rate", symbol="SR", section="ELEC_CHAR",
                value_typ=disp, unit="V/µs", condition="RL=2k",
                spec_type=SpecType.TYPICAL, engineering_class=EngineeringClass.PERFORMANCE
            )

        if key == 'offset_voltage':
            val = extras['vos_typ']
            disp, unit = self._auto_scale(val, 'V')
            return GeneratedParameter(
                key=key, label="Input Offset Voltage", symbol="Vos", section="ELEC_CHAR",
                value_typ=self._round(disp, 2), value_max=self._round(disp*2, 2), unit=unit,
                spec_type=SpecType.MAX_LIMIT, engineering_class=EngineeringClass.PERFORMANCE
            )

        if key == 'input_bias_current':
            val = extras['bias_typ']
            disp, unit = self._auto_scale(val, 'A')
            return GeneratedParameter(
                key=key, label="Input Bias Current", symbol="Ib", section="ELEC_CHAR",
                value_typ=self._round(disp, 2), value_max=self._round(disp*2, 2), unit=unit,
                spec_type=SpecType.MAX_LIMIT, engineering_class=EngineeringClass.PERFORMANCE
            )
            
        if key == 'input_offset_current':
            val = extras['ios_typ']
            disp, unit = self._auto_scale(val, 'A')
            return GeneratedParameter(
                key=key, label="Input Offset Current", symbol="Ios", section="ELEC_CHAR",
                value_typ=self._round(disp, 2), value_max=self._round(disp*2, 2), unit=unit,
                spec_type=SpecType.MAX_LIMIT, engineering_class=EngineeringClass.PERFORMANCE
            )

        if key == 'voltage_noise_density':
            val = extras['en_typ']
            disp, unit = self._auto_scale(val, 'V')
            return GeneratedParameter(
                key=key, label="Voltage Noise Density", symbol="en", section="ELEC_CHAR",
                value_typ=self._round(disp, 1), unit=f"{unit}/√Hz", condition="f=1kHz",
                spec_type=SpecType.TYPICAL, engineering_class=EngineeringClass.PERFORMANCE
            )
            
        if key == 'cmrr':
            val = extras['cmrr_typ']
            return GeneratedParameter(
                key=key, label="Common Mode Rejection Ratio", symbol="CMRR", section="ELEC_CHAR",
                value_min=int(val-10), value_typ=int(val), unit="dB",
                spec_type=SpecType.MIN_LIMIT, engineering_class=EngineeringClass.PERFORMANCE
            )

        return None

    # ------------------------------------------------------------------------
    # 4. BASE VALUES
    # ------------------------------------------------------------------------
    def calculate_base_value(self, key: str, limits: Any, context: GenerationContext) -> Any:
        pkg_spec = context.extras['pkg_specs']
        if key == 'package_code': return context.package
        if key == 'dimensions_lxwxh': return pkg_spec['dims']
        if key == 'power_dissipation': return pkg_spec['pd_max']
        if key == 'thermal_resistance_ja': return pkg_spec['rth_ja']
        return None

    # ------------------------------------------------------------------------
    # 5. CORRELATIONS
    # ------------------------------------------------------------------------
    def apply_correlations(self, result: DatasheetResult) -> None:
        """Apply secondary correlations"""
        extras = result.context.extras
        
        for param in result.parameters:
            if param.key == 'settling_time':
                gbw = extras['gbw_typ']
                # ts (0.01%) ~ 0.7 / GBW  (Single pole approx)
                ts_sec = 0.7 / gbw 
                disp, unit = self._auto_scale(ts_sec, 's')
                param.value_typ = self._round(disp, 1)
                param.unit = unit

    # ------------------------------------------------------------------------
    # UTILS
    # ------------------------------------------------------------------------
    def _auto_scale(self, value: float, base_unit: str) -> Tuple[float, str]:
        if value is None or value == 0: return 0, base_unit
        
        prefixes = [(1e-12,'p'), (1e-9,'n'), (1e-6,'µ'), (1e-3,'m'), (1,''), (1e6,'M')]
        val_abs = abs(value)
        best_scale = 1.0
        best_prefix = ''
        
        for scale, prefix in prefixes:
            if val_abs >= scale:
                # Avoid scaling 0.005A to 5mA if standard is A, but usually valid
                if val_abs < scale * 1000:
                    best_scale = scale
                    best_prefix = prefix
        
        # Cleanup
        if best_prefix == '' and base_unit == 'V' and val_abs < 1.0: # Prefer mV for small volts
             return value * 1000, 'mV'

        return value / best_scale, f"{best_prefix}{base_unit}"

    def _round(self, val, digits):
        if val is None: return None
        return round(val, digits)

Overwriting synthetic_pipeline/opamp_strategy.py


# חלק ה- main

In [ ]:
"""
Production Runner - הרצת ייצור דאטה סינטטית
גרסה נקייה לייצור (Production Ready)
"""

import json
import os
import time
import threading
import cv2
import numpy as np
import random
from tqdm import tqdm

from datetime import datetime
from typing import Dict, List, Any
from dataclasses import asdict, dataclass, field
from concurrent.futures import ThreadPoolExecutor, as_completed
from html2image import Html2Image

# 1. Logic & DB
from synthetic_pipeline.logic import DatasheetGenerator
from synthetic_pipeline.strategies import UNIFIED_COMPONENT_DB, MODEL_TO_USE

# 2. Strategies
from synthetic_pipeline.resistor_strategy import ResistorStrategy
from synthetic_pipeline.capacitor_strategy import CapacitorStrategy
from synthetic_pipeline.diode_strategy import DiodeStrategy
from synthetic_pipeline.mosfet_strategy import MosfetStrategy
from synthetic_pipeline.voltage_Regulator_strategy import VoltageRegulatorStrategy
from synthetic_pipeline.inductor_strategy import InductorStrategy
from synthetic_pipeline.bjt_strategy import BJTStrategy
from synthetic_pipeline.opamp_strategy import OpampStrategy

# 3. Generators
from synthetic_pipeline.generators import (
    DatasheetHtmlRenderer, 
    DatasheetJSONRenderer, 
    MarketingGenerator,
    ScanAugmentor
)

# ============================================================================
# DATA VALIDATION
# ============================================================================

class DataValidator:
    """מנגנון ולידציה לסינון דאטה פגומה"""
    
    @staticmethod
    def validate_marketing_data(data: Dict) -> tuple[bool, List[str]]:
        errors = []
        if not data:
            return False, ["Empty data"]
            
        required_fields = ['marketing_name', 'manufacturer', 'description']
        for field in required_fields:
            if field not in data or not data[field]:
                errors.append(f"Missing or empty field: {field}")
        
        if 'marketing_name' in data and len(data['marketing_name']) < 3:
            errors.append("Marketing name too short")
        if 'description' in data and len(data['description']) < 50:
            errors.append("Description too short")
        
        invalid_markers = ['error', 'failed', 'null', 'undefined', '###', 'generic comp']
        for field, value in data.items():
            if isinstance(value, str):
                for marker in invalid_markers:
                    if marker.lower() in value.lower():
                        errors.append(f"Invalid content in {field}: contains '{marker}'")
        
        return len(errors) == 0, errors
    
    @staticmethod
    def validate_technical_data(data: Dict) -> tuple[bool, List[str]]:
        errors = []
        if not data:
            return False, ["Empty technical data"]
        
        if 'parameters' in data:
            params = data['parameters']
            if not params or len(params) == 0:
                errors.append("No parameters found")
            
            for param in params:
                if not isinstance(param, dict): continue
                if 'label' not in param or not param['label']:
                    errors.append("Parameter missing label")
                
                has_value = (
                    ('value_min' in param and param['value_min'] is not None) or
                    ('value_typ' in param and param['value_typ'] is not None) or
                    ('value_max' in param and param['value_max'] is not None)
                )
                if not has_value:
                    errors.append(f"Parameter {param.get('label', 'unknown')} has no values")
        
        return len(errors) == 0, errors


# ============================================================================
# JSONL WRITER
# ============================================================================

class JSONLWriter:
    def __init__(self, filepath: str):
        self.filepath = filepath
        self.lock = threading.Lock()
        
    def append(self, data: Dict) -> bool:
        try:
            with self.lock:
                with open(self.filepath, 'a', encoding='utf-8') as f:
                    json_str = json.dumps(data, ensure_ascii=False, default=str)
                    f.write(json_str + '\n')
            return True
        except Exception:
            return False


# ============================================================================
# PRODUCTION STATISTICS
# ============================================================================

@dataclass
class ProductionStats:
    total_attempted: int = 0
    total_success: int = 0
    total_failed: int = 0
    fast_mode: int = 0
    high_quality_mode: int = 0
    validation_failures: int = 0
    component_breakdown: Dict[str, int] = field(default_factory=dict)
    
    def add_success(self, comp_type: str, mode: str):
        self.total_success += 1
        if mode == 'fast': self.fast_mode += 1
        else: self.high_quality_mode += 1
        self.component_breakdown[comp_type] = self.component_breakdown.get(comp_type, 0) + 1
    
    def add_failure(self, comp_type: str):
        self.total_failed += 1
        self.validation_failures += 1
    
    def print_summary(self):
        print("\n" + "="*60)
        print("📊 PRODUCTION SUMMARY")
        print("="*60)
        print(f"Total Attempted:       {self.total_attempted}")
        print(f"✅ Successful:         {self.total_success}")
        print(f"❌ Failed:             {self.total_failed}")
        if self.total_attempted > 0:
            print(f"Success Rate:          {(self.total_success/self.total_attempted*100):.1f}%")
        print(f"\n🚀 Fast Mode:          {self.fast_mode}")
        print(f"💎 High Quality Mode:  {self.high_quality_mode}")
        print(f"⚠️  Validation Fails:   {self.validation_failures}")
        if self.component_breakdown:
            print(f"\nComponent Breakdown:")
            for comp, count in self.component_breakdown.items():
                print(f"   {comp:20s}: {count}")
        print("="*60 + "\n")


# ============================================================================
# MAIN PRODUCTION FUNCTION
# ============================================================================

def run_full_production(
    num_samples_per_type: int = 10,
    fast_ratio: float = 0.8,
    output_dir: str = "production_output",
    enable_parallel: bool = True,
    max_workers: int = 4,
    components_to_generate: List[str] = None,
    ollama_model: str = MODEL_TO_USE, 
    save_html_files: bool = True,
    save_image_files: bool = True,
    save_individual_json: bool = True
):
    print("🚀 Starting Production Run...")
    
    # 1. Setup Generators & Strategies
    generator = DatasheetGenerator(UNIFIED_COMPONENT_DB)
    
    strategies = {
        "RESISTOR": ResistorStrategy(),
        "CAPACITOR": CapacitorStrategy(),
        "DIODE": DiodeStrategy(),
        "MOSFET": MosfetStrategy(),
        "VOLTAGE_REGULATOR": VoltageRegulatorStrategy(),
        "INDUCTOR": InductorStrategy(),
        "BJT": BJTStrategy(),
        "OPAMP": OpampStrategy()
    }
    
    html_renderer = DatasheetHtmlRenderer()
    json_renderer = DatasheetJSONRenderer()
    
    # --- Image Gen & Augmentor ---
    image_generator = None
    try:
        image_generator = Html2Image(browser='edge') 
    except:
        try: image_generator = Html2Image() # Fallback
        except: pass
            
    # Initialize augmentor but don't crash if imports fail
    augmentor = None
    try: augmentor = ScanAugmentor()
    except: pass

    image_lock = threading.Lock()
    
    if components_to_generate is None:
        components_to_generate = list(strategies.keys())
    
    # Create Directories
    os.makedirs(output_dir, exist_ok=True)
    html_dir = os.path.join(output_dir, "html")
    json_dir = os.path.join(output_dir, "json")
    image_dir = os.path.join(output_dir, "images")
    os.makedirs(html_dir, exist_ok=True)
    os.makedirs(json_dir, exist_ok=True)
    os.makedirs(image_dir, exist_ok=True)
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    jsonl_path = os.path.join(output_dir, f"production_{timestamp}.jsonl")
    valid_jsonl = JSONLWriter(jsonl_path)
    invalid_jsonl = JSONLWriter(jsonl_path.replace('.jsonl', '_invalid.jsonl'))
    
    stats = ProductionStats()
    validator = DataValidator()
    
    num_fast = int(num_samples_per_type * fast_ratio)
    
    # === Single Sample Processing Function ===
    # === Single Sample Processing Function ===
    def process_single_sample(comp_type: str, sample_idx: int, mode: str):
        strategy = strategies.get(comp_type)
        if not strategy: return
        
        try:
            # A. Logic
            result = generator.generate(comp_type, strategy)
            
            # B. Marketing
            marketing_gen = MarketingGenerator(mode=mode, model=ollama_model)
            marketing_data = marketing_gen.generate(result.context)
            
            # C. Validation
            m_valid, _ = validator.validate_marketing_data(marketing_data)
            
            technical_data = {
                'parameters': [asdict(p) for p in result.parameters],
                'component_type': comp_type,
                'sample_id': result.context.sample_id
            }
            t_valid, _ = validator.validate_technical_data(technical_data)
            
            is_valid = m_valid and t_valid
            
            # D. Render JSON & Ground Truth (Before creating record)
            rendered_json_output = json_renderer.render(result, marketing_data)
            
            record = {
                'sample_id': result.context.sample_id,
                'component_type': comp_type,
                'mode': mode,
                'timestamp': datetime.now().isoformat(),
                'is_valid': is_valid,
                'marketing_data': marketing_data,
                'ground_truth': rendered_json_output['ground_truth'], # Critical: Use processed strings
                'raw_technical_data': technical_data
            }
            
            if is_valid:
                valid_jsonl.append(record)
                
                # E. HTML & Files
                html_output = html_renderer.render(result, marketing_data)
                
                # --- FIX: Force White Background ---
                # מונע רקע שחור/שקוף בתמונות
                html_output = html_output.replace("</head>", "<style>body { background-color: white !important; }</style></head>")
                # -----------------------------------

                base_filename = f"datasheet_{result.context.sample_id}"

                if save_html_files:
                    with open(os.path.join(html_dir, f"{base_filename}.html"), "w", encoding="utf-8") as f:
                        f.write(html_output)

                if save_individual_json:
                    with open(os.path.join(json_dir, f"{base_filename}.json"), "w", encoding="utf-8") as f:
                        json.dump(rendered_json_output, f, indent=2, ensure_ascii=False, default=str)

                # F. Image Generation (With Augmentation)
                if image_generator and save_image_files:
                    with image_lock:
                        image_size = (1200, 5000)
                        temp_filename = f"{base_filename}_temp.png"
                        
                        try:
                            image_generator.screenshot(html_str=html_output, save_as=temp_filename, size=image_size)
                            
                            if os.path.exists(temp_filename):
                                img = cv2.imread(temp_filename)
                                if img is not None:
                                    # 1. חיתוך (Crop)
                                    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                                    # Invert gray to find black pixels on white background
                                    gray = 255*(gray < 250).astype(np.uint8) 
                                    coords = cv2.findNonZero(gray)
                                    
                                    if coords is not None:
                                        x, y, w, h = cv2.boundingRect(coords)
                                        # מוודא שלא חותכים יותר מדי (מינימום רוחב)
                                        w = max(w, 500) 
                                        img = img[0:y+h+50, :] 

                                    final_filename = f"{base_filename}.jpg"
                                    final_path = os.path.join(image_dir, final_filename)
                                    
                                    # 2. אסטרטגיית אוגמנטציה (לכלוך)
                                    # -----------------------------------------------
                                    rand_val = random.random()
                                    jpeg_quality = 95 # ברירת מחדל גבוהה

                                    if augmentor:
                                        # מצב 1: נקי לחלוטין (10%)
                                        if rand_val < 0.10:
                                            jpeg_quality = 95
                                        
                                        # מצב 2: לכלוך קל (45%)
                                        elif rand_val < 0.55:
                                            jpeg_quality = random.randint(70, 85)
                                            if random.random() < 0.3:
                                                img = augmentor.apply_blur(img)

                                        # מצב 3: סריקה ישנה / מלוכלכת (45%)
                                        else:
                                            img = augmentor.apply_rotation(img, max_angle=1.5)
                                            if random.random() < 0.8:
                                                img = augmentor.apply_scan_artifacts(img)
                                            if random.random() < 0.4:
                                                img = augmentor.apply_salt_and_pepper(img, prob=0.002)
                                            if random.random() < 0.4:
                                                img = augmentor.apply_blur(img)
                                            jpeg_quality = random.randint(50, 70)

                                    # 3. שמירה סופית
                                    cv2.imwrite(final_path, img, [int(cv2.IMWRITE_JPEG_QUALITY), jpeg_quality])
                                    os.remove(temp_filename)
                            
                        except Exception:
                            # Clean up if failed
                            if os.path.exists(temp_filename):
                                try: os.remove(temp_filename)
                                except: pass

                stats.add_success(comp_type, mode)
                
            else:
                invalid_jsonl.append(record)
                stats.add_failure(comp_type)
            
            stats.total_attempted += 1
            
        except Exception:
            # Silent fail for production speed, just log stats
            stats.total_failed += 1
            stats.total_attempted += 1

    # === Main Loop ===
    
    total_tasks = len(components_to_generate) * num_samples_per_type
    start_time = time.time()
    
    pbar = tqdm(total=total_tasks, unit="sample", desc="🏭 Generating Data")
    
    if enable_parallel:
        print(f"🔄 Processing with {max_workers} workers...")
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            futures = []
            for comp_type in components_to_generate:
                # Fast samples
                for i in range(num_fast):
                    futures.append(executor.submit(process_single_sample, comp_type, i, 'fast'))
                # High Quality samples
                for i in range(num_fast, num_samples_per_type):
                    futures.append(executor.submit(process_single_sample, comp_type, i, 'high_quality'))
            
            for _ in as_completed(futures):
                pbar.update(1)
    else:
        print("➡️ Processing sequentially...")
        for comp_type in components_to_generate:
            for i in range(num_fast):
                process_single_sample(comp_type, i, 'fast')
                pbar.update(1)
            for i in range(num_fast, num_samples_per_type):
                process_single_sample(comp_type, i, 'high_quality')
                pbar.update(1)
                
    pbar.close()
    
    elapsed = time.time() - start_time
    print(f"\n✅ Done! Total: {stats.total_attempted} | Success: {stats.total_success} | Time: {elapsed:.2f}s")
    print(f"📁 Output: {image_dir}")
    stats.print_summary()


if __name__ == "__main__":
    run_full_production(
        num_samples_per_type=250,  # שנה לכמות הרצויה
        fast_ratio=0.8,
        output_dir="final_dataset",
        enable_parallel=True,
        save_html_files=False,
        save_individual_json=False,
        save_image_files=True
    )
#components_to_generate = ["RESISTOR","CAPACITOR", "DIODE","MOSFET","VOLTAGE_REGULATOR","INDUCTOR","BJT","OPAMP"]
#components_to_generate=None,  # רוץ על כל הרכיבים שמימשת

🚀 Starting Production Run...


🏭 Generating Data:   0%|                                                                 | 0/2000 [00:00<?, ?sample/s]

🔄 Processing with 4 workers...


🏭 Generating Data:  37%|███████████████████▋                                 | 745/2000 [53:51<3:01:12,  8.66s/sample]

# פרומפט ליצירת רכיבים נוספים

🎯 PROFESSIONAL PROMPT: Strategy Implementation for Synthetic Datasheet Generator

You are an Expert System Architect & Device Physicist.
Your task is to implement a Python Strategy Class for a specific electronic component within an existing synthetic data generation framework.

📚 SYSTEM CONTEXT (Codebase Definitions)
You are inheriting from the following abstract base class. You MUST respect these signatures exactly:

```python
class ComponentStrategy(ABC):
    @abstractmethod
    def create_context(self, schema: Dict, requested_corner: Optional[str]) -> GenerationContext:
        pass

    @abstractmethod
    def calculate_base_value(self, key: str, limits: Any, context: GenerationContext) -> Optional[float]:
        pass

    @abstractmethod
    def apply_correlations(self, result: DatasheetResult) -> None:
        pass
   
    @abstractmethod
    def create_custom_parameter(self, key: str, context: GenerationContext, param_def: Dict) -> Optional[GeneratedParameter]:
        # Hook for creating complex parameters manually
        return None
    @dataclass(frozen=True)
class GenerationContext:
    sample_id: str
    component_type: str
    package: str
    archetype: str
    tolerance: float
    process_corner: str = "TYPICAL"
    package_index: Optional[int]
    extras: Dict[str, Any]

@dataclass
class GeneratedParameter:
    key: str
    label: str
    symbol: str
    section: str
    value_min: Optional[float]
    value_typ: Optional[float]
    value_max: Optional[float]
    unit: str
    condition: str
    spec_type: 'SpecType'
    engineering_class: 'EngineeringClass'

class DatasheetResult:
    context: GenerationContext
    parameters: List[GeneratedParameter]

class PhysicalCorrelationEngine:
    """Helper methods for physics-based adjustments"""
    @staticmethod
    def adjust_rds_based_on_vds(vds_value: float, base_rds: float) -> float: ...
    
    @staticmethod
    def adjust_forward_current_based_on_package(package: str, base_current: float) -> float: ...
    
    @staticmethod
    def adjust_capacitance_vs_voltage(cap_base: float, voltage_rating: float, archetype: str) -> float: ...
    
    @staticmethod
    def adjust_esr_based_on_capacitance(cap_value: float, base_esr: float) -> float: ...
    
    @staticmethod
    def adjust_power_dissipation_based_on_package(package: str, base_power: float) -> float: ...

# ------------------------------------------------
Existing Helpers available in the scope:

PhysicalCorrelationEngine (Static methods for physics rules).

GenerationContext (Dataclass with fields: package, archetype, tolerance, package_index, etc.).

GeneratedParameter, DatasheetResult, SpecType, EngineeringClass.

📥 INPUT DATA (Source of Truth) Use the following dictionary schema to define limits and archetypes. ⚠️ CRITICAL: Use ONLY keys defined here. Do not hardcode numbers if they exist in this DB.

JSON

<--- PASTE_COMPONENT_JSON_HERE --->
🛠️ IMPLEMENTATION TASK

1️⃣ Create a Constants Class Name: [Component]Constants (e.g., MosfetConstants) Include:

TOLERANCE_MAP: Dict mapping archetype names to tolerance floats.

PHYSICS_CONSTANTS: Any lookup tables needed.

UNIT_CONVERSION: If special conversions are needed beyond standard ones.

2️⃣ Create the Strategy Class Name: [Component]Strategy (Inherits from ComponentStrategy)

Required Methods Implementation Details:

A. create_context(self, schema: Dict, requested_corner: Optional[str]) -> GenerationContext

Logic:

Randomly select an archetype from schema['archetypes'].

Identify the target package: Look for the 'package_code' parameter in schema['PACKAGE'].

Extract the list of packages for the selected archetype.

Randomly select a package_index and the package string.

Determine tolerance using your Constants class.

Return: GenerationContext populated with these values.

B. create_custom_parameter(self, key: str, context: GenerationContext, param_def: Dict) -> Optional[GeneratedParameter]

Primary Use: Use this hook for the "Main Definition Parameter" (e.g., vdss for MOSFET, inductance for Inductor) if it requires discrete steps (like standard voltage ratings: 600V, 650V) rather than random floats.

Logic: If handling the key, calculate min/typ/max manually and return a full GeneratedParameter.

Return None for all other keys (let the generator handle them).

C. calculate_base_value(self, key: str, limits: Any, context: GenerationContext) -> Optional[float]

Responsibility: Return the raw Typical value (float).

Logic:

If key is package_code, return context.package.

If key is a physical dimension (length, width, etc.), use context.package_index to pick the correct value from limits (if limits is a list).

For electrical parameters: Resolve limits based on context.archetype. If it's a list/range, pick a random value (using uniform distribution).

Handle package-dependent values (like Power Rating) if applicable.

D. apply_correlations(self, result: DatasheetResult) -> None

Responsibility: iterate over result.parameters and refine values based on physics.

Usage: Call methods from PhysicalCorrelationEngine if applicable (e.g., adjust_rds_based_on_vds).

Implement custom physics rules (e.g., "High Voltage -> High Rds(on)", "Small Package -> Low Power").

📝 OUTPUT REQUIREMENTS

Provide pure Python code.

Include necessary imports (random, typing, dataclasses, etc.).

Ensure code is "Copy-Paste" ready for the existing notebook.

Do not redefine the base classes (ComponentStrategy, DatasheetGenerator).

Context:I am developing a synthetic data generator for electronic components (specifically [COMPONENT_NAME]).Previous iterations failed due to:"The Unit Trap": Treating input values (e.g., 2.2) as Base SI units, creating "exploding" components."Identity Crisis": The metadata claims the package is "SOT-23", but the data table lists "CSP" because the code picked a random value from the schema instead of the selected context."Frankenstein Components": Mismatched physical dimensions vs. ratings."Crash Loops": Missing Enums or NoneType math errors.Objective:Refactor the Python code to implement a Robust, Physics-Aware Strategy that maintains Strict SI Units internally, enforces Identity Consistency, and presents Human-Readable Units externally.Refactoring Requirements:0. Setup & ImportsImports: dataclasses, typing, random, math, enum, re.Enums (Must Define): SpecType, EngineeringClass.1. Refactor [COMPONENT_NAME]ConstantsSource of Truth: Create PACKAGE_SPECS mapping Package Name -> Dimensions & Hard Limits (in standard units like uF*V for CV product, Watts for power).2. Refactor [COMPONENT_NAME]StrategyImplement the following architecture strictly:A. create_context: Standard hierarchical selection (Archetype -> Valid Package -> Context). Ensure all primary SI values are initialized in extras (even as None).B. _generate_primary_values (The Smart Normalization Engine):Smart Normalization: When parsing numeric limits from Schema/JSON, check magnitude. If input is > 5.0 (for currents/caps) and not a Supercap/Power Module, assume micro/milli units and convert to SI Base Units (Farad, Ampere).Physics Enforcement: Perform filtering (e.g., $P < P_{max}$) using SI Units.Result: Return a dictionary of locked values in SI Units.C. create_custom_parameter (The Display Layer):Locking: Retrieve the SI value from context.extras (generate & lock if missing).Presentation: NEVER use the SI value directly for value_typ.Auto-Scaling: Pass the SI value to _auto_scale(val, 'F'). Use the returned scaled value and unit for the GeneratedParameter.D. calculate_base_value (Deterministic Lookup & Identity Safety):CRITICAL FIX - Identity Consistency: If the key is 'package_code', 'case_style', or 'outline', you MUST return context.package. Do NOT allow fallback to random selection.Physical Props: Return dimensions/weight from PACKAGE_SPECS.Physics Logic: For dependent parameters (e.g., Min Input Voltage, Power Dissipation), calculate using the locked SI values from context.extras. Handle None values safely.Fallback: Only use _pick_generic as a last resort for non-critical string fields.E. apply_correlations:Apply physics formulas using SI values from context.Safety: Check if dependencies exist before calculating (e.g., if vout is not None).3. Helper Methods (Mandatory)_auto_scale(value, base_unit): Converts 2.2e-6, 'F' -> (2.2, 'µF')._snap_to_e_series(value): Safe log calculation._extract_numeric_list(limits): Safely parses JSON strings/lists._apply_rounding(...): Rounds based on magnitude.Output:Provide the full, clean Python code.Verification: Ensure calculate_base_value explicitly handles package_code before any random fallback.